## EDA / Data Preprocessing
This notebook goes over scraping the event results from Olympedia.org, it currently only consists of the event results for the men's and women's track and field events but not including relay team races. Since I am trying to classify positive or negative PED use, the relay event results won't provide the model with relevant data. 

This notebook starts by loading in the necessary libraries for exploratory data analysis and for reading in .pdf/.csv files.

I then create two lists, holding the names in the athlete dataframe and the names in the Doping dataframe. I used Fuzzy Wuzzy to find partial matches on athlete names and append both names from the seperate dataframes to a dictionary. I was able to use the dictionary to replace the names in the athlete's dataframe afterwards.

One complete dataframe was created with an added feature 'flagged' indicating whether or not the athlete has tested positive for PED use.

From there, I begin scraping the result tables for the individual track events and merge the results with the complete dataframe by matching names. The rows being dropped are primarily due to the athlete not starting/finishing the race. This may also be due to the athlete being disqualified for a false start, not staying in the correct lane during the race, or PED use.

The dataframe is then preprocessed for modeling.
- filling missing values
- converting categorical features to numerical values
- converting event results in minutes to seconds so all events are on the same scale
- removing athlete records that do not have any results

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd
import numpy as np
import pdfplumber
import tabula
import seaborn as sns
from Olympic_PED_use.src import functions as fn
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

from bs4 import BeautifulSoup
import certifi
import urllib3
import re
from csv import DictReader, DictWriter
import datetime as dt

import glob
import fuzzywuzzy

pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 500)

#### Reading the athlete_events.csv file that contains athlete records (height, weight, age, sex, etc.)

In [207]:
athlete_df = pd.read_csv('../data/athletes_dataset/athlete_events.csv')
athlete_df.columns = [x.lower() for x in athlete_df.columns]

In [208]:
athlete_df = athlete_df.loc[athlete_df.sport == 'Athletics']

#### Removing team events as they won't provide data on specific athletes.
#### Removing marathon races since they only provide one heat of results, I obtained event results with more than one heat.
#### Removing events that are combined track and field events since the scoring is different than of the regular track and field events.

In [209]:
athlete_df = athlete_df.loc[athlete_df.year > 2002]
athlete_df = athlete_df.loc[athlete_df.year < 2018]
athlete_df = athlete_df.loc[athlete_df.event != "Athletics Men's 4 x 100 metres Relay"]
athlete_df = athlete_df.loc[athlete_df.event != "Athletics Men's 4 x 400 metres Relay"]
athlete_df = athlete_df.loc[athlete_df.event != "Athletics Women's 4 x 100 metres Relay"]
athlete_df = athlete_df.loc[athlete_df.event != "Athletics Women's 4 x 400 metres Relay"]
athlete_df = athlete_df.loc[athlete_df.event != "Athletics Men's Marathon"]
athlete_df = athlete_df.loc[athlete_df.event != "Athletics Women's Marathon"]
athlete_df = athlete_df.loc[athlete_df.event != "Athletics Women's 4 x 100 metres Relay"]
athlete_df = athlete_df.loc[athlete_df.event != "Athletics Women's Heptathlon"]
athlete_df = athlete_df.loc[athlete_df.event != "Athletics Men's Decathlon"]

In [210]:
doping_df = fn.create_wiki_doping()

In [211]:
athlete_names = list(athlete_df.name.unique())
doping_names = list(doping_df.name.unique())

#### Since there are different spellings and punctuations for the athlete names in each dataset, I am going to use fuzzy wuzzy to return a list of matches above 65 percent. I append the original name, matched name, and the match ratio to a new dictionary. Next, I will print the keys of the dictionary and create a new dictionary of names to replace in the doping dataframe.

In [212]:
def match_names(term, list_names, min_score=0):
    max_score = -1
    max_name = ''
    for x in list_names:
        score = fuzz.ratio(term, x)
        if (score > min_score) & (score > max_score):
            max_name = x
            max_score = score
    return (max_name, max_score)

In [99]:
dict_list = []
for x in doping_names:
    match = match_names(x, athlete_names, 65)
    dict_ = {}
    dict_.update({'doping_name': x})
    dict_.update({'match_name': match[0]})
    dict_.update({'score': match[1]})
    dict_list.append(dict_)

Names from the event results being matched to the names in the athlete_events.csv

In [189]:
for k,v in athlete_events_names_to_change.items():
    print('"{}":"{}",'.format(v,k))

"":"A. G. Kruger",
"Aadam Ismaeel Khamis":"Aadam Khamis",
"Aaron Nigel Armstrong":"Aaron Armstrong",
"Aaron Brown":"Aaron Brown",
"Tarek Boukensa":"Aauri Bokesa",
"Ababel Yeshaneh Birhane":"Ababel Yeshaneh",
"Abadi Hadis Embaye":"Abadi Hadis",
"Abbas Samimi":"Abbas Samimi",
"Abigail 'Abbey' D'Agostino":"Abbey D'Agostino",
"Abdal Salam Al-Dabaji":"Abdal Salam Al-Dabaji",
"Abdalelah Haroun Hassan":"Abdalelah Haroun",
"Abdalla Abdelgadir El-Sheikh":"Abdalla Abdelgadir",
"Abdalla Yousif":"Abdalla Yousif",
"Abdel Aziz Merzougui Noureddine":"Abdel Aziz Merzougui",
"Abdelati El-Guesse":"Abdelati El-Guesse",
"Abdelatif Chemlal":"Abdelatif Chemlal",
"Abdelaziz Ennaji El-Idrissi":"Abdelaziz Ennaji El-Idrissi",
"Abdul Aziz Zakari":"Abdelhak Zakaria",
"Abdelhakim Maazouz":"Abdelhakim Maazouz",
"Abdelkader Hachlaf":"Abdelkader Hachlaf",
"Abdellah Falil":"Abdellah Falil",
"Abdelmalik Lahoulou":"Abdelmalik Lahoulou",
"Abderrahim Al-Goumri":"Abderrahim Goumri",
"Abdihakim "Abdi" Abdirahman":"Abdi Abdi

In [10]:
athlete_events_names_to_change = {
                "Aadam Ismaeel Khamis":"Aadam Khamis",
                "Aaron Nigel Armstrong":"Aaron Armstrong",
                "Aaron Brown":"Aaron Brown",
                "Ababel Yeshaneh Birhane":"Ababel Yeshaneh",
                "Abadi Hadis Embaye":"Abadi Hadis",
                "Abbas Samimi":"Abbas Samimi",
                "Abigail 'Abbey' D'Agostino":"Abbey D'Agostino",
                "Abdal Salam Al-Dabaji":"Abdal Salam Al-Dabaji",
                "Abdalelah Haroun Hassan":"Abdalelah Haroun",
                "Abdalla Abdelgadir El-Sheikh":"Abdalla Abdelgadir",
                "Abdalla Yousif":"Abdalla Yousif",
                "Abdel Aziz Merzougui Noureddine":"Abdel Aziz Merzougui",
                "Abdelati El-Guesse":"Abdelati El-Guesse",
                "Abdelatif Chemlal":"Abdelatif Chemlal",
                "Abdelaziz Ennaji El-Idrissi":"Abdelaziz Ennaji El-Idrissi",
                "Abdelhakim Maazouz":"Abdelhakim Maazouz",
                "Abdelkader Hachlaf":"Abdelkader Hachlaf",
                "Abdellah Falil":"Abdellah Falil",
                "Abdelmalik Lahoulou":"Abdelmalik Lahoulou",
                "Abderrahim Al-Goumri":"Abderrahim Goumri",
                'Abdihakim "Abdi" Abdirahman':"Abdi Abdirahman",
                "Abdi Waiss Mouhyadin":"Abdi Waiss Mouhyadin",
                "Abdinasir Said Ibrahim":"Abdinasir Ibrahim",
                "Abdoulaye Wagne":"Abdoulaye Wagne",
                "Abdoullah Bamoussa":"Abdoullah Bamoussa",
                "Abdul Aziz Zakari":"Abdul Aziz Zakari",
                "Abdul Buhari":"Abdul Buhari",
                "Abdul Rahman Sultan Al-Nubi Faraj":"Abdul Rahman Al-Nubi",
                "Abdul Rahman Ahmed Suleiman":"Abdul Rahman Suleiman",
                "Abdul Rashid":"Abdul Rashid",
                "Abdul Wahab Zahiri":"Abdul Wahab Zahiri",
                "Abdulaziz Ladan Mohammed":"Abdulaziz Ladan",
                "Abdulla Mohamed Hussein":"Abdulla Mohamed Hussein",
                "Abdullah Abkar Mohammed":"Abdullah Abkar Mohammed",
                "Abdullah Abdulaziz Al-Joud":"Abdullah Al-Joud",
                "Abdullah Said Al-Sooli":"Abdullah Al-Sooli",
                "Abeba Aregawi Gebretsadik":"Abeba Aregawi",
                "Abraham Chebii Koskei":"Abraham Chebii",
                "Abraham Chepkirwok":"Abraham Chepkirwok",
                "Abraham Naibei Cheroben":"Abraham Cheroben",
                "Abraham Kiplimo":"Abraham Kiplimo",
                "Abreham Cherkos Feleke":"Abreham Cherkos",
                "Abubakar Abbas Abbas":"Abubakar Abbas",
                "Abubaker Ali Kamal":"Abubaker Ali Kamal",
                "Abubaker Haydar Abdalla":"Abubaker Haydar",
                "Abubaker Kaki Khamis":"Abubaker Kaki Khamis",
                "Achraf Tadili":"Achraf Tadili",
                "Adam Ahmed R. Gemili":"Adam Gemili",
                "Adam Scott Harris":"Adam Harris",
                "Adam Kolasa":"Adam Kolasa",
                "Adam Kszczot":"Adam Kszczot",
                "Adam Leslie Miller":"Adam Miller",
                "Ella Nelson":"Adam Nelson",
                "Adam Ptek":"Adam Ptáček",
                "Adam Rutter":"Adam Rutter",
                "Adama Jammeh":"Adama Jammeh",
                "Adama N'Jie":"Adama N'Jie",
                "Adel Mechaal Mechaal":"Adel Mechaal",
                "Adelina Gavril":"Adelina Gavrilă",
                "Adem Hecini":"Adem Hecini",
                "Adil El-Kaouch":"Adil Kaouch",
                "Adina Anton":"Adina Anton",
                "Adrian Blincoe":"Adrian Blincoe",
                "Adrian Kent Russell Durant":"Adrian Durant",
                "Adrian Griffith":"Adrian Griffith",
                "Adrian O'Dwyer":"Adrian O'Dwyer",
                "Adriana Fernndez Miranda":"Adriana Fernández",
                "Adriatik Hoxha":"Adriatik Hoxha",
                "Adrien Deghelt":"Adrien Deghelt",
                "Adrienne Power":"Adrienne Power",
                "Adrin Zsolt Annus":"Adrián Annus",
                "Afa Ismail":"Afa Ismail",
                "Afia Neliah Charles":"Afia Charles",
                "Afroditi Skafida":"Afroditi Skafida",
                "Agnes Maryna Samaria":"Agnes Samaria",
                "Agnese Pastare":"Agnese Pastare",
                "Agnieszka Dygacz":"Agnieszka Dygacz",
                "Agnieszka Szwarng":"Agnieszka Szwarnóg",
                "Agn Visockait-Eggerth":"Agnė Visockaitė-Eggerth",
                "Ahmad Hassan Abdullah":"Ahmad Hassan Abdullah",
                "Ahmad Hazer":"Ahmad Hazer",
                "Ahmed Ali":"Ahmed Ali",
                "Ahmed Bader Magour":"Ahmed Bader",
                "Ahmed Yves Douhou":"Ahmed Douhou",
                "Aicha Bilal Fall":"Aicha Fall",
                "Aikaterini Vongoli":"Aikaterini Vongoli",
                "Aim-Issa Nthp":"Aimé-Issa Nthépé",
                "Ainrs Kovals":"Ainārs Kovals",
                "Airin Palyt":"Airinė Palšytė",
                "Aisea Tohi":"Aisea Tohi",
                "Aisha Praught (-Leer)":"Aisha Praught",
                "Aishath Reesha":"Aishath Reesha",
                "Aissata Tour":"Aissata Touré",
                "Aje Wilson":"Ajeé Wilson",
                "Ak Hafiy Tajuddin Rositi":"Ak Hafiy Tajuddin Rositi",
                "Akani Simbine":"Akani Simbine",
                "Akeem Haynes":"Akeem Haynes",
                "Akela Jones":"Akela Jones",
                "Akihiko Nakamura":"Akihiko Nakamura",
                "Akosua Serwaa":"Akosua Serwah",
                "Iman Essa Jasim":"Alaa Jassim",
                "Alan Lawrence Culpepper":"Alan Culpepper",
                "Alan Webb":"Alan Webb",
                "Alana Barber":"Alana Barber",
                "Alana Boyd":"Alana Boyd",
                "Albert Minczr":"Albert Minczér",
                "Alberto Garca Fernndez":"Alberto García",
                "Alberto Paulo":"Alberto Paulo",
                "Aldemir Gomes da Silva Jnior":"Aldemir Gomes",
                "Aldenay Vasallo Machado":"Aldenay Vasallo",
                "Aldwyn Sappleton":"Aldwyn Sappleton",
                "Aleen May Bailey":"Aleen Bailey",
                "Aleesha Barber":"Aleesha Barber",
                "Aleixo-Platini Menga":"Aleixo-Platini Menga",
                "Alejandra Garca Flood":"Alejandra García",
                "Alejandra Ortega Sols":"Alejandra Ortega",
                "Alejandro Manuel Crdeas Robles":"Alejandro Cárdenas",
                "Alejandro Surez Velzquez":"Alejandro Suárez",
                "Aleksander Tammert":"Aleksander Tammert",
                "Aleksandr Yaroslavovich Borichevsky":"Aleksandr Borichevsky",
                "Aleksandr Aleksandrovich Derevyagin":"Aleksandr Derevyagin",
                "Aleksandr Ivanovich Glavatsky":"Aleksandr Glavatsky",
                "Aleksandr Krivchinkov":"Aleksandr Krivchinkov",
                "Aleksandr Nikolayevich Malashevich":"Aleksandr Malashevich",
                "Aleksandr Andreyevich Shustov":"Aleksandr Shustov",
                "Aleksandra Viktorovna Antonova":"Aleksandra Antonova",
                "Aleksandra Andreyevna Fedoriva":"Aleksandra Fedoriva",
                "Aleksandra Kotlyarova":"Aleksandra Kotlyarova",
                "Aleksandra Kotlyarova":"Aleksandra Romanova",
                "Aleksandra Vojnevska":"Aleksandra Vojnevska",
                "Aleksey Nikolayevich Dryomin":"Aleksey Dryomin",
                "Aleksey Lesnichy":"Aleksey Lesnichy",
                "Aleksey Pogorelov":"Aleksey Pogorelov",
                "Aleksey Sergeyevich Zagorny":"Aleksey Zagorny",
                "Alemayehu Bezabeh Desta":"Alemayehu Bezabeh",
                "Alemitu Bekele Degfa":"Alemitu Bekele",
                "Alenka Bikar":"Alenka Bikar",
                "Alessandra Picagevicz":"Alessandra Picagevicz",
                "Alessandra Nobre Resende":"Alessandra Resende",
                "Alessandro Gandellini":"Alessandro Gandellini",
                "Alessandro Talotti":"Alessandro Talotti",
                "Alex Amankwah":"Alex Amankwah",
                'Aleksandr "Alex" Averbukh':"Alex Averbukh",
                "Alex Beddoes":"Alex Beddoes",
                'Alexander "Alex" Hartmann':"Alex Hartmann",
                "Alex Rose":"Alex Rose",
                "Alex Wilson":"Alex Wilson",
                "Alex Wright":"Alex Wright",
                "Alexander John":"Alexander John",
                "Alexander Kosenkow":"Alexander Kosenkow",
                "Alexander Lerionka Sampao":"Alexander Sampao",
                "Alexander Vieweg":"Alexander Vieweg",
                "Alexandra Papageorgiou":"Alexandra Papageorgiou",
                "Alexandra Tavernier":"Alexandra Tavernier",
                "Alexandra Tsisiou (Nasta-)":"Alexandra Tsisiou",
                "Alexandros Papadimitriou":"Alexandros Papadimitriou",
                "Alexandros Papamikhail":"Alexandros Papamikhail",
                "Alexia Pappas":"Alexia Pappas",
                "Alfred Kipketer":"Alfred Kipketer",
                "Alfredo Deza Ganoza":"Alfredo Deza",
                "Alhaji Jeng":"Alhaji Jeng",
                "Ali Abdalla Afringi":"Ali Abdalla",
                "Ali Ahmed Al-Amri":"Ali Ahmed Al-Amri",
                "Ali Mohamed Gharib Al-Zinkawi":"Ali Al-Zinkawi",
                "Ali Zayed Mabrouk El-Zaidi":"Ali Mabrouk El-Zaidi",
                "Ali Massaoudi":"Ali Massaoudi",
                "Ali Mohamed Al-Balooshi":"Ali Mohamed Al-Balooshi",
                "Ali Shareef":"Ali Shareef",
                "Aliann Tabitha Omalara Pompey":"Aliann Pompey",
                "Alice Aprot Nawowuna":"Alice Aprot",
                "Alice Matjkov Honov":"Alice Matějková",
                "Alice Schmidt (-Kehaya)":"Alice Schmidt",
                "Alice Jemeli Timbilil":"Alice Timbilil",
                "Alicia Brown":"Alicia Brown",
                "Claudia Alina Cucerzan":"Alina Cucerzan",
                "Alina Ramona Militaru":"Alina Militaru",
                "Alina Gennadyevna Talay":"Alina Talay",
                "Alison Magdalene George":"Alison George",
                "Alissa Kallinikou":"Alissa Kallinikou",
                "Alistair Ian Cragg":"Alistair Cragg",
                "Aliyah Abrams":"Aliyah Abrams",
                "Allan Scott":"Allan Scott",
                "Alleyne Jeremy Francique":"Alleyne Francique",
                "Allison J. Beckford":"Allison Beckford",
                "Allison Pauline Peter":"Allison Peter",
                "Allison Randall":"Allison Randall",
                "Allyson Michelle Felix":"Allyson Felix",
                "Almensh Belete":"Almensh Belete",
                "Alonso Reno Edward Henry":"Alonso Edward",
                "Altobeli Santos da Silva":"Altobeli Silva",
                "Alwyn Abraham Myburgh":"Alwyn Myburgh",
                "Alyn Camara":"Alyn Camara",
                "Alyona Viktorovna Dubitskaya (Grishko-)":"Alyona Dubitskaya",
                "Alysha Newman":"Alysha Newman",
                "Alysia Johnson-Montao":"Alysia Johnson-Montaño",
                "Alyssa Conley":"Alyssa Conley",
                "Alyxandria Treasure":"Alyxandria Treasure",
                "Amadou Ndiaye":"Amadou Ndiaye",
                "Amaechi Morton":"Amaechi Morton",
                "Amaka Ogoegbunam":"Amaka Ogoegbunam",
                "Amalie Hammild Iuel":"Amalie Iuel",
                "Amaliya Sharoyan":"Amaliya Sharoyan",
                "Aman Wote Fete":"Aman Wote",
                "Amanda Yvette Bingson":"Amanda Bingson",
                "Amandine Allou Affou":"Amandine Allou Affoué",
                "Amanmyrat Hommadow":"Amanmyrat Hommadow",
                "Amantle Montsho":"Amantle Montsho",
                "Amanuel Mesel Tikue":"Amanuel Mesel",
                "Amaru Reto Schenkel":"Amaru Schenkel",
                "Amaurys Ral Valle Mencia":"Amaurys Valle",
                "Amber Campbell":"Amber Campbell",
                'Ambwene "Amber" Simukonda':"Amber Simukonda",
                "Ameer Kenneth Webb":"Ameer Webb",
                "Amel Tuka":"Amel Tuka",
                "Amela Terzi":"Amela Terzić",
                "Amin Abraham Paul Nikfar":"Amin Nikfar",
                "Amina Barsham Bakhit":"Amina Bakhit",
                "Amina Bettiche":"Amina Bettiche",
                "Aminata Diouf":"Aminata Diouf",
                "Aminata Kamissoko":"Aminata Kamissoko",
                "Mohammed El-Amine Belferar":"Amine Belferar",
                "Amine El Manaoui":"Amine El Manaoui",
                "Amine Lalou":"Amine Laâlou",
                "Amor Ben Yahia":"Amor Ben Yahia",
                "Amy Atkinson":"Amy Atkinson",
                "Amy Elizabeth Hastings-Cragg":"Amy Hastings-Cragg",
                "Amy Mback Thiam":"Amy Mbacké Thiam",
                "Amy Sne":"Amy Sène",
                "Amy Yoder Begley":"Amy Yoder Begley",
                "Amlie Perrin":"Amélie Perrin",
                "Ana Maria Groza":"Ana Maria Groza",
                "Ana Kilistina Po'uhila":"Ana Po'Uhila",
                "Ana Veronica Rodean":"Ana Rodean",
                "Ana imi":"Ana Šimić",
                "Anas Beshr":"Anas Beshr",
                "Anaso Jobodwana":"Anaso Jobodwana",
                "Anastasia Kelesidou":"Anastasia Kelesidou",
                "Anastasiya Vasilyevna Ivanova-Shvedova":"Anastasiya Ivanova-Shvedova",
                "Anastasiya Aleksandrovna Kapachinskaya":"Anastasiya Kapachinskaya",
                "Anastasiya Valeryevna Kudinova":"Anastasiya Kudinova",
                "Anastasiya Sergeyevna Mironchik-Ivanova":"Anastasiya Mironchik-Ivanova",
                "Anastasiya Sergeyevna Puzakova":"Anastasiya Puzakova",
                "Anastasiya Volodymyrivna Rabcheniuk":"Anastasiya Rabcheniuk",
                "Anastasiya Svechnikova":"Anastasiya Savchenko",
                "Anastasiya Valeryevna Soprunova":"Anastasiya Soprunova",
                "Anastasiya Svechnikova":"Anastasiya Svechnikova",
                "Anastasiya Aleksandrovna Yatsevich":"Anastasiya Yatsevich",
                "Anay Tejeda Quesada":"Anay Tejeda",
                "Ancua Bobocel":"Ancuța Bobocel",
                "Peterson dos Santos":"Anderson Dos Santos",
                "Andra Kareem Manson":"Andra Manson",
                "Andra Hakli":"Andraš Haklić",
                "Andre De Grasse":"Andre De Grasse",
                "Andrea Bettinelli":"Andrea Bettinelli",
                "Andrea Melissa Blackett":"Andrea Blackett",
                "Andrea Bunjes":"Andrea Bunjes",
                "Andrea Mikls":"Andrea Ferris",
                "Andrea Geubelle":"Andrea Geubelle",
                "Andrea Ivanevi":"Andrea Ivančević",
                "Andrea Longo":"Andrea Longo",
                "Andrea Seccafien":"Andrea Seccafien",
                "Andrea Bunjes":"Andreas Bube",
                "Andreas Erm":"Andreas Erm",
                "Andreas Thorkildsen":"Andreas Thorkildsen",
                "Andreas Vojta":"Andreas Vojta",
                "Andreea Arsine":"Andreea Arsine",
                "Andreea Luiza Ogrzeanu":"Andreea Ogrăzeanu",
                "Andrei Marius Gag":"Andrei Gag",
                "Andrew Evans":"Andrew Evans",
                "Andrew Fisher":"Andrew Fisher",
                "Andrew Hinds":"Andrew Hinds",
                "Andrew Rock":"Andrew Howe",
                "Andrew Duncan Lemoncello":"Andrew Lemoncello",
                "Andrew Owusu":"Andrew Osagie",
                "Andrew Owusu":"Andrew Owusu",
                "Andrew Riley":"Andrew Riley",
                "Andrew Wheating":"Andrew Wheating",
                "Andrey Viktorovich Churilo":"Andrey Churilo",
                "Andrey Anatolyevich Mikhnevich":"Andrey Mikhnevich",
                "Andrey Mikhaylovich Talashko":"Andrey Talashko",
                "Andrey Sergeyevich Yepishin":"Andrey Yepishin",
                "Andriana Veselinova Banova":"Andriana Banova",
                "Andrius Gudius":"Andrius Gudžius",
                "Andriy Volodymyrovych Makarchev":"Andriy Makarchev",
                "Andriy Oleksiyovych Protsenko":"Andriy Protsenko",
                "Andriy Serhiyovych Semenov":"Andriy Semenov",
                "Andriy Volodymyrovych Sokolovskiy":"Andriy Sokolovskiy",
                "Andrus Vrnik":"Andrus Värnik",
                "Andr Bucher":"André Bucher",
                "Andr Domingos da Silva":"André Da Silva",
                "Andr Hhne":"André Höhne",
                "Andr Olivier":"André Olivier",
                "Andrs Arroyo":"Andrés Arroyo",
                'Andrew James "Andy" Baddeley':"Andy Baddeley",
                "Andr Bucher":"Andy Butchart",
                "Andy Gonzlez Nez":"Andy González",
                "Andy Grant":"Andy Grant",
                "Anel Oosthuizen":"Anel Oosthuizen",
                "Aneliya Yordanova-Kumanova":"Aneliya Yordanova-Kumanova",
                "Aneka Drahotov":"Anežka Drahotová",
                "Angela Moroanu":"Angela Moroșanu",
                "Angela Patricia Whyte":"Angela Whyte",
                "Angelika Cichocka":"Angelika Cichocka",
                "Angeliki Tsiolakoudi":"Angeliki Tsiolakoudi",
                "Anhel Alberta Cape":"Anhel Cape",
                "Ani Khachikyan":"Ani Khachikyan",
                "Anicka Newell":"Anicka Newell",
                "Anier Bou":"Anier Boué",
                "Anik Klovics":"Anikó Kálovics",
                "Anil Kumar Sangwan":"Anil Kumar Sangwan",
                "Anis Viktorovich Ananenko":"Anis Ananenko",
                "Anis Selmouni":"Anis Selmouni",
                "Anita Brgger (-Belz)":"Anita Brägger",
                "Anita Liepia (Strte-)":"Anita Liepiņa",
                "Anita Mrton":"Anita Márton",
                "Anita Wodarczyk":"Anita Włodarczyk",
                "Anjelina Nadai Lohalith":"Anjelina Nadai Lohalith",
                "Anju Bobby George (Markose-)":"Anju Bobby George",
                "Anju Takamizawa":"Anju Takamizawa",
                "Ankit Sharma":"Ankit Sharma",
                "Anna Aleksandrovna Alminova":"Anna Alminova",
                "Anna Mikhaylovna Avdeyeva":"Anna Avdeyeva",
                "Anna Vladimirovna Bulgakova":"Anna Bulgakova",
                "Anna Ivanovna Drabenya":"Anna Drabenya",
                "Anna Foitidou":"Anna Foitidou",
                "Anna Iljutenko":"Anna Iljuštšenko",
                "Anna Jagaciak-Michalska":"Anna Jagaciak",
                "Anna Jakubczak-Pawelec":"Anna Jakubczak",
                "Anna Katarzyna Kiebasiska":"Anna Kiełbasińska",
                "Anna Mykolavna Kornuta":"Anna Kornuta",
                "Anna Mozgunova (Brel-)":"Anna Mozgunova",
                "Anna Marta Olichwierczuk-Jesie":"Anna Olichwierczuk-Jesień",
                "Anna Pacholak (-Guzowska)":"Anna Pacholak",
                "Anna Viktorovna Pyatykh":"Anna Pyatykh",
                "Anna Rogowska":"Anna Rostkowska",
                "Anna Rh":"Anna Rüh",
                "Anna Skidan":"Anna Skidan",
                "Anna Therese Sderberg":"Anna Söderberg",
                "Anna Willard (-Pierce)":"Anna Willard",
                "Anna-Emilie Mller":"Anna-Emilie Møller",
                "Marie Annabelle Jennifer Lascar":"Annabelle Lascar",
                "Anne Johanna Halkivaha":"Anne Halkivaha",
                "Anne Zagr":"Anne Zagré",
                'Anneliese "Annie" Rubie':"Annie Rubie",
                "Annika Roloff":"Annika Roloff",
                "Annika Suthe":"Annika Suthe",
                "Anninos Markoullidis":"Anninos Markoullidis",
                "Annsert Whyte":"Annsert Whyte",
                "Anselmo Gomes da Silva":"Anselmo Da Silva",
                "Anson Henry":"Anson Henry",
                "Antar Zerguelaine":"Antar Zerguelaïne",
                "Anthonique Strachan":"Anthonique Strachan",
                "Anthony N. Famiglietti":"Anthony Famiglietti",
                "Anthony Michael Romaniw":"Anthony Romaniw",
                "Antigoni Ntrismpioti":"Antigoni Ntrismpioti",
                "Antje Mldner-Schmidt":"Antje Möldner-Schmidt",
                "Antoine Xavier Adams":"Antoine Adams",
                "Antoine Gakeme":"Antoine Gakeme",
                "Antonn alsk":"Anton Galkin",
                "Anton Kumn":"Anton Kučmín",
                "Anton Andreyevich Lyuboslavsky":"Anton Lyuboslavsky",
                "Antonella Palmisano":"Antonella Palmisano",
                "Antonia Stergiou":"Antonia Stergiou",
                "Antonietta Di Martino":"Antonietta Di Martino",
                "Antonina Vladimirovna Krivoshapka":"Antonina Krivoshapka",
                "Antonina Oleksandrivna Yefremova":"Antonina Yefremova",
                "Antonio Abadia Beci":"Antonio Abadía",
                "Antonio Alkana":"Antonio Alkana",
                "AntonioDavid Jimnez Pentinel":"Antonio Jiménez",
                "Antonio Alkana":"Antonio Reina",
                "Antoniya Yordanova (Stoyanova-)":"Antoniya Yordanova",
                "Antony Chemut":"Antony Chemut",
                "Antonn alsk":"Antonín Žalský",
                "Antti Hermanni Ruuskanen":"Antti Ruuskanen",
                "Antwon Toritseju Hicks":"Antwon Hicks",
                "Anzhela Anatolivna Balakhonova":"Anzhela Balakhonova",
                "Anzhelika Viktorivna Shevchenko":"Anzhelika Shevchenko",
                "Anta Hinriksdttir":"Aníta Hinriksdóttir",
                "Apostolos Parellis":"Apostolos Parellis",
                "April Steiner-Bennett":"April Steiner-Bennett",
                "Arabelly Orjuela Snchez":"Arabelly Orjuela",
                "Arantxa King":"Arantxa King",
                "Arasay Mara Thondike Santovenia":"Arasay Thondike",
                "Aretha Dawn Hill-Thurmond":"Aretha Hill-Thurmond",
                "Ari Pekka Mannio":"Ari Mannio",
                "Arialis Josefa Gandulla Martnez":"Arialis Gandulla",
                "Ariane Friedrich (Tempel-)":"Ariane Friedrich",
                "Ariannis Vichy Saladre":"Ariannis Vichy",
                "Arismendy Peguero Thicket":"Arismendy Peguero",
                "Arnaldo Lus Isaas Abrantes":"Arnaldo Abrantes",
                "Arne Gabius":"Arne Gabius",
                "Arnis Rumbenieks":"Arnis Rumbenieks",
                "Arnold Sorina":"Arnold Sorina",
                "Aron Kifle":"Aron Kifle",
                "Cherono Koech":"Aron Koech",
                "Arsen Sargsyan":"Arsen Sargsyan",
                "Artem Borysovych Rubanko":"Artem Rubanko",
                "Arthmon Hatungimana":"Arthémon Hatungimana",
                "Artur Brzozowski":"Artur Brzozowski",
                "Artur Noga":"Artur Noga",
                "Arturo Casado Alda":"Arturo Casado",
                "Arturo Chvez Korfiatis":"Arturo Chávez",
                "Artyom Andreyevich Dyatlov":"Artyom Dyatlov",
                "Artyom Viktorovich Kosinov":"Artyom Kosinov",
                "Asafa Kehine Powell":"Asafa Powell",
                "Asami Tanno (-Chiba)":"Asami Tanno",
                "Asenate Manoa":"Asenate Manoa",
                "Ashley Kelly":"Ashley Kelly",
                "Ashraf Amgad El-Seify":"Ashraf El-Seify",
                "Maria McCambridge":"Aska Cambridge",
                "Askale Tiksa Benti":"Askale Tiksa",
                "Asmae Leghzaoui":"Asmae Leghzaoui",
                "Asmir Kolainac":"Asmir Kolašinac",
                "Assunta Legnante":"Assunta Legnante",
                "Astrid Kumbernuss":"Astrid Kumbernuss",
                "Mubarak Ata Mubarak":"Ata Mubarak",
                "Athanasia Perra":"Athanasia Perra",
                "Athina Papagianni":"Athina Papagianni",
                "Ato Jabari Boldon":"Ato Boldon",
                "Ato Modibo-Stephens":"Ato Modibo",
                "Aubrey Smith":"Aubrey Smith",
                "Augustine Kiprono Choge":"Augustine Choge",
                "Augusto Dutra de Oliveira":"Augusto Dutra",
                "Augusto Ramos Soares":"Augusto Soares",
                "Augusto Jos Stanley Netepczuk":"Augusto Stanley",
                "Aunese Curreen":"Aunese Curreen",
                "Aurelia Trywiaska-Kollasch":"Aurelia Trywiańska-Kollasch",
                "Aurelie Alcindor":"Aurelie Alcindor",
                "Auriol Dongmo Mekemnang":"Auriol Dongmo Mekemnang",
                "Karl Axel Hrstedt":"Axel Härstedt",
                "Axelle Dauwens":"Axelle Dauwens",
                "Ayad Lamdassem El Mouhcin":"Ayad Lamdassem",
                "Ayako Kimura":"Ayako Kimura",
                "Hassan Ayanleh Souleiman":"Ayanleh Souleiman",
                "Ayanna Alexander":"Ayanna Alexander",
                "Ayanna Hutchinson":"Ayanna Hutchinson",
                "Ayman Bulatovna Kozhakhmetova":"Ayman Kozhakhmetova",
                "Ayomide Folorunso":"Ayomide Folorunso",
                "Ayuko Suzuki":"Ayuko Suzuki",
                "Azat Makhlyafovich Rakipov":"Azat Rakipov",
                "Aziz Lahbabi":"Aziz Lahbabi",
                "Aziz Ouhadi":"Aziz Ouhadi",
                "Azneem Ahmed":"Azneem Ahmed",
                "Ada Diop":"Aïda Diop",
                "Ada Sellam":"Aïda Sellam",
                "Assata Soulama":"Aïssata Soulama",
                "Babur Shakirdzhanov":"Babur Shakirdzhanov",
                "Bachana Khorava":"Bachana Khorava",
                "Bachir Mahamat":"Bachir Mahamat",
                "Bahaa Al-Farra":"Bahaa Al-Farra",
                "Bahadur Singh Sagoo":"Bahadur Singh",
                "Bai Xue":"Bai Xue",
                "Baljinder Singh":"Baljinder Singh",
                "Balzs Baji":"Balázs Baji",
                "Bamab Napo":"Bamab Napo",
                "Baraah Awadallah Marouane":"Baraah Awadallah",
                "Barakat Mubarak Al-Harthi":"Barakat Al-Harthi",
                "Barbara Kovcs":"Barbara Kovács",
                "Barbara Madejczyk":"Barbara Madejczyk",
                "Barbara Angela Parker":"Barbara Parker",
                "Barbara Petrhn":"Barbara Petráhn",
                "Barbara Pierre":"Barbara Pierre",
                "Barbara Szab":"Barbara Szabó",
                "Barbara piler":"Barbara Špiler",
                "Barbora Dibelkov":"Barbora Dibelková",
                "Barbora potkov":"Barbora Špotáková",
                "Bartosz Osewski":"Bartosz Osewski",
                "Bashar Omar Ibrahim":"Bashar Omar",
                "Bashir Abdi":"Bashir Abdi",
                "Basma Al-Eshosh":"Basma Al-Eshosh",
                "Baslio Emdio de Moraes Jnior":"Basílio De Moraes Júnior",
                "Batgereliin Mngntuyaa":"Batgereliin Möngöntuyaa",
                "Baya Rahouli":"Baya Rahouli",
                "Bayano Ali Kamani":"Bayano Kamani",
                "Beate Schrott":"Beate Schrott",
                "Beatrice Chepkoech Sitonik":"Beatrice Chepkoech",
                "Beatrice Roini Liua Faumuina":"Beatrice Faumuina",
                "Beatriz Pascual Rodrguez":"Beatriz Pascual",
                "Beauty Nazum Nahar":"Beauty Nazmun Nahar",
                'Rebecca "Becky" Holliday':"Becky Holliday",
                "Bedan Karoki Muchiri":"Bedan Karoki Muchiri",
                "Belal Mansoor Ali":"Belal Mansoor Ali",
                "Beleynesh Oljira Jemane":"Belaynesh Oljira",
                "Beln Adaluz Casetta":"Belén Casetta",
                'Benjamin Floyd "Ben" Blankenship':"Ben Blankenship",
                'Benjamin "Ben" St. Lawrence':"Ben St. Lawrence",
                "Ben Youssef Mit":"Ben Youssef Meïté",
                "Benedetta Ceccarelli":"Benedetta Ceccarelli",
                "Beniamin Kuciski":"Beniamin Kuciński",
                "Benik Abramyan":"Benik Abramyan",
                "Benita Jaye Willis-Johnson":"Benita Willis-Johnson",
                "Benjamin Kiplagat":"Benjamin Kiplagat",
                "Manuel Benjamn Enzema Owo":"Benjamín Enzema",
                "Benjamn Snchez Bermejo":"Benjamín Sánchez",
                "Benn Harradine":"Benn Harradine",
                "Bennie Jamaal Brazell":"Bennie Brazell",
                "Benson Kiplagat Seurei":"Benson Seurei",
                "Bereket Desta Gebretsadik":"Bereket Desta",
                "Bergur Ingi Petursson":"Bergur Pétursson",
                "Berhane Adere Debala":"Berhane Adere",
                "Berhanu Alemu":"Berhanu Alemu",
                "Bernard Rollen Williams, III":"Bernard Williams Iii",
                "Bernardo Baloyes Navas":"Bernardo Baloyes",
                "Berta Castells Franco":"Berta Castells",
                "Bertrand Moulinet":"Bertrand Moulinet",
                "Besu Sado Beko":"Besu Sado",
                "Beth Potter":"Beth Potter",
                "Betlhem Desalegn Belayneh":"Betlhem Desalegn Belayneh",
                "Betsy Saina":"Betsy Saina",
                "Bettina Mller (-Weissina)":"Bettina Müller",
                "Betty Heidler":"Betty Heidler",
                "Beverly McDonald (-Stewart)":"Beverly Mcdonald",
                "Beverly Sue Ramos":"Beverly Ramos",
                "Bian Ka":"Bian Ka",
                "Bianca Kappler":"Bianca Kappler",
                "Bianca Stuart":"Bianca Stuart",
                "Bibiro Ali Taher":"Bibiro Ali Taher",
                "Bilal Tabti":"Bilal Tabti",
                "Bilisuma Shugi Gelassa":"Bilisuma Shugi Gelassa",
                "Biljana Mitrovi-Topi":"Biljana Mitrović-Topić",
                "Ella Nelson":"Billy Nelson",
                "Binnaz Uslu":"Binnaz Uslu",
                "Birhan Getahun":"Birhan Getahun",
                "Birhanu Yemataw Balew":"Birhanu Balew",
                "Tayna Lawrence":"Bj Lawrence",
                "Blessing Okagbare (-Ighoteguonor)":"Blessing Okagbare",
                "Bo Fanfang":"Bo Fanfang",
                "Boaz Kiplagat Lalang":"Boaz Lalang",
                "Bobby Aloysius":"Bobby Aloysius",
                "Bogdan Vitalyevich Pishchalnikov":"Bogdan Pishchalnikov",
                "Gabriel Bogdan rus":"Bogdan Țărus",
                "Boitumelo Masilo":"Boitumelo Masilo",
                "Boleslav Skhirt'ladze":"Boleslav Skhirt'Ladze",
                "Boniface Kiprop Toroitich":"Boniface Kiprop",
                "Boris Berian":"Boris Berian",
                "Borja Vivas Jimnez":"Borja Vivas",
                "Bouchra Chaabi":"Bouchra Chaabi",
                "Bounkou Camara":"Bounkou Camara",
                "Botjan Bu":"Boštjan Buč",
                "Brad Walker":"Brad Walker",
                "Brahim Al-Kaazouzi":"Brahim Kaazouzi",
                "Brahim Taleb":"Brahim Taleb",
                "Braian Ezequiel Toledo":"Braian Toledo",
                "Bralon Shazam Taplin":"Bralon Taplin",
                "Bram Som":"Bram Som",
                "Brandon Jones":"Brandon Jones",
                "Brandon McBride":"Brandon Mcbride",
                "Brandon Allan Simpson":"Brandon Simpson",
                "Brandon Valentine-Parris":"Brandon Valentine-Parris",
                "Branko Zorko":"Branko Zorko",
                "Edward Ray Breaux Greer":"Breaux Greer",
                "Brenda Martinez":"Brenda Martinez",
                "Brenda N. Taylor":"Brenda Taylor",
                "Brendan Kyle Akeem Christian":"Brendan Christian",
                "Brendan Cole":"Brendan Cole",
                "Brendon Rodney":"Brendon Rodney",
                "Brenessa Thompson":"Brenessa Thompson",
                "Brett Morse":"Brett Morse",
                "Brett Robinson":"Brett Robinson",
                "Brian Dzingai":"Brian Dzingai",
                "Brian Johnson":"Brian Johnson",
                "Bridget Franek":"Bridget Franek",
                "Brigita Langerholc":"Brigita Langerholc",
                "Brigita Virbalyt-Dimien":"Brigita Virbalytė-Dimšienė",
                "Brigitte Mara Merlano Pjaro":"Brigith Merlano",
                "Brigitte Ann Foster-Hylton":"Brigitte Foster-Hylton",
                "Brittany Borman":"Brittany Borman",
                "Brittany Crew":"Brittany Crew",
                "Bronwyn Louise Eagles":"Bronwyn Eagles",
                "Brenessa Thompson":"Bronwyn Thompson",
                "Brooke Stratton":"Brooke Stratton",
                "Bruno Dominix Hortelano Roig":"Bruno Hortelano",
                "Bryan Barnett":"Bryan Barnett",
                "Btissam Boucif Lakhouad":"Btissam Lakhouad",
                "Buoban Pamang":"Buoban Pamang",
                "Burcu Ayhan (-Yksel)":"Burcu Ayhan",
                "Burger Lambrechts":"Burger Lambrechts",
                "Burkheart Ellis Jr.":"Burkheart Ellis",
                "Buthayna Eid Al-Yaqoubi":"Buthayna Al-Yaqoubi",
                "Byeon Yeong-Jun":"Byeon Yeong-Jun",
                "Byron Efrn Piedra Avils":"Byron Piedra",
                "Byron Robinson":"Byron Robinson",
                "Brbara Roco Comba":"Bárbara Comba",
                "Batrice Alice Kamuchanga":"Béatrice Alice Kamuchanga",
                "Branger Aymard Bosse":"Béranger Bosse",
                "Caimin Christian Douglas":"Caimin Douglas",
                "Cale Simmons":"Cale Simmons",
                "Caleb Mwangangi Ndiku":"Caleb Ndiku",
                "California Molefe":"California Molefe",
                'Li Loong "Calvin" Kang':"Calvin Kang",
                'Cameron "Cam" Levins':"Cam Levins",
                "Candeer Klner Ouz":"Candeğer Kılınçer Oğuz",
                "Candice Henrietta Scott":"Candice Scott",
                "Caridad Jerez Castellanos":"Caridad Jerez",
                "Carina Horn":"Carina Horn",
                "Carl Andrew Myerscough":"Carl Myerscough",
                "Carla Salom Rocha":"Carla Rocha",
                "Carla Cristina Paquete do Sacramento":"Carla Sacramento",
                "Carles Castillejo Salvador":"Carles Castillejo",
                "Carline Muir":"Carline Muir",
                "Carlos Garca Garca":"Carlos García",
                "Carlos Rafael Jorge":"Carlos Jorge",
                "Carlos Tobalina Aspirez":"Carlos Tobalina",
                "Carlos Vliz Wilburt":"Carlos Véliz",
                "Mara Carlota Castrejana Fernndez":"Carlota Castrejana",
                "Carmelita Jeter":"Carmelita Jeter",
                "Carmen Ruth Douma-Hussar":"Carmen Douma-Hussar",
                "Carol Mokola":"Carol Mokola",
                "Carol Ann Rodrguez Taylor":"Carol Rodríguez",
                "Madeleine Carole Kaboud Mebam":"Carole Kaboud Mebam",
                "Carolin Tamara Hingst":"Carolin Hingst",
                "Carolin Nytra (-Dietrich)":"Carolin Nytra",
                "Carina Horn":"Carolina Torres",
                "Caroline Bonde Holm":"Caroline Bonde Holm",
                "Carrie Anne Tollefson":"Carrie Tollefson",
                "Carsten Schlangen":"Carsten Schlangen",
                "Carvin Nkanata":"Carvin Nkanata",
                "Casey Malone":"Casey Malone",
                "Casey Garnett Vincent":"Casey Vincent",
                "Agnes Catherine Chikwakwa":"Catherine Chikwakwa",
                "Marcelle Cecilia Bouele Bondo":"Cecilia Bouele",
                "Cejhae Colin Greene":"Cejhae Greene",
                "Celiangeli Morales Melndez":"Celiangelí Morales",
                "Celma da Graa Soares Bonfim":"Celma Bonfim",
                "Chahinez Nasri":"Chahinez Nasri",
                "Chala Beyo Techo":"Chala Beyo",
                "Chamleunesouk Ao-Oudomphonh":"Chamleunesouk Ao-Oudomphonh",
                "Chan Ming Tai":"Chan Ming Tai",
                "Chan Seyha":"Chan Seyha",
                "Chancel Ilunga Sankuru":"Chancel Ilunga Sankuru",
                "Chandra Lavon Sturrup":"Chandra Sturrup",
                "Chandra Kala Thapa":"Chandra Thapa",
                "Chang Chunfeng":"Chang Chunfeng",
                "Chang Ming-Huang":"Chang Ming-Huang",
                "Chanice Chase-Taylor":"Chanice Chase-Taylor",
                "Charlene Maria Attard":"Charlene Attard",
                "Charlene Woitha":"Charlene Woitha",
                "Charles Tyrone Allen":"Charles Allen",
                "Charel Grethen":"Charles Grethen",
                "Charles Jock":"Charles Jock",
                "Charles Waweru Kamathi":"Charles Kamathi",
                "Charles Yosei Muneria":"Charles Muneria",
                "Charles Philibert-Thiboutot":"Charles Philibert-Thiboutot",
                "Charlie Gruber":"Charlie Gruber",
                "Charline Mathias":"Charline Mathias",
                "Charlotta Emma Katarina Fougberg":"Charlotta Fougberg",
                "Charlotte Wingfield":"Charlotte Wingfield",
                "Chauncy Master":"Chauncy Master",
                "Chauness Chauzje Choosha":"Chauzje Choosha",
                "Chelsea Hammond":"Chelsea Hammond",
                "Chelsea Alicia Hayes":"Chelsea Hayes",
                "Chelsea Jaensch":"Chelsea Jaensch",
                "Chen Chieh":"Chen Chieh",
                "Chen Haijian":"Chen Haijian",
                "Chen Qi":"Chen Qi",
                "Chen Yang":"Chen Yang",
                "Cheng Wen":"Cheng Wen",
                "Cherono Koech":"Cherono Koech",
                "Cheryl Leigh Webb":"Cheryl Webb",
                "Chiang Wai Hung":"Chiang Wai Hung",
                "Chiara Rosa":"Chiara Rosa",
                "Chijindu Andre E. O. Ujah":"Chijindu Ujah",
                "Chinatsu Mori":"Chinatsu Mori",
                'Chinonyelum Pius "Chinonye" Ohadugha':"Chinonye Ohadugha",
                "Chinwe Okoro":"Chinwe Okoro",
                "Chisato Fukushima":"Chisato Fukushima",
                "Choe Byeong-Kwang":"Choe Byeong-Kwang",
                "Choi Yun-Hui":"Choi Yun-Hui",
                'Christopher "Chris" Bennett':"Chris Bennett",
                'Christopher "Chris" Erickson':"Chris Erickson",
                "Christine Kalmer":"Chris Lambert",
                "Chris Joey Lloyd":"Chris Lloyd",
                "Chris Meke Walasi":"Chris Meke Walasi",
                "Christopher 'Chris' O'Hare":"Chris O'Hare",
                "Charles Jock":"Chris Pinnock",
                "Christin Hussong":"Chris Rawlinson",
                "Richard Thompson":"Chris Thompson",
                "Chrishuna Williams":"Chris Williams",
                "Nils Winter":"Chris Winter",
                "Chrishuna Williams":"Chrishuna Williams",
                "Christabel Nettey":"Christabel Nettey",
                "Christania Williams":"Christania Williams",
                "Christian Belz":"Christian Belz",
                "Christian Cantwell":"Christian Cantwell",
                "Christian Sean Malcolm":"Christian Malcolm",
                "Christian Nicolay":"Christian Nicolay",
                "Christian Nsiah":"Christian Nsiah",
                "Christian Obrist":"Christian Obrist",
                "Christian Reif":"Christian Reif",
                "Christian Smith":"Christian Smith",
                "Christin Hussong":"Christin Hussong",
                "Christina Epps":"Christina Epps",
                "Christina Hering":"Christina Hering",
                "Christina Obergfll":"Christina Obergföll",
                "Christina Illum Scherwin":"Christina Scherwin",
                "Christina Schwanitz":"Christina Schwanitz",
                "Ljubica Christina Vukicevic (-Demidov)":"Christina Vukicevic",
                "Christine Amertil (-Ling)":"Christine Amertil",
                "Christine Arron":"Christine Arron",
                "Christine Botlogetswe":"Christine Botlogetswe",
                "Christine Sonali Merrill":"Christine Sonali Merrill",
                "Christoph Harting":"Christoph Harting",
                "Christopher Linke":"Christophe Lemaître",
                "Christopher Lima da Costa":"Christopher Da Costa",
                "Christopher Linke":"Christopher Linke",
                "Christy Udoh":"Christy Udoh",
                'Christin "Christy" Wurth-Thomas':"Christy Wurth-Thomas",
                "Chu Yafei":"Chu Yafei",
                "Churandy Thomas Martina":"Churandy Martina",
                "Ciara Everard":"Ciara Everard",
                "Ciara Mageean":"Ciara Mageean",
                "Ciaran O'Lionair":"Ciaran Ó'Lionáird",
                "Cindy Billaud":"Cindy Billaud",
                "Cindy Ofili":"Cindy Ofili",
                "Cindy Roleder":"Cindy Roleder",
                "Cisiane Dutra Lopes":"Cisiane Dutra",
                "Claire Woods-Tallent":"Claire Woods-Tallent",
                "Clarence Munyai":"Clarence Munyai",
                "Clarissa Claretti":"Clarissa Claretti",
                "Clarisse Rasoarizay":"Clarisse Cruz",
                "Claudia Balderrama Ibaez":"Claudia Balderrama",
                "Claudia Mihaela Bobocea":"Claudia Bobocea",
                "Claudia Coslovich":"Claudia Coslovich",
                "Claudia Coslovich":"Claudia Gesell",
                "Claudio Licciardello":"Claudio Licciardello",
                "Cleopatra Ayesha Borel-Brown":"Cleopatra Borel-Brown",
                "Cludio Roberto de Sousa":"Cláudio Sousa",
                "Cllia Reuse-Rard":"Clélia Rard-Reuse",
                "Coby Miller":"Coby Miller",
                "Colleen Quigley":"Colleen Quigley",
                "Collis Birmingham":"Collis Birmingham",
                "Concepcin Montaner Coll":"Concepción Montaner",
                "Conor McCullough":"Conor Mccullough",
                "Conrad Williams":"Conrad Williams",
                "Cora Alicto":"Cora Alicto",
                "Corinna Harrer":"Corinna Harrer",
                "Cornel Fredericks":"Cornel Fredericks",
                "Cornelis Sibe":"Cornelis Sibe",
                "Courtney Babcock":"Courtney Babcock",
                "Courtney Frerichs":"Courtney Frerichs",
                "Courtney Carl Williams":"Courtney Williams",
                "Craig Kinsley":"Craig Kinsley",
                "Craig Stuart Mottram":"Craig Mottram",
                "Craig Keith Pickering":"Craig Pickering",
                "Cristina Ioana Bujin":"Cristina Bujin",
                "Cristina Casandra (Iloc-)":"Cristina Casandra",
                "Cristina Llovera Rossell":"Cristina Llovera",
                "Crystal Emmanuel":"Crystal Emmanuel",
                "Csongor Olten":"Csongor Olteán",
                "Curt Raymond Clausen":"Curt Clausen",
                "Curtis J. Moss":"Curtis Moss",
                "Cuthbert Nyasango":"Cuthbert Nyasango",
                "Cydonie Camille Mothersill (-Modibo)":"Cydonie Mothersill",
                "Cynthia Maduengele Bolingo Mbongo":"Cynthia Bolingo",
                "Cyrus Dean Hostetler":"Cyrus Hostetler",
                "Cdric El-Idrissi":"Cédric El-Idrissi",
                "Cdric Marie Carlos Thrse Van Branteghem":"Cédric Van Branteghem",
                "Cline Roseline Laporte":"Céline Laporte",
                "Dace Ruskule":"Dace Ruskule",
                "Dada brahimov":"Dadaş İbrahimov",
                "Dadi Denis":"Dadi Denis",
                "Dafne Schippers":"Dafne Schippers",
                "Daham Najim Bashir":"Daham Najim Bashir",
                "Dai Tamesue":"Dai Tamesue",
                "Daichi Sawano":"Daichi Sawano",
                "Dailenys Alcntara Pacheco":"Dailenys Alcántara",
                "Dailis Caballero Vega":"Dailis Caballero",
                "Daim Perni Figueroa":"Daimí Pernía",
                "Daina Levy":"Daina Levy",
                "Daisuke Matsunaga":"Daisuke Matsunaga",
                "Dalal Masfir Al-Harith":"Dalal Al-Harith",
                "Dale Stevenson":"Dale Stevenson",
                "Dalilah Muhammad":"Dalila Rugama",
                "Damar Hopeton Forbes":"Damar Forbes",
                "Kobala Vithanage Damayanthi Dharsha":"Damayanthi Dharsha",
                "Damian Czykier":"Damian Czykier",
                "Damien Birkinhead":"Damien Birkinhead",
                "Damjan Zlatnar":"Damjan Zlatnar",
                "Dana Abdul Razak (Hussain-)":"Dana Abdul Razak",
                "Dana Elvira Cervantes Garca":"Dana Cervantes",
                "Dana Lee-Anne Ellis":"Dana Ellis",
                "Dana Velkov":"Dana Velďáková",
                "Danah Al-Nasrallah":"Danah Al-Nasrallah",
                "Dane Hyatt":"Dane Hyatt",
                "Daniel Andjar Ponce":"Daniel Andújar",
                "Daniel Everton Bailey":"Daniel Bailey",
                "Daniel Dbrowski":"Daniel Dąbrowski",
                "Daniele Greco":"Daniel Grueso",
                "Daniel Jasinski":"Daniel Jasinski",
                "Daniel Lincoln":"Daniel Lincoln",
                "Daniela Bruno Stoffel Cardoso":"Daniela Cardoso",
                "Elena Daniela Crlan":"Daniela Cîrlan",
                "Daniela Lyubenova Yordanova":"Daniela Yordanova",
                "Daniele Caimmi":"Daniele Caimmi",
                "Daniele Meucci":"Daniele Meucci",
                "Danijel Furtula":"Danijel Furtula",
                "Danilson Ricciuli":"Danilson Ricciuli",
                "Danniel Thomas (-Dodd)":"Danniel Thomas",
                "Danny D'Souza":"Danny D'Souza",
                'Daniel "Danny" Ecker':"Danny Ecker",
                'Daniel Peter "Danny" Talbot':"Danny Talbot",
                "Danuta Urbanik":"Danuta Urbanik",
                "Daria Korczyska (Onysko-)":"Daria Korczyńska",
                "Dariusz Stanisaw Ku":"Dariusz Kuć",
                "Dariya Derkach":"Dariya Derkach",
                "Darlan Romani":"Darlan Romani",
                "Darlenis Obregn Mulato":"Darlenis Obregón",
                "Darrel Rondel Brown":"Darrel Brown",
                "Darrell Hill":"Darrell Hill",
                "Darrell Wesh":"Darrell Wesh",
                "Darren Andrew Campbell":"Darren Campbell",
                "Darren Gilford":"Darren Gilford",
                "Darvin Edwards":"Darvin Edwards",
                "Darvis Darell Patton":"Darvis Patton",
                "Darya Igorevna Klishina":"Darya Klishina",
                "Darya Maslova":"Darya Maslova",
                "Darya Pchelnik":"Darya Pchelnik",
                "Darya Vitalyevna Pishchalnikova":"Darya Pishchalnikova",
                "Daryll Saskia Neita":"Daryll Neita",
                "Dathan James Ritzenhein":"Dathan Ritzenhein",
                "Daundre Barnaby":"Daundre Barnaby",
                "David Adley Smith Snchez, II":"David Adley Smith",
                "David Brisseault":"David Brisseault",
                "David Bustos Gonzlez":"David Bustos",
                "David Canal Valero":"David Canal",
                "David Fiegen":"David Fiegen",
                "David Freeman":"David Freeman",
                "David Gillick":"David Gillick",
                "David Ilariani":"David Ilariani",
                "David Kimutai Rotich":"David Kimutai",
                "David McNeill":"David Mcneill",
                "David Storl":"David Storl",
                "David Verburg":"David Söderberg",
                "David Scott Torrence":"David Torrence",
                "Davide Manenti":"Davide Manenti",
                "Dawid Tomala":"Dawid Tomala",
                "Dawit Seyaum Biratu":"Dawit Seyaum",
                "Dawit Wolde Arega":"Dawit Wolde",
                "Jaime Dayron Mrquez Gutirrez":"Dayron Márquez",
                "Deajah Stevens":"Deajah Stevens",
                "Dean Horace Griffiths":"Dean Griffiths",
                "DeAnna Marie Price":"Deanna Price",
                'Deborah "Debbie" Ferguson-McKenzie':"Debbie Ferguson-Mckenzie",
                'Deborah Anne "Debbie" Sosimenko':"Debbie Sosimenko",
                "Dedeh Erawati":"Dedeh Erawati",
                "Deirdre Ryan":"Deirdre Ryan",
                "Dejene Berhanu":"Dejene Berhanu",
                "Deji Aliu":"Deji Aliu",
                "Delloreen Alverna Ennis-London":"Delloreen Ennis-London",
                "Delphine Bertille Atangana Bodo":"Delphine Atangana",
                "Demetrius Rashad Pinder":"Demetrius Pinder",
                "Demma Daba Bikila":"Demma Daba",
                "Denia Caballero Ponce":"Denia Caballero",
                "Denika Kassim":"Denika Kassim",
                "Denis Sergeyevich Alekseyev":"Denis Alekseyev",
                "Denis Langlois":"Denis Langlois",
                "Denis Vaslyevich Simanovich":"Denis Simanovich",
                "Denisa erbova-Rosolov":"Denisa Ščerbová-Rosolová",
                "Denise Hinrichs":"Denise Hinrichs",
                "Denys Serhiyovych Yurchenko":"Denys Yurchenko",
                "Derek Mandell":"Derek Mandell",
                "Deresse Mekonnen Tsigu":"Deresse Mekonnen",
                "Derrick Atkins":"Derrick Atkins",
                "Derrick L. Peterson":"Derrick Peterson",
                "Derval O'Rourke (-O'Leary)":"Derval O'Rourke",
                "Kelsie Hendry":"Desiree Henry",
                "Desiree Rossit":"Desiree Rossit",
                "Desislav Gunev":"Desislav Gunev",
                "Despina Zapounidou":"Despina Zapounidou",
                "Detlef Bock":"Detlef Bock",
                "Deuce Carter":"Deuce Carter",
                "Devilert Arsene Kimbembe":"Devilert Arsene Kimbembe",
                "Devon Allen":"Devon Allen",
                "Diamara Planell Cruz":"Diamara Planell",
                "Diana Aydosova":"Diana Aydosova",
                "Diana Khubeseryan":"Diana Khubeseryan",
                "Diana Martn Gimnez":"Diana Martín",
                "Diana Sujew":"Diana Sujew",
                "Diane Borg":"Diane Borg",
                "Diane S. Cummins":"Diane Cummins",
                "Diane Nukuri-Johnson":"Diane Nukuri-Johnson",
                "Dickson Marwa Mkami":"Dickson Mkami",
                "Didier Kiki":"Didier Kiki",
                "Diego Ferreira":"Diego Del Real",
                "Daniel Estrada Coz":"Diego Estrada",
                "Diego Ferreira":"Diego Ferreira",
                "Diego Armando Palomeque Echevarra":"Diego Palomeque",
                "Diego Ruz Sanz":"Diego Ruiz",
                "Dieudonn Disi":"Dieudonné Disi",
                "Digna Luz Murillo Moreno":"Digna Murillo",
                "Dilshod Nazarov":"Dilshod Nazarov",
                "Dimitra Dova":"Dimitra Dova",
                "Dimitriana Surdu":"Dimitriana Surdu",
                "Dimitrios Filindras":"Dimitrios Filindras",
                "Dimitrios Chondrokoukis":"Dimitrios Khondrokoukis",
                "Dimitrios Serelis":"Dimitrios Serelis",
                "Dion Kwasi Crabbe":"Dion Crabbe",
                "Dipna Lim Prasad":"Dipna Lim Prasad",
                "Ejowvokoghene Divine Oduduru":"Divine Oduduru",
                "Djibo Idrissa Ousseini":"Djibo Idrissa Ousseini",
                "Djnbou Dant":"Djénébou Danté",
                "Dmitri Markov":"Dmitri Markov",
                "Dmitri Marin":"Dmitri Marşin",
                "Dmitrijs Jurkevis":"Dmitrijs Jurkevičs",
                "Dmitrijs Mikvis":"Dmitrijs Miļkēvičs",
                "Dmitry Koblov":"Dmitry Bogdanov",
                "Dmitry Koblov":"Dmitry Koblov",
                "Dmitry Sivakov":"Dmitry Nabokov",
                "Dmitry Igoryevich Shevchenko":"Dmitry Shevchenko",
                "Dmitry Sivakov":"Dmitry Sivakov",
                "Dmitry Andreyevich Starodubtsev":"Dmitry Starodubtsev",
                "Dmytro Oleksiyovych Dem'ianiuk":"Dmytro Dem'Ianiuk",
                "Dmytro Leonidovych Hlushchenko":"Dmytro Hlushchenko",
                "Dmytro Volodymyrovych Kosynskiy":"Dmytro Kosynskiy",
                "Dmytro Valeriyovych Yakovenko":"Dmytro Yakovenko",
                "Dobrinka Vaskova Shalamanova":"Dobrinka Shalamanova",
                "Dolores Checa Porcel":"Dolores Checa",
                "Domingas Embana Togna":"Domingas Togna",
                "Dominic Demeritte":"Dominic Demeritte",
                "Dominic Laurence Johnson":"Dominic Johnson",
                "Dominique Scott":"Dominique Scott",
                "Donald Eugene Sanford":"Donald Sanford",
                "Anilda Thomas":"Donald Thomas",
                "Dong Jimin":"Dong Jimin",
                "Dong Xiaoqin":"Dong Xiaoqin",
                'Donald "Donn" Cabral':"Donn Cabral",
                "Donna Karen Fraser":"Donna Fraser",
                "Danny D. McFarlane":"Donna Macfarlane",
                'Dorothea "Dora" Kalpakidou':"Dora Kalpakidou",
                "Dorcus Inzikuru":"Dorcus Inzikuru",
                "Eyawomano Doreen Amata":"Doreen Amata",
                "Dorian Armand Scott":"Dorian Scott",
                "Dorian ollaku":"Dorian Çollaku",
                "Dou Zhaobo":"Dou Zhaobo",
                "Dragan Mustapi":"Dragan Mustapić",
                "Dragan Peri":"Dragan Perić",
                "Dragana Tomaevi":"Dragana Tomašević",
                "Dragutin Topi":"Dragutin Topić",
                "Drita Islami":"Drita Islami",
                "Randolph Duane Ross":"Duane Ross",
                "Duane Renard Solomon":"Duane Solomon",
                "Dudley S. Dorival":"Dudley Dorival",
                "Dudu Karakaya":"Dudu Karakaya",
                "Dulce Mara Rodrguez de la Cruz":"Dulce María Rodríguez",
                "Dutee Chand":"Dutee Chand",
                "Dwain Anthony Chambers":"Dwain Chambers",
                "Dwight Thomas":"Dwight Thomas",
                "Dylan Armstrong":"Dylan Armstrong",
                "Dniel Kiss":"Dániel Kiss",
                "Dborah Lizeth Rodrguez Guelmo":"Déborah Rodríguez",
                "Dng Th Vit Anh":"Dương Thị Việt Anh",
                "Ebba Anna Jungmark":"Ebba Jungmark",
                "Ebrahim Rahimian":"Ebrahim Rahimian",
                "Ebru Kavaklolu":"Ebru Kavaklıoğlu",
                "Eddie Jermoni Lovett":"Eddie Lovett",
                "Eddy de Lpine":"Eddy De Lépine",
                "Edi Silva Maia":"Edi Maia",
                "Edi Pono":"Edi Ponoš",
                "Edidiong Ofonime Odiong":"Edidiong Odiong",
                "Edina Fsti":"Edina Füsti",
                "Edis Elkasevi":"Edis Elkasević",
                "Edit Vri":"Edit Vári",
                "Edith Chewanjel Masai":"Edith Masai",
                "Edivaldo Isaas Carvalho vora Monteiro":"Edivaldo Monteiro",
                "Eduard Antonio Villanueva":"Eduard Villanueva",
                "Eduardo Buenavista":"Eduardo Buenavista",
                "Eduardo Ivn Rodrguez Ramallo":"Eduardo Iván Rodríguez",
                "Edy Jakariya":"Edy Jakariya",
                "Egle Uljas":"Egle Uljas",
                "Egl Balinait":"Eglė Balčiūnaitė",
                "Egl Staiinait":"Eglė Staišiūnaitė",
                "Eha Rnne":"Eha Rünne",
                "Ehsan Haddadi":"Ehsan Haddadi",
                "Ehsan Mohajer Shojaei":"Ehsan Mohajer Shojaei",
                "Eike Onnen":"Eike Onnen",
                "Eiki Takahashi":"Eiki Takahashi",
                "Eileen O'Keeffe":"Eileen O'Keeffe",
                "Eilish McColgan":"Eilish Mccolgan",
                "Eirini Kokkinariou":"Eirini Kokkinariou",
                "Eirini Chrysovalado Terzoglou":"Eirini Terzoglou",
                "Eirini Vasileiou":"Eirini Vasileiou",
                "Eivind Prestegrd Henriksen":"Eivind Henriksen",
                "El Hassan El-Abbassi":"El Hassan El-Abbassi",
                "Elaine Thompson":"Elaine Thompson",
                "Elama Fa'atonu":"Elama Fa'Atonu",
                "Eldred Henry":"Eldred Henry",
                "Eleanor Patterson":"Eleanor Patterson",
                "Elena Lucia Guerra Barbero":"Elena Guerra",
                "Elena Romagnolo":"Elena Iagăr",
                "Elena Mirela Lavric":"Elena Mirela Lavric",
                "Elena Andreea Panuroiu":"Elena Panțuroiu",
                "Elena Popescu":"Elena Popescu",
                "Elena Romagnolo":"Elena Romagnolo",
                "Eleni Artymata":"Eleni Artymata",
                "Eleni Filandra":"Eleni Filandra",
                "Eleni Teloni (Demetriou-)":"Eleni Teloni",
                "Eleonora Anna  Giorgi":"Eleonora Giorgi",
                "Eli Johana Moreno Valencia":"Eli Johana Moreno",
                "Eliane Martins":"Eliane Martins",
                "Marie Eliane Saholinirina":"Eliane Saholinirina",
                "Eliecith Palacios Santos":"Eliecith Palacios",
                "Elijah Motonei Manangoi":"Elijah Manangoi",
                "Elina Dmitriyevna Mikhina":"Elina Mikhina",
                "Eline Berings":"Eline Berings",
                "Elis Lapenmal":"Elis Lapenmal",
                "Elisa Cusma Piccione":"Elisa Cusma",
                "Elisabeth Eberl":"Elisabeth Eberl",
                "Elisabeth Victoire Mandaba":"Elisabeth Mandaba",
                "Elisabetta Marin":"Elisabetta Marin",
                "Elisavet Pesiridou":"Elisavet Pesiridou",
                "Eliseo Martn Omenat":"Eliseo Martín",
                "Elisngela Maria Adriano":"Elisângela Adriano",
                "Mary Elizabeth Zaragoza Guerrini":"Elizabeth Zaragoza",
                "Elizna Naud":"Elizna Naudé",
                "Ella Nelson":"Ella Nelson",
                "Florina Pierdevar":"Ellina Zvereva",
                "Elliot Levi Giles":"Elliot Giles",
                "Eloise Wellings (Poppet-)":"Eloise Wellings",
                "Elroy Gelant":"Elroy Gelant",
                "Elva Dryer (Martinez-)":"Elva Dryer",
                "Elvan Abeylegesse":"Elvan Abeylegesse",
                "Emanuele Abate":"Emanuele Abate",
                "Emel Dereli":"Emel Dereli",
                "Emilia Ankiewicz":"Emilia Ankiewicz",
                "Emilia Mikue Ondo":"Emilia Mikue Ondo",
                "Emiliano Lasa Snchez":"Emiliano Lasa",
                "Emily Kaye Freeman":"Emily Freeman",
                "Emily Infeld":"Emily Infeld",
                "Emir Bekri":"Emir Bekrić",
                "Emma Anna-Maria Green-Tregaro":"Emma Green-Tregaro",
                "Emma Wade":"Emma Wade",
                "Emmanuel Kwame Dasor":"Emmanuel Dasor",
                "Emmanuel Matadi":"Emmanuel Matadi",
                "Endurance Ojokolo":"Endurance Ojokolo",
                "English Gardner":"English Gardner",
                "Enrique Llanos":"Enrique Llanos",
                "Ercment Olgundeniz":"Ercüment Olgundeniz",
                "Eric Javier Alejandro":"Eric Alejandro",
                "Erik Cade":"Eric Cray",
                "Eric Gillis":"Eric Gillis",
                "Eric Matthias":"Eric Matthias",
                "Eric Nkansah Appiah":"Eric Nkansah",
                "Erica Ashley McLain":"Erica Mclain",
                "Erick Rodrguez Lezcano":"Erick Rodríguez",
                "Erik Balnuweit":"Erik Balnuweit",
                "Erik Cade":"Erik Cadée",
                "Erik Tysse":"Erik Tysse",
                "Craig Kinsley":"Erika Kinsey",
                "Erin Leigh Donohue":"Erin Donohue",
                "Erin Gilreath":"Erin Gilreath",
                "Erin Teschuk":"Erin Teschuk",
                "Erison George Hurtault":"Erison Hurtault",
                "Erkinzhon Tulkunovich Isakov":"Erkinzhon Isakov",
                "Erma-Gene Evans":"Erma-Gene Evans",
                "Ernest Ndissipou":"Ernest Ndissipou",
                "Ersin Tacir":"Ersin Tacir",
                "Ese Brume":"Ese Brume",
                "Eseosa Fostine Desalu":"Eseosa Desalu",
                "Esko Olavi Mikkola":"Esko Mikkola",
                "Essa Ismail Rashed":"Essa Ismail Rashed",
                "Manuel Esteban Soto Ruz":"Esteban Soto",
                "Ester Balassini":"Ester Balassini",
                "Esther Guerrero Puigdevall":"Esther Guerrero",
                "Esthera Petre":"Esthera Petre",
                "Estie Wittstock":"Estie Wittstock",
                "Etenesh Diro Neda":"Etenesh Diro",
                "Etimoni Timuani":"Etimoni Timuani",
                "Eugenio Rossi":"Eugenio Rossi",
                "Eunice Barber":"Eunice Barber",
                "Eunice Jepkoech Sum":"Eunice Sum",
                "Eusebio Cceres Lpez":"Eusebio Cáceres",
                "Evan Dunfee":"Evan Dunfee",
                "Evangeleen Ikelap":"Evangeleen Ikelap",
                "Evangelia Xinou":"Evangelia Xinou",
                "Evans Marie":"Evans Marie",
                "Evdokia Tsamoglou":"Evdokia Tsamoglou",
                "Evelin Rivera Blanquet":"Evelin Rivera",
                "Marily dos Santos":"Evelyn Dos Santos",
                "Ewa Nikola Swoboda":"Ewa Swoboda",
                "Ezinne Okparaebo":"Ezinne Okparaebo",
                "Ezra Kenyoke Sambu":"Ezra Sambu",
                "Eref Apak":"Eşref Apak",
                "Fabian Heinle":"Fabian Heinle",
                "Fabiana dos Santos Mores":"Fabiana Morães",
                "Fabiana de Almeida Murer":"Fabiana Murer",
                "Fabiano Joseph Naasi":"Fabiano Naasi",
                "Fabiano Peanha":"Fabiano Peçanha",
                "Fabienne Fraez":"Fabienne Féraez",
                "Fabienne Kohlmann":"Fabienne Kohlmann",
                "Fabienne Schlumpf":"Fabienne Schlumpf",
                "Fabio Cerutti":"Fabio Cerutti",
                "Louis Steven Fabrice Coiffic":"Fabrice Coiffic",
                "Fabrice Lapierre":"Fabrice Lapierre",
                "Fadrique Ignacio Iglesias Mendizbal":"Fadrique Iglesias",
                "Fadwa Al-Bouza":"Fadwa Al-Bouza",
                "Fadwa Sidi Madane":"Fadwa Sidi Madane",
                "Faith Macharia":"Faith Macharia",
                "Fana Ashby":"Fana Ashby",
                "Fanuel Kenosi":"Fanuel Kenosi",
                "Fartun Abukar Omar":"Fartun Abukar Omar",
                "Fathia Ali Bouraleh":"Fathia Ali Bouraleh",
                "Fatih Avan":"Fatih Avan",
                "Fatima Abdullah Suleiman Al-Dahman":"Fatima Suleiman Dahman",
                "Fatma Nagib El-Sharnouby":"Fatma El-Sharnouby",
                "Fatmata Fofanah":"Fatmata Fofanah",
                "Fatou Bintou Fall":"Fatou Bintou Fall",
                "Fatou Tiyana":"Fatou Tiyana",
                "Fawaz Al-Shammari":"Fawaz Al-Shammari",
                "Fedrick Dacres":"Fedrick Dacres",
                "Felicia ilea-Moldovan":"Felicia Țilea-Moldovan",
                "Felipa Alicia Palacios Hinestroza":"Felipa Palacios",
                "Felipe Vivancos Ruiz":"Felipe Vivancos",
                "Felisha Johnson":"Felisha Johnson",
                "Felismina Cavela":"Felismina Cavela",
                "Felix Kikwai Kibore":"Felix Kibore",
                "Femi Seun Ogunode":"Femi Ogunode",
                "Femke Pluim":"Femke Pluim",
                "Feng Bin":"Feng Bin",
                "Feodosiy Ciumacenco":"Feodosiy Ciumacenco",
                "Ferguson Cheruiyot Rotich":"Ferguson Rotich",
                "Brenda Martinez":"Fernanda Martins",
                "Fernando Carro Morillo":"Fernando Carro",
                "Fernando Pereira de Almeida":"Fernando De Almeida",
                "Fernando Lumain":"Fernando Lumain",
                "Feta Ahamada":"Feta Ahamada",
                "Filip Ingebrigtsen":"Filip Ingebrigtsen",
                "Filip Mihaljevi":"Filip Mihaljević",
                "Filipo Muller":"Filipo Muller",
                "Filippo Campioli":"Filippo Campioli",
                "Filiz Kadoan":"Filiz Kadoğan",
                "Fionnuala Britton-McCormack":"Fionnuala Britton-Mccormack",
                "Flings Joyner Owusu-Agyapong":"Flings Owusu-Agyapong",
                "Flor Vsquez":"Flor Vásquez",
                "Flora Rendoumi":"Flora Rendoumi",
                "Florent Lacasse":"Florent Lacasse",
                "Floria Guei":"Floria Gueï",
                "Florian Carvalho":"Florian Carvalho",
                "Florian Orth":"Florian Orth",
                "Florina Pierdevar":"Florina Pierdevară",
                "Flo Khnert":"Floé Kühnert",
                "Flvia Maria de Lima":"Flávia De Lima",
                "Fong Yee Pui":"Fong Yee Pui",
                "Fouad El-Kaam":"Fouad El-Kaam",
                "Ene Franca Idoko (-Isaac)":"Franca Idoko",
                "Franciela das Graas Krasucki":"Franciela Krasucki",
                "Francine Niyonizigiye":"Francine Niyonizigiye",
                "Francis Manioru":"Francis Manioru",
                "Francis Obiorah Obikwelu":"Francis Obikwelu",
                "Francisco Arcilla Aller":"Francisco Arcilla",
                "Franck Dannique Elemba Owaka":"Franck Elemba",
                "Francky-Edgard Mbotto":"Francky-Edgard Mbotto",
                "Franka Dietzsch":"Franka Dietzsch",
                "Franka Magali":"Franka Magali",
                'Frank "Frankie" Fredericks':"Frankie Fredericks",
                "Franklin Andrs Nazareno Macas":"Franklin Nazareno",
                "Frantz Kruger":"Frantz Kruger",
                "Freddy Gonzlez Fonten":"Freddy González",
                "Fumilay da Fonseca":"Fumilay Da Fonseca",
                "Fbio Gomes da Silva":"Fábio Gomes",
                "Gable Garenamotse":"Gable Garenamotse",
                "Gabriel Mvumvure":"Gabriel Mvumvure",
                "Gabriela Petrova":"Gabriela Petrova",
                "Gabriela Stafford":"Gabriela Stafford",
                "Gil Roberts":"Gail Devers",
                "Gakologelwang Lesiba Masheto":"Gakologelwang Masheto",
                "Galin Kostadinov":"Galin Kostadinov",
                "Galina Yevgenyevna Bogomolova":"Galina Bogomolova",
                "Galina Mityaeva":"Galina Mityaeva",
                "Galina Pedan":"Galina Pedan",
                "Gamze Bulut":"Gamze Bulut",
                "Ganapathi Krishnan":"Ganapathi Krishnan",
                "Gao Shuying":"Gao Shuying",
                "Gao Xinglong":"Gao Xinglong",
                "Gao Yang":"Gao Yang",
                "Gareth Robert Warburton":"Gareth Warburton",
                "Garfield Darien":"Garfield Darien",
                "Gary Kikaya Senga":"Gary Kikaya",
                "Gary Thomas Reed":"Gary Reed",
                'Foo Ee "Gary" Yeo':"Gary Yeo",
                "Gaspar Jos Pereira Arajo":"Gaspar Araújo",
                "Gayane Chiloyan":"Gayane Chiloyan",
                "Gcinile Moyane":"Gcinile Moyane",
                "Gebre Egziabher Gebremariam":"Gebre Egziabher Gebremariam",
                "Geena Gall (-Lara)":"Geena Gall",
                "Geiner Alonso Mosquera Becerra":"Geiner Mosquera",
                "Geisa Rafaela Arcanjo":"Geisa Arcanjo",
                "Geisa Aparecida Muniz Coutinho":"Geisa Coutinho",
                "Gelete Burka Bati":"Gelete Burka",
                "Genevieve LaCaze":"Genevieve Lacaze",
                "Genevive Lalonde":"Geneviève Lalonde",
                "Gennady Sergeyevich Chernovol":"Gennady Chernovol",
                "Gennady Viktorovich Moroz":"Gennady Moroz",
                "Jamila Genzeb Shumi Regasa":"Genzeb Shumi",
                'Geoffrey "Geoff" Harris':"Geoff Harris",
                "Geoffrey Kipsang Kamworor":"Geoffrey Kamworor",
                "Geoffrey Kusuro":"Geoffrey Kusuro",
                "Georganne Rochelle Moline":"Georganne Moline",
                "George Washington Kitchens, Jr.":"George Kitchens",
                "Georgi Stoyanov Ivanov":"Georgi Ivanov",
                "Georgia Tsilingkiri":"Georgia Tsilingkiri",
                "Georgina Tth":"Georgina Tóth",
                "Georgios Theodoridis":"Georgios Theodoridis",
                "Georgy Nikolayevich Sheyko":"Georgy Sheyko",
                "Geovanna Irusta (Morejon-)":"Geovana Irusta",
                "Gerald Phiri":"Gerald Phiri",
                "Geraldine Lititia Pillay":"Geraldine Pillay",
                "Gerhard Mayer":"Gerardo Martínez",
                "Gerd Kanter":"Gerd Kanter",
                "Gergely Horvth":"Gergely Horváth",
                "Gerhard Mayer":"Gerhard Mayer",
                "Germn Pablo Chiaraviglio Ermcora":"Germán Chiaraviglio",
                "Germn Lujn Lauro":"Germán Lauro",
                "Geronimo Goeloe":"Geronimo Goeloe",
                'Gerrit-Jan "Gert-Jan" Liefers':"Gert-Jan Liefers",
                "Gesa Felicitas Krause":"Gesa Felicitas Krause",
                "Ghada Ali":"Ghada Ali",
                "Gharid Ghrouf":"Gharid Ghrouf",
                "Gheorghe Calin Guet":"Gheorghe Gușet",
                "Ghfran Al-Mouhamad":"Ghfran Mouhamad",
                "Ghyd Kermeliss Holly Olonghot":"Ghyd Olonghot",
                "Gia Gerisha Lewis-Smallwood":"Gia Lewis-Smallwood",
                "Gian Nicola Berardi":"Gian Nicola Berardi",
                "Gianmarco Tamberi":"Gianmarco Tamberi",
                "Gina Bass":"Gina Bass",
                "Gina Lckenkemper":"Gina Lückenkemper",
                "Ginou Etienne":"Ginou Etienne",
                "Gintaras Andriukeviius":"Gintaras Andriuškevičius",
                "Giordano Benedetti":"Giordano Benedetti",
                "Giorgio Rubino":"Giorgio Rubino",
                "Giovanni Orlando Codrington":"Giovanni Lanaro",
                "Gisele Lima de Oliveira":"Gisele Oliveira",
                "Gita Vaniva Dodova":"Gita Dodova",
                "Giuseppe Maffei":"Giuseppe Maffei",
                "Gladys Nataly Landaverde Hernndez":"Gladys Landaverde",
                "Gladys Thompson":"Gladys Thompson",
                "Glody Dube":"Glody Dube",
                "Gloria Asumnu":"Gloria Asumnu",
                "Gloria Hooper":"Gloria Hooper",
                "Goitom Kifle":"Goitom Kifle",
                "Goldie Katherine Dinah Sayers":"Goldie Sayers",
                "Gong Lijiao":"Gong Lijiao",
                "Goran Nava":"Goran Nava",
                "Gordon Teokotai Heather":"Gordon Heather",
                "Grace Claxton":"Grace Claxton",
                "Grace E. Upshaw":"Grace Upshaw",
                "Grace Wanjiru Njue":"Grace Wanjiru",
                "Graciela Martins":"Graciela Martins",
                "Grant Robison":"Grant Robison",
                "Grayna Prokopek-Janacek":"Grażyna Prokopek-Janacek",
                "Greggmar Orial Swift":"Greggmar Swift",
                "Gregor Cankar":"Gregor Cankar",
                "Gregor Traber":"Gregor Traber",
                "Grzegorz Sudo":"Gregory Sedoc",
                "Gretta Taslakian (-Yerevanian)":"Gretta Taslakian",
                "Grzegorz Sposb":"Grzegorz Sposób",
                "Grzegorz Sudo":"Grzegorz Sudoł",
                "Gu Yuan":"Gu Yuan",
                "Guan Yingnan":"Guan Yingnan",
                "Gudaf Tsegay Desta":"Gudaf Tsegay",
                "Guilherme Henrique Cobbo":"Guilherme Cobbo",
                "Guillermo Martnez Lpez":"Guillermo Martínez",
                "Gulfiya Raifovna Khanafeyeva":"Gulfiya Khanafeyeva",
                "Gunta Latieva-udare":"Gunta Latiševa-Čudare",
                "Gurmeet Singh":"Gurmeet Singh",
                "Gustavo Cuesta Rosario":"Gustavo Cuesta",
                "Guzel Shavkatovna Khubbiyeva":"Guzel Khubbiyeva",
                "Guni Valur Gunason":"Guðni Guðnason",
                "Gwen Berry":"Gwen Berry",
                "Gyula Csaba Duds":"Gyula Dudás",
                "Gbor Dobos":"Gábor Dobos",
                "Gbor Istvn Mt":"Gábor Máté",
                "Grard Koban":"Gérard Kobéané",
                "Gza Pauer":"Géza Pauer",
                "Glcan Mngr":"Gülcan Mıngır",
                "Gnther Weidlinger":"Günther Weidlinger",
                "Habitam Alemu":"Habitam Alemu",
                "Hadhari Djaffar Saindou":"Hadhari Djaffar",
                "Hafsatu Kamara":"Hafsatu Kamara",
                "Hagen Pohle":"Hagen Pohle",
                "Haidar Nasser Abdul Shaheed":"Haidar Nasser",
                "Hailemariyam Amare Tegen":"Hailemariyam Amare",
                "Hais Welday Ghebrat":"Hais Welday",
                "Hajar Saeed Saad Sowaid Al-Khaldi":"Hajar Al-Khaldi",
                "Abdusalam Hala Gezah":"Hala Gezah",
                "Haley Louise McGregor":"Haley Mcgregor",
                "Haley Nicole Nemra":"Haley Nemra",
                "Halil Akka":"Halil Akkaş",
                "Halima Hachlaf":"Halima Hachlaf",
                "Halima Nakaayi":"Halima Nakaayi",
                "Halimat Oyinoza Ismaila":"Halimat Ismaila",
                "Halyna Yaroslavivna Obleshchuk":"Halyna Obleshchuk",
                "Mohamed Daud Mohamed":"Hamada Mohamed",
                "Hamdan Awdah Al-Bishi":"Hamdan Al-Bishi",
                "Hamid Ezzine":"Hamid Ezzine",
                "Hamid Reza Zooravand":"Hamid Reza Zooravand",
                "Hamish Carson":"Hamish Carson",
                "Hamish Peacock":"Hamish Peacock",
                "Hamoud Abdallah Said Al-Dalhami":"Hamoud Abdallah Al-Dalhami",
                "Hamza Ali":"Hamza Alić",
                "Han Yucheng":"Han Yucheng",
                "Hanami Sekine":"Hanami Sekine",
                "Hanane Ouhaddou":"Hanane Ouhaddou",
                "Hanna Demydova":"Hanna Demydova",
                "Hanna Volodymyrivna Hatsko-Fedusova":"Hanna Hatsko-Fedusova",
                "Hanna Knyazyeva-Minenko":"Hanna Knyazyeva-Minenko",
                "Hanna Valerivna Mishchenko":"Hanna Mishchenko",
                "Hanna Oleksandrivna Plotitsyna":"Hanna Plotitsyna",
                "Hanna Serhivna Titimets":"Hanna Titimets",
                "Hannah Jane England (-Gunn)":"Hannah England",
                "Hannes Kirchler":"Hannes Hopley",
                "Hannes Kirchler":"Hannes Kirchler",
                'Gerhardus "Hardus" Pienaar':"Hardus Pienaar",
                "Hari Kumar Rimal":"Hari Kumar Rimal",
                "Harmon Harmon":"Harmon Harmon",
                "Harouna Garba":"Harouna Garba",
                "Harumi Hiroyama (Suzuki-)":"Harumi Hiroyama",
                "Harwant Kaur":"Harwant Kaur",
                "Hasan Mahboob Ali":"Hasan Mahboob",
                "Hassan Chani":"Hassan Chani",
                "Hassan Hirt":"Hassan Hirt",
                "Hassan Mead":"Hassan Mead",
                "Hassan Mohamed Mahmoud":"Hassan Mohamed Mahmoud",
                "Hassan Saaid":"Hassan Saaid",
                "Hassan Taftian":"Hassan Taftian",
                "Hassanine Al-Sbai":"Hassanine Sbai",
                "Hatem Ghoula":"Hatem Ghoula",
                "Hawanatu Bangura":"Hawanatu Bangura",
                "Hayat Lambarki":"Hayat Lambarki",
                "Hayle brahimov":"Hayle İbrahimov",
                "Hayley Tullett (Parry-)":"Hayley Tullett",
                "Hazel Mae Clark-Riley":"Hazel Clark-Riley",
                "Hazel-Ann Regis":"Hazel-Ann Regis",
                "Heather Barbara Samuel":"Heather Samuel",
                "Heather Steacy":"Heather Steacy",
                "Zenaide Vieira":"Heber Viera",
                "Hedda Hynne":"Hedda Hynne",
                "Hederson Alves Estefani":"Hederson Estefani",
                "Heide Seerling (-Quinn)":"Heide Seÿerling",
                "Helen Teresa Pattinson-Clitheroe":"Helen Pattinson-Clitheroe",
                "Helena Javornik":"Helena Javornik",
                "Heli Maarit Koivula-Kruger":"Heli Koivula-Kruger",
                "Hendrik Mokganyetsi":"Hendrick Mokganyetsi",
                "Hendrick Ramaala":"Hendrick Ramaala",
                "Henricho Bruintjies":"Henricho Bruintjies",
                "Henrik Ingebrigtsen":"Henrik Ingebrigtsen",
                "Henry C. Dagmil":"Henry Dagmil",
                "Henry Max Frayne":"Henry Frayne",
                "Herbert A. McGregor":"Herbert Mcgregor",
                "Hermenegildo Leite":"Hermenegildo Leite",
                "Hicham Bellani":"Hicham Bellani",
                "Hicham Bouchicha":"Hicham Bouchicha",
                "Higor Silva Alves":"Higor Alves",
                "Hilary Stellingwerff (Edmondson-)":"Hilary Stellingwerff",
                "Hilda Jepchumba Kibet":"Hilda Kibet",
                "Hillary Bor":"Hillary Bor",
                "Hind Dhiba (Chahyd-)":"Hind Déhiba",
                "Hinikissa Albertine Ndikert":"Hinikissa Ndikert",
                "Hiroki Ogita":"Hiroki Ogita",
                "Hiroyasu Tsuchie":"Hiroyasu Tsuchie",
                "Hiskel Tewelde":"Hiskel Tewelde",
                "Hitomi Niiya":"Hitomi Niiya",
                "Hiwot Ayalew Yeder":"Hiwot Ayalew",
                "Holder Ocante da Silva":"Holder Da Silva",
                "Holly Bethan Bleasdale-Bradshaw":"Holly Bleasdale-Bradshaw",
                "Homiyu Tesfaye Heyi":"Homiyu Tesfaye",
                "Hortense Clementine Bewouda Kamni":"Hortense Béwouda",
                "Houleye Ba":"Houleye Ba",
                "Hristina Risteska":"Hristina Risteska",
                "Hsiang Chun-Hsien":"Hsiang Chun-Hsien",
                "Hu Kai":"Hu Kai",
                "Huang Bokai":"Huang Bokai",
                "Huang Changzhou":"Huang Changzhou",
                "Huang Haiqiang":"Huang Haiqiang",
                "Huang Qiuyan":"Huang Qiuyan",
                "Huang Qun":"Huang Qun",
                "Huang Shih-Feng":"Huang Shih-Feng",
                "Huang Xiaoxiao":"Huang Xiaoxiao",
                "Hudson Santos de Souza":"Hudson De Souza",
                "Hussain Jamaan Al-Hamdah":"Hussain Al-Hamdah",
                "Hussain Taher Al-Sabee":"Hussain Taher Al-Sabee",
                "Hctor Cotto Gonzlez":"Héctor Cotto",
                "Hlder Mendes Abreu Ornelas":"Hélder Ornelas",
                "Hseyin Atc":"Hüseyin Atıcı",
                "Ian Dobson":"Ian Dobson",
                "Ian Scott Waltz":"Ian Waltz",
                "Iaroslav Musinschi":"Iaroslav Musinschi",
                "Ibrahim M. Al-Hamaidi":"Ibrahim Al-Hamaidi",
                "Ibrahim Tondi":"Ibrahim Tondi",
                "Ibrahim Turay":"Ibrahim Turay",
                "Ibrahima Wade":"Ibrahima Maïga",
                "Ibrahima Wade":"Ibrahima Wade",
                "Idrissa Adam":"Idrissa Adam",
                "Idrissa Sanou":"Idrissa Sanou",
                "Ieva Zunda":"Ieva Zunda",
                "Juan Ignacio Cerra":"Ignacio Guerra",
                "Anthony Ignisious Gaisah Essuman":"Ignisious Gaisah",
                "Igor Vyacheslavovich Astapkovich":"Igor Astapkovich",
                "Igor Bychkov":"Igor Bychkov",
                "Igor Piotr Janik":"Igor Janik",
                "Igor Primc":"Igor Primc",
                "Igor Alekseyevich Vinichenko":"Igor Vinichenko",
                "Igors Sokolovs":"Igors Sokolovs",
                "Ihor Valeriyovych Bodrov":"Ihor Bodrov",
                "Ihor Ivanovych Tuhay":"Ihor Tuhay",
                "Ildar Salikhovich Minshin":"Ildar Minshin",
                "Ilian Ananiev Efremov":"Iliyan Efremov",
                "Illias Fifa Temesmani":"Illias Fifa",
                "Ilya Valeryevich Konovalov":"Ilya Konovalov",
                "Ilya Yevgenyevich Korotkov":"Ilya Korotkov",
                "Ilze Gribule":"Ilze Gribule",
                "Iman Essa Jasim":"Iman Essa",
                "Indira Terrero Letuce":"Indira Terrero",
                "Indr Jakubaityt":"Indrė Jakubaitytė",
                "Ineta Radvia":"Ineta Radēviča",
                "Inga Eduardovna Abitova":"Inga Abitova",
                "Inga Juodekien (Petrauskait-)":"Inga Juodeškienė",
                "Inga Stasiulionyt":"Inga Stasiulionytė",
                "Ingeborg Lvnes":"Ingeborg Løvnes",
                "Ingo Schultz":"Ingo Schultz",
                "Ingrid Johana Hernndez Castillo":"Ingrid Hernández",
                "Ingrid Yahoska Narvaez":"Ingrid Yahoska",
                "Inika McPherson":"Inika Mcpherson",
                "Inna Liubomirova Eftimova":"Inna Eftimova",
                "Ina Gliznua":"Inna Gliznuţa",
                "Inna Oleksandrivna Kashyna":"Inna Kashyna",
                "Inna Polukina":"Inna Poluškina",
                "Inna Oleksandrivna Saienko":"Inna Saienko",
                "Santa Ins Melchor Huiza":"Inés Melchor",
                "Ins Pereira Henriques":"Inês Henriques",
                "Iolanta Yevgenyevna Ulyeva":"Iolanta Ulyeva",
                "Ion Emilianov":"Ion Emilianov",
                "Irache Quintanal Franco":"Irache Quintanal",
                "Irene Jelagat":"Irene Jelagat",
                "Irfan Kolothum Thodi":"Irfan Kolothum Thodi",
                "Irina Andreyevna Davydova":"Irina Davydova",
                "Irina Andreyevna Gordeyeva":"Irina Gordeyeva",
                "Irina Sergeyevna Khabarova":"Irina Khabarova",
                "Irina Aleksandrovna Khudoroshkina":"Irina Khudoroshkina",
                "Irina Nikolayevna Korzhanenko":"Irina Korzhanenko",
                "Irina Andreyevna Litvinenko-Ektova":"Irina Litvinenko-Ektova",
                "Irina Mikitenko (Volynskaya-)":"Irina Mikitenko",
                "Irina Aleksandrovna Obedina":"Irina Obedina",
                "Irina Cristiana Carreira Rodrigues":"Irina Rodrigues",
                "Irina Stasyuk-Chernushenko":"Irina Stasyuk-Chernushenko",
                "Irina Ivanovna Tarasova":"Irina Tarasova",
                "Irina Leonidovna Vaskovskaya":"Irina Vaskovskaya",
                "Irina Gennadyevnya Yakoltsevich (-Zhuk)":"Irina Yakoltsevich",
                "Irina Vasilyevna Yatchenko":"Irina Yatchenko",
                "Iris Mara Fuentes-Pila Ortz":"Iris Fuentes-Pila",
                "Iryna Ihorivna Herashchenko":"Iryna Herashchenko",
                "Iryna Volodymyrivna Klymets":"Iryna Klymets",
                "Iryna Stanislavivna Mykhalchenko":"Iryna Mykhalchenko",
                "Iryna Oleksandrivna Novozhylova":"Iryna Novozhylova",
                "Irne Pusterla":"Irène Pusterla",
                "Isa Phillips":"Isa Phillips",
                "Isaac Makwala":"Isaac Makwala",
                "Isaac Makwala":"Isaac Palma",
                "Isaac Ntiamoah":"Isaac Silafau",
                "Isaac Kiprono Songok":"Isaac Songok",
                "Isabel Checa Porcel":"Isabel Checa",
                "Isabel le Roux":"Isabel Le Roux",
                "Isabel Macas Chow":"Isabel Macías",
                "Isabelle Sandstedt Pedersen":"Isabelle Pedersen",
                "Isaiah Msibi":"Isaiah Msibi",
                "Isamu Fujisawa":"Isamu Fujisawa",
                "Isbel Luaces McKullock":"Isbel Luaces",
                "Ishmail Dudu Kamara":"Ishmail Kamara",
                "Isiah Kiplangat Koech":"Isiah Koech",
                "Isiah Young":"Isiah Young",
                "Isidora Andrea Jimnez Ibacache":"Isidora Jiménez",
                "Isidro Montoya Valencia":"Isidro Montoya",
                "Isireli Naikelekelevesi":"Isireli Naikelekelevesi",
                "Issam Nima":"Issam Nima",
                "Itay Magidi":"Itay Magidi",
                "Iva Strakov":"Iva Straková",
                "Ivan Tarasovych Heshko":"Ivan Heshko",
                "Ivan Horvat":"Ivan Horvat",
                "Ivan Ivanov":"Ivan Ivanov",
                "Ivan Antonovich Trotsky":"Ivan Trotsky",
                "Ivan Yuryevich Tukhtachov":"Ivan Tukhtachov",
                "Ivan Horvat":"Ivan Ukhov",
                "Ivan Viktorovich Yushkov":"Ivan Yushkov",
                "Ivan Viktorovich Zaytsev":"Ivan Zaytsev",
                "Ivana Brkljai":"Ivana Brkljačić",
                "Ivana Roman":"Ivana Rožman",
                "Ivanique Kemp":"Ivanique Kemp",
                "Ivano Bucci":"Ivano Bucci",
                "Ivet Miroslavova Lalova-Collio":"Ivet Lalova-Collio",
                "Iveta Putalov":"Iveta Putalová",
                "Izmir Smajlaj":"Izmir Smajlaj",
                "J'maal Alexander":"J'Maal Alexander",
                "Jaak-Heinrich Jagor":"Jaak-Heinrich Jagor",
                "Jack Peter Green":"Jack Green",
                "Jack Howard":"Jack Howard",
                "Jackie Baumann":"Jackie Baumann",
                "Jack Howard":"Jackie Edwards",
                'Jaclyn "Jackie" Jeschelnig (-Ulm)':"Jackie Jeschelnig",
                'Jackson "Jacko" Gill':"Jacko Gill",
                "Jackson Amadeo Quionez Vernaza":"Jackson Quiñónez",
                "Jacob Araptany":"Jacob Araptany",
                "Jacob Kiplimo":"Jacob Kiplimo",
                "Thamsanqa Jacob Rozani":"Jacob Rozani",
                "Jacques Freitag":"Jacques Freitag",
                "Jade Latoya Bailey":"Jade Bailey",
                "Jade Linsey Johnson":"Jade Johnson",
                "Jade Louise Lally":"Jade Lally",
                "Jadel Abdul Ghani Gregrio":"Jadel Gregório",
                "Jahvid Andre Best":"Jahvid Best",
                "Jailma Sales de Lima":"Jailma Lima",
                "Jakub Czaja":"Jakub Czaja",
                "Jakub Holua":"Jakub Holuša",
                "Jakub Bogdan Jelonek":"Jakub Jelonek",
                "Jakub Vadlejch":"Jakub Vadlejch",
                "Jamal Wilson":"Jamal Wilson",
                "Jamele Mason":"Jamele Mason",
                "James Carter":"James Carter",
                "James William Dolphin":"James Dolphin",
                "James Lee Ellington":"James Ellington",
                "James Theophilus Grayman":"James Grayman",
                "James Kwalia Chepkurui":"James Kwalia",
                "James Nolan":"James Nolan",
                "James Nyang Chiengjiek":"James Nyang Chiengjiek",
                "James Parker":"James Parker",
                "Jamial St. John Rolle":"Jamial Rolle",
                "Jamile Sara Samuel":"Jamile Samuel",
                "Javon Francis":"Jamy Franco",
                "Jan Kudlika":"Jan Kudlička",
                "Jan Marcell":"Jan Marcell",
                "Jan Sekpona":"Jan Sekpona",
                "Jan Zakrzewski":"Jan Zakrzewski",
                "Jan elezn":"Jan Železný",
                "Sapna Punia":"Jana Pittman",
                "Jana Velkov":"Jana Velďáková",
                "Jane Wanjiku Gakunyi":"Jane Wanjiku",
                "Janeil Bellille":"Janeil Bellille",
                "Janelle Redhead":"Janelle Redhead",
                "Janet Achola":"Janet Achola",
                "Janet Amponsah":"Janet Amponsah",
                "Janet Cherobon-Bawcom":"Janet Cherobon-Bawcom",
                "Jang Jeong-Yeon":"Jang Jeong-Yeon",
                "Janice Alatoa":"Janice Alatoa",
                "Janieve Russell":"Janieve Russell",
                "Zane Robertson":"Janus Robberts",
                "Jarbas Mascarenhas Jnior":"Jarbas Mascarenhas Júnior",
                "Jared Connaughton":"Jared Connaughton",
                "Jared Lewis":"Jared Lewis",
                "Jarmila Klimeov":"Jarmila Klimešová",
                "Jaroslav itansk":"Jaroslav Žitňanský",
                "Jarred Daniel Rome":"Jarred Rome",
                "Jarrion Lawson":"Jarrion Lawson",
                "Jarrod Bannister":"Jarrod Bannister",
                "Jasmin Salihovi":"Jasmin Salihović",
                "Jasmine Camacho-Quinn":"Jasmine Quinn",
                "Jasminka Guber":"Jasminka Guber",
                "Jason John Gardener":"Jason Gardener",
                "Jason Morgan":"Jason Morgan",
                "Jason Ron Stewart":"Jason Stewart",
                "Jason Tunks":"Jason Tunks",
                "Jason Edwin Young":"Jason Young",
                "Javier Bermejo Merino":"Javier Bermejo",
                "Javier Adolfo Carriqueo Inostroza":"Javier Carriqueo",
                "Javier Cienfuegos Pinilla":"Javier Cienfuegos",
                "Marie Gayot":"Javier Gazol",
                "Jayson Jones":"Jayson Jones",
                "Jaysuma Saidy Ndure":"Jaysuma Saidy Ndure",
                "Jean Marie Okutu Kouletio":"Jean Marie Okutu",
                "Jean-Claude Rabbath":"Jean-Claude Rabbath",
                "Jean-Jacques Nkouloukidi":"Jean-Jacques Nkouloukidi",
                "Jean-Patrick Nduwimana":"Jean-Patrick Nduwimana",
                "Jean-Yves Esparon":"Jean-Yves Esparon",
                "Jeanine Assani Issouf":"Jeanine Assani Issouf",
                "Jeanelle Scheper":"Jeannelle Scheper",
                "Jeannette Boahemaa Kwakye":"Jeannette Kwakye",
                'Jeffrey "Jeff" Riseley':"Jeff Riseley",
                "Jeffery Gibson":"Jeffery Gibson",
                "Jeffrey Michael Julmis":"Jeffrey Julmis",
                "Jehue Gordon":"Jehue Gordon",
                "Jeimy Julissa Bernrdez Santos":"Jeimmy Bernárdez",
                "Jemma Louise Simpson":"Jemma Simpson",
                "Jenna Prandini":"Jenna Prandini",
                "Jennifer Madu":"Jennifer Madu",
                "Jennifer Wenth":"Jennifer Wenth",
                'Jennifer "Jenny" Blundell':"Jenny Blundell",
                "Jenny Elbe":"Jenny Elbe",
                "Jenny Margareta Kallur":"Jenny Kallur",
                "Jenny Keni":"Jenny Keni",
                "Jeon Yeong-Eun":"Jeon Yeong-Eun",
                "Jeong Hye-Rim":"Jeong Hye-Rim",
                "Jeong Sang-Jin":"Jeong Sang-Jin",
                "Jeong Sun-Ok":"Jeong Sun-Ok",
                "Jere Petrud Bergius":"Jere Bergius",
                "Jeremy Bascom":"Jeremy Bascom",
                "Jeremy Dodson":"Jeremy Dodson",
                "Jeremy Richard Scott":"Jeremy Scott",
                "Jermaine Gonzales":"Jermaine Gonzales",
                "Jeroen D'hoedt":"Jeroen D'Hoedt",
                "Claes Jesper Fritz":"Jesper Fritz",
                "Jesse Jay Tamangrow":"Jesse Tamangrow",
                "Jesse Daniel Williams":"Jesse Williams",
                "Michael Aguilar":"Jessica Aguilera",
                "Jessica Hancco Merma":"Jessica Hancco",
                "Jessica Inchude":"Jessica Inchude",
                "Jessica O'Connell":"Jessica O'Connell",
                "Jessica Lorene Smith":"Jessica Smith",
                "Jessica Anne Zelinka":"Jessica Zelinka",
                "Jess Farias de Lima":"Jessé De Lima",
                "Jess Espaa Cobo":"Jesús España",
                "Jeena elnova-Prokopuka":"Jeļena Čelnova-Prokopčuka",
                "Jhoanis Carlos Portilla Cardenas":"Jhoanis Portilla",
                "Ji Wei":"Ji Wei",
                "Jiang Jing":"Jiang Jing",
                "Jidou Ould Khaye El-Moctar":"Jidou El-Moctar",
                'Jillian Mary "Jill" Camarena-Williams':"Jill Camarena-Williams",
                "Jillian Schwartz":"Jillian Schwartz",
                "Jim Sveny":"Jim Svenøy",
                "Jimmy Anak Ahar":"Jimmy Anak Ahar",
                "Jimmy Vicaut":"Jimmy Vicaut",
                "Jing Xuezhu":"Jing Xuezhu",
                "Jinson Johnson":"Jinson Johnson",
                "Jip Vastenburg":"Jip Vastenburg",
                "Ji Malysa":"Jiří Malysa",
                "Ji Muk":"Jiří Mužík",
                "Ji Vojtk":"Jiří Vojtík",
                "Joachim Brchner Olsen":"Joachim Olsen",
                "Joana Ribeiro Costa":"Joana Costa",
                "Joanna Fiodorow":"Joanna Fiodorow",
                "Joanna Jwik":"Joanna Jóźwik",
                "Joanna Linkiewicz":"Joanna Linkiewicz",
                "Joanna Aleksandra Piwowarska":"Joanna Piwowarska",
                "Joanna Winiewska":"Joanna Wiśniewska",
                "Joanne Cuddihy":"Joanne Cuddihy",
                "Joanne Dow":"Joanne Dow",
                "Jodie Alicia Williams":"Jodie Williams",
                "Joel Baden":"Joel Baden",
                "Joel Milburn":"Joel Milburn",
                "Joel Japheth Phillip":"Joel Phillip",
                "Janelle Redhead":"Joel Redhead",
                "Joelma das Neves Sousa":"Joelma Sousa",
                "Joeri Jansen":"Joeri Jansen",
                "Johan Tobias Cronje":"Johan Cronje",
                "Johan Lukas Wissman":"Johan Wissman",
                "Johanna Katriina Manninen":"Johanna Manninen",
                "Johannes Vetter":"Johannes Vetter",
                "John Ampomah":"John Ampomah",
                "John Carl Godina":"John Godina",
                "John Bray Henwood":"John Henwood",
                "John Howard":"John Howard",
                "John Kemboi Kibowen":"John Kibowen",
                "John Kibet Koech":"John Koech",
                "JohnCheruiyot Korir":"John Korir",
                "John Paul Mayock":"John Mayock",
                "John Hamilton Nunn":"John Nunn",
                "John Robert Oosthuizen":"John Robert Oosthuizen",
                "John Ruuka":"John Ruuka",
                "John Yuda Msuri":"John Yuda",
                "Johnathan Knight Cabral":"Johnathan Cabral",
                "Johnson Kubisa":"Johnson Kubisa",
                "Joice Maduaka":"Joice Maduaka",
                "Jolanta Dukure":"Jolanta Dukure",
                "Jonathan Borle":"Jonathan Borlée",
                "Jonathan James Chimier":"Jonathan Chimier",
                "Jonathan Johnson":"Jonathan Johnson",
                "Jonathan Williams":"Jonathan Williams",
                "Jonathan Craig Wyatt":"Jonathan Wyatt",
                "Jonathan Jens-Erik strand":"Jonathan Åstrand",
                "Joanne Dow":"Jonni Lowe",
                "Jordan Mageo":"Jordan Mageo",
                "Jordin Andrade":"Jordin Andrade",
                "Jorge Esteban Balliengo":"Jorge Balliengo",
                "Jorge Yadin Fernndez Hernndez":"Jorge Fernández",
                "Jorge Paula":"Jorge Paula",
                "Jorge Torres":"Jorge Torres",
                "Josanne Lucille Lucas":"Josanne Lucas",
                "Josef Prorok":"Josef Prorok",
                "Josef Robertson":"Josef Robertson",
                "Joseph Andy Lui":"Joseph Andy Lui",
                "Joseph Batangdon":"Joseph Batangdon",
                "Joseph Mwengi Mutua":"Joseph Mutua",
                "Joseph-Berlioz Randriamihaja":"Joseph-Berlioz Randriamihaja",
                "Josephine Nnkiruka Onyia":"Josephine Onyia",
                "Josephine Terlecki":"Josephine Terlecki",
                'Joshua "Josh" Ilustre':"Josh Ilustre",
                'Joshua "Josh" McAdams':"Josh Mcadams",
                "Moushaumi Robinson":"Josh Robinson",
                "Joshua Kiprui Cheptegei":"Joshua Kiprui Cheptegei",
                "Joshua James Ross":"Joshua Ross",
                "Josleidy Ribalta Alfonso":"Josleidy Ribalta",
                "Jos Alessandro Bernardo Baggio":"José Alessandro Bagio",
                "Jos Carlos Herrera Vargas":"José Carlos Herrera",
                "Jos Carlos Gomes Moreira":"José Carlos Moreira",
                "Jos David Domnguez Guimera":"José David Domínguez",
                "Jos David Galvn Martnez":"José David Galván",
                "Jos Gregorio Pea Trejo":"José Gregorio Peña",
                "Jos Luis Blanco Quevedo":"José Luis Blanco",
                "Jos Lus Gaspar Hernndez":"José Luis Gaspar",
                "Jos Manuel Martnez Fernndez":"José Manuel Martínez",
                "Jos Mara Raymundo Cox":"José María Raymundo",
                "Jos Reynaldo Bencosme de Leon":"José Reynaldo Bencosme De Leon",
                "Jos Ros Ortega":"José Ríos",
                "Joy Amechi Eze":"Joy Eze",
                "Joy Nakhumicha Sakari":"Joy Nakhumicha Sakari",
                "Joyce Chepkirui":"Joyce Chepkirui",
                "Jozef Repk":"Jozef Repčík",
                "Joo Manuel Pires":"João Manuel Pires",
                "Joo Baptista N'Tyamba":"João N'Tyamba",
                "Joo Vtor de Oliveira":"João Vítor De Oliveira",
                "Jolle Sandrine Mbumi Nkouindjin":"Joëlle Mbumi Nkouindjin",
                "Juan Carlos de la Ossa Yunta":"Juan Carlos De La Ossa",
                "Juan Carlos Higuero Mate":"Juan Carlos Higuero",
                "Juan Carlos Romero Bernal":"Juan Carlos Romero",
                "Juan Ignacio Cerra":"Juan Ignacio Cerra",
                "Juan Luis Barrios Nieves":"Juan Luis Barrios",
                "Juan Manuel Cano Ceres":"Juan Manuel Cano",
                "Juan Manuel Molina Morote":"Juan Manuel Molina",
                "Juan Miguel Zeledn":"Juan Miguel Zeledón",
                "Juan Pedro Toledo Domnguez":"Juan Pedro Toledo",
                "Juan Luis Sainfleur Hernndez":"Juan Sainfleur",
                "Juan Pierre van Deventer":"Juan Van Deventer",
                "Juana Rosario Arrendel":"Juana Rosario Arrendel",
                "Judith Pl Roig":"Judit Plá",
                "Judit Varga":"Judit Varga",
                "Jukka Pekka Sakari Keskisalo":"Jukka Keskisalo",
                "Julia Helen Bleasdale":"Julia Bleasdale",
                "Julia Dubina":"Julia Dubina",
                "Julia Fischer":"Julia Fischer",
                "Julian Francis Matthews":"Julian Matthews",
                "Julian Reus":"Julian Reus",
                "Julian Jrummi Walsh":"Julian Walsh",
                "Julian Weber":"Julian Weber",
                "Julian Wruck":"Julian Wruck",
                "Juliane Sprenger-Afflerbach":"Juliane Sprenger-Afflerbach",
                "Julie Culley":"Julie Culley",
                "Julie Labonte":"Julie Labonté",
                "Julien Fivaz":"Julien Fivaz",
                "Juliet Chekwel":"Juliet Chekwel",
                "Juliet Itoya Ebhomhenye":"Juliet Itoya",
                "Julio Csar Salazar Enrquez":"Julio César Salazar",
                "Julio Ren Martnez Sicn":"Julio René Martínez",
                "Julius Mutekanga":"Julius Mutekanga",
                "Julius Kiplagat Yego":"Julius Yego",
                "Jun Osakada":"Jun Osakada",
                'Jurica "Juraj" Grabui':"Juraj Grabušić",
                "Jurgen Themen":"Jurgen Themen",
                "Jurij Rovan":"Jurij Rovan",
                "Justin John Anlezark":"Justin Anlezark",
                "Justin Thomas Chaston":"Justin Chaston",
                "Justin Alexander Gatlin":"Justin Gatlin",
                "Justin Rodhe":"Justin Rodhe",
                "Justine Bayiga":"Justine Bayiga",
                "Justine Fedronic":"Justine Fedronic",
                "Justine Palframan":"Justine Palframan",
                "Justine Gail Robbeson":"Justine Robbeson",
                "Justyn Paris Ashby Warner":"Justyn Warner",
                "Justyna wity":"Justyna Święty",
                "Jutamass Thavoncharoen":"Jutamass Thavoncharoen",
                "Juttaporn Krasaeyan":"Juttaporn Krasaeyan",
                "Jrme Bortoluzzi":"Jérôme Bortoluzzi",
                "Jrme Clavier":"Jérôme Clavier",
                "Jrme Marcel Crews":"Jérôme Crews",
                "Jssica de Barros Augusto":"Jéssica Augusto",
                "Jlia Takcs Nyerges":"Júlia Takács",
                'Jlianna "Jlia" Tudja':"Júlia Tudja",
                "Jlio Csar Miranda de Oliveira":"Júlio César Oliveira",
                "Jnis Leitis":"Jānis Leitis",
                "Kabange Mupopo":"Kabange Mupopo",
                "Kabelo Kgosiemang":"Kabelo Kgosiemang",
                "Kadiatou Camara":"Kadiatou Camara",
                "Kadra Mohamed Dembil":"Kadra Mohamed",
                "Kaftien Antoine Gomis":"Kafétien Gomis",
                "Kai Freddie Selvon":"Kai Selvon",
                "Kaila McKnight":"Kaila Mcknight",
                "Kaina Darina Martnez":"Kaina Martínez",
                "Kaingaue David":"Kaingaue David",
                "Kaire Leibak":"Kaire Leibak",
                "Kaitinano Mwemweata":"Kaitinano Mwemweata",
                "Kakianako Nariki":"Kakianako Nariki",
                "Kaliese Spencer":"Kaliese Spencer",
                "Kalliopi Ouzouni":"Kalliopi Ouzouni",
                "Kaltoum Bouaasayriya":"Kaltoum Bouaasayriya",
                "Kaltouma Nadjina":"Kaltouma Nadjina",
                "Kamal Boulahfane":"Kamal Boulahfane",
                "Kamia Yousufi":"Kamia Yousufi",
                "Kamiel Maase":"Kamiel Maase",
                "Kamil Kryski":"Kamil Kryński",
                "Kamila Liwinko (Stepaniuk-)":"Kamila Lićwinko",
                "Kamila Skolimowska":"Kamila Skolimowska",
                "Kanchi Maya Koju":"Kanchi Maya Koju",
                "Kierre Beckles":"Kanika Beckles",
                "Andr Bucher":"Kara Goucher",
                "Kara Estelle Patterson-Winger":"Kara Patterson-Winger",
                "Kareem Streete-Thompson":"Kareem Streete-Thompson",
                "Karen Pamela Gallardo Pinto":"Karen Gallardo",
                "Kariem Hussein":"Kariem Hussein",
                "Kariman Abuljadayel":"Kariman Abuljadayel",
                "Karin Mayr-Krifka (-Knoll-)":"Karin Mayr-Krifka",
                "Karin Melis (Mey-)":"Karin Melis",
                "Karina Vnukova":"Karina Vnukova",
                "Karitaake Tewaaki":"Karitaake Tewaaki",
                "Karol Zalewski":"Karol Zalewski",
                "Karolina Koeczek":"Karolina Kołeczek",
                "Karoline Bjerkeli Grvdal":"Karoline Bjerkeli Grøvdal",
                "Karsten Dilla":"Karsten Dilla",
                "Karsten Kobs":"Karsten Kobs",
                "Karsten Warholm":"Karsten Warholm",
                "Katalin Divs":"Katalin Divós",
                "Katarzyna Kowalska":"Katarzyna Kowalska",
                "Kate Dennison (-Rooney)":"Kate Dennison",
                "Kate Grace":"Kate Grace",
                "Kate O'Neill (-Tenforde)":"Kate O'Neill",
                "Kate Amelia Reed":"Kate Reed",
                "Kateryna Oleksandrivna Derun":"Kateryna Derun",
                "Kateryna Yevhenivna Karsak":"Kateryna Karsak",
                "Kateina Baurov":"Kateřina Baďurová",
                "Kateina echov":"Kateřina Čechová",
                "Kateina afrnkov":"Kateřina Šafránková",
                "Katharina Molitor":"Katharina Molitor",
                "Kathrin Klaas":"Kathrin Klaas",
                "Kathryn Mitchell":"Kathryn Mitchell",
                'Katherine "Kathy" Butler':"Kathy Butler",
                "Katura Marae":"Katura Marae",
                "Katy Sealy":"Katy Sealy",
                "Kauiza Moreira Venncio":"Kauiza Venâncio",
                "Kaur Kivistik":"Kaur Kivistik",
                "Kaveh Sadegh Mousavi":"Kaveh Mousavi",
                "Kayoko Fukushi":"Kayoko Fukushi",
                "Kazuya Shiojiri":"Kazuya Shiojiri",
                "Kefasi Chitsala":"Kefasi Chitsala",
                "Kei Takase":"Kei Takase",
                "Keila da Silva da Costa":"Keila Costa",
                "Keisuke Nozawa":"Keisuke Nozawa",
                "Kellie Wapshott":"Kellie Suttle",
                "Kellie Wapshott":"Kellie Wapshott",
                "Kellion Knibb":"Kellion Knibb",
                "Kelly-Ann Kaylene Baptiste":"Kelly-Ann Baptiste",
                "Kelsey Card":"Kelsey Card",
                "Kelsey Nakanelua":"Kelsey Nakanelua",
                "Kelsey-Lee Roberts":"Kelsey-Lee Roberts",
                "Kelsie Ahbe":"Kelsie Ahbe",
                "Kelsie Hendry":"Kelsie Hendry",
                "Kemal Mei":"Kemal Mešić",
                "Kemar Dwain Hyman":"Kemar Hyman",
                "Ashani Kemarley Brown":"Kemarley Brown",
                "Kemel Oneil Thompson":"Kemel Thompson",
                "Oluwakemi 'Kemi' Adekoya":"Kemi Adekoya",
                "Kemoy Campbell":"Kemoy Campbell",
                "Ken Yoshizawa":"Ken Yoshizawa",
                "Kendra Clarke":"Kendra Clarke",
                "Kenia Marsha Sinclair":"Kenia Sinclair",
                "Kenji Fujimitsu":"Kenji Fujimitsu",
                "Kenji Narisako":"Kenji Narisako",
                "Kenneth Nathaniel Medwood":"Kenneth Medwood",
                "Kensuke Takezawa":"Kensuke Takezawa",
                "Kerri-Ann Mitchell":"Kerri-Ann Mitchell",
                "Kerron Stewart":"Kerron Stewart",
                "Kerry O'Flaherty":"Kerry O'Flaherty",
                "Keshorn Walcott":"Keshorn Walcott",
                "Keston Bledman":"Keston Bledman",
                "Keturah Orji":"Keturah Orji",
                "Kevin S. Eastler":"Kevin Eastler",
                "Kevin Lpez Yerga":"Kevin López",
                "Nevin Yant":"Kevin Rans",
                "Kevin Roy Sullivan":"Kevin Sullivan",
                "Khadevis Kekoai Tobi Robinson":"Khadevis Robinson",
                "Khaido Alexouli":"Khaido Alexouli",
                "Mohamed Khaled Belabbas":"Khaled Belabbas",
                "Khalid Habash Mohammed Al-Suwaidi":"Khalid Habash Al-Suwaidi",
                "Khalil Ahmed Al-Hanahneh":"Khalil Al-Hanahneh",
                "Khamica Bingham":"Khamica Bingham",
                "Khamis Abdullah Seifeddine":"Khamis Abdullah Seifeddine",
                "Khoudir Aggoune":"Khoudir Aggoune",
                "Khristina Tancheva Georgieva":"Khristina Georgieva",
                "Khristina Kokotou":"Khristina Kokotou",
                "Khristoforos Merousis":"Khristos Khiotis",
                "Khrysoula Anagnostopoulou":"Khrysoula Anagnostopoulou",
                "Khrysoula Goudenoudi":"Khrysoula Goudenoudi",
                "Khrystyna Petrivna Stuy":"Khrystyna Stuy",
                "Khushbir Kaur":"Khushbir Kaur",
                "Kia Davis":"Kia Davis",
                "Kibw Rasheid Johnson":"Kibwé Johnson",
                "Kidane Tadesse Habtesilase":"Kidane Tadesse",
                "Kieng Samorn":"Kieng Samorn",
                "Kierre Beckles":"Kierre Beckles",
                "Kilakone Siphonexay":"Kilakone Siphonexay",
                "Kim Oscar Amb":"Kim Amb",
                'Kimberly "Kim" Barrett':"Kim Barrett",
                "Kim Collins":"Kim Collins",
                "Kim Deok-Hyeon":"Kim Deok-Hyeon",
                "Kim Gevaert":"Kim Gevaert",
                "Kim Gyeong-Ae":"Kim Gyeong-Ae",
                "Kim Hyeon-Seop":"Kim Hyeon-Seop",
                "Kim Juhl Christensen":"Kim Juhl Christensen",
                "Kim Gevaert":"Kim Kreiner",
                "Kim Kuk-Young":"Kim Kuk-Young",
                "Kim Mi-Jeong":"Kim Mi-Jeong",
                "Kim Mi-Jeong":"Kim Mickle",
                "Leigh Smith":"Kim Smith",
                "Kim Yu-Seok":"Kim Yu-Seok",
                "Kimberly Garca Len":"Kimberly García",
                'Kimberly "Kimmie" Williams':"Kimmie Williams",
                "Kineke Alicia Alexander":"Kineke Alexander",
                "Kirill Gennadiyevich Ikonnikov":"Kirill Ikonnikov",
                "Kirill Olegovich Sosunov":"Kirill Sosunov",
                "Kirsi Pauliina Mykknen":"Kirsi Mykkänen",
                "Kirsi Eva Marika Valasti":"Kirsi Valasti",
                "Kirsten Bolm":"Kirsten Bolm",
                "Kirsten Nieuwendam":"Kirsten Nieuwendam",
                "Kitson Kapiriel":"Kitson Kapiriel",
                "Klodiana Shala":"Klodiana Shala",
                "Klberson Davide":"Kléberson Davide",
                "Koichiro Morioka":"Koichiro Morioka",
                "Koji Alexander Murofushi":"Koji Murofushi",
                "Kokoutse Fabrice Dabla":"Kokoutse Fabrice Dabla",
                "Kolyo Neshev":"Kolyo Neshev",
                "Kondwani Chiwina":"Kondwani Chiwina",
                "Konomi Kai":"Konomi Kai",
                "Konrad Bukowiecki":"Konrad Bukowiecki",
                "Konstantin Igorevich Borichevsky":"Konstantin Borichevsky",
                "Konstantin Georgiyevich Shabanov":"Konstantin Shabanov",
                "Konstantina Efedaki":"Konstantina Efedaki",
                "Konstantinos Stathelakos":"Konstantinos Stathelakos",
                "Konstanze Klosterhalfen":"Konstanze Klosterhalfen",
                "Korene Amoy Hinds":"Korene Hinds",
                'Konstantinos "Kostas" Baniotis':"Kostas Baniotis",
                'Konstantinos "Kostas" Filippidis':"Kostas Filippidis",
                "Kota Murayama":"Kota Murayama",
                "Krishna Poonia":"Krishna Poonia",
                "Meri Kristiina Mkel":"Kristiina Mäkelä",
                "Kristijan Efremov":"Kristijan Efremov",
                "Kristin Gierisch":"Kristin Gierisch",
                "Kristin L. Heaston":"Kristin Heaston",
                "Kristina Pronzhenko":"Kristina Pronzhenko",
                "Kristina Saltanovi (-Dias)":"Kristina Saltanovič",
                "Kristine Harutyunyan":"Kristine Harutyunyan",
                "Kristof Beyens":"Kristof Beyens",
                "Krisztina Molnr":"Krisztina Molnár",
                "Krisztina Papp":"Krisztina Papp",
                "Krisztin Pars":"Krisztián Pars",
                "Krystian Zalewski":"Krystian Zalewski",
                "Krystyna Danilczyk-Zabawska":"Krystyna Danilczyk-Zabawska",
                "Ksenija Balta":"Ksenija Balta",
                "Kseniya Aleksandrovna Priyemko-Detsuk":"Kseniya Priyemko-Detsuk",
                "Kumi Otoshi":"Kumi Otoshi",
                "Kumiko Ikeda":"Kumiko Ikeda",
                "Kumiko Okada":"Kumiko Okada",
                "Kurtis Marschall":"Kurtis Marschall",
                "Ware Kutre Dulecha":"Kutre Dulecha",
                "Kyle Scott Alcorn":"Kyle Alcorn",
                "Kyle Greaux":"Kyle Greaux",
                "Kyriakos Ioannou":"Kyriakos Ioannou",
                "Kne N'Doye":"Kène N'Doye",
                "Kvin Borle":"Kévin Borlée",
                "Kvin Campion":"Kévin Campion",
                "Kvin Menaldo":"Kévin Ménaldo",
                "Kvlcm Kaya Salman":"Kıvılcım Kaya",
                "Lacena Golding-Clarke":"Lacena Golding-Clarke",
                'Lachlan "Lach" Renshaw':"Lach Renshaw",
                "Lacy Janson":"Lacy Janson",
                "Ladji Doucour":"Ladji Doucouré",
                "Laenly Phoutthavong":"Laenly Phoutthavong",
                "Lai Chun Ho":"Lai Chun Ho",
                "Laia Forcadell Arenas":"Laia Forcadell",
                "Laila Ferrer e Silva":"Laila Ferrer",
                "Lajos Imre Krthy":"Lajos Kürthy",
                "Lalita Shivaji Babar":"Lalita Babar",
                "Lamin Tucker":"Lamin Tucker",
                "Lance Brooks":"Lance Brooks",
                "Lankantien Lamboni":"Lankantien Lamboni",
                "Lanni Marchant":"Lanni Marchant",
                "Larisa Nikolayevna Kruglova":"Larisa Kruglova",
                "Lars Brgeling":"Lars Börgeling",
                "Lars Peter Riedel":"Lars Riedel",
                "LaShauntea Moore":"Lashauntea Moore",
                "LaShawn Merritt":"Lashawn Merritt",
                "LaTasha Lavon Colander-Richardson (-Clark)":"Latasha Colander-Richardson",
                "Latifa Essarokh":"Latifa Essarokh",
                "Lauma Grva":"Lauma Grīva",
                "Laura Gerraughty (-Ekstrand)":"Laura Gerraughty",
                "Laura Muir":"Laura Muir",
                "Laura Reynolds":"Laura Reynolds",
                "Lisa Jane Weightman":"Laura Weightman",
                "Laura de Witte":"Laura Whittle",
                "Lauren Boden-Wells":"Lauren Boden-Wells",
                "Lauren Katherine Hewitt":"Lauren Hewitt",
                "Laurence Manfrdi":"Laurence Manfrédi",
                "Lauryn Chenet Williams":"Lauryn Williams",
                "Laverne M. Eve":"Laverne Eve",
                "Laverne M. Eve":"Laverne Jones",
                "LaVonne Celeste Idlette":"Lavonne Idlette",
                "Lanece Clarke":"Lawrence Clarke",
                "Lawrence Amaechi A. Okoye":"Lawrence Okoye",
                "Laye Abdullayeva":"Layeş Abdullayeva",
                "Le Roux Hamman":"Le Roux Hamman",
                "Leah Nugent":"Leah Nugent",
                "Lebogang Shange":"Lebogang Shange",
                "Lecabela Dias da Fonseca Quaresma":"Lecabela Quaresma",
                "Lee Da-Seul":"Lee Da-Seul",
                "Lee Dae-Ro":"Lee Dae-Ro",
                "Lee Hup Wei":"Lee Hup Wei",
                "Lee Jae-Hun":"Lee Jae-Hun",
                "Lee Jeong-Eun":"Lee Jeong-Eun",
                "Lee Jeong-Jun":"Lee Jeong-Jun",
                "Lee McConnell":"Lee Mcconnell",
                "Lee Mi-Yeong":"Lee Mi-Yeong",
                "Leford Green":"Leford Green",
                'Eleftherios "Lefteris" Thanopoulos':"Lefteris Thanopoulos",
                "Lehann Fourie":"Lehann Fourie",
                "Lei Lei Win":"Lei Lei Win",
                "Leigh Ignatius Julius":"Leigh Julius",
                "Leigh Smith":"Leigh Smith",
                "Leila Maryam Ben Youssef":"Leila Ben Youssef",
                "Lena Urbaniak":"Lena Urbaniak",
                "Lenka Ledvinov":"Lenka Ledvinová",
                "Lenka Masn":"Lenka Masná",
                "Lennie Waite":"Lennie Waite",
                'Leonard "Leo" Myles-Mills':"Leo Myles-Mills",
                "Leonard Essau Korir":"Leonard Essau Korir",
                "Leonardo Nelson Price":"Leonardo Price",
                "Leonid Vladimirovich Andreyev":"Leonid Andreyev",
                "Leonid Alekseyevich Cherevko":"Leonid Cherevko",
                "Leontia Kallenou":"Leontia Kallenou",
                "Lerissa Henry":"Lerissa Henry",
                "Lerma Elmira Bulauitan-Gabito":"Lerma Bulauitan-Gabito",
                "Leslie Arthur Copeland":"Leslie Copeland",
                "Leslie Djhone":"Leslie Djhone",
                "Lesyan Mayor Gil":"Lesyaní Mayor",
                "Letitia Alma Vriesde":"Letitia Vriesde",
                "Levente Csillag":"Levente Csillag",
                "Levern Donaline Spencer":"Levern Spencer",
                "Levi Asher Cadogan":"Levi Cadogan",
                "Lewis Simon Banda":"Lewis Banda",
                'Alexis "Lexi" Weeks':"Lexi Weeks",
                "Leyla Rajabi":"Leyla Rajabi",
                "Li Fengfeng":"Li Fengfeng",
                "Li Jinzhe":"Li Jinzhe",
                "Li Ling":"Li Ling",
                "Li Lingwei":"Li Lingwei",
                "Li Meiju":"Li Meiju",
                "Li Rongxiang":"Li Rongxiang",
                "Li Runrun":"Li Runrun",
                "Li Wen-Hua (-Tsai-Yi)":"Li Wen-Hua",
                "Li Xiangyu":"Li Xiangyu",
                "Li Xiaohong":"Li Xiaohong",
                "Li Xuemei":"Li Xuemei",
                "Li Yanfeng":"Li Yanfeng",
                "Li Yanmei":"Li Yanmei",
                "Li Zhenzhu":"Li Zhenzhu",
                "Li Zhilong":"Li Zhilong",
                "Liadagmis Povea Rodrguez":"Liadagmis Povea",
                "Liang Shuyan":"Liang Shuyan",
                "Liaqat Ali":"Liaqat Ali",
                "Sinia Ergoti":"Libania Grenot",
                "Libor Charfreitag":"Libor Charfreitag",
                "Libor Malina":"Libor Malina",
                "Lidia Chojecka (-Okniska)":"Lidia Chojecka",
                "Lidiane Lopes":"Lidiane Lopes",
                'Liesbeth Jantina "Lieja" Tunks-Koeman':"Lieja Tunks-Koeman",
                "Liemarvin Bonevacia":"Liemarvin Bonevacia",
                "Liina Laasma":"Liina Laasma",
                "Likourgos-Stefanos Tsakonas":"Likourgos Tsakonas",
                "Patrina Allen":"Liliana Allen",
                "Liliana Estefania Neto":"Liliana Neto",
                "Liliana Zagacka":"Liliana Zagacka",
                "Lilit Harutyunyan":"Lilit Harutyunyan",
                "Liliya Dusmetova":"Liliya Dusmetova",
                "Liliya Dusmetova":"Liliya Shobukhova",
                "Lin Chia-Ying":"Lin Chia-Ying",
                "Lin Ching-Hsuan":"Lin Ching-Hsuan",
                "Lina Grinikait (-Samuole)":"Lina Grinčikaitė",
                "Linda Ferga-Khodadin":"Linda Ferga-Khodadin",
                "Linda Maria Sandblom":"Linda Sandblom",
                "Linda Stahl":"Linda Stahl",
                "Linden Hall":"Linden Hall",
                "Lindsay Hanekom":"Lindsay Hanekom",
                "Lindsey Anderson (Olson-)":"Lindsey Anderson",
                "Karl Linus Thrnblad":"Linus Thörnblad",
                "Lisa Corrigan":"Lisa Corrigan",
                "Lisa Jane Dobriskey (Soos-)":"Lisa Dobriskey",
                "Lisa Mayer":"Lisa Mayer",
                "Lisa Vasa Misipeka (-Allen)":"Lisa Misipeka",
                "Lisa Uhl (Koll-)":"Lisa Uhl",
                "Lisneidy Ins Veita Crdova":"Lisneidy Veitía",
                "Lissa Mary Audrey Labiche":"Lissa Labiche",
                "Liu Feiliang":"Liu Feiliang",
                "Liu Qing":"Liu Qing",
                "Liu Shiying":"Liu Shiying",
                "Liu Tingting":"Liu Tingting",
                "Liu Xiangrong":"Liu Xiangrong",
                "Liu Xiaosheng":"Liu Xiaosheng",
                "Liu Yang":"Liu Yang",
                "Liu Yinghui":"Liu Yinghui",
                "Liu Yunfeng":"Liu Yunfeng",
                "Liudmyla Viktorivna Kovalenko":"Liudmyla Kovalenko",
                "Luiza Gega":"Liz Gleadle",
                "Lloyd Zvasiya":"Lloyd Zvasiya",
                "Logan Perry Cunningham":"Logan Cunningham",
                "Lois Maikel Martnez Gonzlez":"Lois Maikel Martínez",
                'Lori "Lolo" Jones':"Lolo Jones",
                "Lopez (Lopepe-) Lomong":"Lopez Lomong",
                "Loree Smith":"Loree Smith",
                "Lorenzo Povegliano":"Lorenzo Povegliano",
                "Lornah Jebiwot Kiplagat":"Lornah Kiplagat",
                "Lorraine Amanda Shaw":"Lorraine Shaw",
                "Lorraine Efe Ugen":"Lorraine Ugen",
                "Lorne Dorcas Bazolo":"Lorène Bazolo",
                "Louis Tsatoumas":"Louis Tsatoumas",
                "Louise Carton":"Louise Carton",
                "Lovisa Tora Yvonne Lindh":"Lovisa Lindh",
                "Lu Huihui":"Lu Huihui",
                "Luan Gabriel":"Luan Gabriel",
                'Lucas "Luc" Bruchet':"Luc Bruchet",
                "Lucas Jakubczyk":"Lucas Jakubczyk",
                "Lucia Chandamale":"Lucia Chandamale",
                "Lucia Hrivnk Klocov":"Lucia Hrivnák Klocová",
                "Luciana Alves dos Santos":"Luciana Dos Santos",
                "Luciana de Paula Mendes":"Luciana Mendes",
                "Lucie Pelantov":"Lucie Pelantová",
                "Lucie Vrbensk":"Lucie Vrbenská",
                "Lucie krobkova-Martincov":"Lucie Škrobákova-Martincová",
                "Lucimar Teodoro":"Lucimar Moura",
                "Lucimar Teodoro":"Lucimar Teodoro",
                "Lucy Wangui Kabuu":"Lucy Kabuu",
                "Lucy Van Dalen-Oliver":"Lucy Van Dalen-Oliver",
                "Luis Alberto Marco Contreras":"Luis Alberto Marco",
                "Luis Felipe Mliz Linares":"Luis Felipe Méliz",
                "Luis Fernando Garca Bechinie":"Luis Fernando García",
                "Luis Fernando Lpez Erazo":"Luis Fernando López",
                "Luis Ral Luna Lemus":"Luis Luna",
                "Luis Miguel Martn Berlanas":"Luis Miguel Martín",
                "Louis Tsatoumas":"Luis Ostos",
                "Lus Joel Castro Rivera":"Luis Rivera",
                "Louise Carton":"Luis Tristán",
                "Luiza Gega":"Luiza Gega",
                "Luka Janei":"Luka Janežič",
                "Lukas Weihaidinger":"Lukas Weißhaidinger",
                "Luke Kendall Adams":"Luke Adams",
                "Luke Bezzina":"Luke Bezzina",
                "Luke Arron Cutts":"Luke Cutts",
                "Luke Mathews":"Luke Mathews",
                "Luk Melich":"Lukáš Melich",
                "Luk Milo":"Lukáš Milo",
                "Luleseged Wale":"Luleseged Wale",
                "Lutimar Abreu Paes":"Lutimar Paes",
                "Liu Feiliang":"Luís Feiteira",
                "Lus Manuel Teixeira S":"Luís Manuel Sá",
                "Lydia Chebet Rotich":"Lydia Rotich",
                "Lynique Prinsloo":"Lynique Prinsloo",
                "Lynsey Sharp":"Lynsey Sharp",
                "Lyubov Gennadyevna Perepelova":"Lyubov Perepelova",
                "Lyudmila Sergeyevna Kolchanova":"Lyudmila Kolchanova",
                "Gabriela Petrova":"Lyudmila Petrova",
                "Lszl Boros":"László Boros",
                "Lzaro Eduardo Borges Ried":"Lázaro Borges",
                "La Sprunger":"Léa Sprunger",
                "L Vn Dng L":"Lê Văn Dương Lê",
                "Lna Mze":"Līna Mūze",
                "Ma Ning":"Ma Ning",
                "Ma Xuejun":"Ma Xuejun",
                "Mabel Gay Tamayo":"Mabel Gay",
                "Machel Cedenio":"Machel Cedenio",
                "Madara Palameika":"Madara Palameika",
                "Madeleine Pape":"Madeleine Pape",
                "Madeline Hills (Heiner-)":"Madeline Hills",
                "Mae Koime":"Mae Koime",
                "Magaly Bonilla Sols":"Magaly Bonilla",
                "Maggie Malone":"Maggie Malone",
                "Jan Magnus Arvidsson":"Magnus Arvidsson",
                "Magnus Kirt":"Magnus Kirt",
                "Mahamoud Farah":"Mahamoud Farah",
                "Mahau Camargo Suguimati":"Mahau Suguimati",
                "Mahmoud Samimi":"Mahmoud Samimi",
                "Maho Hanaoka":"Maho Hanaoka",
                "Maik'o Gogoladze":"Maik'O Gogoladze",
                "Mailn Vargas Escalona":"Mailín Vargas",
                "Maizurah Abdul Rahim":"Maizurah Abdul",
                "Maja Mihalinec":"Maja Mihalinec",
                "Marine Ghazaryan":"Majed Ghazal",
                "Majed Saeed Sultan":"Majed Saeed Sultan",
                "Makelesi Bulikiobo-Batimala":"Makelesi Bulikiobo-Batimala",
                "Makoura Keita":"Makoura Keita",
                "Maksim Sergeyevich Dyldin":"Maksim Dyldin",
                "Maksim Aleksandrovich Lynsha":"Maksim Lynsha",
                "Maksim Viktorovich Sidorov":"Maksim Sidorov",
                "Maksym Anatoliyovych Mazuryk":"Maksym Mazuryk",
                "Malachi John Davis":"Malachi Davis",
                "Malaika Mihambo":"Malaika Mihambo",
                "Malik Khaled Ahmed Louahla":"Malik Louahla",
                "Malika Al-Akkaoui":"Malika Akkaoui",
                "Malindi Elmore":"Malindi Elmore",
                "Malte Mohr":"Malte Mohr",
                "Malwina Kopron":"Malwina Kopron",
                "Mamadou Barry":"Mamadou Barry",
                "Mamadou Kasse Hanne":"Mamadou Kassé Hanne",
                "Mame Tacko Diouf":"Mame Tacko Diouf",
                "Mandeep Kaur":"Mandeep Kaur",
                "Manish Singh Rawat":"Manish Singh",
                "Manjula Kumara Pathiranage Wijesekera":"Manjula Kumara Wijesekera",
                "Manpreet Kaur":"Manpreet Kaur",
                "Manuel Andr Antonio":"Manuel Antonio",
                "Manuel Damio":"Manuel Da Silva",
                "Manuel Damio":"Manuel Damião",
                "Manuel Jos Fuenmayor Berra":"Manuel Fuenmayor",
                "Manuel Martnez Gutirrez":"Manuel Martínez",
                "Manuel Olmedo Villar":"Manuel Olmedo",
                "Manula Montebrun":"Manuèla Montebrun",
                "Maoulida Daroueche":"Maoulida Daroueche",
                "Marc Burns":"Marc Burns",
                "Marc Schneeberger":"Marc Schneeberger",
                "Marcel Lomnick":"Marcel Lomnický",
                "Marcel Johannes van der Westen":"Marcel Van Der Westen",
                "Marcela Valeria Britos":"Marcela Britos",
                "Marcell Dek Nagy":"Marcell Deák Nagy",
                "Marcelo Adrin Pugliese":"Marcelo Pugliese",
                "Marcin Jdrusiski":"Marcin Jędrusiński",
                "Marcin Krukowski":"Marcin Krukowski",
                "Marcin Lewandowski":"Marcin Lewandowski",
                "Marcin Waldemar Marciniszyn":"Marcin Marciniszyn",
                "Marcin Andrzej Nowak":"Marcin Nowak",
                "Marcin Micha Starzak":"Marcin Starzak",
                "Marcin Urba":"Marcin Urbaś",
                "Marco Antonio Rodrguez Pardo":"Marco Antonio Rodríguez",
                "Marco Antonio Verni Lippi":"Marco Antonio Verni",
                "Marco Cribari":"Marco Cribari",
                "Marco Paulo Fortes":"Marco Fortes",
                "Marco Andrea Giungi":"Marco Giungi",
                "Marco Lingua":"Marco Lingua",
                "Marco Torrieri":"Marco Torrieri",
                "Marco Lingua":"Marcos Chuva",
                "Marcus la Grange":"Marcus La Grange",
                "Mardrea Elaine Hyman":"Mardrea Hyman",
                "Marek Niit":"Marek Niit",
                "Marek Plawgo":"Marek Plawgo",
                "Mareks rents":"Mareks Ārents",
                "Marestella Torres":"Marestella Torres",
                "Margaret Adetutu Adeoye":"Margaret Adeoye",
                "Margaret Bamgbose":"Margaret Bamgbose",
                "Margaret Okayo":"Margaret Maury",
                "Margaret Okayo":"Margarita Mukashova",
                "Margherita Magnani":"Margherita Magnani",
                "Margret Hassan":"Margret Hassan",
                "Marharyta Serhivna Dorozhon":"Marharyta Dorozhon",
                "Marharyta Tverdokhlib":"Marharyta Tverdokhlib",
                "Maria Andrejczyk":"Maria Andrejczyk",
                "Maria Belimpasaki":"Maria Belimpasaki",
                "Maria Benedicta Chigbolu":"Maria Benedicta Chigbolu",
                "Maria Bernard":"Maria Bernard",
                "Maria Epiph Ikelap":"Maria Ikelap",
                "Maria Laura Almiro":"Maria Laura Almirão",
                "Maria Leonor Ribeiro Tavares":"Maria Leonor Tavares",
                "Nelia Martins":"Maria Martins",
                "Maria McCambridge":"Maria Mccambridge",
                "Maria Lynn Michta-Coffey":"Maria Michta-Coffey",
                "Maria Natalia Londa":"Maria Natalia Londa",
                "Maria Protopapa":"Maria Protopapa",
                "Mariam Kromah":"Mariam Kromah",
                "Mariama Mamoudou Ittatou":"Mariama Mamoudou Ittatou",
                "Marian Joan Burnett":"Marian Burnett",
                "Mariana Cress":"Mariana Cress",
                "Mariana Dinkova Dimitrova":"Mariana Dimitrova",
                "Mariana Solomon":"Mariana Solomon",
                "Maribel Gonalves da Conceio":"Maribel Gonçalves",
                'Mariam "Marie" Kevkhishvili-Machavariani':"Marie Kevkhishvili",
                "Marie McMahon-Davenport":"Marie Mcmahon-Davenport",
                "Marie Poissonnier (-Inocencio)":"Marie Poissonnier",
                "Marie Polli":"Marie Polli",
                "Marie-Jose Ta Lou":"Marie-Josée Ta Lou",
                "Marie-Jolle Conjungo":"Marie-Joëlle Conjungo",
                "Marie-Laurence Jungfleisch":"Marie-Laurence Jungfleisch",
                "Marie-Lyne Joseph":"Marie-Lyne Joseph",
                "Marielle Hall":"Marielle Hall",
                "Mariely Snchez Hichez":"Mariely Sánchez",
                "Mariem Alaoui Selsouli":"Mariem Alaoui Selsouli",
                "Marija Martinovi-estak":"Marija Martinović-Šestak",
                "Marika Popowicz-Drapaa":"Marika Popowicz-Drapała",
                "Marilia Grigoriou":"Marilia Grigoriou",
                "Maria Cioncan":"Marina Aitova",
                "Alina Armas":"Marina Arzamasova",
                "Marina Lapina":"Marina Lapina",
                "Marina Marghiev-Nicienco":"Marina Marghiev-Nicișenco",
                "Marina Vladimirovna Maslyonko":"Marina Maslyonko",
                "Marina Munan":"Marina Munćan",
                "Marina Tomi":"Marina Tomić",
                "Marine Ghazaryan":"Marine Ghazaryan",
                "Marius Bakken":"Mario Bazán",
                "Mario Pestano Garca":"Mario Pestano",
                "Marion Buisson":"Marion Buisson",
                "Marion Lotout":"Marion Lotout",
                "Marios Evangelou":"Marios Evangelou",
                "Mariana Solomon":"Marisol Romero",
                "Maritza Guamn":"Maritza Guamán",
                "Maritza Rafaela Poncio Tzul":"Maritza Poncio",
                "Marius Bakken":"Marius Bakken",
                "Marius avelskis":"Marius Šavelskis",
                "Marius ikas":"Marius Žiūkas",
                "Mria Czakov":"Mariya Abakumova",
                "Mariya Aleksandrovna Bespalova":"Mariya Bespalova",
                "Mariya Dobrinova Dimitrova":"Mariya Dimitrova",
                "Mariya Yurivna Shatalova":"Mariya Shatalova",
                "Mariya Viktorovna Smolyachkova":"Mariya Smolyachkova",
                "Mariya Telushkina":"Mariya Telushkina",
                "Mariya Ivanovna Yakovenko":"Mariya Yakovenko",
                "Marin Bokor":"Marián Bokor",
                "Mark Boswell":"Mark Boswell",
                "Mark Carroll":"Mark Carroll",
                "Mark William Dry":"Mark Dry",
                "Mark English":"Mark English",
                "Mark Anthony Lewis-Francis":"Mark Lewis-Francis",
                "Markino Buckley":"Markino Buckley",
                "Marko Aleksejev":"Marko Aleksejev",
                "Markus Esser":"Markus Esser",
                "Markus Mnch":"Markus Münch",
                "Marina Munan":"Marla Runyan",
                "Marlena Hilari Wesh":"Marlena Wesh",
                "Marlon Ronald Devonish":"Marlon Devonish",
                "Marlyse Nsourou":"Marlyse Nsourou",
                "Marquise Derell Goodwin":"Marquise Goodwin",
                "Marshevet Dante Hooker (-Myers)":"Marshevet Hooker",
                "Marta Domnguez Azpeleta":"Marta Domínguez",
                "Makoura Keita":"Marta Freitas",
                "Marta Jeschke":"Marta Jeschke",
                "Marta Menda Valencia":"Marta Mendía",
                "Marian Oprea":"Marta Onofre",
                "Marthe Koala":"Marthe Koala",
                "Martial Mbandjock":"Martial Mbandjock",
                "Martin Kupper":"Martin Kupper",
                "Martin Kuera":"Martin Kučera",
                "Martin Mari":"Martin Marić",
                "Martin Irungu Mathathi":"Martin Mathathi",
                "Martin Prll":"Martin Pröll",
                "Martin Wierig":"Martin Wierig",
                "Martina Hranov-Daniov":"Martina Hrašnová-Danišová",
                "Martina Pretelli":"Martina Pretelli",
                "Martina Strutz":"Martina Strutz",
                "Matija estak":"Martina Šestáková",
                'Martina "Martinka" Ratej':"Martinka Ratej",
                "Martyn John Bernard":"Martyn Bernard",
                "Marua ernjul":"Maruša Černjul",
                "Marvin Anderson":"Marvin Anderson",
                "Marvin Bracy":"Marvin Bracy",
                "Kariem Hussein":"Marwa Hussein",
                'Nkemdilim "Mary" Onyali-Omagbemi':"Mary Onyali-Omagbemi",
                "Maryan Nuh Muse":"Maryan Muse",
                "Maryna V'iacheslavivna Kylypko":"Maryna Kylypko",
                "Maryna Serhivna Maidanova":"Maryna Maidanova",
                "Marzia Caravelli":"Marzia Caravelli",
                "Mazoon Khalfan Saleh Al-Alawi":"Marzoon Al-Alawi",
                "Mara Abel Diguez":"María Abel",
                "Mara del Mar Jover Prez":"María Del Mar Jover",
                "Mara del Rosario Snchez Guerrero":"María Del Rosario Sánchez",
                "Mariela Gonzlez Torres":"María González",
                "Mara Guadalupe Snchez Magaa":"María Guadalupe Sánchez",
                "Mara Jos Povs Novella":"María José Povés",
                "Mara Romary Rifka Gonzlez":"María Romary Rifka",
                "Masashi Eriguchi":"Masashi Eriguchi",
                "Masato Naito":"Masato Naito",
                "Masato Yokota":"Masato Yokota",
                "Masbah Ahmmed":"Masbah Ahmmed",
                "Mason Finley":"Mason Finley",
                "Masoud Azizi":"Masoud Azizi",
                "Masumi Fuchise":"Masumi Fuchise",
                "Mateusz Przybylko":"Mateusz Przybylko",
                "Matheus Facho Inocncio":"Matheus Inocêncio",
                "Mathieu Bilodeau":"Mathews Binu",
                "Mathias Ntawulikura":"Mathias Ntawulikura",
                "Mathieu Gnanligo Fousseni":"Mathieu Gnanligo",
                "Mathilde Andraud":"Mathilde Andraud",
                "Matic Osovnikar":"Matic Osovnikar",
                "Matija Kranjc":"Matija Kranjc",
                "Matija estak":"Matija Šestak",
                'Matthew "Matt" Douglas':"Matt Douglas",
                'Matthew Lloyd "Matt" Hudson-Smith':"Matt Hudson-Smith",
                'Matthew "Matt" Hughes':"Matt Hughes",
                'Matthew "Matt" Tegenkamp':"Matt Tegenkamp",
                "Matteo Galvan":"Matteo Galvan",
                "Matteo Giupponi":"Matteo Giupponi",
                "Matteo Villani":"Matteo Villani",
                "Matthew Denny":"Matthew Denny",
                "Matthias Bhler":"Matthias Bühler",
                "Matthias de Zordo":"Matthias De Zordo",
                "Matti Viljami Mononen":"Matti Mononen",
                "Juha Matti Nrhi":"Matti Närhi",
                "Matylda Szlzak-Kowal":"Matylda Szlęzak-Kowal",
                "Mat Bubenik":"Matúš Bubeník",
                "Maureen Jelagat Maiyo":"Maureen Jelagat Maiyo",
                "Maureen Koster":"Maureen Koster",
                "Maurice Greene":"Maurice Greene",
                "Maurice Mitchell":"Maurice Mitchell",
                "Maurice Andre Wignall":"Maurice Wignall",
                "Mauricio Jos Arteaga Snchez":"Mauricio Arteaga",
                "Maya Rehberg":"Maya Rehberg",
                "Pavel Maslk":"Maykel Massó",
                "Mayookha Mathalikunnel Johny":"Mayookha Johny",
                "Mayra Carolina Herrera Prez":"Mayra Herrera",
                "Matea Matoevi":"Mayte Martínez",
                "Mayumi Kawasaki":"Mayumi Kawasaki",
                "Maziah Mahusin":"Maziah Mahusin",
                "Mala de Paula Machado":"Maíla Machado",
                "Masa Rejepowa":"Maýsa Rejepowa",
                "Magorzata Houb":"Małgorzata Hołub",
                "Magorzata Pskit":"Małgorzata Pskit",
                "Magorzata Zadura":"Małgorzata Zadura",
                "Morgan Mitchell":"Megan Metcalfe",
                "Megan Simmonds":"Megan Simmonds",
                "Megumi Tanaka (-Oshima)":"Megumi Tanaka",
                "Mehboob Ali":"Mehboob Ali",
                "Mekdes Bekele Tadesse":"Mekdes Bekele",
                "Mekonnen Gebremedhin Woldegeorgis":"Mekonnen Gebremedhin",
                "Melanie Seeger":"Melanie Seeger",
                "Melik Janoyan":"Melik Janoyan",
                "Melina Dawn Hamilton (-Holmes)":"Melina Hamilton",
                "Melissa Murillo Rivas":"Melisa Murillo",
                "Melissa Corrine Bishop":"Melissa Bishop",
                "Melissa Boekelman":"Melissa Boekelman",
                "Melissa Breen":"Melissa Breen",
                "Meng Yan":"Meng Yan",
                "Menzi Dlamini":"Menzi Dlamini",
                "Meraf Bahta Ogbagaber":"Meraf Bahta",
                "Mercedes Chilla Lpez":"Mercedes Chilla",
                "Mercy Cherono Koech":"Mercy Cherono",
                "Mercy Wanjiku Njoroge":"Mercy Njoroge",
                "Mergen Mmmedow":"Mergen Mämmedow",
                "Meritzer Williams":"Meritzer Williams",
                "Merja Tuulikki Korpela":"Merja Korpela",
                "Merlene Joyce Ottey-Page":"Merlene Ottey-Page",
                "Mert Atl":"Mert Atlı",
                "Merve Aydn":"Merve Aydın",
                "Mervyn Richard Luckwell":"Mervyn Luckwell",
                "Merve Aydn":"Meryem Akdağ",
                "Meselech Melkamu Haileyesus":"Meselech Melkamu",
                "Meskerem Legesse":"Meskerem Legesse",
                "Mestawat Tadesse":"Mestawat Tadesse",
                "Mestawet Tufa Demisse":"Mestawet Tufa",
                "Mesud Pezer":"Mesud Pezer",
                "Michael Aguilar":"Michael Aguilar",
                "Michael Buchleitner":"Michael Buchleitner",
                "Michael Bultheel":"Michael Bultheel",
                "Michael Audley Frater":"Michael Frater",
                "Michael Herrera Flores":"Michael Herrera",
                "Michael Walter Mathieu":"Michael Mathieu",
                "Michael Friedrich Mllenbeck":"Michael Möllenbeck",
                "Michael John Rimmer":"Michael Rimmer",
                "Michael Robertson":"Michael Robertson",
                "Michael Kosgei Rotich":"Michael Rotich",
                "Michaela Kargbo":"Michaela Kargbo",
                "Sara Michaela Meijer":"Michaela Meijer",
                "Michal Balner":"Michal Balner",
                "Michal Kabelka":"Michal Kabelka",
                "Michal neberger":"Michal Šneberger",
                "Micha Bieniek":"Michał Bieniek",
                "Micha Haratyk":"Michał Haratyk",
                "Michaela Hrub":"Michel Tornéus",
                "Michelle Ballentine":"Michelle Ballentine",
                "Michelle Banga Moudzoula":"Michelle Banga Moudzoula",
                "Michelle Carey":"Michelle Carey",
                "Michelle Carey":"Michelle Carter",
                "Michelle Finn":"Michelle Finn",
                "Michelle-Lee Raquel Ahye":"Michelle-Lee Ahye",
                "Mickal Hanany":"Mickaël Hanany",
                "Micol Cattaneo":"Micol Cattaneo",
                "Miguel Lpez":"Miguel López",
                "Miguel S. Pate":"Miguel Pate",
                "Miguel Quesada Velasco":"Miguel Quesada",
                "Miguel Lpez":"Miguel Ángel López",
                "Miguel ngel Rodrguez Gallegos":"Miguel Ángel Rodríguez",
                "Mihaela Maria Botezan":"Mihaela Botezan",
                "Michaela Hrub":"Mihaela Neacșu",
                "Mihkel Kukk":"Mihkel Kukk",
                "Miho Sugimori":"Miho Sugimori",
                "Mihter Wendolin":"Mihter Wendolin",
                "Mika Yoshikawa (Oshima-)":"Mika Yoshikawa",
                "Mikaela Johanna Emilia Ingberg":"Mikaela Ingberg",
                "Mike Fenner":"Mike Fenner",
                'Michael "Mike" Hartfield':"Mike Hartfield",
                "Mikel Kevon Thomas":"Mikel Thomas",
                "Mikhail Sergeyevich Lipsky":"Mikhail Lipsky",
                "Mikhailis Anastasakis":"Mikhailis Anastasakis",
                "Mikhalis Stamatogiannis":"Mikhalis Stamatogiannis",
                "Mikko Antti Juhani Lahtio":"Mikko Lahtio",
                "Mikko Tapio Latvala":"Mikko Latvala",
                "Mikul Konopka":"Mikuláš Konopka",
                "Milan Habork":"Milan Haborák",
                "Milan Risti":"Milan Ristić",
                "Milan Trajkovic":"Milan Trajkovic",
                "Milena Milaevi":"Milena Milašević",
                "Miles Ukaoma":"Miles Ukaoma",
                "Miloslav Konopka":"Miloslav Konopka",
                "Miloud Rahmouni":"Miloud Rahmouni",
                "Miltiadis Tentoglou":"Miltiadis Tentoglou",
                "Milngela Franchesca Rosales Davila":"Milángela Rosales",
                'Amina "Mina" At Hammou':"Mina Aït Hammou",
                "Mindaugas Norbutas":"Mindaugas Norbutas",
                "Minna Marianne Nikkanen":"Minna Nikkanen",
                "Minori Hayakari":"Minori Hayakari",
                'Miroslav "Miran" Vodovnik':"Miran Vodovnik",
                "Miranda Melville":"Miranda Melville",
                "Mireille Dereboba-Ngaisset":"Mireille Dereboba-Ngaisset",
                "Mireille Florine Nguimgo":"Mireille Nguimgo",
                "Mirela Maniani-Tzelili":"Mirela Maniani-Tzelili",
                "Miriam Bobkov":"Miriam Bobková",
                "Martina Strutz":"Mirna Ortíz",
                "Miroslav Guzdek":"Miroslav Guzdek",
                "Misaki Onishi":"Misaki Onishi",
                "Misleydis Gonzlez Tamayo":"Misleydis González",
                'Mitchell "Mitch" Kealey':"Mitch Kealey",
                "Mitko Tsenov":"Mitko Tsenov",
                "Mitsuhiro Sato":"Mitsuhiro Sato",
                "Miyuki Uehara":"Miyuki Uehara",
                "Moa Elin Marianne Hjelmer":"Moa Hjelmer",
                "Moacir Zimmermann":"Moacir Zimmermann",
                "Mohamad Al-Garni":"Mohamad Al-Garni",
                "Mohamad Siraj Tamim":"Mohamad Siraj Tamim",
                "Mohamed Abu Abdullah":"Mohamed Abu Abdullah",
                "Mohamed Issa Al-Thawadi":"Mohamed Al-Thawadi",
                "Mohamed Ameur":"Mohamed Ameur",
                "Mohamed Amyn":"Mohamed Amyn",
                "Mohamed El-Hachimi":"Mohamed El-Hachimi",
                "Mohamed Fakhri Ismail":"Mohamed Fakhri Ismail",
                "Mohamed Khaldi":"Mohamed Fathalla",
                "Mohamed Khaldi":"Mohamed Khaldi",
                "Mohamed Lamine Dansoko":"Mohamed Lamine Dansoko",
                "Mohamed Daud Mohamed":"Mohamed Mohamed",
                "Mohamed Moustaoui":"Mohamed Moustaoui",
                "Mohamed Al-Sghaier":"Mohamed Sghaier",
                "Mohammad Anas Yahiya":"Mohammad Anas Yahiya",
                "Mohammad Arzandeh":"Mohammad Arzandeh",
                "Mohammad Shamsuddin":"Mohammad Shamsuddin",
                "Mohammed Abukhousa":"Mohammed Abukhousa",
                "Mohammed Ahmed":"Mohammed Ahmed",
                "Mohammed Ahmed Al-Yafaee":"Mohammed Al-Yafaee",
                "Mohamed Khaldi":"Mohammed Khuwalidi",
                "Mohammed Rageh":"Mohammed Rageh",
                "Mohammed Othman H. Shaween":"Mohammed Shaween",
                "Mohan Khan":"Mohan Khan",
                "Mohan Khan":"Mohsen Anany",
                "Moise Joseph":"Moise Joseph",
                "Molly Huddle (-Benninger)":"Molly Huddle",
                "Monder Rizki":"Monder Rizki",
                "Monica Iagr-Dinescu":"Monica Iagăr-Dinescu",
                "Monica Stoian":"Monica Stoian",
                "Monika Joanna Bejnar":"Monika Bejnar",
                "Monika Georgieva Gachevska":"Monika Gachevska",
                "Monika Niedersttter":"Monika Niederstätter",
                "Liesbeth Monique Jansen":"Monique Jansen",
                "Montell Marcelle M. Douglas":"Montell Douglas",
                "Montserrat Pujol Joval":"Montserrat Pujol",
                "Moonika Aava":"Moonika Aava",
                "Morgan Uceny":"Morgan Lake",
                "Morgan Mitchell":"Morgan Mitchell",
                "Morgan Uceny":"Morgan Uceny",
                "Morten Jensen":"Morten Jensen",
                "Moses Kamut":"Moses Kamut",
                "Moses Ndiema Kipsiro":"Moses Kipsiro",
                "Moses Martin Kurong":"Moses Kurong",
                "Moses Ndiema Masai":"Moses Masai",
                "Moses Moeketsi Mosuhli":"Moses Mosop",
                "Mosito Lehata":"Mosito Lehata",
                'Mostafa Al-Ismaili "Smaili"':"Mostafa Smaili",
                "Mouhssin Al-Chehibi":"Mouhssin Chehibi",
                "Moukheld Mahil F. Al-Outaibi":"Moukheld Al-Outaibi",
                "Moumi Sbergu":"Moumi Sébergué",
                "Mounir Yemmouni":"Mounir Yemmouni",
                "Mourad Marofit":"Mourad Marofit",
                "Moussa Aouanouk":"Moussa Aouanouk",
                "Moussa Camara":"Moussa Camara",
                "Moussa Dembele":"Moussa Dembele",
                "Mowen Boino":"Mowen Boino",
                "Muhammad Sajid Ahmad":"Muhammad Sajid Ahmad",
                "Mujinga Elodie Kambundji":"Mujinga Kambundji",
                "Muktar Edris Awel":"Muktar Edris",
                "Mulern Jean":"Mulern Jean",
                "Mulugeta Wendimu Gebere":"Mulugeta Wendimu",
                "Mumin Booqora Gala":"Mumin Gala",
                "Muna Kalameya Durka":"Muna Durka",
                "Muna Jabir Adam Ahmed":"Muna Jabir Adam",
                "Muna Lee":"Muna Lee",
                "Muriel Coneo Paredes":"Muriel Coneo",
                "Murielle Ahour":"Murielle Ahouré",
                "Musa Audu":"Musa Amer",
                "Musa Hajdari":"Musa Hajdari",
                "Musaeb Abdulrahman Balla":"Musaeb Abdulrahman Balla",
                "Hassan Mustafa Mohamed":"Mustafa Mohamed",
                "Myint Swe Li":"Myint Swe Li",
                "Mykhailo Mykolaiovych Knysh":"Mykhailo Knysh",
                "Mykola Mykolaiovych Labovskiy":"Mykola Labovskiy",
                "Mykyta Vasylovych Nesterenko":"Mykyta Nesterenko",
                "Myriam Lonie Mani":"Myriam Mani",
                "Myriam Soumar":"Myriam Soumaré",
                "Mrcio Simo de Souza":"Márcio De Souza",
                "Marcio Soares Teles":"Márcio Teles",
                "Mria Czakov":"Mária Czaková",
                "Mria Glikov":"Mária Gáliková",
                "Mt Helebrandt":"Máté Helebrandt",
                "Mrt Israel":"Märt Israel",
                "Mlanie Blouin":"Mélanie Blouin",
                "Melanie Skotnik-Melfort":"Mélanie Skotnik-Melfort",
                "Mlina Robert-Michon":"Mélina Robert-Michon",
                "Mnica Equihua Solarzano":"Mónica Equihua",
                "Mris Urtns":"Māris Urtāns",
                "Nabie Foday Fofanah":"Nabie Foday Fofanah",
                "Nabil Mohammed Al-Garbi":"Nabil Al-Garbi",
                "Nabil Madi":"Nabil Madi",
                "Nader Abdullah Al-Massri":"Nader Al-Massri",
                "Nadezhda Nikolayevna Ostapchuk":"Nadezhda Ostapchuk",
                "Nadia Mohamed Ejjafini":"Nadia Ejjafini",
                "Nadia Noujani":"Nadia Noujani",
                "Nadine Beckel":"Nadine Beckel",
                "Nadine Faustin-Parker":"Nadine Faustin-Parker",
                "Nadine Gonska":"Nadine Gonska",
                "Nadine Hentschke":"Nadine Hentschke",
                "Nadine Hildebrand":"Nadine Hildebrand",
                "Nadine Kleinert-Schmitt":"Nadine Kleinert-Schmitt",
                "Nadine Mller":"Nadine Müller",
                "Nadine Mller":"Nadine Rohr",
                "Nadine Visser":"Nadine Visser",
                "Nadiya Dusanova":"Nadiya Dusanova",
                "Nadiya Ivanivna Prokopuk-Borovska":"Nadiya Prokopuk-Borovska",
                "Nadjim Manseur":"Nadjim Manseur",
                "Nafissa Souleymane":"Nafissa Souleymane",
                "Nagihan Karadere":"Nagihan Karadere",
                "Nagmeldin Ali Abubakr":"Nagmeldin Ali",
                "Nahida Mahour Bacha Touhami":"Nahida Touhami",
                "Nahom Mesfin Tariku":"Nahom Mesfin",
                "Nadine Gonska":"Naide Gomes",
                "Naiel Santiago D'Almeida":"Naiel D'Almeida",
                "Najma Parveen":"Najma Parveen",
                "Namakoe Nkhasi":"Namakoe Nkhasi",
                "Nancy Chepkwemoi":"Nancy Chepkwemoi",
                "Naoki Tsukahara":"Naoki Tsukahara",
                "Naoyuki Daigo":"Naoyuki Daigo",
                "Narcisa Marisol Landzuri Bentez":"Narcisa Landázuri",
                "Naroa Agirre Kamio":"Naroa Agirre",
                "Nasria Baghdad-Azadj":"Nasria Baghdad-Azaïdj",
                "Maya Rehberg":"Nastja Reiberger",
                "Natalia Artic-Stratulat":"Natalia Artic-Stratulat",
                "Nataliya Berkut":"Natalia Cercheş",
                "Natalia Duc Soler":"Natalia Ducó",
                "Natalia Rodrguez Martnez":"Natalia Rodríguez",
                "Natalie Saville":"Natalie Saville",
                "Nataliya Asanova":"Nataliya Asanova",
                "Nataliya Berkut":"Nataliya Berkut",
                "Nataliya Viktorivna Fokina-Semenova":"Nataliya Fokina-Semenova",
                "Nataliya Evgenyevna Ivoninskaya":"Nataliya Ivoninskaya",
                "Nataliya Valeryevna Khoronenko-Mikhnevich":"Nataliya Khoronenko-Mikhnevich",
                "Nataliya Petrovna Khrushchelyeva":"Nataliya Khrushchelyeva",
                "Nataliya Stanislavovna Koreyvo":"Nataliya Koreyvo",
                "Nataliya Mikhaylovna Kresova-Rusakova":"Nataliya Kresova-Rusakova",
                "Nataliya Petrivna Kushch-Mazuryk":"Nataliya Kushch-Mazuryk",
                "Nataliya Oleksivna Lupu":"Nataliya Lupu",
                "Nataliya Andreyevna Murinovich":"Nataliya Murinovich",
                "Nataliya Olehivna Pohrebniak":"Nataliya Pohrebniak",
                "Nataliya Oleksandrivna Pryshchepa":"Nataliya Pryshchepa",
                "Nataliya Mikhaylovna Shimchuk":"Nataliya Shimchuk",
                "Nataliya Yevhenivna Strohova":"Nataliya Strohova",
                "Nataliya Vyacheslavovna Torshina-Alimzhanova":"Nataliya Torshina-Alimzhanova",
                "Nataliya Vitalyevna Yevdokimova":"Nataliya Yevdokimova",
                "Nataliya Olehivna Zolotukhina":"Nataliya Zolotukhina",
                "Natasha Laren Mayers":"Natasha Mayers",
                "Natasha Wodak":"Natasha Wodak",
                'Nathan "Nate" Brannen':"Nate Brannen",
                "Nathalie De Vos":"Nathalie De Vos",
                "Natoya Goule":"Natoya Goule",
                "Nauraj Singh Randhawa Amarjit Singh":"Nauraj Singh Randhawa",
                "Malbas Jamous Nawal El-Jack":"Nawal El-Jack",
                "Nazar Begliyev":"Nazar Begliyev",
                "Nazar Viktorovych Kovalenko":"Nazar Kovalenko",
                "Nazmizan Mohamad":"Nazmizan Mohamad",
                "Ndiss Kaba Badji":"Ndiss Kaba Badji",
                "Ndye Fatou Seck Souma":"Ndèye Fatou Soumah",
                "Nebiat Habtemariam Measho":"Nebiat Habtemariam",
                "Ned Justeen Azemia":"Ned Azemia",
                "Nedad Mulabegovi":"Nedžad Mulabegović",
                "Neelam Jaswant Singh Dogra":"Neelam Jaswant Singh Dogra",
                "Neil Weare":"Neil Weare",
                "Neisha Bernard-Thomas":"Neisha Bernard-Thomas",
                "Nelia Martins":"Nelia Martins",
                "Nelia Ihorivna Neporadna":"Nelia Neporadna",
                "Nelkis Teresa Casabona Gonzlez":"Nelkis Casabona",
                "Nelson Kipkosgei Cherutich":"Nelson Cherutich",
                "Nelson Stone":"Nelson Stone",
                "Nenad Lonar":"Nenad Lončar",
                "Nercelys Desire Soto Soto":"Nercely Soto",
                "Neringa Aidietyt":"Neringa Aidietytė",
                "Nethaneel Joseph Mitchell-Blake":"Nethaneel Mitchell-Blake",
                "Nevena Mineva Dimitrova":"Nevena Mineva",
                "Nevin Yant":"Nevin Yanıt",
                "Ngerak Florencio":"Ngerak Florencio",
                'Ngonidzashe "Ngoni" Makusha':"Ngoni Makusha",
                "Nguse Tesfaldet Amlosom":"Nguse Tesfaldet",
                "Nguyn Thnh Ngng":"Nguyễn Thành Ngưng",
                "Nguyn Th Huyn":"Nguyễn Thị Huyền",
                "Nguyn Th Thanh Phc":"Nguyễn Thị Thanh Phúc",
                "Nguyn nh Cng":"Nguyễn Đình Cương",
                "Nicholas Kiplagat Bett":"Nicholas Bett",
                "Nicholas Kemboi":"Nicholas Kemboi",
                'Nicholas Jean "Nick" Buckfield':"Nick Buckfield",
                "Nadine Mller":"Nick Miller",
                'Nicholas "Nick" Symmonds':"Nick Symmonds",
                "Nickel Ashmeade":"Nickel Ashmeade",
                "Nickiesha Wilson":"Nickiesha Wilson",
                "Niclas Emanuel Sandells":"Niclas Sandells",
                'Niconnor "Nico" Alexander':"Nico Alexander",
                "Nicola Ciotti":"Nicola Ciotti",
                "Nicola Trentin":"Nicola Trentin",
                "Nicola Vizzoni":"Nicola Vizzoni",
                "Nicolai Portelli":"Nicolai Portelli",
                "Nicolas Assat":"Nicolas Aïssat",
                "Nicolas Figre":"Nicolas Figère",
                "Nicolas Guigon":"Nicolas Guigon",
                "Nicolas Macrozonaris":"Nicolas Macrozonaris",
                "Nicole Bchler":"Nicole Büchler",
                "Nicole Wendy Forrester":"Nicole Forrester",
                "Nicole Ramalalanirina (-Valrie)":"Nicole Ramalalanirina",
                "Nicole Sifuentes (Edwards-)":"Nicole Sifuentes",
                "Nicole M. Teter":"Nicole Teter",
                "Nicoleen Cronje":"Nicolene Cronje",
                "Lenua Nicoleta Grdinaru-Grasu":"Nicoleta Grădinaru-Grasu",
                "Nigel Levine":"Nigel Levine",
                "Nigina Sharipova":"Nigina Sharipova",
                'Nikolia "Niki" Mitropoulou':"Niki Mitropoulou",
                'Nikita "Niki" Palli':"Niki Palli",
                "Niki Panetta":"Niki Panetta",
                "Niki Xanthou":"Niki Xanthou",
                "Nikita Viktorovich Filippov":"Nikita Filippov",
                "Nikki Jayne Hamblin":"Nikki Hamblin",
                "Nikkita Holder":"Nikkita Holder",
                "Niklas Bo Arrhenius":"Niklas Arrhenius",
                "Niko Verakauta":"Niko Verekauta",
                "Nikola Tomekov-Brejchov":"Nikola Tomečková-Brejchová",
                "Nikolas Skarvelis":"Nikolas Skarvelis",
                "Nikolay Atanasov Atanasov":"Nikolay Atanasov",
                "Nikolay Anatolyevich Chavkin":"Nikolay Chavkin",
                "Nikoleta Kyriakopoulou":"Nikoleta Kyriakopoulou",
                "Nikolett Szab":"Nikolett Szabó",
                "Nikolina Horvat":"Nikolina Horvat",
                "Nils Brembach":"Nils Brembach",
                "Nils Winter":"Nils Winter",
                "Nilson de Oliveira Andr":"Nilson André",
                "Nimet Karaku":"Nimet Karakuş",
                "Nina Kolari":"Nina Kolarič",
                "Nirinaharifidy Ramilijaona":"Nirinaharifidy Ramilijaona",
                "Nirmala Sheoran":"Nirmala Sheoran",
                "Niurka Montalvo Amaro":"Niurka Montalvo",
                "Nixon Kiplimo Chepseba":"Nixon Chepseba",
                "Noah Akwu":"Noah Akwu",
                "Nobuharu Asahara":"Nobuharu Asahara",
                "Noelle Montcalm":"Noelle Montcalm",
                "Noelly Mankatu Bibiche":"Noelly Mankatu Bibiche",
                "Nooa Takooa":"Nooa Takooa",
                "Noor Hussain Al-Malki":"Noor Al-Malki",
                "Nooralotta Maria Neziri":"Nooralotta Neziri",
                "Nora Ada Bicet Juan":"Nora Aída Bicet",
                "Noraseela Mohd Khalid":"Noraseela Mohd Khalid",
                "Norica Cmpean":"Norica Cîmpean",
                "Norma Gonzlez Camilde":"Norma González",
                "Nolie Yarigo":"Noélie Yarigo",
                "Nungruthai Chaipetch":"Nungruthai Chaipetch",
                "Nuray Tezeta Srekli (Desalgn-Dengersa-)":"Nuray Tezeta Sürekli",
                "Nura Fernndez Domnguez":"Nuría Fernández",
                "Nwanneka Mariauchenna Okwelogu":"Nwanneka Okwelogu",
                "Ndia Semedo":"Nédia Semedo",
                "Nbia Aparecida Soares":"Núbia Soares",
                "O'Dayne Richards":"O'Dayne Richards",
                "Obadele Olutosin Thompson":"Obadele Thompson",
                "Joseph Obinna Metu":"Obinna Metu",
                "Ockert Diderich Cilliers":"Ockert Cilliers",
                "Odile Ahouanwanou":"Odile Ahouanwanou",
                "Ogho-Ogene Omano Egwero":"Ogho-Oghene Egwero",
                "Okilani Tinilau":"Okilani Tinilau",
                "Okkert Brits":"Okkert Brits",
                "Oksana Luneva":"Oksana Luneva",
                "Oksana Yurivna Shkurat":"Oksana Shkurat",
                "Oksana Pavlovna Udmurtova":"Oksana Udmurtova",
                "Oksana Aleksandrovna Yarygina":"Oksana Yarygina",
                "Oksana Aleksandrovna Yesipchuk":"Oksana Yesipchuk",
                "Oksana Volodymyrivna Zakharchuk":"Oksana Zakharchuk",
                "Ola Sesay":"Ola Sesay",
                "Oldika Mareov":"Oldřiška Marešová",
                "Ole Hesselbjerg":"Ole Hesselbjerg",
                "Oleg Olegovich Mishukov":"Oleg Mishukov",
                "Oleg Normatov":"Oleg Normatov",
                "Oleg Sergeyevich Kulkov":"Oleg Sergeyev",
                "Oleg Borisovich Zhuravlyov":"Oleg Zhuravlyov",
                "Oleksandra Ryzhkova":"Oleksandr Dryhol",
                "Oleksandr Mykolaiovych Korchmid":"Oleksandr Korchmid",
                "Oleksandra Ryzhkova":"Oleksandr Krykun",
                "Aleksandar Rakovi":"Oleksandr Nartov",
                "Oleksandr Serhiyovych P'iatnytsia":"Oleksandr P'Iatnytsia",
                "Inna Oleksandrivna Saienko":"Oleksandra Stadniuk",
                "Oleksiy Serhiyovych Semenov":"Oleksiy Semenov",
                "Oleksiy Mykolaiovych Sokyrskiy":"Oleksiy Sokyrskiy",
                "Olena Anatolivna Antonova":"Olena Antonova",
                "Olena Kholosha":"Olena Kholosha",
                "Olesea Cojuhari":"Olesea Cojuhari",
                "Olesia Ivanivna Povkh":"Olesia Povkh",
                "Olga Yevgenyevna Bludova-Safronova":"Olga Bludova-Safronova",
                "Olga Bolova":"Olga Bolşova",
                "Olga Mikhaylovna Burova-Chernyavskaya (Davydova-)":"Olga Burova-Chernyavskaya",
                "Olga Cristea":"Olga Cristea",
                "Olga Ivanovna Kurban":"Olga Ivanova",
                "Olga Kaidantzi":"Olga Kaidantzi",
                "Olga Pavlovna Kravtsova":"Olga Kravtsova",
                "Olga Sergeyevna Kuzenkova":"Olga Kuzenkova",
                "Olga Sergeyevna Ryabinkina":"Olga Ryabinkina",
                "Olga Aleksandrovna Sergeyenko-Sudareva":"Olga Sergeyenko-Sudareva",
                "Olga Shchukina":"Olga Shchukina",
                "Olga Alekseyevna Tereshkova":"Olga Tereshkova",
                "Olga Mikhaylovna Tsander":"Olga Tsander",
                "Olga-Anastasia Vasdeki":"Olga Vasdeki",
                "Olha Anatolivna Bibik":"Olha Bibik",
                "Olha Yevhenivna Ivankova":"Olha Ivankova",
                "Olha Oleksandrivna Liakhova":"Olha Liakhova",
                "Olha Mykolavna Skrypak":"Olha Skrypak",
                "Olha Oleksandrivna Yakovenko":"Olha Yakovenko",
                "Olive Mary Loughnane":"Olive Loughnane",
                "Oliver Dziubak":"Oliver Dziubak",
                "Olivera Jevti":"Olivera Jevtić",
                "Olivia Celia McKoy":"Olivia Mckoy",
                "Olivier Irabaruta":"Olivier Irabaruta",
                "Oliver Sanou":"Olivier Sanou",
                "Olli-Pekka Karjalainen":"Olli-Pekka Karjalainen",
                "Olusoji Adetokumbo Fasuba":"Olusoji Fasuba",
                "Oluwatobiloba Ayomide Amusan":"Oluwatobiloba Amusan",
                "Om Prakash Singh Karhana":"Om Prakash Singh Karhana",
                "Omar Jouma Bilal Al-Salfa":"Omar Al-Salfa",
                "Omar Cisneros Bonora":"Omar Cisneros",
                "Omar Ahmed Abdel Latif El-Ghazaly":"Omar El-Ghazaly",
                "Omar Segura Mendoza":"Omar Segura",
                "Omolara Omotosho":"Omolara Omotosho",
                "Onalenna Oabona Baloyi":"Onalenna Baloyi",
                "Orukpe Eraiyokan":"Orukpe Eraiyokan",
                "Osbourne Moxey":"Osbourne Moxey",
                "Oscar Leonard Carl Pistorius":"Oscar Pistorius",
                "Tommi Oskari Frsn":"Oskari Frösén",
                "Santtu Oskari Mr":"Oskari Mörö",
                "Osleidys Menndez Sez":"Osleidys Menéndez",
                "Marily dos Santos":"Osmar Dos Santos",
                "Oumar Loum":"Oumar Loum",
                "Oxana Juravel":"Oxana Juravel",
                "Pablo Andrs Pietrobelli":"Pablo Pietrobelli",
                "Pamela Dutkiewicz":"Pamela Dutkiewicz",
                "Pamela Chardene Mouele-Mboussi":"Pamela Mouele-Mboussi",
                "Panagiota Tsinopoulou":"Panagiota Tsinopoulou",
                "Panagiotis Sarris":"Panagiotis Sarris",
                "Panagiotis Stroubakos":"Panagiotis Stroubakos",
                "Role Prezelj":"Paola Pérez",
                "Paolo Csar Yurivilca Caldern":"Paolo Yurivilca",
                "Paraskevi Papakhristou":"Paraskevi Papakhristou",
                "Paraskevi Theodorou":"Paraskevi Theodorou",
                "Park Chil-Seong":"Park Chil-Seong",
                "Park Jae-Myeong":"Park Jae-Myeong",
                "Park Tae-Gyeong":"Park Tae-Gyeong",
                "Pascal Martinot-Lagarde":"Pascal Martinot-Lagarde",
                "Paskar Owor":"Paskar Owor",
                'Patrick O. "Pat" Jarrett':"Pat Jarrett",
                "Patrick Tuara":"Pat Taea",
                "Patience Okon George":"Patience Okon George",
                "Patric Suter":"Patric Suter",
                "Patricia Sarrapio Martn":"Patricia Sarrapio",
                "Patricia Sylvester (Cornwall-)":"Patricia Sylvester",
                "Patrick George Johnson":"Patrick Johnson",
                "Patrick Tiernan":"Patrick Tiernan",
                "Patrick Tuara":"Patrick Tuara",
                "Arne Patrik Kristiansson (-Klft)":"Patrik Kristiansson",
                "Patrina Allen":"Patrina Allen",
                "Patrycja Wyciszkiewicz":"Patrycja Wyciszkiewicz",
                "Patryk Dobek":"Patryk Dobek",
                "Patrcia Mbengani Bravo Mamona":"Patrícia Mamona",
                "Patsy Serafina Akeli":"Patsy Akeli",
                "John Paul Brizzel":"Paul Brizzel",
                "Paul Geoffrey Burgess":"Paul Burgess",
                "Paul Williams":"Paul Di Bella",
                "Paul Hession":"Paul Hession",
                "Paul Williams":"Paul Williams",
                "Paula Leena Huhtaniemi (-Tarvainen)":"Paula Huhtaniemi",
                "Paula Jane Radcliffe (-Lough)":"Paula Radcliffe",
                "Paulina Buziak":"Paulina Buziak",
                "Paulina Guba":"Paulina Guba",
                "Pauline Kwalea":"Pauline Kwalea",
                "Pauline Pousse":"Pauline Pousse",
                "Paulo Amotun Lokoro":"Paulo Amotun",
                "Paul Williams":"Paulo Villar",
                "Pauls Pujts":"Pauls Pujāts",
                "Pavel Gennadyevich Boreysha":"Pavel Boreysha",
                "Pavel Leonidovich Chumachenko":"Pavel Chumachenko",
                "Pavel Aleksandrovich Gerasimov":"Pavel Gerasimov",
                "Pavel Eduardovich Krivitsky":"Pavel Krivitsky",
                "Pavel Maslk":"Pavel Maslák",
                "Pavel Vladimirovich Potapovich":"Pavel Potapovich",
                "Pavel Aleksandrovich Trenikhin":"Pavel Trenikhin",
                "Pavla Hamkov (-Rybov)":"Pavla Hamáčková",
                "Pawe Czapiewski":"Paweł Czapiewski",
                "Pawe Fajdek":"Paweł Fajdek",
                "Pawe Rakoczy":"Paweł Rakoczy",
                "Pawe Wojciechowski":"Paweł Wojciechowski",
                "Pedro Daniel Gmez Cruz":"Pedro Gómez",
                "Pedrya Seymour":"Pedrya Seymour",
                "Peninah Arusei Jerop":"Peninah Arusei",
                "Peoria Koshiba":"Peoria Koshiba",
                "Perdita Felicien":"Perdita Felicien",
                "Periklis Iakovakis":"Periklis Iakovakis",
                'Perresha Alexandra "Perri" Shakes-Drayton':"Perri Shakes-Drayton",
                "Philip Enrique Perseus Karlstrm":"Perseus Karlström",
                "Peruth Chemutai":"Peruth Chemutai",
                "Petar Nachev Dachev":"Petar Dachev",
                "Peter Bol":"Peter Bol",
                "Peter O. Emelieze":"Peter Emelieze",
                "Peter Esenwein":"Peter Esenwein",
                "Peter Hork":"Peter Horák",
                "Peter Joseph Nowill":"Peter Nowill",
                "Peter Sack":"Peter Sack",
                "Peter Zupanc":"Peter Zupanc",
                "Petr Frydrych":"Petr Frydrych",
                "Petr Stehlk":"Petr Stehlík",
                "Petr Svoboda":"Petr Svoboda",
                "Petra Fontanive":"Petra Fontanive",
                "Petrina May Price":"Petrina Price",
                "Pezhman Ghalehnoei":"Pezhman Ghalehnoei",
                "Phara Anacharsis":"Phara Anacharsis",
                "Philaylack Sackpraseuth":"Philaylack Sackpraseuth",
                "Philip Milanov":"Philip Milanov",
                "Philipp Bandi":"Philipp Bandi",
                "Philippa Kate Roles":"Philippa Roles",
                "Philip Martin Dunn":"Phillip Tarus",
                "Phobay Kutu-Akoi":"Phobay Kutu-Akoi",
                "Phouk Sopheak":"Phouk Sopheak",
                "Phumlile Ndzinisa":"Phumlile Ndzinisa",
                "Pia Tajnikar":"Pia Tajnikar",
                "Pierre Browne":"Pierre Browne",
                "Pierre-Ambroise Bosse":"Pierre-Ambroise Bosse",
                "Pieter Hendricks de Villiers":"Pieter De Villiers",
                "Pieter Desmet":"Pieter Desmet",
                "Pieter-Jan Hannes":"Pieter-Jan Hannes",
                "Piotr Buciarski":"Piotr Buciarski",
                "Piotr Arkadiusz Klimczak":"Piotr Klimczak",
                "Piotr Lisek":"Piotr Lisek",
                "Piotr Maachowski":"Piotr Małachowski",
                "Piotr Grzegorz Rysiukiewicz":"Piotr Rysiukiewicz",
                "Poh Seng Song":"Poh Seng Song",
                "Pol Moya Betriu":"Pol Moya",
                "Polat Kemboi Arkan":"Polat Arıkan",
                "Polina Viktorovna Repina":"Polina Repina",
                "Povilas Mykolaitis":"Povilas Mykolaitis",
                "Poina Jeizarova":"Poļina Jeļizarova",
                "Pramila Rijal":"Pramila Rijal",
                "Predrag Filipovi":"Predrag Filipović",
                "Preeja Sreedharan":"Preeja Sreedharan",
                "Prenam Pesse":"Prenam Pesse",
                "Primo Kozmus":"Primož Kozmus",
                "Prince Moses Mumba":"Prince Moses Mumba",
                "Mara Princesa Oliveros Bohrquez":"Princesa Oliveros",
                "Krista DuChene":"Priscah Cherono",
                "Priscilla Frederick":"Priscilla Frederick",
                "Prodromos Katsantonis":"Prodromos Katsantonis",
                "Przemysaw Czajkowski":"Przemysław Czajkowski",
                "Przemysaw Jdrzej Czerwiski":"Przemysław Czerwiński",
                "Pyotr Aleksandrovich Brayko":"Pyotr Brayko",
                "Pnar Saka":"Pınar Saka",
                "Qin Qiang":"Qin Qiang",
                "Qin Wangping":"Qin Wangping",
                "Queen Quedith Earth Harrison":"Queen Harrison",
                "Quentin Bigot":"Quentin Bigot",
                "Quentin Rew":"Quentin Rew",
                "Raasin G. McIntosh":"Raasin Mcintosh",
                "Rababe Arafi":"Rababe Arafi",
                "Rabah Aboud":"Rabah Aboud",
                "Rabah Yousif":"Rabah Yousif",
                "Rabangaki Nawai":"Rabangaki Nawai",
                "Rabia Ashiq":"Rabia Ashiq",
                "Rabia Makhloufi":"Rabia Makhloufi",
                "Rabiou Guero Gao":"Rabiou Guero Gao",
                "Rachael Nachula":"Rachael Nachula",
                "Rachel Seaman (Lavalle-)":"Rachel Seaman",
                "Rachel Tallent":"Rachel Tallent",
                "Rachel Yurkovich":"Rachel Yurkovich",
                "Rachid Chouhal":"Rachid Chouhal",
                "Rachidatou Seini Maikido":"Rachidatou Seini Maikido",
                "Radek Juka":"Radek Juška",
                "Radoslava Yankova Mavrodieva":"Radoslava Mavrodieva",
                "Radosaw Popawski":"Radosław Popławski",
                "Rafa Augustyn":"Rafał Augustyn",
                "Rafa Omelko":"Rafał Omelko",
                "Rafith Rodrguez Lleneres":"Rafith Rodríguez",
                "Rahamatou Dram":"Rahamatou Dramé",
                "Raigo Toompuu":"Raigo Toompuu",
                "Raivydas Stanys":"Raivydas Stanys",
                "Rajendra Bahadur Bhandari":"Rajendra Bahadur Bhandari",
                "Rakia Al-Gassra":"Rakia Al-Gassra",
                "Ralf Bartels":"Ralf Bartels",
                "Ramil Kamilyevich Aritkulov":"Ramil Aritkulov",
                "Ramil Guliyev":"Ramil Guliyev",
                "Ramon Gittens":"Ramon Gittens",
                "Anna Ramona Papaioannou":"Ramona Papaioannou",
                "Raquel Gonzlez Campos":"Raquel González",
                "Rashid Shafi Al-Dosari":"Rashid Al-Dosari",
                "Rachid Ziar":"Rashid Ramzi",
                "Rasmus Mgi":"Rasmus Mägi",
                "Raven Saunders":"Raven Saunders",
                "Raymond Eugene Higgs":"Raymond Higgs",
                "Raymond Kibet":"Raymond Kibet",
                "Emiliano Lasa Snchez":"Raysa Sánchez",
                "Ral-Roland Spank":"Raúl Spank",
                "Rebekka Haase":"Rebekka Haase",
                "Recep elik":"Recep Çelik",
                "Michael Reese Hoffa":"Reese Hoffa",
                "Regan Lamble":"Regan Lamble",
                "Regina George Grause":"Regina George",
                "Reginaldo Micha Ndong":"Reginaldo Ndong",
                "Regine Tugade":"Regine Tugade",
                "Reinaldo Proenza Rivo":"Reinaldo Proenza",
                "Reinhardt van Rensburg":"Reinhardt Van Rensburg",
                "Ren Mengqian":"Ren Mengqian",
                "Renata Pli":"Renata Pliś",
                "Rene Herrera":"Rene Herrera",
                "Renny Quow":"Renny Quow",
                "Rens Blom":"Rens Blom",
                "Ren Herms":"René Herms",
                "Ren Kalmer":"René Kalmer",
                "Rene Eykens":"Renée Eykens",
                "Restituta Joseph Kemi":"Restituta Joseph",
                "Reyare Thomas":"Reyare Thomas",
                "Reyes Estvez Lpez":"Reyes Estévez",
                "Reynier Mea Berenguer":"Reynier Mena",
                "Reza Ghasemi":"Reza Ghasemi",
                "Rena-Flor Okori":"Reïna-Flor Okori",
                "Rhonda Watkins":"Rhonda Watkins",
                "Rhydian Cowley":"Rhydian Cowley",
                "Rhys Williams":"Rhys Williams",
                "Ricardo Chambers":"Ricardo Chambers",
                "Richard Phillips":"Richard Phillips",
                "Richard Ringer":"Richard Ringer",
                "Richard Thompson":"Richard Thompson",
                "Ricardo Ramos Vargas":"Richard Vargas",
                "Richson Simeon":"Richson Simeon",
                "Ricky Robertson":"Ricky Robertson",
                "Ricky Robertson":"Ricky Soos",
                "Rima Kashafutdinova":"Rima Kashafutdinova",
                "Rima Taha Farid":"Rima Taha",
                "Ristananna Tracey":"Ristananna Tracey",
                "Riste Pandev":"Riste Pandev",
                "Risto Mtas":"Risto Mätas",
                "Rita Ramanauskait":"Rita Ramanauskaitė",
                "Rita Rcsei":"Rita Récsei",
                "Dimitra Dova":"Rita Turova",
                "Coby Miller":"Rob Mullett",
                "Robert Gary":"Roba Gari",
                'Robert Adrian "Robby" Andrews':"Robby Andrews",
                "Robert Gary":"Robert Gary",
                "Robert Harting":"Robert Harting",
                "Robert Markus Hggblom":"Robert Häggblom",
                "Robert Kajuga":"Robert Kajuga",
                "Leif Robert Kronberg":"Robert Kronberg",
                "Robert Lathouwers":"Robert Lathouwers",
                "Robert Renner":"Robert Renner",
                "Robert Sobera":"Robert Sobera",
                "Robert Urbanek":"Robert Urbanek",
                "Robert Wolski":"Robert Wolski",
                "Roberto Caracciolo Mandje (-Mandje)":"Roberto Caracciolo Mandje",
                "Robert Gary":"Roberto García",
                "Roberto Janet Durruty":"Roberto Janet",
                "Roberto Sawyers Furtado":"Roberto Sawyers",
                "Roberto Skyers Prez":"Roberto Skyers",
                "Robin Erewa":"Robin Erewa",
                "Robina Muqim Yaar":"Robina Muqim Yaar",
                "Rocco van Rooyen":"Rocco Van Rooyen",
                "Roco Florido Florido":"Rocío Florido",
                "Roderick Genki Dean":"Roderick Genki Dean",
                "Rodman Teltull":"Rodman Teltull",
                "Rohan Handunpurage Pradeep Kumara":"Rohan Pradeep Kumara",
                "Roisin McGettigan-Dumas":"Roisin Mcgettigan",
                "Roland Nmeth":"Roland Németh",
                "Roland Robert Varga":"Roland Varga",
                "Rolando Cruz Palacios Castillo":"Rolando Palacios",
                "Jaime Rolando Saquipay Pai":"Rolando Saquipay",
                "Rolands trobinders":"Rolands Štrobinders",
                "Romain Mesnil":"Romain Mesnil",
                "Roman Valeriyovych Avramenko":"Roman Avramenko",
                "Roman William Cress":"Roman Cress",
                "Roman Fricke":"Roman Fricke",
                "Roman Novotn":"Roman Novotný",
                "Roman Rozna":"Roman Rozna",
                "Roman Petrovich Smirnov":"Roman Smirnov",
                "Roman Virastiuk":"Roman Virastiuk",
                "Romana Dubnov (Block-)":"Romana Dubnová",
                "Romana Malov":"Romana Maláčová",
                "Romina Natalia Maggi":"Romina Maggi",
                "Romrio Martins Leito":"Romário Leitão",
                "Ronald Pognon":"Ronald Bennett",
                "Ronald Joseph Forbes":"Ronald Forbes",
                "Ronald Musagala":"Ronald Julião",
                "Ronald Chebolei Kwemoi":"Ronald Kwemoi",
                "Ronald Musagala":"Ronald Musagala",
                "Ronald Pognon":"Ronald Pognon",
                "Ronalds Arjs":"Ronalds Arājs",
                "Rondell Kelvin Sorillo":"Rondel Sorrillo",
                "Ronnie Ash":"Ronnie Ash",
                "Ronny Nilsen":"Ronny Nilsen",
                "Rosa Mara Morat Rodrguez":"Rosa María Morató",
                "Lalanirina Rosa Rakotozafy":"Rosa Rakotozafy",
                "Rosa Mara Morat Rodrguez":"Rosa Rodríguez",
                "Rose Mary Almanza Blanco":"Rose Mary Almanza",
                "Rose Nathike Lokonyen":"Rose Nathike Lokonyen",
                "Rose Marie Richmond":"Rose Richmond",
                "Rosefelo Siosi":"Rosefelo Siosi",
                "Rosemar Maria Coelho Neto":"Rosemar Coelho",
                "Rosibel Garca Mena":"Rosibel García",
                "Roslinda Samsu":"Roslinda Samsu",
                "Ross Arthur Millington":"Ross Millington",
                "Ross Anthony Murray":"Ross Murray",
                "Rossella Giordano":"Rossella Giordano",
                "Rosevitha Okou":"Rosvitha Okou",
                "Rossella Giordano":"Rosângela Santos",
                "Rouhollah Askari Gandomani":"Rouhollah Askari",
                "Romana Malov":"Roxana Díaz",
                "Roxana Elisabeta Brc":"Roxana Elisabeta Bîrcă",
                "Roxroy Cato":"Roxroy Cato",
                "Role Prezelj":"Rožle Prezelj",
                "Ruben Ramolefi":"Ruben Ramolefi",
                "Rubie Joy Gabriel":"Rubie Joy Gabriel",
                "Rubn Palomeque Barrera":"Rubén Palomeque",
                "Perennes Paulette Ruddy Zang-Milama":"Ruddy Zang-Milama",
                "Rudolf Gtz":"Rudolf Götz",
                "Rudy Winkler":"Rudy Winkler",
                "Rui Pedro S Alves da Silva":"Rui Pedro Da Silva",
                'Rukayatu "Ruky" Abdulai':"Ruky Abdulai",
                "Rumyana Dimitrova Karapetrova":"Rumyana Karapetrova",
                "Ruslan Abbasov":"Ruslan Abbasov",
                "Ruslan Hryhorovych Dmytrenko":"Ruslan Dmytrenko",
                "Ruslan Mikhaylovich Mashchenko":"Ruslan Mashchenko",
                "Ruslan Volodymyrovych Yeremenko":"Ruslan Yeremenko",
                "Ruslana Petrivna Tsykhotska":"Ruslana Tsykhotska",
                "Russel Ward Roman":"Russel Roman",
                "Rustam Barotovich Kuvatov":"Rustam Kuvatov",
                "Ruswahl Samaai":"Ruswahl Samaai",
                "Rutger Smith":"Rutger Smith",
                "Ruth Bisibori Nyangau":"Ruth Bosibori",
                "Ruth Sophia Spelmeyer":"Ruth Spelmeyer",
                "Ruwida El-Hubti":"Ruwida El-Hubti",
                "Ryan Bailey":"Ryan Bailey",
                "Ryan Brathwaite":"Ryan Brathwaite",
                "Ryan Crouser":"Ryan Crouser",
                "Ryan Gregson":"Ryan Gregson",
                "Ryan Keith Whiting":"Ryan Whiting",
                "Ryo Matsuda":"Ryo Matsuda",
                "Ryohei Arai":"Ryohei Arai",
                "Ryota Yamagata":"Ryota Yamagata",
                "Rytis Sakalauskas":"Rytis Sakalauskas",
                "Ryuji Ono":"Ryuji Ono",
                "Rnelle Lamote":"Rénelle Lamote",
                "Rbert Fazekas":"Róbert Fazekas",
                "Rbert tefko":"Róbert Štefko",
                "Sabina Asenjo lvarez":"Sabina Asenjo",
                "Sabina Veit":"Sabina Veit",
                "Sabine Zimmer-Krantz":"Sabine Zimmer-Krantz",
                "Sabrina Mockenhaupt":"Sabrina Mockenhaupt",
                "Sachiko Konishi":"Sachiko Konishi",
                "Sadaf Siddiqui":"Sadaf Siddiqui",
                "Saeed Ali Al-Adhreai":"Saeed Ali Al-Adhreai",
                "Sage Watson":"Sage Watson",
                "Sahana Kumari":"Sahana Kumari",
                "Sahily Diago Mesa":"Sahily Diago",
                "Saily Viart Despaigne":"Saily Viart",
                "Sajjad Hashemi Ahangari":"Sajjad Hashemi",
                "Sajjad Moradi":"Sajjad Moradi",
                "Salamtou Hassane":"Salamtou Hassane",
                "Yaqoob Salem Eid Yaqoob":"Salem Eid Yaqoob",
                "Salem Mubarak M. Al-Yami":"Salem Mubarak Al-Yami",
                "Salhate Djamalidine":"Salhate Djamalidine",
                "Salim Keddar":"Salim Keddar",
                "Salim Sdiri":"Salim Sdiri",
                "Salima El Ouali Al-Alami":"Salima El Ouali Alami",
                'Selina "Sally" Barsosio':"Sally Barsosio",
                "Salome Nyirarukundo":"Salome Nyirarukundo",
                "Ryan Crouser":"Sam Crouser",
                "Sam McEntee":"Sam Mcentee",
                "Samia Yusuf Omar":"Samia Omar",
                "Samir Moussaoui":"Samir Moussaoui",
                "Samson Kiflemariam":"Samson Kiflemariam",
                "Mabu Joseph Samson Ngoepe":"Samson Ngoepe",
                "Samuel Coco-Viloin":"Samuel Coco-Viloin",
                "Samuel Adelebari Francis":"Samuel Francis",
                "Samuel Ireri Gathimba":"Samuel Gathimba",
                "Samwel Mwera Chegere":"Samwel Mwera",
                "Samwel Kwaangu Shauri":"Samwel Shauri",
                "San Naing":"San Naing",
                "Sanaa Koubaa":"Sanaa Koubaa",
                "Sara Aerts":"Sandra Arenas",
                "Sandra Elisabeth Eriksson":"Sandra Eriksson",
                "Sandra Gomis":"Sandra Gomis",
                "Sandra Lpez Reyes":"Sandra López",
                "Sandra Perkovi":"Sandra Perković",
                "Sandrine Thibaud-Kangni":"Sandrine Thiébaud-Kangni",
                "Sanja Gavrilovi":"Sanja Gavrilović",
                "Sanna Abubkheet":"Sanna Bkheet",
                "Sanni Marja Anniina Utriainen":"Sanni Utriainen",
                "Santino Kenyi":"Santino Kenyi",
                "Sapna Punia":"Sapna Punia",
                "Sara Gambetta":"Sara Gambetta",
                "Sara Kolak":"Sara Kolak",
                "Sara Kolak":"Sara Moreira",
                "Sara Louise Treacy":"Sara Treacy",
                "Sarah Amer Attar":"Sarah Attar",
                "Sarah Louise Claxton":"Sarah Claxton",
                "Farah Jacques":"Sarah Jamieson",
                "Sarah Lahti":"Sarah Lahti",
                "Sarah-Lynn Wells":"Sarah Wells",
                "Saraswati Bhattarai":"Saraswati Bhattarai",
                "Saraswati Amit Dey-Saha":"Saraswati Dey-Saha",
                "Saruba Colley":"Saruba Colley",
                "Sasha Springer-Jones":"Sasha Springer-Jones",
                "Satomi Kubokura":"Satomi Kubokura",
                "Satoru Tanigawa":"Satoru Tanigawa",
                "Saud Al-Zaabi":"Saud Al-Zaabi",
                'Voisava "Savva" Lika':"Savva Lika",
                "Savvas Panavoglou":"Savvas Panavoglou",
                "Scott Martin":"Scott Martin",
                "Scott Russell":"Scott Russell",
                "Sean Michael Baldock":"Sean Baldock",
                "Sean Nana A. K. Safo-Antwi":"Sean Safo-Antwi",
                "Sean Wroe":"Sean Wroe",
                "Sebastian Bayer":"Sebastian Bayer",
                "Sebastian Ernst":"Sebastian Ernst",
                "Sebastian Bayer":"Sebastian Rodger",
                "Sebastin Martos Roa":"Sebastián Martos",
                "Seema Antil":"Seema Antil",
                "Seilala Mang Sua":"Seilala Sua",
                "Seito Yamamoto":"Seito Yamamoto",
                "Sekou Kaba":"Sekou Kaba",
                "Selahattin obanolu":"Selahattin Çobanoğlu",
                "Selim Bayrak":"Selim Bayrak",
                "Selim Nurudeen":"Selim Nurudeen",
                "Selina Christina Bchel":"Selina Büchel",
                "Selma Kajan":"Selma Kajan",
                "Seltana At Hammou":"Seltana Aït Hammou",
                "Semiha Mutlu":"Semiha Mutlu",
                "Semoy Kee-Ann Hackett":"Semoy Hackett",
                "Senbere Teferi":"Senbere Teferi",
                "Sentayehu Ejigu Tamerat":"Sentayehu Ejigu",
                "Seren Bundy-Davies":"Seren Bundy-Davies",
                "Sergey Chernov":"Sergey Chernov",
                "Sergey Viktorovich Kolomoyets":"Sergey Kolomoyets",
                "Sergey Mikhaylovich Kucheryanu":"Sergey Kucheryanu",
                "Sergey Pakura":"Sergey Pakura",
                "Sergey Sergeyevich Zasimovich":"Sergey Zasimovich",
                "Serghei Marghiev":"Serghei Marghiev",
                "Sergio Fernndez Roda":"Sergio Fernández",
                "Serhiy Oleksandrovych Demidiuk":"Serhiy Demidiuk",
                "Serhiy Petrovych Lebid":"Serhiy Lebid",
                "Serhiy Volodimirovich Smelyk":"Serhiy Smelyk",
                "Seth Kwame Amoo":"Seth Amoo",
                "Moriam Seun Adigun":"Seun Adigun",
                "Shadae Lawrence":"Shadae Lawrence",
                'Folashade "Shade" Abugan':"Shade Abugan",
                "Shadrack Kipchirchir":"Shadrack Kipchirchir",
                "Shaka Sola":"Shaka Sola",
                "Shalaya Kipp":"Shalaya Kipp",
                "Shamar Sands":"Shamar Sands",
                "Shana Amanda Cox":"Shana Cox",
                "Shanahan Sakofalefatu Sanitoa":"Shanahan Sanitoa",
                "Shane Brathwaite":"Shane Brathwaite",
                "Shani Marks":"Shani Marks",
                "Shanice Craft":"Shanice Craft",
                "Anilda Thomas":"Shanieka Thomas",
                "Shannon Nicole Rowbury":"Shannon Rowbury",
                "Shara Nerissa Proctor":"Shara Proctor",
                "Susan Mary Scott":"Sharolyn Scott",
                "Sharon Day (-Monroe)":"Sharon Day",
                "Sharon Firisua":"Sharon Firisua",
                "Shaun Patrick Bownes":"Shaun Bownes",
                "Shavez Hart":"Shavez Hart",
                "Alana Barber":"Shawn Barber",
                "Shawn Crawford":"Shawn Crawford",
                "Alan Lawrence Culpepper":"Shayne Culpepper",
                "Sheila Reid":"Sheila Reid",
                "Shelayna Elicia  Oskan-Clarke (Gumbs-)":"Shelayna Oskan-Clarke",
                "Shelbi Vaughan":"Shelbi Vaughan",
                "Shelby Houlihan":"Shelby Houlihan",
                "Shelley Jean Newman (Drew-)":"Shelley Newman",
                "Shelly-Ann Fraser-Pryce":"Shelly-Ann Fraser-Pryce",
                "Sheniqua Monique Ferguson":"Sheniqua Ferguson",
                "Shermaine Williams":"Shermaine Williams",
                "Sherone Anmarica Simpson":"Sherone Simpson",
                "Sherridan Kirk":"Sherridan Kirk",
                "Sherry Ingrid Veronica Fletcher":"Sherry Fletcher",
                "Shevon Kaiwana Stoddart":"Shevon Stoddart",
                "Shi Dongpeng":"Shi Dongpeng",
                "Shi Na":"Shi Na",
                "Shifana Ali":"Shifana Ali",
                "Shingo Suetsugu":"Shingo Suetsugu",
                "Shinichi Terano":"Shinichi Terano",
                "Shinji Takahira":"Shinji Takahira",
                "Shinoona Salah Al-Habsi":"Shinoona Al-Habsi",
                "Shirin Akter":"Shirin Akter",
                "Shirley Catherine Webb":"Shirley Webb",
                "Shitaye Eshete Habtegebrel":"Shitaye Eshete",
                "Sho Kawamoto":"Sho Kawamoto",
                "Sholpan Bulatovna Kozhakhmetova":"Sholpan Kozhakhmetova",
                "Shota Iizuka":"Shota Iizuka",
                "Sibusiso Bruno Matsenjwa":"Sibusiso Matsenjwa",
                "Sifan Hassan":"Sifan Hassan",
                "Siham Al-Hilali":"Siham Hilali",
                "Silas Kiplagat":"Silas Kiplagat",
                "Silke Spiegelburg":"Silke Spiegelburg",
                "Silvano Chesani":"Silvano Chesani",
                "Silvia Felipo Sue":"Silvia Felipo",
                "Silvia Eduardo Panguana":"Silvia Panguana",
                "Silvia Salis":"Silvia Salis",
                "Silvia Weissteiner":"Silvia Weissteiner",
                "Silvio Ramn Guerra Burbano":"Silvio Guerra",
                "Silvio Schirrmeister":"Silvio Schirrmeister",
                "Silviya Georgieva Danekova":"Silviya Danekova",
                "Simeon Oscar C. Williamson":"Simeon Williamson",
                "Simon Frans Vroemen":"Simon Vroemen",
                "Simon Wachira":"Simon Wachira",
                "Simona La Mantia":"Simona La Mantia",
                "Simone Collio":"Simone Collio",
                "Kemoy Campbell":"Simoya Campbell",
                "Simret Sultan Ghebermichael":"Simret Sultan",
                "Sin Ir-Yong":"Sin Ir-Yong",
                "Sina Schielke":"Sina Schielke",
                "Sinia Ergoti":"Siniša Ergotić",
                "Sinta Ozolia-Kovala (-Sprudzane)":"Sinta Ozoliņa-Kovala",
                "Siraj Williams":"Siraj Williams",
                "Sisay Bezabeh":"Sisay Bezabeh",
                "Sisilia Anisa Levu Seavula":"Sisilia Seavula",
                "Sitora Khamidova":"Sitora Khamidova",
                "Siueni Filimone":"Siueni Filimone",
                "Mary Sloan Siegrist":"Sloan Siegrist",
                'Ismal "Smail" Sghyr':"Smail Sghyr",
                "Snezhana Vladimirovna Yurchenko":"Snezhana Yurchenko",
                "Soe Min Thu":"Soe Min Thu",
                "Sofia Ennaoui":"Sofia Ennaoui",
                "Sofie Natalie Skoog":"Sofie Skoog",
                "Sofine Labidi":"Sofiène Laâbidi",
                "Solange Maril Witteveen":"Solange Witteveen",
                "Solomon Bockarie-Bayoh":"Solomon Bockarie-Bayoh",
                "Sonata Miluauskait (Buksnien-)":"Sonata Milušauskaitė",
                "Sonata Tamoaityt":"Sonata Tamošaitytė",
                "Song Aimin":"Song Aimin",
                "Song Dan":"Song Dan",
                "Song Hongjuan":"Song Hongjuan",
                "Sonia Bisset Poll":"Sonia Bisset",
                "Sonia Malavisi":"Sonia Malavisi",
                "Siraj Williams":"Sonia Williams",
                "Sonja Roman":"Sonja Roman",
                "Sophie Duarte":"Sophie Duarte",
                "Sophie Hitchon":"Sophie Hitchon",
                "Soslan Kerimbekovich Tsirikhov":"Soslan Tsirikhov",
                "Sosthene Taroum Moguenara":"Sosthene Moguenara",
                "Sou Tit Linda":"Sou Tit Linda",
                "Souad At Salem (-Mahour-Bacha)":"Souad Aït Salem",
                "Soufiane El-Bakkali":"Soufiane El-Bakkali",
                "Soufiyan Bouqantar":"Soufiyan Bouqantar",
                "Souhalia Alamou":"Souhalia Alamou",
                "Souksavanh Tonsacktheva":"Souksavanh Tonsacktheva",
                "Souleymane Ould Chebal Moctar":"Souleymane Ould Chebal",
                "Sparkle McKnight":"Sparkle Mcknight",
                "Spas Bukhalov":"Spas Bukhalov",
                'Spiridon "Spyros" Lempesis':"Spyros Lempesis",
                "Srabani Nanda":"Srabani Nanda",
                "Bo Staffan Strand":"Staffan Strand",
                "Stanislav Anatoliyovych Melnykov":"Stanislav Melnykov",
                "Stanislav Sajdok":"Stanislav Sajdok",
                "Stanislav Vladimirovich Tivonchik":"Stanislav Tivonchik",
                "Stanley Gbagbeke":"Stanley Gbagbeke",
                "Stanley Joseph":"Stanley Joseph",
                "Stanly Jos del Carmen Cruz":"Stanly Del Carmen",
                "Staislavs Olijars":"Staņislavs Olijars",
                "Stefan Brits":"Stefan Brits",
                "Stefany Mildred Coronado Gemio":"Stefany Coronado",
                "Steffen Uliczka":"Steffen Uliczka",
                'Stefanie "Steffi" Nerius':"Steffi Nerius",
                'Stylianos "Stelios" Dimotsios':"Stelios Dimotsios",
                "Stella Chesang":"Stella Chesang",
                'Styliani "Stella" Papadopoulou':"Stella Papadopoulou",
                'Styliani "Stella" Pilatou':"Stella Pilatou",
                'Styliani "Stella" Tsikouna':"Stella Tsikouna",
                "Stella-Iro Ledaki":"Stella-Iro Ledaki",
                "Stephan Louw":"Stephan Louw",
                "Stephan Steding":"Stephan Steding",
                "Stephanie Karenmonica Brown-Trafton":"Stephanie Brown-Trafton",
                "Stephanie Lynn McCann":"Stephanie Mccann",
                "Stephanie Reilly (O'Reilly-)":"Stephanie Reilly",
                "Stephanie April Twell":"Stephanie Twell",
                "Stephen McNelly Jones":"Stephen Jones",
                "Stephen Mokoka":"Stephen Mokoka",
                "Stephen Mokoka":"Stephen Mozia",
                "Stephan Steding":"Stephen Sáenz",
                'Stephen James "Steve" Backley':"Steve Backley",
                "Denise Lewis":"Steve Lewis",
                "Steven Solomon":"Steven Solomon",
                "Stipe uni":"Stipe Žunić",
                "Stuart James Farquhar":"Stuart Farquhar",
                "Stuart Rendell":"Stuart Rendell",
                "Stuart Bradley Stokes":"Stuart Stokes",
                "Stphane Buckland":"Stéphane Buckland",
                "Stphanie Falzon":"Stéphanie Falzon",
                "Su Bingtian":"Su Bingtian",
                "Su Wai'bou Sanneh":"Su Wai'Bou Sanneh",
                "Su Xinyue":"Su Xinyue",
                "Su Yiping":"Su Yiping",
                "Subenrat Insaeng":"Subenrat Insaeng",
                "Sudha Singh":"Sudha Singh",
                "Sudirman Hadi":"Sudirman Hadi",
                "Suguru Osako":"Suguru Osako",
                "Sukhrob Khodzhayev":"Sukhrob Khodzhayev",
                "Sultan Abdulmajeed E. Al-Habshi":"Sultan Abdulmajeed Al-Habshi",
                "Sultan Khamis Zaman":"Sultan Khamis Zaman",
                "Sultan Mubarak M. Al-Dawoodi":"Sultan Mubarak Al-Dawoodi",
                "Sultan Saeed":"Sultan Saeed",
                "Sultan Tucker":"Sultan Tucker",
                "Sultana Frizell":"Sultana Frizell",
                "Sumaira Zahoor":"Sumaira Zahoor",
                "R. M. Sumeda Ranasinghe":"Sumeda Ranasinghe",
                "Sun Yawei":"Sun Yawei",
                "Sun Yingjie":"Sun Yingjie",
                "Sunayna Wahi":"Sunayna Wahi",
                "Sunette Stella Viljoen":"Sunette Viljoen",
                "Supanara Suksawat, Na Ayutthaya":"Supanara Suksawat",
                "Supriyati Sutono":"Supriyati Sutono",
                "Surendra Kumar Singh":"Surendra Singh",
                "Surita Febbraio (Loots-)":"Surita Febbraio",
                "Suryo Agung Wibowo":"Suryo Wibowo",
                "Susan Kuijken (-Krumins)":"Susan Kuijken",
                "Susan Mary Scott":"Susan Scott",
                "Susan Mary Scott":"Susana Costa",
                "Sabina Veit":"Susana Feitor",
                "Susanna Elisabeth Kallur":"Susanna Kallur",
                "Susanne Keil":"Susanne Keil",
                "Kameradin Susanthika Jayasinghe":"Susanthika Jayasinghe",
                'Suzanne N. "Suzy" Powell-Roos':"Suzy Powell-Roos",
                "Svatoslav Ton":"Svatoslav Ton",
                "Sven Martin Skagestad":"Sven Martin Skagestad",
                "Rumen Dimitrov":"Svetla Dimitrova",
                "Svetlana Aleksandrovna Bodritskaya":"Svetlana Bodritskaya",
                "Svetlana Bolshakova":"Svetlana Bolshakova",
                "Svetlana Vasilyevna Klyuka":"Svetlana Klyuka",
                "Svetlana Vladimirovna Krivelyova":"Svetlana Krivelyova",
                "Svetlana Mikhaylovna Kudelich":"Svetlana Kudelich",
                "Svetlana Pessova":"Svetlana Pessova",
                "Svetlana Mikhaylovna Radzivil":"Svetlana Radzivil",
                "Svetlana Fyodorovna Sudak-Torun":"Svetlana Sudak-Torun",
                "Svitlana Mykhailivna Mamieieva":"Svitlana Mamieieva",
                "Svitlana Volodymyrivna Shmidt":"Svitlana Shmidt",
                "Sydney McLaughlin":"Sydney Mclaughlin",
                "Sylwester Bednarek":"Sylwester Bednarek",
                "Sylwia Ejdys (-Tomaszewska)":"Sylwia Ejdys",
                "Sylwia Korzeniowska":"Sylwia Korzeniowska",
                "Szymon Zikowski":"Szymon Ziółkowski",
                "Sbastien Gattuso":"Sébastien Gattuso",
                "Srgio Vieira Galdino":"Sérgio Galdino",
                "Slvia Isabel Cruz":"Sílvia Cruz",
                "Sren Ludolph":"Sören Ludolph",
                "T'erea Brown":"T'Erea Brown",
                "Taavi Peetre":"Taavi Peetre",
                "Tabarie Joil Henry":"Tabarie Henry",
                "Tabia Hasina Charles":"Tabia Charles",
                "Tafese Seboka":"Tafese Seboka",
                "Tahesia Gaynell Harrigan-Scott":"Tahesia Harrigan-Scott",
                "Tahmina Kohistani":"Tahmina Kohistani",
                "Taina Halasima":"Taina Halasima",
                "Taina Maria Uppa-Kolkkala":"Taina Uppa-Kolkkala",
                "Takashi Eto":"Takashi Eto",
                "Takayo Kondo":"Takayo Kondo",
                "Takayuki Kishimoto":"Takayuki Kishimoto",
                "Takayuki Matsumiya":"Takayuki Matsumiya",
                "Takayuki Tanii":"Takayuki Tanii",
                "Arata Fujiwara":"Takeshi Fujiwara",
                "Takumi Saito":"Takumi Saito",
                "Tamara Salaki":"Tamara Salaški",
                "Tameka Williams":"Tameka Williams",
                "Jared Lewis":"Tamsyn Lewis",
                "Tams Kazi":"Tamás Kazi",
                "Tams Margl":"Tamás Margl",
                "Tan Jian":"Tan Jian",
                "Tanel Laanme":"Tanel Laanmäe",
                "Ibrahim Tanko Braimah":"Tanko Braimah",
                "Tanya-Gee Blake":"Tanya Blake",
                "Tanya Holliday":"Tanya Holliday",
                "Tanya Vladimirova Koleva-Stefanova":"Tanya Koleva-Stefanova",
                "Tarasue Barnett":"Tarasue Barnett",
                "Tarek Boukensa":"Tarek Boukensa",
                "Tareq Mubarak Taher":"Tareq Mubarak Taher",
                "Tarik Bouguetab":"Tarik Bouguetaïb",
                "Tariq Ahmed Al-Amri":"Tariq Ahmed Al-Amri",
                "Tarmo Jallai":"Tarmo Jallai",
                "Taryn Suttie":"Taryn Suttie",
                "Tark Langat Akda":"Tarık Akdağ",
                'Anastasios "Tasos" Gousis':"Tasos Gousis",
                "Tatenda Tsumba":"Tatenda Tsumba",
                "Tatiele Roberta de Carvalho":"Tatiele De Carvalho",
                "Tatjana Jelaa (-Mirkovi )":"Tatjana Jelača",
                "Tatjana Lofamakanda Pinto":"Tatjana Pinto",
                "Tatyana Valeryevna Andrianova":"Tatyana Andrianova",
                "Tatyana Borisova":"Tatyana Borisova",
                "Tatyana Valeryevna Dektyareva":"Tatyana Dektyareva",
                "Tatyana Vasilyevna Kalmykova":"Tatyana Kalmykova",
                "Tatyana Vladimirovna Kholodovich":"Tatyana Kholodovich",
                "Tatyana Valeryevna Sibileva":"Tatyana Sibileva",
                "Tatyana Yefimenko":"Tatyana Yefimenko",
                "Tavevele Noa":"Tavevele Noa",
                "Tavis Bailey":"Tavis Bailey",
                "Taylor Milne":"Taylor Milne",
                "Tayna Lawrence":"Tayna Lawrence",
                "Teemu Sakari Wirkkala":"Teemu Wirkkala",
                "Tega Peter Odele":"Tega Odele",
                "Teja Melink":"Teja Melink",
                "Tejitu Daba Chalchissa":"Tejitu Daba",
                "Teklemariam Medhin Weldeslassie":"Teklemariam Medhin",
                "Teklit Teweldebrhan":"Teklit Teweldebrhan",
                "Temalangeni Mbali Dlamini":"Temalangeni Dlamini",
                "Teodoro Vega Morales":"Teodoro Vega",
                "Tepa Aki Ilari Reinikainen":"Tepa Reinikainen",
                "Teray Smith":"Teray Smith",
                "Tereapii Tapoki":"Tereapii Tapoki",
                "Teresa Cristina Lopes Machado":"Teresa Machado",
                "Teresa Nzola Meso Ba":"Teresa Nzola Meso",
                "Teresa Vaill":"Teresa Vaill",
                "Teresita Natvidad Collado Ramos":"Teresita Collado",
                "Tereza apkov":"Tereza Čapková",
                "Tero Mikael Jrvenp":"Tero Järvenpää",
                "Tero Kristian Pitkmki":"Tero Pitkämäki",
                "Tessa Johanna van Schagen":"Tessa Van Schagen",
                "Tetiana Mykolavna Liakhovych":"Tetiana Liakhovych",
                "Tetiana Hryhorivna Petliuk":"Tetiana Petliuk",
                "Tetiana Shchurenko (Nikolaeva-)":"Tetiana Shchurenko",
                "Tetiana Tkalich":"Tetiana Tkalich",
                "Tetsuya Tateno":"Tetsuya Tateno",
                "Tewodros Shiferaw":"Tewodros Shiferaw",
                "Teymur Qasmov":"Teymur Qasımov",
                "Tezdzhan Naimova":"Tezdzhan Naimova",
                "Thamer Kamal Ali":"Thamer Kamal Ali",
                "Thea LaFond":"Thea Lafond",
                "Theo Piniau":"Theo Piniau",
                "Thet Zaw Win":"Thet Zaw Win",
                "Thiago do Rosario Andr":"Thiago André",
                "Thomas Ayeko":"Thomas Ayeko",
                "Thomas Barr":"Thomas Barr",
                "Thomas Chamney":"Thomas Chamney",
                "Thomas Rhler":"Thomas Röhler",
                "Thuso Mpuang":"Thuso Mpuang",
                "Lance Brooks":"Tia Brooks",
                "Tia-Adana Belle":"Tia-Adana Belle",
                "Tiandra Tamika Ponteen":"Tiandra Ponteen",
                "Tianna Madison-Bartoletta":"Tianna Madison-Bartoletta",
                "Tiffany Adaez Porter (Ofili-)":"Tiffany Porter",
                "Tiffany Marthenia Ross-Williams":"Tiffany Ross-Williams",
                "Tigest Getent Mekonen":"Tigest Getent",
                "Tigist Assefa":"Tigist Assefa",
                "Tigest Gashaw Belay":"Tigist Gashaw",
                "Tiidrek Nurme":"Tiidrek Nurme",
                "Tihomir Ivaylo Ivanov":"Tikhomir Ivanov",
                "Tilak Ram Tharu":"Tilak Ram Tharu",
                "Till Helmke":"Till Helmke",
                'Timothy B. "Tim" Berrett':"Tim Berrett",
                "Tim Broe":"Tim Broe",
                "Tim Lobinger":"Tim Lobinger",
                'Timothy "Tim" Nedow':"Tim Nedow",
                "Tina arman":"Tim Seaman",
                "Timi Hilly Garstang":"Timi Garstang",
                "Timicka Clarke":"Timicka Clarke",
                'Jin Wei "Timothee" Yap':"Timothee Yap",
                "Timothy Kiptanui Too":"Timothy Kiptanui",
                "Timothy Toroitich":"Timothy Toroitich",
                "Tina arman":"Tina Čarman",
                "Tina utej":"Tina Šutej",
                "Tino Hber":"Tino Häber",
                "Tintu Luka":"Tintu Luka",
                "Tiombe Hurd":"Tiombé Hurd",
                "Tisha Felice Waller":"Tisha Waller",
                "Tjipekapora Herunga":"Tjipekapora Herunga",
                "Tlotliso Gift Leotlela":"Tlotliso Leotlela",
                "Tobias Dahm":"Tobias Dahm",
                "Tobias Scherbarth":"Tobias Scherbarth",
                "Tobias Benjamin Unger":"Tobias Unger",
                "Todd Matthews Jouda":"Todd Matthews Jouda",
                "Toea Wisil":"Toea Wisil",
                "Tom Compernolle":"Tom Compernolle",
                "Thomas Barr":"Tom Farrell",
                'Thomas Benjamin "Tom" Lancashire':"Tom Lancashire",
                'Tomas "Tom" Walsh':"Tom Walsh",
                "Tomasz Majewski":"Tomasz Majewski",
                "Tomasz ukasz Szymkowiak":"Tomasz Szymkowiak",
                "Tomomi Abiko":"Tommi Evilä",
                "Tomomi Abiko":"Tomomi Abiko",
                "Tom Jank":"Tomáš Janků",
                "Tom Stank":"Tomáš Staněk",
                "Tonje Angelsen":"Tonje Angelsen",
                'Anthony "Tony" Wamulwa':"Tony Wamulwa",
                "Tora L. Harris":"Tora Harris",
                "Tim Broe":"Tori Bowie",
                "Darvin Edwards":"Torri Edwards",
                "Torsten Schmidt":"Torsten Schmidt",
                'Olutoyin "Toyin" Augustus':"Toyin Augustus",
                "Traves Jamie Smikle":"Traves Smikle",
                "Trayvon Jaquez Bromell":"Trayvon Bromell",
                'Harold "Tre" Houston':"Tre Houston",
                "Trecia-Kaye Smith":"Trecia Smith",
                "Tora L. Harris":"Tremaine Harris",
                "Trevell Marcell Quinley":"Trevell Quinley",
                "Trevor Barron":"Trevor Barron",
                "Trevor Barron":"Trevor Barry",
                "Trevorvano Mackey":"Trevorvano Mackey",
                "Tricia Flores":"Tricia Flores",
                "Tricia Roberts":"Tricia Roberts",
                "Trihas Gebre Aunoon":"Trihas Gebre",
                "Trine Pilskog":"Trine Pilskog",
                "Trish Bartholomew":"Trish Bartholomew",
                "Tristan Thomas":"Tristan Thomas",
                "Troy Douglas":"Troy Douglas",
                "Tsanko Rosenov Arnaudov":"Tsanko Arnaudov",
                "Tsepang Gladys Sello":"Tsepang Sello",
                "Tsholofelo Thipe (Selemela-)":"Tsholofelo Thipe",
                "Tsvetanka Mincheva Khristova":"Tsvetanka Khristova",
                "Tsvetelina Petrova Kirilova":"Tsvetelina Kirilova",
                "Tuba Gven":"Tuğba Güvenç",
                "Dudu Karakaya":"Tuğba Karakaya",
                "Tue ahutolu":"Tuğçe Şahutoğlu",
                "Tyler Christopher":"Tyler Christopher",
                "Tynia Gaither":"Tynia Gaither",
                "Tyrone Damien Edgar":"Tyrone Edgar",
                "Loree Smith":"Tyrone Smith",
                "Tyson Lashane Gay":"Tyson Gay",
                "Zuzana Schindlerov":"Tânia Spindler",
                "Tnde Vaszi":"Tünde Vaszi",
                "Trkan Erimi (-zata)":"Türkan Erişmiş",
                "Uchenna Emedolu":"Uchenna Emedolu",
                "Ulrike Urbansky (-Leyckes)":"Ulrike Urbansky",
                "Urszula Piwnicka (Jasiska-)":"Urszula Piwnicka",
                "Usain St. Leo Bolt":"Usain Bolt",
                "Vadim Anatolyevich Devyatovsky":"Vadim Devyatovsky",
                "Vadim Hranovschi":"Vadim Hranovschi",
                "Vadim Sergeyevich Strogalyev":"Vadim Strogalyev",
                "Vadims Vasievskis":"Vadims Vasiļevskis",
                "Vadym Rostyslavovych Slobodeniuk":"Vadym Slobodeniuk",
                "Valentina Kibalnikova":"Valentina Kibalnikova",
                "Valentina Nazarova":"Valentina Nazarova",
                "Valentyna Oleksandrivna Horpynych-Zhudina":"Valentyna Horpynych-Zhudina",
                "Valentyna Oleksandrivna Myronchuk":"Valentyna Myronchuk",
                "Valentna Gotovska (Tolstika-)":"Valentīna Gotovska",
                "Valerie Kasanita Adams-Vili (-Price)":"Valerie Adams-Vili",
                "Valeriya Valeryevna Zabruskova":"Valeriya Zabruskova",
                "Valery Pisarev":"Valery Borisov",
                "Valery Vitalyevich Svyatokho":"Valery Svyatokho",
                'Valentina Ivanovna "Valya" Tsybulskaya':"Valya Tsybulskaya",
                "Valrijs olnerovis":"Valērijs Žolnerovičs",
                "Vanda Juhsz":"Vanda Juhász",
                'Vano "Vane" Stojanov':"Vane Stojanov",
                "Vanessa Boslak":"Vanessa Boslak",
                "Vanessa Isabella Vandy":"Vanessa Vandy",
                "Vanja Perii":"Vanja Perišić",
                "Vania Emilova Stambolova":"Vanya Stambolova",
                "Vardan Pahlevanyan":"Vardan Pahlevanyan",
                "Vashti Cunningham":"Vashti Cunningham",
                "Vasily Borisovich Kaptyukh":"Vasily Kaptyukh",
                "Veerle Dejaeghere":"Veerle Dejaeghere",
                "Venelina Ivanova Veneva-Mateeva":"Venelina Veneva-Mateeva",
                "Venera Getova":"Venera Getova",
                "Vera Lcia Mendes Barbosa":"Vera Barbosa",
                "Vera Begi":"Vera Begić",
                "Verena Sailer":"Verena Sailer",
                "Veronica Angella Campbell-Brown":"Veronica Campbell-Brown",
                "Veronica Inglese":"Veronica Inglese",
                "Veronica Nyaruai Wanjiru":"Veronica Nyaruai",
                "Lenka Masn":"Veronika Mosina",
                "Veronika Dmitriyevna Shutkova":"Veronika Shutkova",
                "Vernica Abigail Colindres Garca":"Verónica Colindres",
                "Vesa Tapani Rantanen":"Vesa Rantanen",
                "Vicente Lenlson de Lima":"Vicente De Lima",
                "Victor Kibet":"Victor Kibet",
                "Victoria Mitchell":"Victoria Mitchell",
                "Victoria Palacios Carillo":"Victoria Palacios",
                "Vida Anim":"Vida Anim",
                "Vijona Kryeziu":"Vijona Kryeziu",
                "Vikas Shive Gowda":"Vikas Gowda",
                "Viktor Mikhaylovich Burayev":"Viktor Burayev",
                "Viktor Andriyovych Kuznietsov":"Viktor Kuznietsov",
                "Viktor Vladimirovich Leptikov":"Viktor Leptikov",
                "Viktor Damyanov Ninov":"Viktor Ninov",
                "Viktoriya Vladimirovna Gurova-Valyukevich":"Viktoriya Gurova-Valyukevich",
                "Viktoriya Viktorivna Tkachuk":"Viktoriya Tkachuk",
                "Viktoriya Viktorovna Zyabkina":"Viktoriya Zyabkina",
                "Viktria Madarsz":"Viktória Madarász",
                "Ville Johannes Tiisanoja":"Ville Tiisanoja",
                "Vincent Kiplangat Kosgei":"Vincent Kiplangat Kosgei",
                "Vincent Le Dauphin":"Vincent Le Dauphin",
                "Vincent Mumo Kiilu":"Vincent Mumo",
                "Vincent Zouaoui-Dandrieux":"Vincent Zouaoui-Dandrieux",
                "Vincenza Cal":"Vincenza Calì",
                "Viola Chelagat Kibiwot":"Viola Kibiwot",
                "Viola Cheptoo Lagat":"Viola Lagat",
                "Violeta Guzmn Magaa":"Violeta Guzmán",
                "Viorica Sndica igu":"Viorica Țigău",
                "Vira Viktorivna Rebryk":"Vira Rebryk",
                "Vira Mykhalivna Zozulia":"Vira Zozulia",
                "Virgen Benavides Muoz":"Virgen Benavides",
                "Virgil Hodge":"Virgil Hodge",
                "Virgilijus Alekna":"Virgilijus Alekna",
                "Visa Veli Hongisto":"Visa Hongisto",
                "Taina Halasima":"Vita Palamar",
                "Vitalij Kozlov":"Vitalij Kozlov",
                "Vitaliy Mykolaiovych Butrym":"Vitaliy Butrym",
                "Vitaly Vladimirovich Shkurlatov":"Vitaly Shkurlatov",
                "Vivian Peters-Chukwuemeka":"Vivian Chukwuemeka",
                'Vladislava "Vlada" Ovcharenko':"Vlada Ovcharenko",
                "Vladimir Alekseyevich Kanaykin":"Vladimir Kanaykin",
                "Vitalij Kozlov":"Vladimir Kozlov",
                "Vladimr Maka":"Vladimir Malyavin",
                "Vladimir Viktorovich Parvatkin":"Vladimir Parvatkin",
                "Vladimr Maka":"Vladimír Maška",
                "Vladimra Maltov-Rackov (Palyzov-)":"Vladimíra Malátová-Racková",
                "Vladyslav Yuriyovych Piskunov":"Vladyslav Piskunov",
                "Voldemrs Lsis":"Voldemārs Lūsis",
                "Volodymyr Viktorovych Ziuskov":"Volodymyr Ziuskov",
                "Vonette Dixon":"Vonette Dixon",
                "Vyacheslav Muravyov":"Vyacheslav Muravyov",
                "Vyacheslav Muravyov":"Vyacheslav Sakayev",
                "Vyacheslav Vasilyevich Shabunin":"Vyacheslav Shabunin",
                "Vyacheslav Nikolayevich Voronin":"Vyacheslav Voronin",
                "Vronique Mang":"Véronique Mang",
                "Vctor Manuel Castillo Petit":"Víctor Castillo",
                "Vctor Abel Fatecha Riveros":"Víctor Fatecha",
                "Vctor Garca Blazquez":"Víctor García",
                "Vtor Hugo Silva Mouro dos Santos":"Vítor Hugo Dos Santos",
                "Vitzslav Vesel":"Vítězslav Veselý",
                "Vra Pospilov-Cechlov":"Věra Pospíšilová-Cechlová",
                "V Th Hng":"Vũ Thị Hương",
                "Wallace Spearmon, Jr.":"Wallace Spearmon",
                "Walter L. Davis":"Walter Davis",
                "Walter Dix":"Walter Dix",
                "Wan Kin Yee":"Wan Kin Yee",
                "Wang Chunyu":"Wang Chunyu",
                "Wang Hao":"Wang Hao",
                "Wang Jianan":"Wang Jianan",
                "Wang Jing":"Wang Jing",
                "Wang Lina":"Wang Lina",
                "Wang Yu":"Wang Yu",
                "Wang Zheng":"Wang Zheng",
                "Wanida Boonwan":"Wanida Boonwan",
                "Wanner Miller Moreno":"Wanner Miller",
                "Warren Fraser":"Warren Fraser",
                "Warren Fraser":"Warren Weir",
                "Waseelah Fadhl Saad":"Waseelah Saad",
                "Wataru Yazawa":"Wataru Yazawa",
                "Wayne Courtney Davis":"Wayne Davis",
                "Wayne Snyman":"Wayne Snyman",
                "Wei Yongli":"Wei Yongli",
                "Wenda Nel (Theron-)":"Wenda Nel",
                "Wendy Gabriela Cornejo Aliaga":"Wendy Cornejo",
                "Werknesh Kidane":"Werknesh Kidane",
                "Wesley Joel Vzquez Vzquez":"Wesley Vázquez",
                "Weynay Ghebresilasie":"Weynay Ghebresilasie",
                "Whitney Jordan Ashley":"Whitney Ashley",
                "Widad Mendil":"Widad Mendil",
                "Brenda Martinez":"Wilfredo Martínez",
                "Wilfried Bingangoye":"Wilfried Bingangoye",
                "Hua Wilfried Serge Koffi":"Wilfried Koffi",
                "Wilhem Belocian":"Wilhem Belocian",
                "William Donald Hamlyn-Harris":"William Hamlyn-Harris",
                "Williams Collazo Gutirrez":"Williams Collazo",
                "Wilma Anna Helena Murto":"Wilma Murto",
                "Wilson Kipkemei Busienei":"Wilson Kipkemei Busienei",
                "Wilson Kiprop":"Wilson Kiprop",
                "Winder Bolivar Cuevas Prez":"Winder Cuevas",
                "Winneth Dube":"Winneth Dube",
                "Winnie Nanyondo":"Winnie Nanyondo",
                "Winny Chebet":"Winny Chebet",
                "Winston Oudkerk George":"Winston George",
                "Wioletta Magdalena Janowska-Frankiewicz":"Wioletta Janowska-Frankiewicz",
                "Wioletta Potpa":"Wioletta Potępa",
                "Wojciech Nowicki":"Wojciech Nowicki",
                "Wojciech Theiner":"Wojciech Theiner",
                "Wolfram Mller":"Wolfram Müller",
                "Woo Sang-Hyeok":"Woo Sang-Hyeok",
                "Woroud Sawalha":"Woroud Sawalha",
                "Wu Shuijiao":"Wu Shuijiao",
                "Wu Tao":"Wu Tao",
                "Xavier James":"Xavier James",
                "dison Xavier Moreno Cruz":"Xavier Moreno",
                "Xie Limei":"Xie Limei",
                "Xie Wenjun":"Xie Wenjun",
                "Xie Zhenye":"Xie Zhenye",
                "Xue Changrui":"Xue Changrui",
                "Xue Fei":"Xue Fei",
                "Xue Juan":"Xue Juan",
                "Yacnier Luis Gonzlez":"Yacnier Luis",
                "Yacob Jarso Kintra":"Yacob Jarso",
                "Maritza Guamn":"Yadira Guamán",
                "Yadisleidy Pedroso Gonzlez":"Yadisleidy Pedroso",
                "Yahya Berrabah":"Yahya Berrabah",
                "Yaime Prez Tellez":"Yaimé Pérez",
                "Yainelis Ribeaux Rustafull":"Yainelis Ribeaux",
                "Yamil Aldama Pozo (-Dodds)":"Yamilé Aldama",
                "Yancarlos Martnez Martnez":"Yancarlos Martínez",
                "Yanet Cruz Cruz":"Yanet Cruz",
                "Yang Mingxia":"Yang Mingxia",
                "Yang Yansheng":"Yang Yansheng",
                "Yang Yaozu":"Yang Yaozu",
                "Yania Ferrales Monteagudo":"Yania Ferrales",
                "Yanina Dislavovna Provalinskaya-Karolchik":"Yanina Provalinskaya-Karolchik",
                "Yaniuvis Lpez Sago":"Yaniuvis López",
                "Yann Domenech":"Yann Domenech",
                "Yao Jie":"Yao Jie",
                "Yarelis Barrios Castaeda":"Yarelis Barrios",
                "Yarianna Martnez Iglesias":"Yarianna Martínez",
                "Yasemin Can":"Yasemin Can",
                "Yasser Fathi Ibrahim Farag":"Yasser Farag",
                "Yassine Bensghir":"Yassine Bensghir",
                "Yassine Hethat":"Yassine Hethat",
                "Yazaldes Valdemar Nascimento Alfonso":"Yazaldes Nascimento",
                "Yegor Vladimirovich Nikolayev":"Yegor Nikolayev",
                "Yehualeye Beletew":"Yehualeye Beletew Mitiku",
                "Yeimer Lpez Garca":"Yeimer López",
                "Yeison Javier Rivas Rivas":"Yeison Rivas",
                "Yekaterina Aleksandrovna Bakhvalova":"Yekaterina Bakhvalova",
                "Yekaterina Eduardovna Bikert":"Yekaterina Bikert",
                "Yekaterina Viktorovna Galitskaya":"Yekaterina Galitskaya",
                "Yekaterina Sergeyevna Kondratyeva":"Yekaterina Kondratyeva",
                "Yekaterina Aleksandrovna Kostetskaya":"Yekaterina Kostetskaya",
                "Yekatarina Viktorovna Poplavskaya":"Yekaterina Poplavskaya",
                "Yekaterina Tunguskova":"Yekaterina Tunguskova",
                "Yekaterina Yuryevna Yevseyeva":"Yekaterina Yevseyeva",
                "Yelena Vladimirovna Arzhakova":"Yelena Arzhakova",
                "Yelena Bobrovskaya":"Yelena Bobrovskaya",
                "Yelena Ivanovna Bolsun":"Yelena Bolsun",
                "Yelena Anatolyevna Churakova":"Yelena Churakova",
                "Yelena Yevgenyevna Koshcheyeva":"Yelena Koshcheyeva",
                "Yelena Nikolayevna Kuznetsova":"Yelena Kuznetsova",
                "Yelena Viktorovna Matoshko":"Yelena Matoshko",
                "Yelena Stanislavovna Nagovitsyna":"Yelena Nagovitsyna",
                "Yelena Parfyonova":"Yelena Parfyonova",
                "Yelena Smolyanova":"Yelena Smolyanova",
                "Yelena Smolyanova":"Yelena Soboleva",
                "Yelena Anatolyevna Zadorozhnaya":"Yelena Zadorozhnaya",
                "Yeliz Ay (-Yldz)":"Yeliz Ay",
                "Yelizaveta Andriyanovna Grechishnikova":"Yelizaveta Grechishnikova",
                "Yelizaveta Valeryevna Savlinis":"Yelizaveta Savlinis",
                "Yemane Haileselassie":"Yemane Haileselassie",
                "Yenew Alamirew Getahun":"Yenew Alamirew",
                "Yennifer Padilla Gonzlez":"Yenifer Padilla",
                "Yenima Arencibia Barrisontes":"Yenima Arencibia",
                "Yerzhan Askarov":"Yerzhan Askarov",
                "Yeseida Isaid Carrillo Torres":"Yeseida Carrillo",
                "Yesenia  Ivania Miranda":"Yesenia Miranda",
                "Yevgeniya Nikolayevna Kolodko":"Yevgeniya Kolodko",
                "Yevgeniya Yevgeniyevna Polyakova":"Yevgeniya Polyakova",
                "Yevgeny Valeryevich Borisov":"Yevgeny Borisov",
                "Yevgeny Labutov":"Yevgeny Labutov",
                "Yevgeny Vasilyevich Meleshchenko":"Yevgeny Meleshchenko",
                "Yevgeny Nikolayevich Misyulya":"Yevgeny Misyulya",
                "Yevgeny Gennadyevich Pechonkin":"Yevgeny Pechonkin",
                "Yevhen Valeriyovych Vynohradov":"Yevhen Vynohradov",
                "Yevheniya Valerivna Snihur":"Yevheniya Snihur",
                "Yidiel Islay Contreras Garca":"Yidiel Contreras",
                "Yiech Pur Biel":"Yiech Pur Biel",
                "Yigrem Demelash":"Yigrem Demelash",
                "Yin Anna":"Yin Anna",
                "Yipsi Moreno Gonzlez":"Yipsi Moreno",
                "Yirisleydi Lisbet Ford Carnonell":"Yirisleydi Ford",
                "Yoandys Alberto Lescay Prado":"Yoandys Lescay",
                "Yoann Kowal":"Yoann Kowal",
                "Yoel Hernndez Reyes":"Yoel Hernández",
                "Yohan Blake":"Yohan Blake",
                "Yoko Shibui":"Yoko Shibui",
                "Marie Yolene Raffin":"Yolène Raffin",
                "Yomara Hinestroza Murillo":"Yomara Hinestroza",
                "Yonas Kinde":"Yonas Kifle",
                "Yordan Luis O'Farril Olivera":"Yordan O'Farril",
                "Yoshihide Kiryu":"Yoshihide Kiryu",
                "Yoshitaka Iwamizu":"Yoshitaka Iwamizu",
                "Yosiry Urrutia Chaverra":"Yosiry Urrutia",
                "Youba Mohamed Ould H'Mede":"Youba Ould H'Meïde",
                "Youcef Abdi":"Youcef Abdi",
                "Young Talkmore Nyongani":"Young Talkmore Nyongani",
                "Youns Es-Salhi":"Younés Es-Salhi",
                "Yousef Ahmed Al-Masrahi":"Yousef Masrahi",
                "Youssef Baba":"Youssef Baba",
                "Youssef Saad Kamel":"Youssef Saad Kamel",
                "Yuan Qiqi":"Yuan Qiqi",
                "Yuan Yu Fang":"Yuan Yu Fang",
                "Yudelkis Fernndez Luis":"Yudelkis Fernández",
                "Yuka Murofushi":"Yuka Murofushi",
                "Yuka Takashima":"Yuka Takashima",
                "Yuki Ebihara":"Yuki Ebihara",
                "Yuki Matsushita":"Yuki Matsushita",
                "Yuki Sato":"Yuki Sato",
                "Yuki Yamaguchi":"Yuki Yamaguchi",
                "Yuki Yamazaki":"Yuki Yamazaki",
                "Yukifumi Murakami":"Yukifumi Murakami",
                "Yukiko Akaba":"Yukiko Akaba",
                "Yulenmis Aguilar Martnez":"Yulenmis Aguilar",
                "Yuliana Y. Martinez Perez":"Yuliana Perez",
                "Yuliya Vladimirovna Balykina":"Yuliya Balykina",
                "Yuliya Igorevna Chermoshanskaya":"Yuliya Chermoshanskaya",
                "Yuliya Alekseyevna Golubchikova":"Yuliya Golubchikova",
                "Yuliya Aleksandrovna Gushchina":"Yuliya Gushchina",
                "Yuliya Andreyevna Kondakova":"Yuliya Kondakova",
                "Yuliya Vladimirovna Korol":"Yuliya Korol",
                "Yuliya Sergeyevna Leontyuk":"Yuliya Leontyuk",
                "Yuliya Sergeyevna Nosova-Pechonkina":"Yuliya Nosova-Pechonkina",
                "Yuliya Anatolivna Olishevska":"Yuliya Olishevska",
                "Yumileidi Cumb Jay":"Yumileidi Cumbá",
                "Yun Seung-Hyun":"Yun Seung-Hyun",
                "Yunaika Crawford Rogert":"Yunaika Crawford",
                "Yuniel Hernndez Solar":"Yuniel Hernández",
                "Yunio Lastre Hechavarra":"Yunior Lastre",
                "Yuri Floriani":"Yuri Floriani",
                "Yuriko Kobayashi":"Yuriko Kobayashi",
                "Yuriy Hryhorovych Bilonoh":"Yuriy Bilonoh",
                "Yuriy Oleksandrovych Krymarenko":"Yuriy Krymarenko",
                "Yury Vladimirovich Voronkin":"Yury Voronkin",
                "Yusmay Bicet Planas":"Yusmay Bicet",
                "Yusneysi Santiusti Caballero":"Yusneysi Santiusti",
                "Yusuke Suzuki":"Yusuke Suzuki",
                "Yuta Shitara":"Yuta Shitara",
                "Yuzo Kanemaru":"Yuzo Kanemaru",
                "Yves Niar":"Yves Niaré",
                "Yvette Lewis":"Yvette Lewis",
                "Yvonne Harrison Castro":"Yvonne Harrison",
                "Yvonne Trevino Hayek":"Yvonne Treviño",
                "Zahra Bani":"Zahra Bani",
                "Zaidatul Husniah Zulkifli":"Zaidatul Husniah Binti Zulkifli",
                "Zakia Mrisho Mohamed":"Zakia Mrisho",
                "Zalina Marghiev (-Petrivskaya)":"Zalina Marghiev",
                "Zamira Shamiliyevna Amirova":"Zamira Amirova",
                "Zamzam Mohamed Farah":"Zamzam Mohamed Farah",
                "Zane Robertson":"Zane Robertson",
                "Zara Georgia Northover":"Zara Northover",
                "Zeinab Bakour":"Zeinab Bakour",
                "Zemzem Ahmed Deko":"Zemzem Ahmed",
                "Zenaide Vieira":"Zenaide Vieira",
                "Zhang Guirong":"Zhang Guirong",
                "Zhang Guowei":"Zhang Guowei",
                "Zhang Hao":"Zhang Hao",
                "Zhang Jun":"Zhang Jun",
                "Zhang Li":"Zhang Li",
                "Zhang Peimeng":"Zhang Peimeng",
                "Zhang Wenxiu":"Zhang Wenxiu",
                "Zhang Xiaoyi":"Zhang Xiaoyi",
                "Zhang Xiaoyu":"Zhang Xiaoyu",
                "Zhang Xinyan":"Zhang Xinyan",
                "Zhang Yingying":"Zhang Yingying",
                "Zhanna Yurivna Pintusevych-Blok (Tarnopolska-)":"Zhanna Pintusevych-Blok",
                "Zhao Yanni":"Zhao Yanni",
                "Zhao Yingying":"Zhao Yingying",
                "Zheng Xingjuan":"Zheng Xingjuan",
                "Zhou Can":"Zhou Can",
                "Zhou Yang":"Zhou Yang",
                "Zhu Hongjun":"Zhu Hongjun",
                "Zhu Yanmei":"Zhu Yanmei",
                "Zigismunds Sirmais":"Zigismunds Sirmais",
                "Zinaida Sendrit":"Zinaida Sendriūtė",
                "Zita Ajkler":"Zita Ajkler",
                "Zoe Buckman":"Zoe Buckman",
                "Zoe Lianne Derham":"Zoe Derham",
                "Zoltn Kvg":"Zoltán Kővágó",
                "Zouhair Aouad":"Zouhair Aouad",
                "Zouhair El-Ouardi":"Zouhair El-Ouardi",
                "Zourah Mohamed Ali":"Zourah Ali",
                "Zsolt Biber":"Zsolt Biber",
                "Zsolt Szeglet":"Zsolt Szeglet",
                "Zuleima Aramendiz Meja":"Zuleima Araméndiz",
                "Zulema Fuentes-Pila Ortz":"Zulema Fuentes-Pila",
                "Zulia Ins Calatayud Torres":"Zulia Calatayud",
                "Zurian Hechavarra Martn":"Zurian Hechavarría",
                "Zuzana Kovikov-Hlavoov":"Zuzana Kováčiková-Hlavoňová",
                "Zuzana Malkov":"Zuzana Malíková",
                "Zuzana Schindlerov":"Zuzana Schindlerová",
                "Zuzana Malkov":"Zuzanna Radecka",
                "lvaro de Arriba Lpez":"Álvaro De Arriba",
                "lvaro Fernndez Cerezo":"Álvaro Fernández",
                "lvaro Martn Uriol":"Álvaro Martín",
                "lvaro Rodrguez Melero":"Álvaro Rodríguez",
                "ngel David Rodrguez Garca":"Ángel David Rodríguez",
                "ngel Mullera Rodrguez":"Ángel Mullera",
                "ngela Mara Figueroa Palacios":"Ángela Figueroa",
                "Angela Moroanu":"Ángela Tenorio",
                "sds Hjlmsdttir":"Ásdís Hjálmsdóttir",
                "Tnde Szab":"Éder Souza",
                "Edson Luciano Ribeiro":"Édson Ribeiro",
                "Elisabeth Grousselle":"Élisabeth Grousselle",
                "lodie Gugan":"Élodie Guégan",
                "loyse Lesueur":"Éloyse Lesueur",
                "milie Menuet":"Émilie Menuet",
                "milie Mondor":"Émilie Mondor",
                "Jean ric Milazar":"Éric Milazar",
                "rica Rocha de Sena":"Érica Sena",
                "va Krti":"Éva Kürti",
                "va Orbn":"Éva Orbán",
                "inn Bjrn orsteinsson":"Óðinn Björn Þorsteinsson",
                "zlem Kaya":"Özlem Kaya",
                "rsula Ruz Prez":"Úrsula Ruiz",
                "elena Rabowa":"Ýelena Rýabowa",
                "rey Edda Elsdttir":"Þórey Edda Elísdóttir",
                "riks Rags":"Ēriks Rags",
                "lham Tanui zbilen":"İlham Özbilen",
                "ukasz Chya":"Łukasz Chyła",
                "ukasz Grzeszczuk":"Łukasz Grzeszczuk",
                "ukasz Michalski":"Łukasz Michalski",
                "ukasz Nowak":"Łukasz Nowak",
                "ukasz Parszczyski":"Łukasz Parszczyński",
                "Stepan Wagner":"Štepán Wagner",
                "tpn Janek":"Štěpán Janáček",
                "tpn Tesak":"Štěpán Tesařík",
                "aneta Glanc":"Żaneta Glanc",
                "an Rudolf":"Žan Rudolf",
                "ivil Vaiciukeviit":"Živilė Vaiciukevičiūtė",
                "liby krov":"Əlibəy Şükürov",
                "tefan Vasilache":"Ștefan Vasilache"}

In [9]:
names_to_replace = {'Abderrahim Goumri': 'Abderrahim Al-Goumri',
                    'Abderrahime Bouramdane': 'Abderrahhime Bouramdane',
                    'Adil Kaouch': 'Adil El-Kaouch',
                    'Adrián Annus': 'Adrin Zsolt Annus',
                    'Aleksey Lesnichiy': 'Aleksey Lesnichy',
                    'Aleksey Voyevodin': 'Aleksey Nikolayevich Voyevodin',
                    'Alemayehu Bezabeh': 'Alemayehu Bezabeh Desta',
                    'Alemitu Bekele': 'Alemitu Bekele Degfa',
                    'Alessandra Aguilar': 'Alessandra Aguilar Morn',
                    'Mina Aït Hammou': 'Amina "Mina" At Hammou',
                    'Amina Bakhit': 'Amina Barsham Bakhit',
                    'Amine Laâlou': 'Amine Lalou',
                    'Anastasiya Kapachinskaya': 'Anastasiya Aleksandrovna Kapachinskaya',
                    'Anastasiya Soprunova': 'Anastasiya Valeryevna Soprunova',
                    'Anastasiya Kudinova': 'Anastasiya Valeryevna Kudinova',
                    'Andrey Rudnitskiy': 'Andrey Petrovich Rudnitsky',
                    'Anil Kumar': 'Anil Kumar Sangwan',
                    'Anis Ananenka': 'Anis Viktorovich Ananenko',
                    'Anna Avdeyeva': 'Anna Mikhaylovna Avdeyeva',
                    'Anna Mishchenko': 'Anna Mikhaylovna Avdeyeva',
                    'Anthony Alozie': 'Uzodinma Anthony Alozie',
                    'Antonina Yefremova': 'Antonina Oleksandrivna Yefremova',
                    'Antonio David Jiménez': 'AntonioDavid Jimnez Pentinel',
                    'Anzhelika Shevchenko': 'Anzhelika Viktorivna Shevchenko',
                    'Asafa Powell': 'Asafa Kehine Powell',
                    'Aslı Çakır Alptekin': 'Asl akr (-Alptekin)',
                    'Ato Stephens': 'Ato Modibo-Stephens',
                    'Attila Szabó': 'Attila Szab',
                    'Ayman Kozhakhmetova': 'Ayman Bulatovna Kozhakhmetova',
                    'Aziz Zakari': 'Abdul Aziz Zakari',
                    'Bahar Doğan': 'Bahar Doan',
                    'Barakat Al-Harthi': 'Barakat Mubarak Al-Harthi',
                    'Bernard Williams': 'Bernard Rollen Williams',
                    'Bobby-Gaye Wilkins': 'Bobby-Gaye Wilkins (-Gooden)',
                    'Brian Mariano': 'Brian Vincent Mariano',
                    'Béranger-Aymard Bossé': 'Branger Aymard Bosse',
                    'Christy Ekpukhon': 'Christy Ekpukhon Ihunaegbo',
                    'Daniela Yordanova': 'Daniela Lyubenova Yordanova',
                    'Darya Pishchalnikova': 'Darya Vitalyevna Pishchalnikova',
                    'Denis Alekseyev': 'Denis Sergeyevich Alekseyev',
                    'Dmytro Kosynskyy': 'Dmytro Volodymyrovych Kosynskiy',
                    'Dorian Scott': 'Dorian Armand Scott',
                    'Dmitri Marşin': 'Dmitri Marin',
                    'Edi Maia': 'Edi Silva Maia',
                    'Elena Arzhakova': 'Yelena Vladimirovna Arzhakova',
                    'Elena Lashmanova': 'Yelena Anatolyevna Lashmanova',
                    'Esref Apak': 'Eref Apak',
                    'Eva-Maria Gradwohl': 'Eva-Maria Gradwohl (Wilfing-)',
                    'Fani Halkia': 'Fani Khalkia',
                    'Femi Ogunode': 'Femi Seun Ogunode',
                    'Folashade Abugan': 'Folashade "Shade" Abugan',
                    'Gareth Warburton': 'Gareth Robert Warburton',
                    'Geisa Arcanjo': 'Geisa Rafaela Arcanjo',
                    'Gergely Horváth': 'Gergely Horvth',
                    'Ghfran Mouhamad': 'Ghfran Al-Mouhamad',
                    'Gloria Kemasuode(Gloria Ubiebor)': 'Gloria E. Kemasuode (-Ubiebor)',
                    'Géza Pauer': 'Gza Pauer',
                    'Hanna Drabenia': 'Anna Ivanovna Drabenya',
                    'Hamza Driouch': 'Hamza Al-Driouch',
                    'Hezekiél Sepeng': 'Hezekiel Sello Sepeng',
                    'Hrysopiyi Devetzi': 'Khrysopigi "Pigi" Devetzi',
                    'Hussain Al-Hamdah': 'Hussain Jamaan Al-Hamdah',
                    'Igor Vinichenko': 'Igor Alekseyevich Vinichenko',
                    'Igor Yerokhin': 'Igor Nikolayevich Yerokhin',
                    'Ildar Minshin': 'Ildar Salikhovich Minshin',
                    'Ilya Konovalov': 'Ilya Valeryevich Konovalov',
                    'Inga Abitova': 'Inga Eduardovna Abitova',
                    'Inna Eftimova': 'Inna Liubomirova Eftimova',
                    'Irina Khudoroshkina': 'Irina Aleksandrovna Khudoroshkina',
                    'Irina Yatchenko': 'Irina Vasilyevna Yatchenko',
                    'Jillian Camarena-Williams': 'Jillian Mary "Jill" Camarena-Williams',
                    'Josephine Onyia': 'Josephine Nnkiruka Onyia',
                    'Joshua Ross': 'Joshua James Ross',
                    'Josiane Tito': 'Josiane da Silva Tito',
                    'José Alessandro Bagio': 'Jos Alessandro Bernardo Baggio',
                    'José Luis Blanco': 'Jos Luis Blanco Quevedo',
                    'Joyce Zakari Joy Nakhumicha Sakari': 'Joy Nakhumicha Sakari',
                    'Julien Kapek': 'Julien-Pierre Kapek',
                    'Kagisho Kumbane': 'Ishmael Kagisho Kumbane',
                    'Karin Mey Melis': 'Karin Melis (Mey-)',
                    'Kenta Bell': 'Kenwood Kenta Bell',
                    'Kirill Ikonnikov': 'Kirill Gennadiyevich Ikonnikov',
                    'Kostas Baniotis': 'Konstantinos "Kostas" Baniotis',
                    'Kostas Filippidis': 'Konstantinos "Kostas" Filippidis',
                    'Larbi Bouraada': 'Larbi Bourrada',
                    'LaVerne Jones-Ferrette': 'LaVerne Janet Jones-Ferrette',
                    'Lebogang Phalula': 'Dina Lebo Phalula',
                    'Liza Hunter-Galvan': 'Liza Marie Hunter-Galvan',
                    'Lucimara da Silva': 'Lucimara Silvestre da Silva',
                    'Lyubov Perepelova': 'Lyubov Gennadyevna Perepelova',
                    'Lyudmyla Yosypenko': 'Liudmyla Dmytrivna Iosypenko',
                    'Lü Huihui': 'Lu Huihui',
                    'Marcelo Pugliese': 'Marcelo Adrin Pugliese',
                    'Marina Marghieva': 'Marina Marghiev-Nicienco',
                    'Mariya Bespalova': 'Mariya Aleksandrovna Bespalova',
                    'Mariya Yakovenko': 'Mariya Ivanovna Yakovenko',
                    'Mark Lewis-Francis': 'Mark Anthony Lewis-Francis',
                    'Marta Dominguez': 'Marta Domnguez Azpeleta',
                    'Martin Marić': 'Martin Mari',
                    'Michalis Stamatogiannis': 'Mikhalis Stamatogiannis',
                    'Mihaela Botezan': 'Mihaela Maria Botezan',
                    'Mike Rodgers': 'Michael "Mike" Rodgers',
                    'Milan Haborák': 'Milan Habork',
                    'Mohamed El Hachimi': 'Mohamed El-Hachimi',
                    'Mohammed Shaween': 'Mohammed Othman H. Shaween',
                    'Nadezhda Ostapchuk': 'Nadezhda Nikolayevna Ostapchuk',
                    'Nataliia Lupu': 'Nataliya Oleksivna Lupu',
                    'Nataliya Pyhyda': 'Nataliya Serhivna Pyhyda',
                    'Natasha Mayers': 'Natasha Laren Mayers',
                    'Nevin Yanit': 'Nevin Yant',
                    'Nicolas Figère': 'Nicolas Figre',
                    'Ofentse Mogawane': 'Ofense Iginitius Mogawane',
                    'Oksana Yarygina': 'Oksana Aleksandrovna Yarygina',
                    'Oleksandr Korchmid': 'Oleksandr Mykolaiovych Korchmid',
                    'Oleksiy Shelest': 'Oleksiy Petrovych Shelest',
                    'Olena Antonova': 'Olena Anatolivna Antonova',
                    'Olga Kaniskina': 'Olga Nikolayevna Kaniskina',
                    'Olga Kuzenkova': 'Olga Sergeyevna Kuzenkova',
                    'Olga Tereshkova': 'Olga Alekseyevna Tereshkova',
                    'Oludamola Osayomi': 'Oludamola Bolanle "Damola" Osayomi',
                    'Olutoyin Augustus  (Toyin Augustus)': 'Olutoyin "Toyin" Augustus',
                    'Onalenna Baloyi': 'Onalenna Oabona Baloyi',
                    'Pavel Krivitsky': 'Pavel Eduardovich Krivitsky',
                    'Paweł Rakoczy': 'Pawe Rakoczy',
                    'Pınar Saka': 'Pnar Saka',
                    'Rachid Ghanmouni': 'Rachid El-Ghanmouni',
                    'Recep Çelik': 'Recep elik',
                    'Roland Varga': 'Roland Robert Varga',
                    'Roman Avramenko': 'Roman Valeriyovych Avramenko',
                    'Rosa America Rodríguez': 'Rosa Mara Morat Rodrguez',
                    'Roxana Bârcă': 'Roxana Elisabeta Brc',
                    'Ruqaya Al-Ghasra': 'Rakia Al-Gassra',
                    'Róbert Fazekas': 'Rbert Fazekas',
                    'Sandra Perković': 'Sandra Perkovi',
                    'Seltana Aït Hammou': 'Seltana At Hammou',
                    'Semoy Hackett': 'Semoy Kee-Ann Hackett',
                    'Sergey Kirdyapkin': 'Sergey Aleksandrovich Kirdyapkin',
                    'Sherone Simpson': 'Sherone Anmarica Simpson',
                    'Shelly-Ann Fraser': 'Shelly-Ann Fraser-Pryce',
                    'Stanislav Melnykov': 'Stanislav Anatoliyovych Melnykov',
                    'Sultan Mubarak Al-Dawoodi': 'Sultan Mubarak M. Al-Dawoodi',
                    'Surita Febbraio': 'Surita Febbraio (Loots-)',
                    'Svetlana Klyuka': 'Svetlana Vasilyevna Klyuka',
                    'Svetlana Krivelyova': 'Svetlana Vladimirovna Krivelyova',
                    'Svitlana Shmidt': 'Svitlana Volodymyrivna Shmidt',
                    'Tahesia Harrigan-Scott': 'Tahesia Gaynell Harrigan-Scott',
                    'Tatyana Dektyareva': 'Tatyana Valeryevna Dektyareva',
                    'Tetiana Petliuk': 'Tetiana Hryhorivna Petliuk',
                    'Tetyana Hamera-Shmyrko': 'Tetiana Hamera-Shmyrko',
                    'Tezzhan Naimova': 'Tezdzhan Naimova',
                    'Traves Smikle': 'Traves Jamie Smikle',
                    'Tsholofelo Thipe': 'Tsholofelo Thipe (Selemela-)',
                    'Tyson Gay': 'Tyson Lashane Gay',
                    'Valentin Kruglyakov': 'Valentina Kibalnikova',
                    'Valeriy Borchin': 'Valeriy Viktorovich Borchin',
                    'Vania Stambolova': 'Vania Emilova Stambolova',
                    'Vanja Perišić': 'Vanja Perii',
                    'Venelina Veneva': 'Venelina Ivanova Veneva-Mateeva',
                    'Viktor Burayev': 'Viktor Mikhaylovich Burayev',
                    'Ville Tiisanoja': 'Ville Johannes Tiisanoja',
                    'Vivian Chukwuemeka': 'Vivian Peters-Chukwuemeka',
                    'Vladimir Kanaykin': 'Vladimir Alekseyevich Kanaykin',
                    'Víctor Castillo': 'Vctor Manuel Castillo Petit',
                    'Wallace Spearmon': 'Wallace Spearmon, Jr.',
                    'Walter Davis': 'Walter L. Davis',
                    'Yarelys Barrios': 'Yarelis Barrios Castaeda',
                    'Yekaterina Kostetskaya': 'Yekaterina Aleksandrovna Kostetskaya',
                    'Yelena Churakova': 'Yelena Anatolyevna Churakova',
                    'Yelena Prokhorova': 'Yelena Parfyonova',
                    'Yelizaveta Grechishnikova': 'Yelizaveta Andriyanovna Grechishnikova',
                    'Yelyzaveta Bryzhina': 'Yelyzaveta Viktorivna Bryzhina',
                    'Yevhen Vynohradov': 'Yevhen Valeriyovych Vynohradov',
                    'Yolanda Caballero': 'Yolanda Beatriz Caballero Prez',
                    'Yousef Masrahi': 'Yousef Ahmed Al-Masrahi',
                    'Yuliya Balykina': 'Yuliya Vladimirovna Balykina',
                    'Yuriy Bilonoh': 'Yuriy Hryhorovych Bilonoh',
                    'Zalina Marghieva': 'Zalina Marghiev (-Petrivskaya)',
                    'Zoltán Kővágó': 'Zoltn Kvg',
                    'Zohar Zimro': 'Zohar Zemiro',
                    'Zudikey Rodríguez': 'Zudikey Rodrguez Nez',
                    'Ángel Mullera': 'ngel Mullera Rodrguez',
                    'Ümmü Kiraz': 'mm Kiraz',
                    'Živilė Balčiūnaitė': 'ivil Balinait'}

In [213]:
athlete_df.name = athlete_df.name.replace(athlete_events_names_to_change)
doping_df.name = doping_df.name.replace(names_to_replace)
doping_df.name = doping_df.name.replace(athlete_events_names_to_change)

#### Defining two new name lists to see how many match. Before changing any of the names I was able to match on about 90 athlete names.

In [214]:
athlete_names = list(athlete_df.name.unique())
doping_names = list(doping_df.name.unique())

In [215]:
matched = []
for x in range(len(doping_names)):
    if doping_names[x] in athlete_names:
        matched.append(doping_names[x])
print('Found {} matches\n\n'.format(len(matched)))

Found 238 matches




In [216]:
athlete_df.duplicated('name').sum()

2114

In [217]:
athlete_df = athlete_df.drop_duplicates('name')

In [218]:
len(athlete_df)

4817

#### Inserting the 'flagged' column to hold binary values indicating PED use

In [219]:
doping_df.insert(4, 'flagged', 1)

In [220]:
athlete_df = pd.merge(athlete_df, doping_df[['name', 'flagged']], how='left', on='name')

In [221]:
athlete_df.flagged = athlete_df.flagged.fillna(value=0)
athlete_df.flagged.value_counts()

0.0    4579
1.0     241
Name: flagged, dtype: int64

In [222]:
athlete_df = athlete_df.drop_duplicates()

In [223]:
athlete_df.flagged.value_counts()

0.0    4579
1.0     238
Name: flagged, dtype: int64

### Merging Event Results
Below I will begin to scrape the results of the men's track events and merge them with the athlete dataframe on matching names.

#### Scraping the results from the men's 100m dash 2004-Athens

In [224]:
athens_100m_men = fn.olympic_query('athens', '2004', '100m-men')
mens_100m_04 = fn.olympic_scraper(athens_100m_men)
mens_100m_04 = fn.content_cleaner(mens_100m_04)
mens_100m_04.columns = ['rank', 'name', 'mens_100m_04']
mens_100m_04.insert(3, "event", "Athletics Men's 100 metres")
mens_100m_04.mens_100m_04 = [str(x) for x in mens_100m_04.mens_100m_04]
mens_100m_04.name = [x.strip() for x in mens_100m_04.name]
mens_100m_04.mens_100m_04 = [x.strip() for x in mens_100m_04.mens_100m_04]
mens_100m_04.mens_100m_04 = [float(x) for x in mens_100m_04.mens_100m_04]


athlete_df = pd.merge(athlete_df, mens_100m_04[['name', 'mens_100m_04', 'event']], how='left', on=['name','event'], suffixes=(None, '_100m_mens_athens'))

#### Scraping the results from the men's 100m dash 2008-Beijing

In [225]:
beijing_100m_men = fn.olympic_query('beijing', '2008', '100m-men')
mens_100m_08 = fn.olympic_scraper(beijing_100m_men)
mens_100m_08 = fn.content_cleaner(mens_100m_08)
mens_100m_08.columns = ['rank', 'name', 'mens_100m_08']
mens_100m_08.insert(3, "event", "Athletics Men's 100 metres")
mens_100m_08.mens_100m_08 = [str(x) for x in mens_100m_08.mens_100m_08]
mens_100m_08.mens_100m_08 = [x.strip() for x in mens_100m_08.mens_100m_08]
mens_100m_08.name = [x.strip() for x in mens_100m_08.name]
mens_100m_08.mens_100m_08 = [x.strip('DPG /') for x in mens_100m_08.mens_100m_08]
mens_100m_08.mens_100m_08 = [float(x) for x in mens_100m_08.mens_100m_08]


athlete_df = pd.merge(athlete_df, mens_100m_08[['name', 'mens_100m_08', 'event']], how='left', on=['name', 'event'], suffixes=(None, '_100m_mens_beijing'))


#### Scraping the results from the men's 100m dash 2012-London
Removing rows with missing values, this is either due to an update on doping sanctions or the athlete did not finish/start the race.

In [226]:
london_100m_men = fn.olympic_query('london', '2012', '100m-men')
mens_100m_12 = fn.olympic_scraper(london_100m_men)
mens_100m_12 = fn.content_cleaner(mens_100m_12)
mens_100m_12.columns = ['rank', 'name', 'mens_100m_12']
mens_100m_12.insert(3, "event", "Athletics Men's 100 metres")
mens_100m_12.mens_100m_12 = [str(x) for x in mens_100m_12.mens_100m_12]
mens_100m_12.name = [x.strip() for x in mens_100m_12.name]
mens_100m_12.mens_100m_12 = [x.strip() for x in mens_100m_12.mens_100m_12]
mens_100m_12 = mens_100m_12.drop([31,86,87])
mens_100m_12.mens_100m_12 = [float(x) for x in mens_100m_12.mens_100m_12]



athlete_df = pd.merge(athlete_df, mens_100m_12[['name', 'mens_100m_12', 'event']], how='left', on=['name', 'event'], suffixes=(None, '_100m_mens_london'))


#### Scraping the results from the men's 100m dash 2016-Rio

In [227]:
rio_100m_men = fn.olympic_query('rio', '2016', '100m-men')
mens_100m_16 = fn.olympic_scraper(rio_100m_men)
mens_100m_16 = fn.content_cleaner(mens_100m_16)
mens_100m_16.columns = ['rank', 'name', 'mens_100m_16']
mens_100m_16.insert(3, "event", "Athletics Men's 100 metres")
mens_100m_16.mens_100m_16 = [str(x) for x in mens_100m_16.mens_100m_16]
mens_100m_16.name = [x.strip() for x in mens_100m_16.name]
mens_100m_16.mens_100m_16 = [x.strip() for x in mens_100m_16.mens_100m_16]
mens_100m_16.mens_100m_16 = [float(x) for x in mens_100m_16.mens_100m_16]


athlete_df = pd.merge(athlete_df, mens_100m_16[['name', 'mens_100m_16', 'event']], how='left', on=['name', 'event'], suffixes=(None, '_100m_mens_rio'))


#### Scraping the results from the men's 200m dash 2004-Athens

In [228]:
athens_200m_men = fn.olympic_query('athens', '2004', '200m-men')
mens_200m_04 = fn.olympic_scraper(athens_200m_men)
mens_200m_04 = fn.content_cleaner(mens_200m_04)
mens_200m_04.columns = ['rank', 'name', 'mens_200m_04']
mens_200m_04.insert(3, "event", "Athletics Men's 200 metres")
mens_200m_04.mens_200m_04 = [str(x) for x in mens_200m_04.mens_200m_04]
mens_200m_04.mens_200m_04 = [x.strip() for x in mens_200m_04.mens_200m_04]
mens_200m_04.mens_200m_04 = [float(x) for x in mens_200m_04.mens_200m_04]
mens_200m_04.name = [x.strip() for x in mens_200m_04.name]


athlete_df = pd.merge(athlete_df, mens_200m_04[['name', 'mens_200m_04', 'event']], how='left', on=['name', 'event'], suffixes=(None, '_200m_mens_athens'))


#### Scraping the results from the men's 200m dash 2008-Beijing
droping rows with missing values

In [229]:
beijing_200m_men = fn.olympic_query('beijing', '2008', '200m-men')
mens_200m_08 = fn.olympic_scraper(beijing_200m_men)
mens_200m_08 = fn.content_cleaner(mens_200m_08)
mens_200m_08.columns = ['rank', 'name', 'mens_200m_08']
mens_200m_08.insert(3, "event", "Athletics Men's 200 metres")
mens_200m_08.mens_200m_08 = [str(x) for x in mens_200m_08.mens_200m_08]
mens_200m_08.mens_200m_08 = [x.strip() for x in mens_200m_08.mens_200m_08]
mens_200m_08.mens_200m_08 = [x[:5] for x in mens_200m_08.mens_200m_08]
mens_200m_08.name = [x.strip() for x in mens_200m_08.name]
mens_200m_08 = mens_200m_08.drop([6,7,67,23,39,71,103,166,
                                 167,168,169,178,219,227,235])
mens_200m_08.mens_200m_08  = [float(x) for x in mens_200m_08.mens_200m_08 ]


athlete_df = pd.merge(athlete_df, mens_200m_08[['name', 'mens_200m_08', 'event']], how='left', on=['name', 'event'], suffixes=(None, '_200m_mens_beijing'))


#### Scraping the results from the men's 200m dash 2012-London

In [230]:
london_200m_men = fn.olympic_query('london', '2012', '200m-men')
mens_200m_12 = fn.olympic_scraper(london_200m_men)
mens_200m_12 = fn.content_cleaner(mens_200m_12)
mens_200m_12.columns = ['rank', 'name', 'mens_200m_12']
mens_200m_12.insert(3, "event", "Athletics Men's 200 metres")
mens_200m_12.mens_200m_12 = [str(x) for x in mens_200m_12.mens_200m_12]
mens_200m_12.mens_200m_12 = [x.strip() for x in mens_200m_12.mens_200m_12]
mens_200m_12.name = [x.strip() for x in mens_200m_12.name]
mens_200m_12 = mens_200m_12.drop([30,31,85,86])
mens_200m_12.mens_200m_12 = [float(x) for x in mens_200m_12.mens_200m_12]


athlete_df = pd.merge(athlete_df, mens_200m_12[['name', 'mens_200m_12', 'event']], how='left', on=['name', 'event'], suffixes=(None, '_200m_mens_london'))


#### Scraping the results from the men's 200m dash 2016-Rio

In [231]:
rio_200m_men = fn.olympic_query('rio', '2016', '200m-men')
mens_200m_16 = fn.olympic_scraper(rio_200m_men)
mens_200m_16 = fn.content_cleaner(mens_200m_16)
mens_200m_16.columns = ['rank', 'name', 'mens_200m_16']
mens_200m_16.insert(3, "event", "Athletics Men's 200 metres")
mens_200m_16.mens_200m_16 = [str(x) for x in mens_200m_16.mens_200m_16]
mens_200m_16.mens_200m_16 = [x.strip() for x in mens_200m_16.mens_200m_16]
mens_200m_16.mens_200m_16 = [float(x) for x in mens_200m_16.mens_200m_16]
mens_200m_16.name = [x.strip() for x in mens_200m_16.name]


athlete_df = pd.merge(athlete_df, mens_200m_16[['name', 'mens_200m_16', 'event']], how='left', on=['name', 'event'], suffixes=(None, '_200m_mens_rio'))


#### Scraping the results from the men's 400m dash 2004-Athens

In [232]:
athens_400m_men = fn.olympic_query('athens', '2004', '400m-men')
mens_400m_04 = fn.olympic_scraper(athens_400m_men)
mens_400m_04 = fn.content_cleaner(mens_400m_04)
mens_400m_04.columns = ['rank', 'name', 'mens_400m_04']
mens_400m_04.insert(3, "event", "Athletics Men's 400 metres")
mens_400m_04.mens_400m_04 = [str(x) for x in mens_400m_04.mens_400m_04]
mens_400m_04.mens_400m_04 = [x.strip() for x in mens_400m_04.mens_400m_04]
mens_400m_04.mens_400m_04 = [float(x) for x in mens_400m_04.mens_400m_04]
mens_400m_04.name = [x.strip() for x in mens_400m_04.name]


athlete_df = pd.merge(athlete_df, mens_400m_04[['name', 'mens_400m_04', 'event']], how='left', on=['name', 'event'], suffixes=(None, '_400m_mens_athens'))


#### Scraping the results from the men's 400m dash 2008-Beijing
dropping rows with missing values

In [233]:
beijing_400m_men = fn.olympic_query('beijing', '2008', '400m-men')
mens_400m_08 = fn.olympic_scraper(beijing_400m_men)
mens_400m_08 = fn.content_cleaner(mens_400m_08)
mens_400m_08.columns = ['rank', 'name', 'mens_400m_08']
mens_400m_08.insert(3, "event", "Athletics Men's 400 metres")
mens_400m_08.mens_400m_08 = [str(x) for x in mens_400m_08.mens_400m_08]
mens_400m_08.mens_400m_08 = [x.strip() for x in mens_400m_08.mens_400m_08]
mens_400m_08.mens_400m_08 = [x[:5] for x in mens_400m_08.mens_400m_08]
mens_400m_08.name = [x.strip() for x in mens_400m_08.name]
mens_400m_08 = mens_400m_08.drop([113,114,122,146])
mens_400m_08.mens_400m_08 = [float(x) for x in mens_400m_08.mens_400m_08]


athlete_df = pd.merge(athlete_df, mens_400m_08[['name', 'mens_400m_08', 'event']], how='left', on=['name', 'event'], suffixes=(None, '_400m_mens_beijing'))


#### Scraping the results from the men's 400m dash 2012-London
dropping rows with missing values

In [234]:
london_400m_men = fn.olympic_query('london', '2012', '400m-men')
mens_400m_12 = fn.olympic_scraper(london_400m_men)
mens_400m_12 = fn.content_cleaner(mens_400m_12)
mens_400m_12.columns = ['rank', 'name', 'mens_400m_12']
mens_400m_12.insert(3, "event", "Athletics Men's 400 metres")
mens_400m_12.mens_400m_12 = [str(x) for x in mens_400m_12.mens_400m_12]
mens_400m_12.mens_400m_12 = [x.strip() for x in mens_400m_12.mens_400m_12]
mens_400m_12.name = [x.strip() for x in mens_400m_12.name]
mens_400m_12 = mens_400m_12.drop([30,78,79,80,81,82])
mens_400m_12.mens_400m_12 = [float(x) for x in mens_400m_12.mens_400m_12]


athlete_df = pd.merge(athlete_df, mens_400m_12[['name', 'mens_400m_12', 'event']], how='left', on=['name', 'event'], suffixes=(None, '_400m_mens_london'))


#### Scraping the results from the men's 400m dash 2016-Rio

In [235]:
rio_400m_men = fn.olympic_query('rio', '2016', '400m-men')
mens_400m_16 = fn.olympic_scraper(rio_400m_men)
mens_400m_16 = fn.content_cleaner(mens_400m_16)
mens_400m_16.columns = ['rank', 'name', 'mens_400m_16']
mens_400m_16.insert(3, "event", "Athletics Men's 400 metres")
mens_400m_16.mens_400m_16 = [str(x) for x in mens_400m_16.mens_400m_16]
mens_400m_16.mens_400m_16 = [x.strip() for x in mens_400m_16.mens_400m_16]
mens_400m_16.mens_400m_16 = [float(x) for x in mens_400m_16.mens_400m_16]
mens_400m_16.name = [x.strip() for x in mens_400m_16.name]


athlete_df = pd.merge(athlete_df, mens_400m_16[['name', 'mens_400m_16', 'event']], how='left', on=['name', 'event'], suffixes=(None, '_400m_mens_rio'))


#### Scraping the results from the men's 800m dash 2004-Athens

In [236]:
athens_800m_men = fn.olympic_query('athens', '2004', '800m-men')
mens_800m_04 = fn.olympic_scraper(athens_800m_men)
mens_800m_04 = fn.content_cleaner(mens_800m_04)
mens_800m_04.columns = ['rank', 'name', 'mens_800m_04']
mens_800m_04.insert(3, "event", "Athletics Men's 800 metres")
mens_800m_04.mens_800m_04 = [str(x) for x in mens_800m_04.mens_800m_04]
mens_800m_04.mens_800m_04 = [x.strip() for x in mens_800m_04.mens_800m_04]
mens_800m_04.name = [x.strip() for x in mens_800m_04.name]
mens_800m_04.mens_800m_04 = [x.replace('h','.')for x in mens_800m_04.mens_800m_04]
mens_800m_04.mens_800m_04 = [x.replace(':','.')for x in mens_800m_04.mens_800m_04]


athlete_df = pd.merge(athlete_df, mens_800m_04[['name', 'mens_800m_04', 'event']], how='left', on=['name', 'event'], suffixes=(None, '_800m_mens_athens'))


#### Scraping the results from the men's 800m dash 2008-Beijing
dropping rows with missing values

In [237]:
beijing_800m_men = fn.olympic_query('beijing', '2008', '800m-men')
mens_800m_08 = fn.olympic_scraper(beijing_800m_men)
mens_800m_08 = fn.content_cleaner(mens_800m_08)
mens_800m_08.columns = ['rank', 'name', 'mens_800m_08']
mens_800m_08.insert(3, "event", "Athletics Men's 800 metres")
mens_800m_08.mens_800m_08 = [str(x) for x in mens_800m_08.mens_800m_08]
mens_800m_08.mens_800m_08 = [x.strip() for x in mens_800m_08.mens_800m_08]
mens_800m_08.name = [x.strip() for x in mens_800m_08.name]
mens_800m_08 = mens_800m_08.drop([114,115,116,130,138,154])
mens_800m_08.mens_800m_08 = [x.replace('h','.')for x in mens_800m_08.mens_800m_08]
mens_800m_08.mens_800m_08 = [x.replace(':','.')for x in mens_800m_08.mens_800m_08]


athlete_df = pd.merge(athlete_df, mens_800m_08[['name', 'mens_800m_08', 'event']], how='left', on=['name', 'event'], suffixes=(None, '_800m_mens_beijing'))


#### Scraping the results from the men's 800m dash 2012-London
dropping rows with missing values

In [238]:
london_800m_men = fn.olympic_query('london', '2012', '800m-men')
mens_800m_12 = fn.olympic_scraper(london_800m_men)
mens_800m_12 = fn.content_cleaner(mens_800m_12)
mens_800m_12.columns = ['rank', 'name', 'mens_800m_12']
mens_800m_12.insert(3, "event", "Athletics Men's 800 metres")
mens_800m_12.mens_800m_12 = [str(x) for x in mens_800m_12.mens_800m_12]
mens_800m_12.mens_800m_12 = [x.strip() for x in mens_800m_12.mens_800m_12]
mens_800m_12.name = [x.strip() for x in mens_800m_12.name]
mens_800m_12.mens_800m_12 = [x.strip('/ DPG') for x in mens_800m_12.mens_800m_12]
mens_800m_12 = mens_800m_12.drop([32,84,85,86,87])
mens_800m_12.mens_800m_12 = [x.replace('h','.')for x in mens_800m_12.mens_800m_12]
mens_800m_12.mens_800m_12 = [x.replace(':','.')for x in mens_800m_12.mens_800m_12]


athlete_df = pd.merge(athlete_df, mens_800m_12[['name', 'mens_800m_12', 'event']], how='left', on=['name', 'event'], suffixes=(None, '_800m_mens_london'))


#### Scraping the results from the men's 800m dash 2016-Rio

In [239]:
rio_800m_men = fn.olympic_query('rio', '2016', '800m-men')
mens_800m_16 = fn.olympic_scraper(rio_800m_men)
mens_800m_16 = fn.content_cleaner(mens_800m_16)
mens_800m_16.columns = ['rank', 'name', 'mens_800m_16']
mens_800m_16.insert(3, "event", "Athletics Men's 800 metres")
mens_800m_16.mens_800m_16 = [str(x) for x in mens_800m_16.mens_800m_16]
mens_800m_16.mens_800m_16 = [x.strip() for x in mens_800m_16.mens_800m_16]
mens_800m_16.name = [x.strip() for x in mens_800m_16.name]
mens_800m_16.mens_800m_16 = [x.replace('h','.')for x in mens_800m_16.mens_800m_16]
mens_800m_16.mens_800m_16 = [x.replace(':','.')for x in mens_800m_16.mens_800m_16]


athlete_df = pd.merge(athlete_df, mens_800m_16[['name', 'mens_800m_16', 'event']], how='left', on=['name', 'event'], suffixes=(None, '_800m_mens_rio'))


#### Scraping the results from the men's 1500m dash 2004-Athens

In [240]:
athens_1500m_men = fn.olympic_query('athens', '2004', '1500m-men')
mens_1500m_04 = fn.olympic_scraper(athens_1500m_men)
mens_1500m_04 = fn.content_cleaner(mens_1500m_04)
mens_1500m_04.columns = ['rank', 'name', 'mens_1500m_04']
mens_1500m_04.insert(3, "event_x", "Athletics Men's 1500 metres")
mens_1500m_04.mens_1500m_04 = [str(x) for x in mens_1500m_04.mens_1500m_04]
mens_1500m_04.mens_1500m_04 = [x.strip() for x in mens_1500m_04.mens_1500m_04]
mens_1500m_04.mens_1500m_04 = [x.replace('h','.')for x in mens_1500m_04.mens_1500m_04]
mens_1500m_04.mens_1500m_04 = [x.replace(':','.')for x in mens_1500m_04.mens_1500m_04]
mens_1500m_04.name = [x.strip() for x in mens_1500m_04.name]



athlete_df = pd.merge(athlete_df, mens_1500m_04[['name', 'mens_1500m_04']], how='left', on='name', suffixes=(None, '_1500m_mens_athens'))


#### Scraping the results from the men's 1500m dash 2008-Beijing
dropping rows with missing values

In [241]:
beijing_1500m_men = fn.olympic_query('beijing', '2008', '1500m-men')
mens_1500m_08 = fn.olympic_scraper(beijing_1500m_men)
mens_1500m_08 = fn.content_cleaner(mens_1500m_08)
mens_1500m_08.columns = ['rank', 'name', 'mens_1500m_08']
mens_1500m_08.insert(3, "event_x", "Athletics Men's 1500 metres")
mens_1500m_08.mens_1500m_08 = [str(x) for x in mens_1500m_08.mens_1500m_08]
mens_1500m_08.mens_1500m_08 = [x.strip() for x in mens_1500m_08.mens_1500m_08]
mens_1500m_08.name = [x.strip() for x in mens_1500m_08.name]
mens_1500m_08.mens_1500m_08 = [x.strip('/ DPG') for x in mens_1500m_08.mens_1500m_08]
mens_1500m_08 = mens_1500m_08.drop([107,108,121,134])
mens_1500m_08.mens_1500m_08 = [x.replace('h','.')for x in mens_1500m_08.mens_1500m_08]
mens_1500m_08.mens_1500m_08 = [x.replace(':','.')for x in mens_1500m_08.mens_1500m_08]


athlete_df = pd.merge(athlete_df, mens_1500m_08[['name', 'mens_1500m_08']], how='left', on='name', suffixes=(None, '_1500m_mens_beijing'))


#### Scraping the results from the men's 1500m dash 2012-London
dropping row with missing value

In [242]:
london_1500m_men = fn.olympic_query('london', '2012', '1500m-men')
mens_1500m_12 = fn.olympic_scraper(london_1500m_men)
mens_1500m_12 = fn.content_cleaner(mens_1500m_12)
mens_1500m_12.columns = ['rank', 'name', 'mens_1500m_12']
mens_1500m_12.insert(3, "event_x", "Athletics Men's 1500 metres")
mens_1500m_12.mens_1500m_12 = [str(x) for x in mens_1500m_12.mens_1500m_12]
mens_1500m_12.mens_1500m_12 = [x.strip() for x in mens_1500m_12.mens_1500m_12]
mens_1500m_12.name = [x.strip() for x in mens_1500m_12.name]
mens_1500m_12.mens_1500m_12 = [x.strip('/ DPG') for x in mens_1500m_12.mens_1500m_12]
mens_1500m_12 = mens_1500m_12.drop([79])
mens_1500m_12.mens_1500m_12 = [x.replace('h','.')for x in mens_1500m_12.mens_1500m_12]
mens_1500m_12.mens_1500m_12 = [x.replace(':','.')for x in mens_1500m_12.mens_1500m_12]


athlete_df = pd.merge(athlete_df, mens_1500m_12[['name', 'mens_1500m_12']], how='left', on='name', suffixes=(None, '_1500m_mens_london'))


#### Scraping the results from the men's 1500m dash 2016-Rio

In [243]:
rio_1500m_men = fn.olympic_query('rio', '2016', '1500m-men')
mens_1500m_16 = fn.olympic_scraper(rio_1500m_men)
mens_1500m_16 = fn.content_cleaner(mens_1500m_16)
mens_1500m_16.columns = ['rank', 'name', 'mens_1500m_16']
mens_1500m_16.insert(3, "event_x", "Athletics Men's 1500 metres")
mens_1500m_16.mens_1500m_16 = [str(x) for x in mens_1500m_16.mens_1500m_16]
mens_1500m_16.mens_1500m_16 = [x.strip() for x in mens_1500m_16.mens_1500m_16]
mens_1500m_16.name = [x.strip() for x in mens_1500m_16.name]
mens_1500m_16.mens_1500m_16 = [x.replace('h','.')for x in mens_1500m_16.mens_1500m_16]
mens_1500m_16.mens_1500m_16 = [x.replace(':','.')for x in mens_1500m_16.mens_1500m_16]


athlete_df = pd.merge(athlete_df, mens_1500m_16[['name', 'mens_1500m_16']], how='left', on='name', suffixes=(None, '_1500m_mens_rio'))


#### Scraping the results from the men's 5000m dash 2004-Athens
dropping row with missing value

In [244]:
athens_5000m_men = fn.olympic_query('athens', '2004', '5000m-men')
mens_5000m_04 = fn.olympic_scraper(athens_5000m_men)
mens_5000m_04 = fn.content_cleaner(mens_5000m_04)
mens_5000m_04.columns = ['rank', 'name', 'mens_5000m_04']
mens_5000m_04.insert(3, "event_x", "Athletics Men's 5000 metres")
mens_5000m_04.mens_5000m_04 = [str(x) for x in mens_5000m_04.mens_5000m_04]
mens_5000m_04.mens_5000m_04 = [x.strip() for x in mens_5000m_04.mens_5000m_04]
mens_5000m_04.name = [x.strip() for x in mens_5000m_04.name]
mens_5000m_04 = mens_5000m_04.drop(35)
mens_5000m_04.mens_5000m_04 = [x.replace('h','.')for x in mens_5000m_04.mens_5000m_04]
mens_5000m_04.mens_5000m_04 = [x.replace(':','.')for x in mens_5000m_04.mens_5000m_04]



athlete_df = pd.merge(athlete_df, mens_5000m_04[['name', 'mens_5000m_04']], how='left', on='name', suffixes=(None, '_5000m_mens_athens'))


#### Scraping the results from the men's 5000m dash 2008-Beijing
dropping rows with missing values

In [245]:
beijing_5000m_men = fn.olympic_query('beijing', '2008', '5000m-men')
mens_5000m_08 = fn.olympic_scraper(beijing_5000m_men)
mens_5000m_08 = fn.content_cleaner(mens_5000m_08)
mens_5000m_08.columns = ['rank', 'name', 'mens_5000m_08']
mens_5000m_08.insert(3, "event_x", "Athletics Men's 5000 metres")
mens_5000m_08.mens_5000m_08 = [str(x) for x in mens_5000m_08.mens_5000m_08]
mens_5000m_08.mens_5000m_08 = [x.strip() for x in mens_5000m_08.mens_5000m_08]
mens_5000m_08.name = [x.strip() for x in mens_5000m_08.name]
mens_5000m_08.mens_5000m_08 = [x.strip('/ DPG') for x in mens_5000m_08.mens_5000m_08]
mens_5000m_08 = mens_5000m_08.drop([14,54,55,56,57,85,98,99,100])
mens_5000m_08.mens_5000m_08 = [x.replace('h','.')for x in mens_5000m_08.mens_5000m_08]
mens_5000m_08.mens_5000m_08 = [x.replace(':','.')for x in mens_5000m_08.mens_5000m_08]


athlete_df = pd.merge(athlete_df, mens_5000m_08[['name', 'mens_5000m_08']], how='left', on='name', suffixes=(None, '_5000m_mens_beijing'))


#### Scraping the results from the men's 5000m dash 2012-London
dropping row with missing value

In [246]:
london_5000m_men = fn.olympic_query('london', '2012', '5000m-men')
mens_5000m_12 = fn.olympic_scraper(london_5000m_men)
mens_5000m_12 = fn.content_cleaner(mens_5000m_12)
mens_5000m_12.columns = ['rank', 'name', 'mens_5000m_12']
mens_5000m_12.insert(3, "event_x", "Athletics Men's 5000 metres")
mens_5000m_12.mens_5000m_12 = [str(x) for x in mens_5000m_12.mens_5000m_12]
mens_5000m_12.mens_5000m_12 = [x.strip() for x in mens_5000m_12.mens_5000m_12]
mens_5000m_12.name = [x.strip() for x in mens_5000m_12.name]
mens_5000m_12.mens_5000m_12 = [x.strip('/ DPG') for x in mens_5000m_12.mens_5000m_12]
mens_5000m_12 = mens_5000m_12.drop([56])
mens_5000m_12.mens_5000m_12 = [x.replace('h','.')for x in mens_5000m_12.mens_5000m_12]
mens_5000m_12.mens_5000m_12 = [x.replace(':','.')for x in mens_5000m_12.mens_5000m_12]


athlete_df = pd.merge(athlete_df, mens_5000m_12[['name', 'mens_5000m_12']], how='left', on='name', suffixes=(None, '_5000m_mens_london'))


#### Scraping the results from the men's 5000m dash 2016-Rio
dropping rows with missing values

In [247]:
rio_5000m_men = fn.olympic_query('rio', '2016', '5000m-men')
mens_5000m_16 = fn.olympic_scraper(rio_5000m_men)
mens_5000m_16 = fn.content_cleaner(mens_5000m_16)
mens_5000m_16.columns = ['rank', 'name', 'mens_5000m_16']
mens_5000m_16.insert(3, "event_x", "Athletics Men's 5000 metres")
mens_5000m_16.mens_5000m_16 = [str(x) for x in mens_5000m_16.mens_5000m_16]
mens_5000m_16.mens_5000m_16 = [x.strip() for x in mens_5000m_16.mens_5000m_16]
mens_5000m_16.name = [x.strip() for x in mens_5000m_16.name]
mens_5000m_16.mens_5000m_16 = [x.strip('DPG/') for x in mens_5000m_16.mens_5000m_16]
mens_5000m_16 = mens_5000m_16.drop([15,66])
mens_5000m_16.mens_5000m_16 = [x.replace('h','.')for x in mens_5000m_16.mens_5000m_16]
mens_5000m_16.mens_5000m_16 = [x.replace(':','.')for x in mens_5000m_16.mens_5000m_16]


athlete_df = pd.merge(athlete_df, mens_5000m_16[['name', 'mens_5000m_16']], how='left', on='name', suffixes=(None, '_5000m_mens_rio'))


#### Scraping the results from the men's 110m hurdles 2004-Athens
dropping row with missing value

In [248]:
athens_110H_men = fn.olympic_query('athens', '2004', '110m-hurdles-men')
mens_110H_04 = fn.olympic_scraper(athens_110H_men)
mens_110H_04 = fn.content_cleaner(mens_110H_04)
mens_110H_04.columns = ['rank', 'name', 'mens_110H_04']
mens_110H_04.insert(3, "event_x", "Athletics Men's 110m Hurdles")
mens_110H_04.mens_110H_04 = [str(x) for x in mens_110H_04.mens_110H_04]
mens_110H_04.mens_110H_04 = [x.strip() for x in mens_110H_04.mens_110H_04]
mens_110H_04.mens_110H_04 = [x.strip('/ DPG') for x in mens_110H_04.mens_110H_04]
mens_110H_04.name = [x.strip() for x in mens_110H_04.name]
mens_110H_04 = mens_110H_04.drop(15)
mens_110H_04.mens_110H_04 = mens_110H_04.mens_110H_04.astype(float)


athlete_df = pd.merge(athlete_df, mens_110H_04[['name', 'mens_110H_04']], how='left', on='name', suffixes=(None, '_110H_mens_athens'))


#### Scraping the results from the men's 110m hurdles 2008-Beijing
dropping rows with missing values

In [249]:
beijing_110H_men = fn.olympic_query('beijing', '2008', '110m-hurdles-men')
mens_110H_08 = fn.olympic_scraper(beijing_110H_men)
mens_110H_08 = fn.content_cleaner(mens_110H_08)
mens_110H_08.columns = ['rank', 'name', 'mens_110H_08']
mens_110H_08.insert(3, "event_x", "Athletics Men's 110m Hurdles")
mens_110H_08.mens_110H_08 = [str(x) for x in mens_110H_08.mens_110H_08]
mens_110H_08.mens_110H_08 = [x[:5] for x in mens_110H_08.mens_110H_08]
mens_110H_08.mens_110H_08 = [x.strip() for x in mens_110H_08.mens_110H_08]
mens_110H_08.mens_110H_08 = [x.strip('/ DPG') for x in mens_110H_08.mens_110H_08]
mens_110H_08.name = [x.strip() for x in mens_110H_08.name]
mens_110H_08 = mens_110H_08.drop([70,71,95,103,144,
                                 145,146,181,182,189])
mens_110H_08.mens_110H_08 = [float(x) for x in mens_110H_08.mens_110H_08]


athlete_df = pd.merge(athlete_df, mens_110H_08[['name', 'mens_110H_08']], how='left', on='name', suffixes=(None, '_110H_mens_beijing'))


#### Scraping the results from the men's 110m hurdles 2012-London
dropping rows with missing values

In [250]:
london_110H_men = fn.olympic_query('london', '2012', '110m-hurdles-men')
mens_110H_12 = fn.olympic_scraper(london_110H_men)
mens_110H_12 = fn.content_cleaner(mens_110H_12)
mens_110H_12.columns = ['rank', 'name', 'mens_110H_12']
mens_110H_12.insert(3, "event_x", "Athletics Men's 110m Hurdles")
mens_110H_12.mens_110H_12 = [str(x) for x in mens_110H_12.mens_110H_12]
mens_110H_12.mens_110H_12 = [x[:7] for x in mens_110H_12.mens_110H_12]
mens_110H_12.mens_110H_12 = [x.strip() for x in mens_110H_12.mens_110H_12]
mens_110H_12.mens_110H_12 = [x.strip('/ DPG') for x in mens_110H_12.mens_110H_12]
mens_110H_12.name = [x.strip() for x in mens_110H_12.name]
mens_110H_12 = mens_110H_12.drop([7,31,32,78,79,80,81,
                                 82,83,84,85,86])


athlete_df = pd.merge(athlete_df, mens_110H_12[['name', 'mens_110H_12']], how='left', on='name', suffixes=(None, '_110H_mens_london'))


#### Scraping the results from the men's 110m hurdles 2016-Rio
dropping row with missing value

In [251]:
rio_110H_men = fn.olympic_query('rio', '2016', '110m-hurdles-men')
mens_110H_16 = fn.olympic_scraper(rio_110H_men)
mens_110H_16 = fn.content_cleaner(mens_110H_16)
mens_110H_16.columns = ['rank', 'name', 'mens_110H_16']
mens_110H_16.insert(3, "event_x", "Athletics Men's 110m Hurdles")
mens_110H_16.mens_110H_16 = [str(x) for x in mens_110H_16.mens_110H_16]
mens_110H_16.mens_110H_16 = [x[:7] for x in mens_110H_16.mens_110H_16]
mens_110H_16.mens_110H_16 = [x.strip() for x in mens_110H_16.mens_110H_16]
mens_110H_16.mens_110H_16 = [x.strip('/ DPG') for x in mens_110H_16.mens_110H_16]
mens_110H_16.name = [x.strip() for x in mens_110H_16.name]
mens_110H_16 = mens_110H_16.drop(7)


athlete_df = pd.merge(athlete_df, mens_110H_16[['name', 'mens_110H_16']], how='left', on='name', suffixes=(None, '_110H_mens_rio'))


#### Scraping the results from the men's 400m hurdles 2004-Athens

In [252]:
athens_400H_men = fn.olympic_query('athens', '2004', '400m-hurdles-men')
mens_400H_04 = fn.olympic_scraper(athens_400H_men)
mens_400H_04 = fn.content_cleaner(mens_400H_04)
mens_400H_04.columns = ['rank', 'name', 'mens_400H_04']
mens_400H_04.insert(3, "event_x", "Athletics Men's 400m Hurdles")
mens_400H_04.mens_400H_04 = [str(x) for x in mens_400H_04.mens_400H_04]
mens_400H_04.mens_400H_04 = [x.strip() for x in mens_400H_04.mens_400H_04]
mens_400H_04.mens_400H_04 = [x.strip('/ DPG') for x in mens_400H_04.mens_400H_04]
mens_400H_04.name = [x.strip() for x in mens_400H_04.name]


athlete_df = pd.merge(athlete_df, mens_400H_04[['name', 'mens_400H_04']], how='left', on='name', suffixes=(None, '_400H_mens_athens'))


#### Scraping the results from the men's 400m hurdles 2008-Beijing
dropping rows with missing values

In [253]:
beijing_400H_men = fn.olympic_query('beijing', '2008', '400m-hurdles-men')
mens_400H_08 = fn.olympic_scraper(beijing_400H_men)
mens_400H_08 = fn.content_cleaner(mens_400H_08)
mens_400H_08.columns = ['rank', 'name', 'mens_400H_08']
mens_400H_08.insert(3, "event_x", "Athletics Men's 400m Hurdles")
mens_400H_08.mens_400H_08 = [str(x) for x in mens_400H_08.mens_400H_08]
mens_400H_08.mens_400H_08 = [x[:7] for x in mens_400H_08.mens_400H_08]
mens_400H_08.mens_400H_08 = [x.strip() for x in mens_400H_08.mens_400H_08]
mens_400H_08.mens_400H_08 = [x.strip('/ DPG') for x in mens_400H_08.mens_400H_08]
mens_400H_08.name = [x.strip() for x in mens_400H_08.name]
mens_400H_08 = mens_400H_08.drop([65,91])


athlete_df = pd.merge(athlete_df, mens_400H_08[['name', 'mens_400H_08']], how='left', on='name', suffixes=(None, '_400H_mens_beijing'))


#### Scraping the results from the men's 400m hurdles 2012-London
dropping rows with missing values

In [254]:
london_400H_men = fn.olympic_query('london', '2012', '400m-hurdles-men')
mens_400H_12 = fn.olympic_scraper(london_400H_men)
mens_400H_12 = fn.content_cleaner(mens_400H_12)
mens_400H_12.columns = ['rank', 'name', 'mens_400H_12']
mens_400H_12.insert(3, "event_x", "Athletics Men's 400m Hurdles")
mens_400H_12.mens_400H_12 = [str(x) for x in mens_400H_12.mens_400H_12]
mens_400H_12.mens_400H_12 = [x[:7] for x in mens_400H_12.mens_400H_12]
mens_400H_12.mens_400H_12 = [x.strip() for x in mens_400H_12.mens_400H_12]
mens_400H_12.mens_400H_12 = [x.strip('/ DPG') for x in mens_400H_12.mens_400H_12]
mens_400H_12.name = [x.strip() for x in mens_400H_12.name]
mens_400H_12 = mens_400H_12.drop([30,31,78,79,80,81])


athlete_df = pd.merge(athlete_df, mens_400H_12[['name', 'mens_400H_12']], how='left', on='name', suffixes=(None, '_400H_mens_london'))


#### Scraping the results from the men's 400m hurdles 2016-Rio
dropping row with missing value

In [255]:
rio_400H_men = fn.olympic_query('rio', '2016', '400m-hurdles-men')
mens_400H_16 = fn.olympic_scraper(rio_400H_men)
mens_400H_16 = fn.content_cleaner(mens_400H_16)
mens_400H_16.columns = ['rank', 'name', 'mens_400H_16']
mens_400H_16.insert(3, "event_x", "Athletics Men's 400m Hurdles")
mens_400H_16.mens_400H_16 = [str(x) for x in mens_400H_16.mens_400H_16]
mens_400H_16.mens_400H_16 = [x[:7] for x in mens_400H_16.mens_400H_16]
mens_400H_16.mens_400H_16 = [x.strip() for x in mens_400H_16.mens_400H_16]
mens_400H_16.mens_400H_16 = [x.strip('/ DPG') for x in mens_400H_16.mens_400H_16]
mens_400H_16.name = [x.strip() for x in mens_400H_16.name]
mens_400H_16 = mens_400H_16.drop(7)

athlete_df = pd.merge(athlete_df, mens_400H_16[['name', 'mens_400H_16']], how='left', on='name', suffixes=(None, '_400H_mens_rio'))


#### Scraping the results from the men's 10000m run 2004-Athens

In [256]:
athens_10000m_men = fn.olympic_query('athens', '2004', '10000m-men')
mens_10000m_04 = fn.olympic_scraper(athens_10000m_men)
mens_10000m_04 = fn.content_cleaner(mens_10000m_04)
mens_10000m_04.columns = ['rank', 'name', 'mens_10000m_04']
mens_10000m_04.insert(3, "event_x", "Athletics Men's 10000 metres")
mens_10000m_04.mens_10000m_04 = [str(x) for x in mens_10000m_04.mens_10000m_04]
mens_10000m_04.mens_10000m_04 = [x.strip() for x in mens_10000m_04.mens_10000m_04]
mens_10000m_04.mens_10000m_04 = [x.replace('h','.')for x in mens_10000m_04.mens_10000m_04]
mens_10000m_04.mens_10000m_04 = [x.replace(':','.')for x in mens_10000m_04.mens_10000m_04]
mens_10000m_04.name = [x.strip() for x in mens_10000m_04.name]


athlete_df = pd.merge(athlete_df, mens_10000m_04[['name', 'mens_10000m_04']], how='left', on='name', suffixes=(None, '_10000m_mens_athens'))


#### Scraping the results from the men's 10000m run 2008-Beijing
dropping rows with missing values

In [257]:
beijing_10000m_men = fn.olympic_query('beijing', '2008', '10000m-men')
mens_10000m_08 = fn.olympic_scraper(beijing_10000m_men)
mens_10000m_08 = fn.content_cleaner(mens_10000m_08)
mens_10000m_08.columns = ['rank', 'name', 'mens_10000m_08']
mens_10000m_08.insert(3, "event_x", "Athletics Men's 10000 metres")
mens_10000m_08.mens_10000m_08 = [str(x) for x in mens_10000m_08.mens_10000m_08]
mens_10000m_08.mens_10000m_08 = [x.strip() for x in mens_10000m_08.mens_10000m_08]
mens_10000m_08.name = [x.strip() for x in mens_10000m_08.name]
mens_10000m_08.mens_10000m_08 = [x.strip('/ DPG') for x in mens_10000m_08.mens_10000m_08]
mens_10000m_08 = mens_10000m_08.drop([35,36,37,38])
mens_10000m_08.mens_10000m_08 = [x.replace('h','.')for x in mens_10000m_08.mens_10000m_08]
mens_10000m_08.mens_10000m_08 = [x.replace(':','.')for x in mens_10000m_08.mens_10000m_08]


athlete_df = pd.merge(athlete_df, mens_10000m_08[['name', 'mens_10000m_08']], how='left', on='name', suffixes=(None, '_10000m_mens_beijing'))


#### Scraping the results from the men's 10000m run 2012-London
dropping rows with missing values

In [258]:
london_10000m_men = fn.olympic_query('london', '2012', '10000m-men')
mens_10000m_12 = fn.olympic_scraper(london_10000m_men)
mens_10000m_12 = fn.content_cleaner(mens_10000m_12)
mens_10000m_12.columns = ['rank', 'name', 'mens_10000m_12']
mens_10000m_12.insert(3, "event_x", "Athletics Men's 10000 metres")
mens_10000m_12.mens_10000m_12 = [str(x) for x in mens_10000m_12.mens_10000m_12]
mens_10000m_12.mens_10000m_12 = [x.strip() for x in mens_10000m_12.mens_10000m_12]
mens_10000m_12.name = [x.strip() for x in mens_10000m_12.name]
mens_10000m_12 = mens_10000m_12.drop([26,27,28])
mens_10000m_12.mens_10000m_12 = [x.replace('h','.')for x in mens_10000m_12.mens_10000m_12]
mens_10000m_12.mens_10000m_12 = [x.replace(':','.')for x in mens_10000m_12.mens_10000m_12]


athlete_df = pd.merge(athlete_df, mens_10000m_12[['name', 'mens_10000m_12']], how='left', on='name', suffixes=(None, '_10000m_mens_london'))


#### Scraping the results from the men's 1000m run 2016-Rio
dropping rows with missing values

In [259]:
rio_10000m_men = fn.olympic_query('rio', '2016', '10000m-men')
mens_10000m_16 = fn.olympic_scraper(rio_10000m_men)
mens_10000m_16 = fn.content_cleaner(mens_10000m_16)
mens_10000m_16.columns = ['rank', 'name', 'mens_10000m_16']
mens_10000m_16.insert(3, "event_x", "Athletics Men's 10000 metres")
mens_10000m_16.mens_10000m_16 = [str(x) for x in mens_10000m_16.mens_10000m_16]
mens_10000m_16.mens_10000m_16 = [x.strip() for x in mens_10000m_16.mens_10000m_16]
mens_10000m_16.name = [x.strip() for x in mens_10000m_16.name]
mens_10000m_16 = mens_10000m_16.drop([32,33])
mens_10000m_16.mens_10000m_16 = [x.replace('h','.')for x in mens_10000m_16.mens_10000m_16]
mens_10000m_16.mens_10000m_16 = [x.replace(':','.')for x in mens_10000m_16.mens_10000m_16]


athlete_df = pd.merge(athlete_df, mens_10000m_16[['name', 'mens_10000m_16']], how='left', on='name', suffixes=(None, '_10000m_mens_rio'))


#### Scraping the results from the men's 20km walk 2004-Athens


In [260]:
athens_mens_20km = fn.olympic_query('athens', '2004', '20km-walk-men')
mens_20km = fn.olympic_scraper(athens_mens_20km)
mens_20km = fn.content_cleaner(mens_20km)
mens_20km.columns = ['rank', 'name', 'mens_20km']
mens_20km.insert(3, "event_x", "Athletics Men's 20km walk")
mens_20km.mens_20km = [str(x) for x in mens_20km.mens_20km]
mens_20km.mens_20km = [x.strip() for x in mens_20km.mens_20km]
mens_20km.mens_20km = [x.strip('/ DPG') for x in mens_20km.mens_20km]
mens_20km.mens_20km = [x.replace('h','.')for x in mens_20km.mens_20km]
mens_20km.mens_20km = [x.replace(':','.')for x in mens_20km.mens_20km]
mens_20km.name = [x.strip() for x in mens_20km.name]


athlete_df = pd.merge(athlete_df, mens_20km[['name', 'mens_20km']], how='left', on='name', suffixes=(None, '_20km_mens_athens'))


#### Scraping the results from the men's 20km walk 2008-Beijing
dropping rows with missing values

In [261]:
beijing_mens_20km = fn.olympic_query('beijing', '2008', '20km-walk-men')
mens_20km_08 = fn.olympic_scraper(beijing_mens_20km)
mens_20km_08 = fn.content_cleaner(mens_20km_08)
mens_20km_08.columns = ['rank', 'name', 'mens_20km_08']
mens_20km_08.insert(3, "event_x", "Athletics Men's 20km walk")
mens_20km_08.mens_20km_08 = [str(x) for x in mens_20km_08.mens_20km_08]
mens_20km_08.mens_20km_08 = [x.strip() for x in mens_20km_08.mens_20km_08]
mens_20km_08.mens_20km_08 = [x.strip('/ DPG') for x in mens_20km_08.mens_20km_08]
mens_20km_08.mens_20km_08 = [x.replace('h','.')for x in mens_20km_08.mens_20km_08]
mens_20km_08.mens_20km_08 = [x.replace(':','.')for x in mens_20km_08.mens_20km_08]
mens_20km_08.name = [x.strip() for x in mens_20km_08.name]
mens_20km_08 = mens_20km_08.drop([49,50])


athlete_df = pd.merge(athlete_df, mens_20km_08[['name', 'mens_20km_08']], how='left', on='name', suffixes=(None, '_20km_mens_beijing'))


#### Scraping the results from the men's 20km walk 2012-London
dropping rows with missing values

In [262]:
london_mens_20km = fn.olympic_query('london', '2012', '20km-walk-men')
mens_20km_12 = fn.olympic_scraper(london_mens_20km)
mens_20km_12 = fn.content_cleaner(mens_20km_12)
mens_20km_12.columns = ['rank', 'name', 'mens_20km_12']
mens_20km_12.insert(3, "event_x", "Athletics Men's 20km walk")
mens_20km_12.mens_20km_12 = [str(x) for x in mens_20km_12.mens_20km_12]
mens_20km_12.mens_20km_12 = [x.strip() for x in mens_20km_12.mens_20km_12]
mens_20km_12.mens_20km_12 = [x.strip('/ DPG') for x in mens_20km_12.mens_20km_12]
mens_20km_12.mens_20km_12 = [x.replace('h','.')for x in mens_20km_12.mens_20km_12]
mens_20km_12.mens_20km_12 = [x.replace(':','.')for x in mens_20km_12.mens_20km_12]
mens_20km_12.name = [x.strip() for x in mens_20km_12.name]
mens_20km_12 = mens_20km_12.drop([48,49,50,51,52,53,54,55])


athlete_df = pd.merge(athlete_df, mens_20km_12[['name', 'mens_20km_12']], how='left', on='name', suffixes=(None, '_20km_mens_london'))


#### Scraping the results from the men's 20km walk 2016-Rio
dropping rows with missing values

In [263]:
rio_mens_20km = fn.olympic_query('rio', '2016', '20km-walk-men')
mens_20km_16 = fn.olympic_scraper(rio_mens_20km)
mens_20km_16 = fn.content_cleaner(mens_20km_16)
mens_20km_16.columns = ['rank', 'name', 'mens_20km_16']
mens_20km_16.insert(3, "event_x", "Athletics Men's 20km walk")
mens_20km_16.mens_20km_16 = [str(x) for x in mens_20km_16.mens_20km_16]
mens_20km_16.mens_20km_16 = [x.strip() for x in mens_20km_16.mens_20km_16]
mens_20km_16.mens_20km_16 = [x.strip('/ DPG') for x in mens_20km_16.mens_20km_16]
mens_20km_16.mens_20km_16 = [x.replace('h','.')for x in mens_20km_16.mens_20km_16]
mens_20km_16.mens_20km_16 = [x.replace(':','.')for x in mens_20km_16.mens_20km_16]
mens_20km_16.name = [x.strip() for x in mens_20km_16.name]
mens_20km_16 = mens_20km_16.drop(range(63,74))


athlete_df = pd.merge(athlete_df, mens_20km_16[['name', 'mens_20km_16']], how='left', on='name', suffixes=(None, '_20km_mens_rio'))


#### Scraping the results from the men's 50km walk 2004-Athens


In [264]:
athens_mens_50km = fn.olympic_query('athens', '2004', '50km-walk-men')
mens_50km_04 = fn.olympic_scraper(athens_mens_50km)
mens_50km_04 = fn.content_cleaner(mens_50km_04)
mens_50km_04.columns = ['rank', 'name', 'mens_50km_04']
mens_50km_04.insert(3, "event_x", "Athletics Men's 50km Walk")
mens_50km_04.mens_50km_04 = [str(x) for x in mens_50km_04.mens_50km_04]
mens_50km_04.mens_50km_04 = [x.strip() for x in mens_50km_04.mens_50km_04]
mens_50km_04.mens_50km_04 = [x.strip('/ DPG') for x in mens_50km_04.mens_50km_04]
mens_50km_04.mens_50km_04 = [x.replace('h','.')for x in mens_50km_04.mens_50km_04]
mens_50km_04.mens_50km_04 = [x.replace(':','.')for x in mens_50km_04.mens_50km_04]
mens_50km_04.name = [x.strip() for x in mens_50km_04.name]


athlete_df = pd.merge(athlete_df, mens_50km_04[['name', 'mens_50km_04']], how='left', on='name', suffixes=(None, '_50km_mens_athens'))


#### Scraping the results from the men's 50km walk 2008-Beijing
dropping rows with missing values

In [265]:
beijing_mens_50km = fn.olympic_query('beijing', '2008', '50km-walk-men')
mens_50km_08 = fn.olympic_scraper(beijing_mens_50km)
mens_50km_08 = fn.content_cleaner(mens_50km_08)
mens_50km_08.columns = ['rank', 'name', 'mens_50km_08']
mens_50km_08.insert(3, "event_x", "Athletics Men's 50km walk")
mens_50km_08.mens_50km_08 = [str(x) for x in mens_50km_08.mens_50km_08]
mens_50km_08.mens_50km_08 = [x.strip() for x in mens_50km_08.mens_50km_08]
mens_50km_08.mens_50km_08 = [x.strip('/ DPG') for x in mens_50km_08.mens_50km_08]
mens_50km_08.mens_50km_08 = [x.replace('h','.')for x in mens_50km_08.mens_50km_08]
mens_50km_08.mens_50km_08 = [x.replace(':','.')for x in mens_50km_08.mens_50km_08]
mens_50km_08.name = [x.strip() for x in mens_50km_08.name]
mens_50km_08 = mens_50km_08.drop(range(47,61))


athlete_df = pd.merge(athlete_df, mens_50km_08[['name', 'mens_50km_08']], how='left', on='name', suffixes=(None, '_50km_mens_beijing'))


#### Scraping the results from the men's 50km walk 2012-London
dropping rows with missing values

In [266]:
london_mens_50km = fn.olympic_query('london', '2012', '50km-walk-men')
mens_50km_12 = fn.olympic_scraper(london_mens_50km)
mens_50km_12 = fn.content_cleaner(mens_50km_12)
mens_50km_12.columns = ['rank', 'name', 'mens_50km_12']
mens_50km_12.insert(3, "event_x", "Athletics Men's 50km walk")
mens_50km_12.mens_50km_12 = [str(x) for x in mens_50km_12.mens_50km_12]
mens_50km_12.mens_50km_12 = [x.strip() for x in mens_50km_12.mens_50km_12]
mens_50km_12.mens_50km_12 = [x.strip('/ DPG') for x in mens_50km_12.mens_50km_12]
mens_50km_12.mens_50km_12 = [x.replace('h','.')for x in mens_50km_12.mens_50km_12]
mens_50km_12.mens_50km_12 = [x.replace(':','.')for x in mens_50km_12.mens_50km_12]
mens_50km_12.name = [x.strip() for x in mens_50km_12.name]
mens_50km_12 = mens_50km_12.drop(range(48,60))


athlete_df = pd.merge(athlete_df, mens_50km_12[['name', 'mens_50km_12']], how='left', on='name', suffixes=(None, '_50km_mens_london'))


#### Scraping the results from the men's 50km walk 2016-Rio
dropping rows with missing values

In [267]:
rio_mens_50km = fn.olympic_query('rio', '2016', '50km-walk-men')
mens_50km_16 = fn.olympic_scraper(rio_mens_50km)
mens_50km_16 = fn.content_cleaner(mens_50km_16)
mens_50km_16.columns = ['rank', 'name', 'mens_50km_16']
mens_50km_16.insert(3, "event_x", "Athletics Men's 50km walk")
mens_50km_16.mens_50km_16 = [str(x) for x in mens_50km_16.mens_50km_16]
mens_50km_16.mens_50km_16 = [x.strip() for x in mens_50km_16.mens_50km_16]
mens_50km_16.mens_50km_16 = [x.strip('/ DPG') for x in mens_50km_16.mens_50km_16]
mens_50km_16.mens_50km_16 = [x.replace('h','.')for x in mens_50km_16.mens_50km_16]
mens_50km_16.mens_50km_16 = [x.replace(':','.')for x in mens_50km_16.mens_50km_16]
mens_50km_16.name = [x.strip() for x in mens_50km_16.name]
mens_50km_16 = mens_50km_16.drop(range(49,80))

athlete_df = pd.merge(athlete_df, mens_50km_16[['name', 'mens_50km_16']], how='left', on='name', suffixes=(None, '_50km_mens_Rio'))


#### Scraping the results from the men's 3000 Steeple-Chase  2004-Athens


In [268]:
athens_mens_3000Steeple_04 = fn.olympic_query('athens', '2004', '3000m-steeplechase-men')
mens_3000Steeple_04 = fn.olympic_scraper(athens_mens_3000Steeple_04)
mens_3000Steeple_04 = fn.content_cleaner(mens_3000Steeple_04)
mens_3000Steeple_04.columns = ['rank', 'name', 'mens_3000Steeple_04']
mens_3000Steeple_04.insert(3, "event_x", "Athletics Men's 3000Steeple_04")
mens_3000Steeple_04.mens_3000Steeple_04 = [str(x) for x in mens_3000Steeple_04.mens_3000Steeple_04]
mens_3000Steeple_04.mens_3000Steeple_04 = [x.strip() for x in mens_3000Steeple_04.mens_3000Steeple_04]
mens_3000Steeple_04.mens_3000Steeple_04 = [x.strip('/ DPG') for x in mens_3000Steeple_04.mens_3000Steeple_04]
mens_3000Steeple_04.mens_3000Steeple_04 = [x.replace('h','.')for x in mens_3000Steeple_04.mens_3000Steeple_04]
mens_3000Steeple_04.mens_3000Steeple_04 = [x.replace(':','.')for x in mens_3000Steeple_04.mens_3000Steeple_04]
mens_3000Steeple_04.name = [x.strip() for x in mens_3000Steeple_04.name]


athlete_df = pd.merge(athlete_df, mens_3000Steeple_04[['name', 'mens_3000Steeple_04']], how='left', on='name', suffixes=(None, '_3000Steeple_mens_athens'))


#### Scraping the results from the men's 3000 Steeple-Chase  2008-Beijing


In [269]:
beijing_mens_3000Steeple_08 = fn.olympic_query('beijing', '2008', '3000m-steeplechase-men')
mens_3000Steeple_08 = fn.olympic_scraper(beijing_mens_3000Steeple_08)
mens_3000Steeple_08 = fn.content_cleaner(mens_3000Steeple_08)
mens_3000Steeple_08.columns = ['rank', 'name', 'mens_3000Steeple_08']
mens_3000Steeple_08.insert(3, "event_x", "Athletics Men's 3000Steeple_08")
mens_3000Steeple_08.mens_3000Steeple_08 = [str(x) for x in mens_3000Steeple_08.mens_3000Steeple_08]
mens_3000Steeple_08.mens_3000Steeple_08 = [x.strip() for x in mens_3000Steeple_08.mens_3000Steeple_08]
mens_3000Steeple_08.mens_3000Steeple_08 = [x.strip('/ DPG') for x in mens_3000Steeple_08.mens_3000Steeple_08]
mens_3000Steeple_08.mens_3000Steeple_08 = [x.replace('h','.')for x in mens_3000Steeple_08.mens_3000Steeple_08]
mens_3000Steeple_08.mens_3000Steeple_08 = [x.replace(':','.')for x in mens_3000Steeple_08.mens_3000Steeple_08]
mens_3000Steeple_08.name = [x.strip() for x in mens_3000Steeple_08.name]


athlete_df = pd.merge(athlete_df, mens_3000Steeple_08[['name', 'mens_3000Steeple_08']], how='left', on='name', suffixes=(None, '_3000Steeple_mens_beijing'))


#### Scraping the results from the men's 3000 Steeple-Chase  2012-London
dropping rows with missing values

In [270]:
london_mens_3000Steeple_12 = fn.olympic_query('london', '2012', '3000m-steeplechase-men')
mens_3000Steeple_12 = fn.olympic_scraper(london_mens_3000Steeple_12)
mens_3000Steeple_12 = fn.content_cleaner(mens_3000Steeple_12)
mens_3000Steeple_12.columns = ['rank', 'name', 'mens_3000Steeple_12']
mens_3000Steeple_12.insert(3, "event_x", "Athletics Men's 3000Steeple_12")
mens_3000Steeple_12.mens_3000Steeple_12 = [str(x) for x in mens_3000Steeple_12.mens_3000Steeple_12]
mens_3000Steeple_12.mens_3000Steeple_12 = [x.strip() for x in mens_3000Steeple_12.mens_3000Steeple_12]
mens_3000Steeple_12.mens_3000Steeple_12 = [x.strip('/ DPG') for x in mens_3000Steeple_12.mens_3000Steeple_12]
mens_3000Steeple_12.mens_3000Steeple_12 = [x.replace('h','.')for x in mens_3000Steeple_12.mens_3000Steeple_12]
mens_3000Steeple_12.mens_3000Steeple_12 = [x.replace(':','.')for x in mens_3000Steeple_12.mens_3000Steeple_12]
mens_3000Steeple_12.name = [x.strip() for x in mens_3000Steeple_12.name]
mens_3000Steeple_12 = mens_3000Steeple_12.drop([13,14,52,53])


athlete_df = pd.merge(athlete_df, mens_3000Steeple_12[['name', 'mens_3000Steeple_12']], how='left', on='name', suffixes=(None, '_3000Steeple_mens_london'))


#### Scraping results from the men's 3000m Steeple-Chase 2016-Rio
removing rows with missing values

In [271]:
rio_mens_3000Steeple_16 = fn.olympic_query('rio', '2016', '3000m-steeplechase-men')
mens_3000Steeple_16 = fn.olympic_scraper(rio_mens_3000Steeple_16)
mens_3000Steeple_16 = fn.content_cleaner(mens_3000Steeple_16)
mens_3000Steeple_16.columns = ['rank', 'name', 'mens_3000Steeple_16']
mens_3000Steeple_16.insert(3, "event_x", "Athletics Men's 3000Steeple_16")
mens_3000Steeple_16.mens_3000Steeple_16 = [str(x) for x in mens_3000Steeple_16.mens_3000Steeple_16]
mens_3000Steeple_16.mens_3000Steeple_16 = [x.strip() for x in mens_3000Steeple_16.mens_3000Steeple_16]
mens_3000Steeple_16.mens_3000Steeple_16 = [x.strip('/ DPG') for x in mens_3000Steeple_16.mens_3000Steeple_16]
mens_3000Steeple_16.mens_3000Steeple_16 = [x.replace('h','.')for x in mens_3000Steeple_16.mens_3000Steeple_16]
mens_3000Steeple_16.mens_3000Steeple_16 = [x.replace(':','.')for x in mens_3000Steeple_16.mens_3000Steeple_16]
mens_3000Steeple_16.name = [x.strip() for x in mens_3000Steeple_16.name]
mens_3000Steeple_16 = mens_3000Steeple_16.drop(range(11,15))


athlete_df = pd.merge(athlete_df, mens_3000Steeple_16[['name', 'mens_3000Steeple_16']], how='left', on='name', suffixes=(None, '_3000Steeple_mens_london'))


#### Scraping the results from the men's discus throw 2004-Athens

In [272]:
athens_mens_discus = fn.olympic_query('athens', '2004', 'discus-throw-men')
mens_discus_04 = fn.olympic_scraper(athens_mens_discus)
mens_discus_04 = fn.content_cleaner(mens_discus_04)
mens_discus_04.columns = ['rank', 'name', 'mens_discus_04']
mens_discus_04.insert(3, "event_x", "Athletics Men's Discus")
mens_discus_04.mens_discus_04 = [str(x) for x in mens_discus_04.mens_discus_04]
mens_discus_04.mens_discus_04 = [x.strip() for x in mens_discus_04.mens_discus_04]
mens_discus_04.mens_discus_04 = [x.strip('/ DPG') for x in mens_discus_04.mens_discus_04]
mens_discus_04.mens_discus_04 = [x.replace('h','.')for x in mens_discus_04.mens_discus_04]
mens_discus_04.mens_discus_04 = [x.replace(':','.')for x in mens_discus_04.mens_discus_04]
mens_discus_04.name = [x.strip() for x in mens_discus_04.name]


athlete_df = pd.merge(athlete_df, mens_discus_04[['name', 'mens_discus_04']], how='left', on='name', suffixes=(None, '_discus_mens_athens'))


#### Scraping the results from the men's discus throw 2008-Beijing

In [273]:
beijing_mens_discus = fn.olympic_query('beijing', '2008', 'discus-throw-men')
mens_discus_08 = fn.olympic_scraper(beijing_mens_discus)
mens_discus_08 = fn.content_cleaner(mens_discus_08)
mens_discus_08.columns = ['rank', 'name', 'mens_discus_08']
mens_discus_08.insert(3, "event_x", "Athletics Men's Discus")
mens_discus_08.mens_discus_08 = [str(x) for x in mens_discus_08.mens_discus_08]
mens_discus_08.mens_discus_08 = [x.strip() for x in mens_discus_08.mens_discus_08]
mens_discus_08.mens_discus_08 = [x.strip('/ DPG') for x in mens_discus_08.mens_discus_08]
mens_discus_08.mens_discus_08 = [x.replace('h','.')for x in mens_discus_08.mens_discus_08]
mens_discus_08.mens_discus_08 = [x.replace(':','.')for x in mens_discus_08.mens_discus_08]
mens_discus_08.name = [x.strip() for x in mens_discus_08.name]


athlete_df = pd.merge(athlete_df, mens_discus_08[['name', 'mens_discus_08']], how='left', on='name', suffixes=(None, '_discus_mens_beijing'))


#### Scraping the results from the men's discus throw 2012-London

In [274]:
london_mens_discus = fn.olympic_query('london', '2012', 'discus-throw-men')
mens_discus_12 = fn.olympic_scraper(london_mens_discus)
mens_discus_12 = fn.content_cleaner(mens_discus_12)
mens_discus_12.columns = ['rank', 'name', 'mens_discus_12']
mens_discus_12.insert(3, "event_x", "Athletics Men's Discus")
mens_discus_12.mens_discus_12 = [str(x) for x in mens_discus_12.mens_discus_12]
mens_discus_12.mens_discus_12 = [x.strip() for x in mens_discus_12.mens_discus_12]
mens_discus_12.mens_discus_12 = [x.strip('/ DPG') for x in mens_discus_12.mens_discus_12]
mens_discus_12.mens_discus_12 = [x.replace('h','.')for x in mens_discus_12.mens_discus_12]
mens_discus_12.mens_discus_12 = [x.replace(':','.')for x in mens_discus_12.mens_discus_12]
mens_discus_12.name = [x.strip() for x in mens_discus_12.name]


athlete_df = pd.merge(athlete_df, mens_discus_12[['name', 'mens_discus_12']], how='left', on='name', suffixes=(None, '_discus_mens_london'))


#### Scraping the results from the men's discus throw 2016-Rio

In [275]:
rio_mens_discus = fn.olympic_query('rio', '2016', 'discus-throw-men')
mens_discus_16 = fn.olympic_scraper(rio_mens_discus)
mens_discus_16 = fn.content_cleaner(mens_discus_16)
mens_discus_16.columns = ['rank', 'name', 'mens_discus_16']
mens_discus_16.insert(3, "event_x", "Athletics Men's Discus")
mens_discus_16.mens_discus_16 = [str(x) for x in mens_discus_16.mens_discus_16]
mens_discus_16.mens_discus_16 = [x.strip() for x in mens_discus_16.mens_discus_16]
mens_discus_16.mens_discus_16 = [x.strip('/ DPG') for x in mens_discus_16.mens_discus_16]
mens_discus_16.mens_discus_16 = [x.replace('h','.')for x in mens_discus_16.mens_discus_16]
mens_discus_16.mens_discus_16 = [x.replace(':','.')for x in mens_discus_16.mens_discus_16]
mens_discus_16.name = [x.strip() for x in mens_discus_16.name]


athlete_df = pd.merge(athlete_df, mens_discus_16[['name', 'mens_discus_16']], how='left', on='name', suffixes=(None, '_discus_mens_rio'))


#### Scraping the results from the men's hammer throw 2004-Rio

In [276]:
athens_mens_hammer = fn.olympic_query('athens', '2004', 'hammer-throw-men')
mens_hammer_04 = fn.olympic_scraper(athens_mens_hammer)
mens_hammer_04 = fn.content_cleaner(mens_hammer_04)
mens_hammer_04.columns = ['rank', 'name', 'mens_hammer_04']
mens_hammer_04.insert(3, "event_x", "Athletics Men's Hammer")
mens_hammer_04.mens_hammer_04 = [str(x) for x in mens_hammer_04.mens_hammer_04]
mens_hammer_04.mens_hammer_04 = [x.strip() for x in mens_hammer_04.mens_hammer_04]
mens_hammer_04.mens_hammer_04 = [x.strip('/ DPG') for x in mens_hammer_04.mens_hammer_04]
mens_hammer_04.mens_hammer_04 = [x.replace('h','.')for x in mens_hammer_04.mens_hammer_04]
mens_hammer_04.mens_hammer_04 = [x.replace(':','.')for x in mens_hammer_04.mens_hammer_04]
mens_hammer_04.name = [x.strip() for x in mens_hammer_04.name]


athlete_df = pd.merge(athlete_df, mens_hammer_04[['name', 'mens_hammer_04']], how='left', on='name', suffixes=(None, '_hammer_mens_athens'))


#### Scraping the results from the men's hammer throw 2008-Beijing
dropping rows with missing values

In [277]:
beijing_mens_hammer = fn.olympic_query('beijing', '2008', 'hammer-throw-men')
mens_hammer_08 = fn.olympic_scraper(beijing_mens_hammer)
mens_hammer_08 = fn.content_cleaner(mens_hammer_08)
mens_hammer_08.columns = ['rank', 'name', 'mens_hammer_08']
mens_hammer_08.insert(3, "event_x", "Athletics Men's Hammer")
mens_hammer_08.mens_hammer_08 = [str(x) for x in mens_hammer_08.mens_hammer_08]
mens_hammer_08.mens_hammer_08 = [x.strip() for x in mens_hammer_08.mens_hammer_08]
mens_hammer_08.mens_hammer_08 = [x.strip('/ DPG') for x in mens_hammer_08.mens_hammer_08]
mens_hammer_08.mens_hammer_08 = [x.replace('h','.')for x in mens_hammer_08.mens_hammer_08]
mens_hammer_08.mens_hammer_08 = [x.replace(':','.')for x in mens_hammer_08.mens_hammer_08]
mens_hammer_08.name = [x.strip() for x in mens_hammer_08.name]
mens_hammer_08 = mens_hammer_08.drop([42,43,44,75,76,77,108,109,110])


athlete_df = pd.merge(athlete_df, mens_hammer_08[['name', 'mens_hammer_08']], how='left', on='name', suffixes=(None, '_hammer_mens_beijing'))


#### Scraping the results from the men's hammer throw 2012-London
dropping rows with missing values

In [278]:
london_mens_hammer = fn.olympic_query('london', '2012', 'hammer-throw-men')
mens_hammer_12 = fn.olympic_scraper(london_mens_hammer)
mens_hammer_12 = fn.content_cleaner(mens_hammer_12)
mens_hammer_12.columns = ['rank', 'name', 'mens_hammer_12']
mens_hammer_12.insert(3, "event_x", "Athletics Men's Hammer")
mens_hammer_12.mens_hammer_12 = [str(x) for x in mens_hammer_12.mens_hammer_12]
mens_hammer_12.mens_hammer_12 = [x.strip() for x in mens_hammer_12.mens_hammer_12]
mens_hammer_12.mens_hammer_12 = [x.strip('/ DPG') for x in mens_hammer_12.mens_hammer_12]
mens_hammer_12.mens_hammer_12 = [x.replace('h','.')for x in mens_hammer_12.mens_hammer_12]
mens_hammer_12.mens_hammer_12 = [x.replace(':','.')for x in mens_hammer_12.mens_hammer_12]
mens_hammer_12.name = [x.strip() for x in mens_hammer_12.name]
mens_hammer_12 = mens_hammer_12.drop([47,48,49])


athlete_df = pd.merge(athlete_df, mens_hammer_12[['name', 'mens_hammer_12']], how='left', on='name', suffixes=(None, '_hammer_mens_london'))


#### Scraping the results from the men's hammer throw 2016-Rio
dropping rows with missing values

In [279]:
rio_mens_hammer = fn.olympic_query('rio', '2016', 'hammer-throw-men')
mens_hammer_16 = fn.olympic_scraper(rio_mens_hammer)
mens_hammer_16 = fn.content_cleaner(mens_hammer_16)
mens_hammer_16.columns = ['rank', 'name', 'mens_hammer_16']
mens_hammer_16.insert(3, "event_x", "Athletics Men's Hammer")
mens_hammer_16.mens_hammer_16 = [str(x) for x in mens_hammer_16.mens_hammer_16]
mens_hammer_16.mens_hammer_16 = [x.strip() for x in mens_hammer_16.mens_hammer_16]
mens_hammer_16.mens_hammer_16 = [x.strip('/ DPG') for x in mens_hammer_16.mens_hammer_16]
mens_hammer_16.mens_hammer_16 = [x.replace('h','.')for x in mens_hammer_16.mens_hammer_16]
mens_hammer_16.mens_hammer_16 = [x.replace(':','.')for x in mens_hammer_16.mens_hammer_16]
mens_hammer_16.name = [x.strip() for x in mens_hammer_16.name]


athlete_df = pd.merge(athlete_df, mens_hammer_16[['name', 'mens_hammer_16']], how='left', on='name', suffixes=(None, '_hammer_mens_rio'))


#### Scraping the results from the mens's shotput 2004-Athens
removing row with missing value

In [280]:
athens_mens_shot = fn.olympic_query('athens', '2004', 'shot-put-men')
mens_shot_04 = fn.olympic_scraper(athens_mens_shot)
mens_shot_04 = fn.content_cleaner(mens_shot_04)
mens_shot_04.columns = ['rank', 'name', 'mens_shot_04']
mens_shot_04.insert(3, "event_x", "Athletics Men's shot")
mens_shot_04.mens_shot_04 = [str(x) for x in mens_shot_04.mens_shot_04]
mens_shot_04.mens_shot_04 = [x.strip() for x in mens_shot_04.mens_shot_04]
mens_shot_04.mens_shot_04 = [x.strip('/ DPG') for x in mens_shot_04.mens_shot_04]
mens_shot_04.mens_shot_04 = [x.replace('h','.')for x in mens_shot_04.mens_shot_04]
mens_shot_04.mens_shot_04 = [x.replace(':','.')for x in mens_shot_04.mens_shot_04]
mens_shot_04.name = [x.strip() for x in mens_shot_04.name]
mens_shot_04 = mens_shot_04.drop(11)


athlete_df = pd.merge(athlete_df, mens_shot_04[['name', 'mens_shot_04']], how='left', on='name', suffixes=(None, '_shot_mens_athens'))


#### Scraping results from the men's shotput 2008-Beijing
removing rows with missing values

In [281]:
beijing_mens_shot = fn.olympic_query('beijing', '2008', 'shot-put-men')
mens_shot_08 = fn.olympic_scraper(beijing_mens_shot)
mens_shot_08 = fn.content_cleaner(mens_shot_08)
mens_shot_08.columns = ['rank', 'name', 'mens_shot_08']
mens_shot_08.insert(3, "event_x", "Athletics Men's Shot")
mens_shot_08.mens_shot_08 = [str(x) for x in mens_shot_08.mens_shot_08]
mens_shot_08.mens_shot_08 = [x.strip() for x in mens_shot_08.mens_shot_08]
mens_shot_08.mens_shot_08 = [x.strip('/ DPG') for x in mens_shot_08.mens_shot_08]
mens_shot_08.mens_shot_08 = [x.replace('h','.')for x in mens_shot_08.mens_shot_08]
mens_shot_08.mens_shot_08 = [x.replace(':','.')for x in mens_shot_08.mens_shot_08]
mens_shot_08.name = [x.strip() for x in mens_shot_08.name]
mens_shot_08 = mens_shot_08.drop([8,50,51,52,53,54,95,96,97,98,99,
                                 140,141,142,143,144])


athlete_df = pd.merge(athlete_df, mens_shot_08[['name', 'mens_shot_08']], how='left', on='name', suffixes=(None, '_shot_mens_beijing'))


#### Scraping results from the men's shotput 2012-London

In [282]:
london_mens_shot = fn.olympic_query('london', '2012', 'shot-put-men')
mens_shot_12 = fn.olympic_scraper(london_mens_shot)
mens_shot_12 = fn.content_cleaner(mens_shot_12)
mens_shot_12.columns = ['rank', 'name', 'mens_shot_12']
mens_shot_12.insert(3, "event_x", "Athletics Men's Shot")
mens_shot_12.mens_shot_12 = [str(x) for x in mens_shot_12.mens_shot_12]
mens_shot_12.mens_shot_12 = [x.strip() for x in mens_shot_12.mens_shot_12]
mens_shot_12.mens_shot_12 = [x.strip('/ DPG') for x in mens_shot_12.mens_shot_12]
mens_shot_12.mens_shot_12 = [x.replace('h','.')for x in mens_shot_12.mens_shot_12]
mens_shot_12.mens_shot_12 = [x.replace(':','.')for x in mens_shot_12.mens_shot_12]
mens_shot_12.name = [x.strip() for x in mens_shot_12.name]
mens_shot_12 = mens_shot_12.drop([49,50,51])


athlete_df = pd.merge(athlete_df, mens_shot_12[['name', 'mens_shot_12']], how='left', on='name', suffixes=(None, '_shot_mens_london'))


#### Scraping results from the men's shotput 2016-Rio
removing rows with missing values

In [283]:
rio_mens_shot = fn.olympic_query('rio', '2016', 'shot-put-men')
mens_shot_16 = fn.olympic_scraper(rio_mens_shot)
mens_shot_16 = fn.content_cleaner(mens_shot_16)
mens_shot_16.columns = ['rank', 'name', 'mens_shot_16']
mens_shot_16.insert(3, "event_x", "Athletics Men's Shot")
mens_shot_16.mens_shot_16 = [str(x) for x in mens_shot_16.mens_shot_16]
mens_shot_16.mens_shot_16 = [x.strip() for x in mens_shot_16.mens_shot_16]
mens_shot_16.mens_shot_16 = [x.strip('/ DPG') for x in mens_shot_16.mens_shot_16]
mens_shot_16.mens_shot_16 = [x.replace('h','.')for x in mens_shot_16.mens_shot_16]
mens_shot_16.mens_shot_16 = [x.replace(':','.')for x in mens_shot_16.mens_shot_16]
mens_shot_16.name = [x.strip() for x in mens_shot_16.name]
mens_shot_16 = mens_shot_16.drop(11)


athlete_df = pd.merge(athlete_df, mens_shot_16[['name', 'mens_shot_16']], how='left', on='name', suffixes=(None, '_shot_mens_rio'))


#### Scraping results form the men's javelin 2004-Athens


In [284]:
athens_mens_javelin = fn.olympic_query('athens', '2004', 'javelin-throw-men')
mens_javelin_04 = fn.olympic_scraper(athens_mens_javelin)
mens_javelin_04 = fn.content_cleaner(mens_javelin_04)
mens_javelin_04.columns = ['rank', 'name', 'mens_javelin_04']
mens_javelin_04.insert(3, "event_x", "Athletics Men's Javelin")
mens_javelin_04.mens_javelin_04 = [str(x) for x in mens_javelin_04.mens_javelin_04]
mens_javelin_04.mens_javelin_04 = [x.strip() for x in mens_javelin_04.mens_javelin_04]
mens_javelin_04.mens_javelin_04 = [x.strip('/ DPG') for x in mens_javelin_04.mens_javelin_04]
mens_javelin_04.mens_javelin_04 = [x.replace('h','.')for x in mens_javelin_04.mens_javelin_04]
mens_javelin_04.mens_javelin_04 = [x.replace(':','.')for x in mens_javelin_04.mens_javelin_04]
mens_javelin_04.name = [x.strip() for x in mens_javelin_04.name]


athlete_df = pd.merge(athlete_df, mens_javelin_04[['name', 'mens_javelin_04']], how='left', on='name', suffixes=(None, '_javelin_mens_athens'))


#### Scraping results form the men's javelin 2008-Beijing
removing rows with missing values

In [285]:
beijing_mens_javelin = fn.olympic_query('beijing', '2008', 'javelin-throw-men')
mens_javelin_08 = fn.olympic_scraper(beijing_mens_javelin)
mens_javelin_08 = fn.content_cleaner(mens_javelin_08)
mens_javelin_08.columns = ['rank', 'name', 'mens_javelin_08']
mens_javelin_08.insert(3, "event_x", "Athletics Men's Javelin")
mens_javelin_08.mens_javelin_08 = [str(x) for x in mens_javelin_08.mens_javelin_08]
mens_javelin_08.mens_javelin_08 = [x.strip() for x in mens_javelin_08.mens_javelin_08]
mens_javelin_08.mens_javelin_08 = [x.strip('/ DPG') for x in mens_javelin_08.mens_javelin_08]
mens_javelin_08.mens_javelin_08 = [x.replace('h','.')for x in mens_javelin_08.mens_javelin_08]
mens_javelin_08.mens_javelin_08 = [x.replace(':','.')for x in mens_javelin_08.mens_javelin_08]
mens_javelin_08.name = [x.strip() for x in mens_javelin_08.name]
mens_javelin_08 = mens_javelin_08.drop([49,87,125])


athlete_df = pd.merge(athlete_df, mens_javelin_08[['name', 'mens_javelin_08']], how='left', on='name', suffixes=(None, '_javelin_mens_beijing'))


#### Scraping results form the men's javelin 2012-London
removing rows with missing values

In [286]:
london_mens_javelin = fn.olympic_query('london', '2012', 'javelin-throw-men')
mens_javelin_12 = fn.olympic_scraper(london_mens_javelin)
mens_javelin_12 = fn.content_cleaner(mens_javelin_12)
mens_javelin_12.columns = ['rank', 'name', 'mens_javelin_12']
mens_javelin_12.insert(3, "event_x", "Athletics Men's Javelin")
mens_javelin_12.mens_javelin_12 = [str(x) for x in mens_javelin_12.mens_javelin_12]
mens_javelin_12.mens_javelin_12 = [x.strip() for x in mens_javelin_12.mens_javelin_12]
mens_javelin_12.mens_javelin_12 = [x.strip('/ DPG') for x in mens_javelin_12.mens_javelin_12]
mens_javelin_12.mens_javelin_12 = [x.replace('h','.')for x in mens_javelin_12.mens_javelin_12]
mens_javelin_12.mens_javelin_12 = [x.replace(':','.')for x in mens_javelin_12.mens_javelin_12]
mens_javelin_12.name = [x.strip() for x in mens_javelin_12.name]
mens_javelin_12 = mens_javelin_12.drop([53,54])


athlete_df = pd.merge(athlete_df, mens_javelin_12[['name', 'mens_javelin_12']], how='left', on='name', suffixes=(None, '_javelin_mens_london'))


#### Scraping results form the men's javelin 2016-Rio
removing rows with missing values

In [287]:
rio_mens_javelin = fn.olympic_query('rio', '2016', 'javelin-throw-men')
mens_javelin_16 = fn.olympic_scraper(rio_mens_javelin)
mens_javelin_16 = fn.content_cleaner(mens_javelin_16)
mens_javelin_16.columns = ['rank', 'name', 'mens_javelin_16']
mens_javelin_16.insert(3, "event_x", "Athletics Men's Javelin")
mens_javelin_16.mens_javelin_16 = [str(x) for x in mens_javelin_16.mens_javelin_16]
mens_javelin_16.mens_javelin_16 = [x.strip() for x in mens_javelin_16.mens_javelin_16]
mens_javelin_16.mens_javelin_16 = [x.strip('/ DPG') for x in mens_javelin_16.mens_javelin_16]
mens_javelin_16.mens_javelin_16 = [x.replace('h','.')for x in mens_javelin_16.mens_javelin_16]
mens_javelin_16.mens_javelin_16 = [x.replace(':','.')for x in mens_javelin_16.mens_javelin_16]
mens_javelin_16.name = [x.strip() for x in mens_javelin_16.name]
mens_javelin_16 

,rank,name,mens_javelin_16,event_x
0,1.,Thomas Rohler,,Athletics Men's Javelin
1,2.,Julius Yego,,Athletics Men's Javelin
2,3.,Keshorn Walcott,,Athletics Men's Javelin
3,4.,Johannes Vetter,,Athletics Men's Javelin
4,5.,Dmytro Kosynskyy,,Athletics Men's Javelin
5,6.,Antti Ruuskanen,,Athletics Men's Javelin
6,7.,Vitezslav Vesely,,Athletics Men's Javelin
7,8.,Jakub Vadlejch,,Athletics Men's Javelin


##### There were no results listed for the men's javelin throw during the Summer Olympic Games in Rio-2016. When going back to the website to make sure, only the athletes and rankings are listed, no results. I will leave this year's results out for the men's javelin throw.

#### Scraping results from men's pole vault 2004-Athens


In [288]:
athens_mens_pole = fn.olympic_query('athens', '2004', 'pole-vault-men')
mens_pole_04 = fn.olympic_scraper(athens_mens_pole)
mens_pole_04 = fn.content_cleaner(mens_pole_04)
mens_pole_04.columns = ['rank', 'name', 'mens_pole_04']
mens_pole_04.insert(3, "event_x", "Athletics Men's pole")
mens_pole_04.mens_pole_04 = [str(x) for x in mens_pole_04.mens_pole_04]
mens_pole_04.mens_pole_04 = [x.strip() for x in mens_pole_04.mens_pole_04]
mens_pole_04.mens_pole_04 = [x.strip('/ DPG') for x in mens_pole_04.mens_pole_04]
mens_pole_04.mens_pole_04 = [x.replace('h','.')for x in mens_pole_04.mens_pole_04]
mens_pole_04.mens_pole_04 = [x.replace(':','.')for x in mens_pole_04.mens_pole_04]
mens_pole_04.name = [x.strip() for x in mens_pole_04.name]


athlete_df = pd.merge(athlete_df, mens_pole_04[['name', 'mens_pole_04']], how='left', on='name', suffixes=(None, '_pole_mens_athens'))


#### Scraping results from men's pole vault 2008-Beijing
removing rows with missing values

In [289]:
beijing_mens_pole = fn.olympic_query('beijing', '2008', 'pole-vault-men')
mens_pole_08 = fn.olympic_scraper(beijing_mens_pole)
mens_pole_08 = fn.content_cleaner(mens_pole_08)
mens_pole_08.columns = ['rank', 'name', 'mens_pole_08']
mens_pole_08.insert(3, "event_x", "Athletics Men's pole")
mens_pole_08.mens_pole_08 = [str(x) for x in mens_pole_08.mens_pole_08]
mens_pole_08.mens_pole_08 = [x.strip() for x in mens_pole_08.mens_pole_08]
mens_pole_08.mens_pole_08 = [x.strip('/ DPG') for x in mens_pole_08.mens_pole_08]
mens_pole_08.mens_pole_08 = [x.replace('h','.')for x in mens_pole_08.mens_pole_08]
mens_pole_08.mens_pole_08 = [x.replace(':','.')for x in mens_pole_08.mens_pole_08]
mens_pole_08.name = [x.strip() for x in mens_pole_08.name]
mens_pole_08 = mens_pole_08.drop([10,11,44,45,46,48,49,50,
                                 82,83,84,86,87,88,120,121,
                                 122,124,125,126])


athlete_df = pd.merge(athlete_df, mens_pole_08[['name', 'mens_pole_08']], how='left', on='name', suffixes=(None, '_pole_mens_beijing'))


#### Scraping results from men's pole vault 2012-London
removing rows with missing values

In [290]:
london_mens_pole = fn.olympic_query('london', '2012', 'pole-vault-men')
mens_pole_12 = fn.olympic_scraper(london_mens_pole)
mens_pole_12 = fn.content_cleaner(mens_pole_12)
mens_pole_12.columns = ['rank', 'name', 'mens_pole_12']
mens_pole_12.insert(3, "event_x", "Athletics Men's pole")
mens_pole_12.mens_pole_12 = [str(x) for x in mens_pole_12.mens_pole_12]
mens_pole_12.mens_pole_12 = [x.strip() for x in mens_pole_12.mens_pole_12]
mens_pole_12.mens_pole_12 = [x.strip('/ DPG') for x in mens_pole_12.mens_pole_12]
mens_pole_12.mens_pole_12 = [x.replace('h','.')for x in mens_pole_12.mens_pole_12]
mens_pole_12.mens_pole_12 = [x.replace(':','.')for x in mens_pole_12.mens_pole_12]
mens_pole_12.name = [x.strip() for x in mens_pole_12.name]
mens_pole_12 = mens_pole_12.drop([11,12,35,36,37,38,39,40,41,
                                 43,45])


athlete_df = pd.merge(athlete_df, mens_pole_12[['name', 'mens_pole_12']], how='left', on='name', suffixes=(None, '_pole_mens_london'))


#### Scraping results from men's pole vault 2016-Rio
removing row with missing value

In [291]:
rio_mens_pole = fn.olympic_query('rio', '2016', 'pole-vault-men')
mens_pole_16 = fn.olympic_scraper(rio_mens_pole)
mens_pole_16 = fn.content_cleaner(mens_pole_16)
mens_pole_16.columns = ['rank', 'name', 'mens_pole_16']
mens_pole_16.insert(3, "event_x", "Athletics Men's Pole")
mens_pole_16.mens_pole_16 = [str(x) for x in mens_pole_16.mens_pole_16]
mens_pole_16.mens_pole_16 = [x.strip() for x in mens_pole_16.mens_pole_16]
mens_pole_16.mens_pole_16 = [x.strip('/ DPG') for x in mens_pole_16.mens_pole_16]
mens_pole_16.mens_pole_16 = [x.replace('h','.')for x in mens_pole_16.mens_pole_16]
mens_pole_16.mens_pole_16 = [x.replace(':','.')for x in mens_pole_16.mens_pole_16]
mens_pole_16.name = [x.strip() for x in mens_pole_16.name]
mens_pole_16 = mens_pole_16.drop(11)


athlete_df = pd.merge(athlete_df, mens_pole_16[['name', 'mens_pole_16']], how='left', on='name', suffixes=(None, '_pole_mens_rio'))

#### Scraping results from men's long jump 2004-Athens


In [292]:
athens_mens_long = fn.olympic_query('athens', '2004', 'long-jump-men')
mens_long_04 = fn.olympic_scraper(athens_mens_long)
mens_long_04 = fn.content_cleaner(mens_long_04)
mens_long_04.columns = ['rank', 'name', 'mens_long_04']
mens_long_04.insert(3, "event_x", "Athletics Men's Long Jump")
mens_long_04.mens_long_04 = [str(x) for x in mens_long_04.mens_long_04]
mens_long_04.mens_long_04 = [x.strip() for x in mens_long_04.mens_long_04]
mens_long_04.mens_long_04 = [x.strip('/ DPG') for x in mens_long_04.mens_long_04]
mens_long_04.mens_long_04 = [x.replace('h','.')for x in mens_long_04.mens_long_04]
mens_long_04.mens_long_04 = [x.replace(':','.')for x in mens_long_04.mens_long_04]
mens_long_04.name = [x.strip() for x in mens_long_04.name]


athlete_df = pd.merge(athlete_df, mens_long_04[['name', 'mens_long_04']], how='left', on='name', suffixes=(None, '_long_mens_athens'))


#### Scraping results from men's long jump 2008-Beijing
removing rows with missing values

In [293]:
beijing_mens_long = fn.olympic_query('beijing', '2008', 'long-jump-men')
mens_long_08 = fn.olympic_scraper(beijing_mens_long)
mens_long_08 = fn.content_cleaner(mens_long_08)
mens_long_08.columns = ['rank', 'name', 'mens_long_08']
mens_long_08.insert(3, "event_x", "Athletics Men's Long Jump")
mens_long_08.mens_long_08 = [str(x) for x in mens_long_08.mens_long_08]
mens_long_08.mens_long_08 = [x.strip() for x in mens_long_08.mens_long_08]
mens_long_08.mens_long_08 = [x.strip('/ DPG') for x in mens_long_08.mens_long_08]
mens_long_08.mens_long_08 = [x.replace('h','.')for x in mens_long_08.mens_long_08]
mens_long_08.mens_long_08 = [x.replace(':','.')for x in mens_long_08.mens_long_08]
mens_long_08.name = [x.strip() for x in mens_long_08.name]
mens_long_08 = mens_long_08.drop([10,50,51,52,91,92,93,132,133,134])


athlete_df = pd.merge(athlete_df, mens_long_08[['name', 'mens_long_08']], how='left', on='name', suffixes=(None, '_long_mens_beijing'))


#### Scraping results from men's long jump 2012-London
removing rows with missing values

In [294]:
london_mens_long = fn.olympic_query('london', '2012', 'long-jump-men')
mens_long_12 = fn.olympic_scraper(london_mens_long)
mens_long_12 = fn.content_cleaner(mens_long_12)
mens_long_12.columns = ['rank', 'name', 'mens_long_12']
mens_long_12.insert(3, "event_x", "Athletics Men's Long Jump")
mens_long_12.mens_long_12 = [str(x) for x in mens_long_12.mens_long_12]
mens_long_12.mens_long_12 = [x.strip() for x in mens_long_12.mens_long_12]
mens_long_12.mens_long_12 = [x.strip('/ DPG') for x in mens_long_12.mens_long_12]
mens_long_12.mens_long_12 = [x.replace('h','.')for x in mens_long_12.mens_long_12]
mens_long_12.mens_long_12 = [x.replace(':','.')for x in mens_long_12.mens_long_12]
mens_long_12.name = [x.strip() for x in mens_long_12.name]
mens_long_12 = mens_long_12.drop([52,53])


athlete_df = pd.merge(athlete_df, mens_long_12[['name', 'mens_long_12']], how='left', on='name', suffixes=(None, '_long_mens_london'))


#### Scraping results from men's long jump 2016-Rio


In [295]:
rio_mens_long = fn.olympic_query('rio', '2016', 'long-jump-men')
mens_long_16 = fn.olympic_scraper(rio_mens_long)
mens_long_16 = fn.content_cleaner(mens_long_16)
mens_long_16.columns = ['rank', 'name', 'mens_long_16']
mens_long_16.insert(3, "event_x", "Athletics Men's Long Jump")
mens_long_16.mens_long_16 = [str(x) for x in mens_long_16.mens_long_16]
mens_long_16.mens_long_16 = [x.strip() for x in mens_long_16.mens_long_16]
mens_long_16.mens_long_16 = [x.strip('/ DPG') for x in mens_long_16.mens_long_16]
mens_long_16.mens_long_16 = [x.replace('h','.')for x in mens_long_16.mens_long_16]
mens_long_16.mens_long_16 = [x.replace(':','.')for x in mens_long_16.mens_long_16]
mens_long_16.name = [x.strip() for x in mens_long_16.name]
                             

athlete_df = pd.merge(athlete_df, mens_long_16[['name', 'mens_long_16']], how='left', on='name', suffixes=(None, '_long_mens_rio'))


#### Scraping results from men's high jump 2004-Athens
removing rows with missing values

In [296]:
athens_mens_high = fn.olympic_query('athens', '2004', 'high-jump-men')
mens_high_04 = fn.olympic_scraper(athens_mens_high)
mens_high_04 = fn.content_cleaner(mens_high_04)
mens_high_04.columns = ['rank', 'name', 'mens_high_04']
mens_high_04.insert(3, "event_x", "Athletics Men's High Jump")
mens_high_04.mens_high_04 = [str(x) for x in mens_high_04.mens_high_04]
mens_high_04.mens_high_04 = [x.strip() for x in mens_high_04.mens_high_04]
mens_high_04.mens_high_04 = [x.strip('/ DPG') for x in mens_high_04.mens_high_04]
mens_high_04.mens_high_04 = [x.replace('h','.')for x in mens_high_04.mens_high_04]
mens_high_04.mens_high_04 = [x.replace(':','.')for x in mens_high_04.mens_high_04]
mens_high_04.name = [x.strip() for x in mens_high_04.name]


athlete_df = pd.merge(athlete_df, mens_high_04[['name', 'mens_high_04']], how='left', on='name', suffixes=(None, '_high_mens_athens'))


#### Scraping results from men's high jump 2008-Beijing
removing rows with missing values

In [297]:
beijing_mens_high = fn.olympic_query('beijing', '2008', 'high-jump-men')
mens_high_08 = fn.olympic_scraper(beijing_mens_high)
mens_high_08 = fn.content_cleaner(mens_high_08)
mens_high_08.columns = ['rank', 'name', 'mens_high_08']
mens_high_08.insert(3, "event_x", "Athletics Men's High Jump")
mens_high_08.mens_high_08 = [str(x) for x in mens_high_08.mens_high_08]
mens_high_08.mens_high_08 = [x.strip() for x in mens_high_08.mens_high_08]
mens_high_08.mens_high_08 = [x.strip('/ DPG') for x in mens_high_08.mens_high_08]
mens_high_08.mens_high_08 = [x.replace('h','.')for x in mens_high_08.mens_high_08]
mens_high_08.mens_high_08 = [x.replace(':','.')for x in mens_high_08.mens_high_08]
mens_high_08.name = [x.strip() for x in mens_high_08.name]
mens_high_08 = mens_high_08.drop([51,91,131])
 

athlete_df = pd.merge(athlete_df, mens_high_08[['name', 'mens_high_08']], how='left', on='name', suffixes=(None, '_high_mens_beijing'))


#### Scraping results from men's high jump 2012-London
removing rows with missing values

In [298]:
london_mens_high = fn.olympic_query('london', '2012', 'high-jump-men')
mens_high_12 = fn.olympic_scraper(london_mens_high)
mens_high_12 = fn.content_cleaner(mens_high_12)
mens_high_12.columns = ['rank', 'name', 'mens_high_12']
mens_high_12.insert(3, "event_x", "Athletics Men's High Jump")
mens_high_12.mens_high_12 = [str(x) for x in mens_high_12.mens_high_12]
mens_high_12.mens_high_12 = [x.strip() for x in mens_high_12.mens_high_12]
mens_high_12.mens_high_12 = [x.strip('/ DPG') for x in mens_high_12.mens_high_12]
mens_high_12.mens_high_12 = [x.replace('h','.')for x in mens_high_12.mens_high_12]
mens_high_12.mens_high_12 = [x.replace(':','.')for x in mens_high_12.mens_high_12]
mens_high_12.name = [x.strip() for x in mens_high_12.name]
mens_high_12 = mens_high_12.drop([45,46,48])


athlete_df = pd.merge(athlete_df, mens_high_12[['name', 'mens_high_12']], how='left', on='name', suffixes=(None, '_high_mens_london'))


#### Scraping results from men's high jump 2016-Rio


In [299]:
rio_mens_high = fn.olympic_query('rio', '2016', 'high-jump-men')
mens_high_16 = fn.olympic_scraper(rio_mens_high)
mens_high_16 = fn.content_cleaner(mens_high_16)
mens_high_16.columns = ['rank', 'name', 'mens_high_16']
mens_high_16.insert(3, "event_x", "Athletics Men's High Jump")
mens_high_16.mens_high_16 = [str(x) for x in mens_high_16.mens_high_16]
mens_high_16.mens_high_16 = [x.strip() for x in mens_high_16.mens_high_16]
mens_high_16.mens_high_16 = [x.strip('/ DPG') for x in mens_high_16.mens_high_16]
mens_high_16.mens_high_16 = [x.replace('h','.')for x in mens_high_16.mens_high_16]
mens_high_16.mens_high_16 = [x.replace(':','.')for x in mens_high_16.mens_high_16]
mens_high_16.name = [x.strip() for x in mens_high_16.name]


athlete_df = pd.merge(athlete_df, mens_high_16[['name', 'mens_high_16']], how='left', on='name', suffixes=(None, '_high_mens_rio'))


#### Scraping results from men's triple jump 2004-Athens
removing rows with missing values

In [300]:
athens_mens_triple = fn.olympic_query('athens', '2004', 'triple-jump-men')
mens_triple_04 = fn.olympic_scraper(athens_mens_triple)
mens_triple_04 = fn.content_cleaner(mens_triple_04)
mens_triple_04.columns = ['rank', 'name', 'mens_triple_04']
mens_triple_04.insert(3, "event_x", "Athletics Men's Triple Jump")
mens_triple_04.mens_triple_04 = [str(x) for x in mens_triple_04.mens_triple_04]
mens_triple_04.mens_triple_04 = [x.strip() for x in mens_triple_04.mens_triple_04]
mens_triple_04.mens_triple_04 = [x.strip('/ DPG') for x in mens_triple_04.mens_triple_04]
mens_triple_04.mens_triple_04 = [x.replace('h','.')for x in mens_triple_04.mens_triple_04]
mens_triple_04.mens_triple_04 = [x.replace(':','.')for x in mens_triple_04.mens_triple_04]
mens_triple_04.name = [x.strip() for x in mens_triple_04.name]


athlete_df = pd.merge(athlete_df, mens_triple_04[['name', 'mens_triple_04']], how='left', on='name', suffixes=(None, '_triple_mens_athens'))


#### Scraping results from men's triple jump 2008-Beijing
removing rows with missing values

In [301]:
beijing_mens_triple = fn.olympic_query('beijing', '2008', 'triple-jump-men')
mens_triple_08 = fn.olympic_scraper(beijing_mens_triple)
mens_triple_08 = fn.content_cleaner(mens_triple_08)
mens_triple_08.columns = ['rank', 'name', 'mens_triple_08']
mens_triple_08.insert(3, "event_x", "Athletics Men's Triple Jump")
mens_triple_08.mens_triple_08 = [str(x) for x in mens_triple_08.mens_triple_08]
mens_triple_08.mens_triple_08 = [x.strip() for x in mens_triple_08.mens_triple_08]
mens_triple_08.mens_triple_08 = [x.strip('/ DPG') for x in mens_triple_08.mens_triple_08]
mens_triple_08.mens_triple_08 = [x.replace('h','.')for x in mens_triple_08.mens_triple_08]
mens_triple_08.mens_triple_08 = [x.replace(':','.')for x in mens_triple_08.mens_triple_08]
mens_triple_08.name = [x.strip() for x in mens_triple_08.name]
mens_triple_08 = mens_triple_08.drop([49,50,88,89,127,128])


athlete_df = pd.merge(athlete_df, mens_triple_08[['name', 'mens_triple_08']], how='left', on='name', suffixes=(None, '_triple_mens_beijing'))


#### Scraping results from men's triple jump 2012-London
removing rows with missing values

In [302]:
london_mens_triple = fn.olympic_query('london', '2012', 'triple-jump-men')
mens_triple_12 = fn.olympic_scraper(london_mens_triple)
mens_triple_12 = fn.content_cleaner(mens_triple_12)
mens_triple_12.columns = ['rank', 'name', 'mens_triple_12']
mens_triple_12.insert(3, "event_x", "Athletics Men's High Jump")
mens_triple_12.mens_triple_12 = [str(x) for x in mens_triple_12.mens_triple_12]
mens_triple_12.mens_triple_12 = [x.strip() for x in mens_triple_12.mens_triple_12]
mens_triple_12.mens_triple_12 = [x.strip('/ DPG') for x in mens_triple_12.mens_triple_12]
mens_triple_12.mens_triple_12 = [x.replace('h','.')for x in mens_triple_12.mens_triple_12]
mens_triple_12.mens_triple_12 = [x.replace(':','.')for x in mens_triple_12.mens_triple_12]
mens_triple_12.name = [x.strip() for x in mens_triple_12.name]
mens_triple_12 = mens_triple_12.drop(38)


athlete_df = pd.merge(athlete_df, mens_triple_12[['name', 'mens_triple_12']], how='left', on='name', suffixes=(None, '_triple_mens_london'))


#### Scraping results from men's triple jump 2016-Rio
removing rows with missing values

In [303]:
rio_mens_triple = fn.olympic_query('rio', '2016', 'triple-jump-men')
mens_triple_16 = fn.olympic_scraper(rio_mens_triple)
mens_triple_16 = fn.content_cleaner(mens_triple_16)
mens_triple_16.columns = ['rank', 'name', 'mens_triple_16']
mens_triple_16.insert(3, "event_x", "Athletics Men's Triple Jump")
mens_triple_16.mens_triple_16 = [str(x) for x in mens_triple_16.mens_triple_16]
mens_triple_16.mens_triple_16 = [x.strip() for x in mens_triple_16.mens_triple_16]
mens_triple_16.mens_triple_16 = [x.strip('/ DPG') for x in mens_triple_16.mens_triple_16]
mens_triple_16.mens_triple_16 = [x.replace('h','.')for x in mens_triple_16.mens_triple_16]
mens_triple_16.mens_triple_16 = [x.replace(':','.')for x in mens_triple_16.mens_triple_16]
mens_triple_16.name = [x.strip() for x in mens_triple_16.name]


athlete_df = pd.merge(athlete_df, mens_triple_16[['name', 'mens_triple_16']], how='left', on='name', suffixes=(None, '_triple_mens_rio'))


## Now I will scrape and combine the results from the womens' events

#### Scraping results from the women's 100m dash 2004-Athens


In [304]:
athens_100m_women = fn.olympic_query('athens', '2004', '100m-women')
womens_100m_04 = fn.olympic_scraper(athens_100m_women)
womens_100m_04 = fn.content_cleaner(womens_100m_04)
womens_100m_04.columns = ['rank', 'name', 'womens_100m_04']
womens_100m_04.insert(3, "event_x", "Athletics Women's 100 metres")
womens_100m_04.womens_100m_04 = [str(x) for x in womens_100m_04.womens_100m_04]
womens_100m_04.name = [x.strip() for x in womens_100m_04.name]
womens_100m_04.womens_100m_04 = [x.strip() for x in womens_100m_04.womens_100m_04]


athlete_df = pd.merge(athlete_df, womens_100m_04[['name', 'womens_100m_04']], how='left', on='name', suffixes=(None, '_100m_womens_athens'))


#### Scraping results from the women's 100m dash 2008-Beijing


In [305]:
beijing_100m_women = fn.olympic_query('beijing', '2008', '100m-women')
womens_100m_08 = fn.olympic_scraper(beijing_100m_women)
womens_100m_08 = fn.content_cleaner(womens_100m_08)
womens_100m_08.columns = ['rank', 'name', 'womens_100m_08']
womens_100m_08.insert(3, "event_x", "Athletics Women's 100 metres")
womens_100m_08.womens_100m_08 = [str(x) for x in womens_100m_08.womens_100m_08]
womens_100m_08.womens_100m_08 = [x.strip() for x in womens_100m_08.womens_100m_08]
womens_100m_08.name = [x.strip() for x in womens_100m_08.name]
womens_100m_08.womens_100m_08 = [x.strip('DPG /') for x in womens_100m_08.womens_100m_08]


athlete_df = pd.merge(athlete_df, womens_100m_08[['name', 'womens_100m_08']], how='left', on='name', suffixes=(None, '_100m_womens_beijing'))


#### Scraping results from the women's 100m dash 2012-London
removing row with missing value

In [306]:
london_100m_women = fn.olympic_query('london', '2012', '100m-women')
womens_100m_12 = fn.olympic_scraper(london_100m_women)
womens_100m_12 = fn.content_cleaner(womens_100m_12)
womens_100m_12.columns = ['rank', 'name', 'womens_100m_12']
womens_100m_12.insert(3, "event_x", "Athletics Women's 100 metres")
womens_100m_12.womens_100m_12 = [str(x) for x in womens_100m_12.womens_100m_12]
womens_100m_12.name = [x.strip() for x in womens_100m_12.name]
womens_100m_12.womens_100m_12 = [x.strip() for x in womens_100m_12.womens_100m_12]
womens_100m_12 = womens_100m_12.drop(121)


athlete_df = pd.merge(athlete_df, womens_100m_12[['name', 'womens_100m_12']], how='left', on='name', suffixes=(None, '_100m_womens_london'))


#### Scraping results from the women's 100m dash 2016-Rio


In [307]:
rio_100m_women = fn.olympic_query('rio', '2016', '100m-women')
womens_100m_16 = fn.olympic_scraper(rio_100m_women)
womens_100m_16 = fn.content_cleaner(womens_100m_16)
womens_100m_16.columns = ['rank', 'name', 'womens_100m_16']
womens_100m_16.insert(3, "event_x", "Athletics Women's 100 metres")
womens_100m_16.womens_100m_16 = [str(x) for x in womens_100m_16.womens_100m_16]
womens_100m_16.name = [x.strip() for x in womens_100m_16.name]
womens_100m_16.womens_100m_16 = [x.strip() for x in womens_100m_16.womens_100m_16]


athlete_df = pd.merge(athlete_df, womens_100m_16[['name', 'womens_100m_16']], how='left', on='name', suffixes=(None, '_100m_womens_rio'))


#### Scraping results from the women's 200m dash 2004-Athens


In [308]:
athens_200m_women = fn.olympic_query('athens', '2004', '200m-women')
womens_200m_04 = fn.olympic_scraper(athens_200m_women)
womens_200m_04 = fn.content_cleaner(womens_200m_04)
womens_200m_04.columns = ['rank', 'name', 'womens_200m_04']
womens_200m_04.insert(3, "event_x", "Athletics Women's 200 metres")
womens_200m_04.womens_200m_04 = [str(x) for x in womens_200m_04.womens_200m_04]
womens_200m_04.womens_200m_04 = [x.strip() for x in womens_200m_04.womens_200m_04]
womens_200m_04.name = [x.strip() for x in womens_200m_04.name]


athlete_df = pd.merge(athlete_df, womens_200m_04[['name', 'womens_200m_04']], how='left', on='name', suffixes=(None, '_200m_womens_athens'))


#### Scraping results from the women's 200m dash 2008-Beijing
removing rows with missing values

In [309]:
beijing_200m_women = fn.olympic_query('beijing', '2008', '200m-women')
womens_200m_08 = fn.olympic_scraper(beijing_200m_women)
womens_200m_08 = fn.content_cleaner(womens_200m_08)
womens_200m_08.columns = ['rank', 'name', 'womens_200m_08']
womens_200m_08.insert(3, "event_x", "Athletics Women's 200 metres")
womens_200m_08.womens_200m_08 = [str(x) for x in womens_200m_08.womens_200m_08]
womens_200m_08.womens_200m_08 = [x.strip() for x in womens_200m_08.womens_200m_08]
womens_200m_08.womens_200m_08 = [x.strip('DPG / ') for x in womens_200m_08.womens_200m_08]
womens_200m_08.womens_200m_08 = [x[:5] for x in womens_200m_08.womens_200m_08]
womens_200m_08.name = [x.strip() for x in womens_200m_08.name]
womens_200m_08 = womens_200m_08.drop([70,79,149,150,159,166])


athlete_df = pd.merge(athlete_df, womens_200m_08[['name', 'womens_200m_08']], how='left', on='name', suffixes=(None, '_200m_womens_beijing'))


#### Scraping results from the women's 200m dash 20012-London
removing rows with missing values

In [310]:
london_200m_women = fn.olympic_query('london', '2012', '200m-women')
womens_200m_12 = fn.olympic_scraper(london_200m_women)
womens_200m_12 = fn.content_cleaner(womens_200m_12)
womens_200m_12.columns = ['rank', 'name', 'womens_200m_12']
womens_200m_12.insert(3, "event_x", "Athletics Women's 200 metres")
womens_200m_12.womens_200m_12 = [str(x) for x in womens_200m_12.womens_200m_12]
womens_200m_12.womens_200m_12 = [x.strip() for x in womens_200m_12.womens_200m_12]
womens_200m_12.name = [x.strip() for x in womens_200m_12.name]
womens_200m_12 = womens_200m_12.drop([84,85])


athlete_df = pd.merge(athlete_df, womens_200m_12[['name', 'womens_200m_12']], how='left', on='name', suffixes=(None, '_200m_womens_london'))


#### Scraping results from the women's 200m dash 2016-Rio


In [311]:
rio_200m_women = fn.olympic_query('rio', '2016', '200m-women')
womens_200m_16 = fn.olympic_scraper(rio_200m_women)
womens_200m_16 = fn.content_cleaner(womens_200m_16)
womens_200m_16.columns = ['rank', 'name', 'womens_200m_16']
womens_200m_16.insert(3, "event_x", "Athletics Women's 200 metres")
womens_200m_16.womens_200m_16 = [str(x) for x in womens_200m_16.womens_200m_16]
womens_200m_16.womens_200m_16 = [x.strip() for x in womens_200m_16.womens_200m_16]
womens_200m_16.name = [x.strip() for x in womens_200m_16.name]


athlete_df = pd.merge(athlete_df, womens_200m_16[['name', 'womens_200m_16']], how='left', on='name', suffixes=(None, '_200m_womens_rio'))


#### Scraping results from the women's 400m dash 2004-Athens


In [312]:
athens_400m_women = fn.olympic_query('athens', '2004', '400m-women')
womens_400m_04 = fn.olympic_scraper(athens_400m_women)
womens_400m_04 = fn.content_cleaner(womens_400m_04)
womens_400m_04.columns = ['rank', 'name', 'womens_400m_04']
womens_400m_04.insert(3, "event_x", "Athletics Women's 400 metres")
womens_400m_04.womens_400m_04 = [str(x) for x in womens_400m_04.womens_400m_04]
womens_400m_04.womens_400m_04 = [x.strip() for x in womens_400m_04.womens_400m_04]
womens_400m_04.name = [x.strip() for x in womens_400m_04.name]


athlete_df = pd.merge(athlete_df, womens_400m_04[['name', 'womens_400m_04']], how='left', on='name', suffixes=(None, '_400m_womens_athens'))


#### Scraping results from the women's 400m dash 2008-Beijing


In [313]:
beijing_400m_women = fn.olympic_query('beijing', '2008', '400m-women')
womens_400m_08 = fn.olympic_scraper(beijing_400m_women)
womens_400m_08 = fn.content_cleaner(womens_400m_08)
womens_400m_08.columns = ['rank', 'name', 'womens_400m_08']
womens_400m_08.insert(3, "event_x", "Athletics Women's 400 metres")
womens_400m_08.womens_400m_08 = [str(x) for x in womens_400m_08.womens_400m_08]
womens_400m_08.womens_400m_08 = [x.strip() for x in womens_400m_08.womens_400m_08]
womens_400m_08.womens_400m_08 = [x.strip('DPG / ') for x in womens_400m_08.womens_400m_08]
womens_400m_08.name = [x.strip() for x in womens_400m_08.name]
womens_400m_08 = womens_400m_08.drop([102,103,124,132])
womens_400m_08.womens_400m_08 = [float(x) for x in womens_400m_08.womens_400m_08]


athlete_df = pd.merge(athlete_df, womens_400m_08[['name', 'womens_400m_08']], how='left', on='name', suffixes=(None, '_400m_womens_beijing'))


#### Scraping results from the women's 400m dash 2012-London
removing rows with missing values

In [314]:
london_400m_women = fn.olympic_query('london', '2012', '400m-women')
womens_400m_12 = fn.olympic_scraper(london_400m_women)
womens_400m_12 = fn.content_cleaner(womens_400m_12)
womens_400m_12.columns = ['rank', 'name', 'womens_400m_12']
womens_400m_12.insert(3, "event_x", "Athletics Men's 400 metres")
womens_400m_12.womens_400m_12 = [str(x) for x in womens_400m_12.womens_400m_12]
womens_400m_12.womens_400m_12 = [x.strip() for x in womens_400m_12.womens_400m_12]
womens_400m_12.womens_400m_12 = [x.strip('DPG / ') for x in womens_400m_12.womens_400m_12]
womens_400m_12.name = [x.strip() for x in womens_400m_12.name]
womens_400m_12 = womens_400m_12.drop([72,73,74,75,76,78,79])
womens_400m_12.womens_400m_12 = [float(x) for x in womens_400m_12.womens_400m_12]


athlete_df = pd.merge(athlete_df, womens_400m_12[['name', 'womens_400m_12']], how='left', on='name', suffixes=(None, '_400m_womens_london'))


#### Scraping results from the women's 400m dash 2016-Rio


In [315]:
rio_400m_women = fn.olympic_query('rio', '2016', '400m-women')
womens_400m_16 = fn.olympic_scraper(rio_400m_women)
womens_400m_16 = fn.content_cleaner(womens_400m_16)
womens_400m_16.columns = ['rank', 'name', 'womens_400m_16']
womens_400m_16.insert(3, "event_x", "Athletics Women's 400 metres")
womens_400m_16.womens_400m_16 = [str(x) for x in womens_400m_16.womens_400m_16]
womens_400m_16.womens_400m_16 = [x.strip() for x in womens_400m_16.womens_400m_16]
womens_400m_16.name = [x.strip() for x in womens_400m_16.name]


athlete_df = pd.merge(athlete_df, womens_400m_16[['name', 'womens_400m_16']], how='left', on='name', suffixes=(None, '_400m_womens_rio'))


#### Scraping results from women's 800m run 2004-Athens
removing rows with missing values

In [316]:
athens_800m_women = fn.olympic_query('athens', '2004', '800m-women')
womens_800m_04 = fn.olympic_scraper(athens_800m_women)
womens_800m_04 = fn.content_cleaner(womens_800m_04)
womens_800m_04.columns = ['rank', 'name', 'womens_800m_04']
womens_800m_04.insert(3, "event_x", "Athletics Women's 800 metres")
womens_800m_04.womens_800m_04 = [str(x) for x in womens_800m_04.womens_800m_04]
womens_800m_04.womens_800m_04 = [x.strip() for x in womens_800m_04.womens_800m_04]
womens_800m_04.womens_800m_04 = [x.replace('h','.')for x in womens_800m_04.womens_800m_04]
womens_800m_04.womens_800m_04 = [x.replace(':','.')for x in womens_800m_04.womens_800m_04]
womens_800m_04.name = [x.strip() for x in womens_800m_04.name]


athlete_df = pd.merge(athlete_df, womens_800m_04[['name', 'womens_800m_04']], how='left', on='name', suffixes=(None, '_800m_womens_athens'))


#### Scraping results from women's 800m run 2008-Beijing
removing rows with missing values

In [317]:
beijing_800m_women = fn.olympic_query('beijing', '2008', '800m-women')
womens_800m_08 = fn.olympic_scraper(beijing_800m_women)
womens_800m_08 = fn.content_cleaner(womens_800m_08)
womens_800m_08.columns = ['rank', 'name', 'womens_800m_08']
womens_800m_08.insert(3, "event_x", "Athletics Women's 800 metres")
womens_800m_08.womens_800m_08 = [str(x) for x in womens_800m_08.womens_800m_08]
womens_800m_08.womens_800m_08 = [x.strip() for x in womens_800m_08.womens_800m_08]
womens_800m_08.womens_800m_08 = [x.strip('DPG / ') for x in womens_800m_08.womens_800m_08]
womens_800m_08.name = [x.strip() for x in womens_800m_08.name]
womens_800m_08 = womens_800m_08.drop([30,55,94,95,125,139])
womens_800m_08.womens_800m_08 = [x.replace('h','.')for x in womens_800m_08.womens_800m_08]
womens_800m_08.womens_800m_08 = [x.replace(':','.')for x in womens_800m_08.womens_800m_08]


athlete_df = pd.merge(athlete_df, womens_800m_08[['name', 'womens_800m_08']], how='left', on='name', suffixes=(None, '_800m_womens_beijing'))


#### Scraping results from women's 800m run 2012-London
removing rows with missing values

In [318]:
london_800m_women = fn.olympic_query('london', '2012', '800m-women')
womens_800m_12 = fn.olympic_scraper(london_800m_women)
womens_800m_12 = fn.content_cleaner(womens_800m_12)
womens_800m_12.columns = ['rank', 'name', 'womens_800m_12']
womens_800m_12.insert(3, "event_x", "Athletics Women's 800 metres")
womens_800m_12.womens_800m_12 = [str(x) for x in womens_800m_12.womens_800m_12]
womens_800m_12.womens_800m_12 = [x.strip() for x in womens_800m_12.womens_800m_12]
womens_800m_12.name = [x.strip() for x in womens_800m_12.name]
womens_800m_12.womens_800m_12 = [x.strip('/ DPG') for x in womens_800m_12.womens_800m_12]
womens_800m_12 = womens_800m_12.drop(range(72,77))
womens_800m_12.womens_800m_12 = [x.replace('h','.')for x in womens_800m_12.womens_800m_12]
womens_800m_12.womens_800m_12 = [x.replace(':','.')for x in womens_800m_12.womens_800m_12]
womens_800m_12

athlete_df = pd.merge(athlete_df, womens_800m_12[['name', 'womens_800m_12']], how='left', on='name', suffixes=(None, '_800m_womens_london'))


#### Scraping results from women's 800m run 2016-Rio


In [319]:
rio_800m_women = fn.olympic_query('rio', '2016', '800m-women')
womens_800m_16 = fn.olympic_scraper(rio_800m_women)
womens_800m_16 = fn.content_cleaner(womens_800m_16)
womens_800m_16.columns = ['rank', 'name', 'womens_800m_16']
womens_800m_16.insert(3, "event_x", "Athletics Women's 800 metres")
womens_800m_16.womens_800m_16 = [str(x) for x in womens_800m_16.womens_800m_16]
womens_800m_16.womens_800m_16 = [x.strip() for x in womens_800m_16.womens_800m_16]
womens_800m_16.womens_800m_16 = [x.replace('h','.')for x in womens_800m_16.womens_800m_16]
womens_800m_16.womens_800m_16 = [x.replace(':','.')for x in womens_800m_16.womens_800m_16]
womens_800m_16.name = [x.strip() for x in womens_800m_16.name]


athlete_df = pd.merge(athlete_df, womens_800m_16[['name', 'womens_800m_16']], how='left', on='name', suffixes=(None, '_800m_womens_rio'))


#### Scraping results from the women's 1500m run 2004-Athens


In [320]:
athens_1500m_women = fn.olympic_query('athens', '2004', '1500m-women')
womens_1500m_04 = fn.olympic_scraper(athens_1500m_women)
womens_1500m_04 = fn.content_cleaner(womens_1500m_04)
womens_1500m_04.columns = ['rank', 'name', 'womens_1500m_04']
womens_1500m_04.insert(3, "event_x", "Athletics Women's 1500 metres")
womens_1500m_04.womens_1500m_04 = [str(x) for x in womens_1500m_04.womens_1500m_04]
womens_1500m_04.womens_1500m_04 = [x.strip() for x in womens_1500m_04.womens_1500m_04]
womens_1500m_04.womens_1500m_04 = [x.replace('h','.')for x in womens_1500m_04.womens_1500m_04]
womens_1500m_04.womens_1500m_04 = [x.replace(':','.')for x in womens_1500m_04.womens_1500m_04]
womens_1500m_04.name = [x.strip() for x in womens_1500m_04.name]


athlete_df = pd.merge(athlete_df, womens_1500m_04[['name', 'womens_1500m_04']], how='left', on='name', suffixes=(None, '_1500m_womens_athens'))


#### Scraping results from the women's 1500m run 2008-Beijing


In [321]:
beijing_1500m_women = fn.olympic_query('beijing', '2008', '1500m-women')
womens_1500m_08 = fn.olympic_scraper(beijing_1500m_women)
womens_1500m_08 = fn.content_cleaner(womens_1500m_08)
womens_1500m_08.columns = ['rank', 'name', 'womens_1500m_08']
womens_1500m_08.insert(3, "event_x", "Athletics Women's 1500 metres")
womens_1500m_08.womens_1500m_08 = [str(x) for x in womens_1500m_08.womens_1500m_08]
womens_1500m_08.womens_1500m_08 = [x.strip() for x in womens_1500m_08.womens_1500m_08]
womens_1500m_08.name = [x.strip() for x in womens_1500m_08.name]
womens_1500m_08.womens_1500m_08 = [x.strip('/ DPG') for x in womens_1500m_08.womens_1500m_08]
womens_1500m_08.womens_1500m_08 = [x.replace('h','.')for x in womens_1500m_08.womens_1500m_08]
womens_1500m_08.womens_1500m_08 = [x.replace(':','.')for x in womens_1500m_08.womens_1500m_08]


athlete_df = pd.merge(athlete_df, womens_1500m_08[['name', 'womens_1500m_08']], how='left', on='name', suffixes=(None, '_1500m_womens_beijing'))


#### Scraping results from the women's 1500m run 2012-London
removing rows with missing values

In [322]:
london_1500m_women = fn.olympic_query('london', '2012', '1500m-women')
womens_1500m_12 = fn.olympic_scraper(london_1500m_women)
womens_1500m_12 = fn.content_cleaner(womens_1500m_12)
womens_1500m_12.columns = ['rank', 'name', 'womens_1500m_12']
womens_1500m_12.insert(3, "event_x", "Athletics Women's 1500 metres")
womens_1500m_12.womens_1500m_12 = [str(x) for x in womens_1500m_12.womens_1500m_12]
womens_1500m_12.womens_1500m_12 = [x.strip() for x in womens_1500m_12.womens_1500m_12]
womens_1500m_12.name = [x.strip() for x in womens_1500m_12.name]
womens_1500m_12.womens_1500m_12 = [x.strip('/ DPG') for x in womens_1500m_12.womens_1500m_12]
womens_1500m_12 = womens_1500m_12.drop([8,77,78])
womens_1500m_12.womens_1500m_12 = [x.replace('h','.')for x in womens_1500m_12.womens_1500m_12]
womens_1500m_12.womens_1500m_12 = [x.replace(':','.')for x in womens_1500m_12.womens_1500m_12]


athlete_df = pd.merge(athlete_df, womens_1500m_12[['name', 'womens_1500m_12']], how='left', on='name', suffixes=(None, '_1500m_womens_london'))


#### Scraping results from the women's 1500m run 2016-Rio


In [323]:
rio_1500m_women = fn.olympic_query('rio', '2016', '1500m-women')
womens_1500m_16 = fn.olympic_scraper(rio_1500m_women)
womens_1500m_16 = fn.content_cleaner(womens_1500m_16)
womens_1500m_16.columns = ['rank', 'name', 'womens_1500m_16']
womens_1500m_16.insert(3, "event_x", "Athletics Women's 1500 metres")
womens_1500m_16.womens_1500m_16 = [str(x) for x in womens_1500m_16.womens_1500m_16]
womens_1500m_16.womens_1500m_16 = [x.strip() for x in womens_1500m_16.womens_1500m_16]
womens_1500m_16.name = [x.strip() for x in womens_1500m_16.name]
womens_1500m_16.womens_1500m_16 = [x.replace('h','.')for x in womens_1500m_16.womens_1500m_16]
womens_1500m_16.womens_1500m_16 = [x.replace(':','.')for x in womens_1500m_16.womens_1500m_16]


athlete_df = pd.merge(athlete_df, womens_1500m_16[['name', 'womens_1500m_16']], how='left', on='name', suffixes=(None, '_1500m_womens_rio'))


#### Scraping results from the women's 1500m run 2004-Athens
removing rows with missing values

In [324]:
athens_5000m_women = fn.olympic_query('athens', '2004', '5000m-women')
womens_5000m_04 = fn.olympic_scraper(athens_5000m_women)
womens_5000m_04 = fn.content_cleaner(womens_5000m_04)
womens_5000m_04.columns = ['rank', 'name', 'womens_5000m_04']
womens_5000m_04.insert(3, "event_x", "Athletics Women's 5000 metres")
womens_5000m_04.womens_5000m_04 = [str(x) for x in womens_5000m_04.womens_5000m_04]
womens_5000m_04.womens_5000m_04 = [x.strip() for x in womens_5000m_04.womens_5000m_04]
womens_5000m_04.womens_5000m_04 = [x.replace('h','.')for x in womens_5000m_04.womens_5000m_04]
womens_5000m_04.womens_5000m_04 = [x.replace(':','.')for x in womens_5000m_04.womens_5000m_04]
womens_5000m_04.name = [x.strip() for x in womens_5000m_04.name]


athlete_df = pd.merge(athlete_df, womens_5000m_04[['name', 'womens_5000m_04']], how='left', on='name', suffixes=(None, '_5000m_womens_athens'))


#### Scraping results from the women's 1500m run 2008-Beijing
removing rows with missing values

In [325]:
beijing_5000m_women = fn.olympic_query('beijing', '2008', '5000m-women')
womens_5000m_08 = fn.olympic_scraper(beijing_5000m_women)
womens_5000m_08 = fn.content_cleaner(womens_5000m_08)
womens_5000m_08.columns = ['rank', 'name', 'womens_5000m_08']
womens_5000m_08.insert(3, "event_x", "Athletics Women's 5000 metres")
womens_5000m_08.womens_5000m_08 = [str(x) for x in womens_5000m_08.womens_5000m_08]
womens_5000m_08.womens_5000m_08 = [x.strip() for x in womens_5000m_08.womens_5000m_08]
womens_5000m_08.name = [x.strip() for x in womens_5000m_08.name]
womens_5000m_08.womens_5000m_08 = [x.strip('/ DPG') for x in womens_5000m_08.womens_5000m_08]
womens_5000m_08 = womens_5000m_08.drop([45,46,77,78])
womens_5000m_08.womens_5000m_08 = [x.replace('h','.')for x in womens_5000m_08.womens_5000m_08]
womens_5000m_08.womens_5000m_08 = [x.replace(':','.')for x in womens_5000m_08.womens_5000m_08]


athlete_df = pd.merge(athlete_df, womens_5000m_08[['name', 'womens_5000m_08']], how='left', on='name', suffixes=(None, '_5000m_womens_beijing'))


#### Scraping results from the women's 1500m run 2012-London
removing rows with missing values

In [326]:
london_5000m_women = fn.olympic_query('london', '2012', '5000m-women')
womens_5000m_12 = fn.olympic_scraper(london_5000m_women)
womens_5000m_12 = fn.content_cleaner(womens_5000m_12)
womens_5000m_12.columns = ['rank', 'name', 'womens_5000m_12']
womens_5000m_12.insert(3, "event_x", "Athletics Women's 5000 metres")
womens_5000m_12.womens_5000m_12 = [str(x) for x in womens_5000m_12.womens_5000m_12]
womens_5000m_12.womens_5000m_12 = [x.strip() for x in womens_5000m_12.womens_5000m_12]
womens_5000m_12.name = [x.strip() for x in womens_5000m_12.name]
womens_5000m_12.womens_5000m_12 = [x.strip('/ DPG') for x in womens_5000m_12.womens_5000m_12]
womens_5000m_12 = womens_5000m_12.drop([49,50])
womens_5000m_12.womens_5000m_12 = [x.replace('h','.')for x in womens_5000m_12.womens_5000m_12]
womens_5000m_12.womens_5000m_12 = [x.replace(':','.')for x in womens_5000m_12.womens_5000m_12]


athlete_df = pd.merge(athlete_df, womens_5000m_12[['name', 'womens_5000m_12']], how='left', on='name', suffixes=(None, '_5000m_womens_london'))


#### Scraping results from the women's 1500m run 2016-Rio
removing row with missing value

In [327]:
rio_5000m_women = fn.olympic_query('rio', '2016', '5000m-women')
womens_5000m_16 = fn.olympic_scraper(rio_5000m_women)
womens_5000m_16 = fn.content_cleaner(womens_5000m_16)
womens_5000m_16.columns = ['rank', 'name', 'womens_5000m_16']
womens_5000m_16.insert(3, "event_x", "Athletics Women's 5000 metres")
womens_5000m_16.womens_5000m_16 = [str(x) for x in womens_5000m_16.womens_5000m_16]
womens_5000m_16.womens_5000m_16 = [x.strip() for x in womens_5000m_16.womens_5000m_16]
womens_5000m_16.name = [x.strip() for x in womens_5000m_16.name]
womens_5000m_16.womens_5000m_16 = [x.strip('DPG/') for x in womens_5000m_16.womens_5000m_16]
womens_5000m_16 = womens_5000m_16.drop(17)
womens_5000m_16.womens_5000m_16 = [x.replace('h','.')for x in womens_5000m_16.womens_5000m_16]
womens_5000m_16.womens_5000m_16 = [x.replace(':','.')for x in womens_5000m_16.womens_5000m_16]


athlete_df = pd.merge(athlete_df, womens_5000m_16[['name', 'womens_5000m_16']], how='left', on='name', suffixes=(None, '_5000m_womens_rio'))


#### Scraping results from the women's 100m hurdles 2004-Athens


In [328]:
athens_100H_women = fn.olympic_query('athens', '2004', '100m-hurdles-women')
womens_100H_04 = fn.olympic_scraper(athens_100H_women)
womens_100H_04 = fn.content_cleaner(womens_100H_04)
womens_100H_04.columns = ['rank', 'name', 'womens_100H_04']
womens_100H_04.insert(3, "event_x", "Athletics Women's 100m Hurdles")
womens_100H_04.womens_100H_04 = [str(x) for x in womens_100H_04.womens_100H_04]
womens_100H_04.womens_100H_04 = [x.strip() for x in womens_100H_04.womens_100H_04]
womens_100H_04.womens_100H_04 = [x.strip('/ DPG') for x in womens_100H_04.womens_100H_04]
womens_100H_04.name = [x.strip() for x in womens_100H_04.name]


athlete_df = pd.merge(athlete_df, womens_100H_04[['name', 'womens_100H_04']], how='left', on='name', suffixes=(None, '_100H_womens_athens'))


#### Scraping results from the women's 100m hurdles 2008-Beijing
removing rows with missing values

In [329]:
beijing_100H_women = fn.olympic_query('beijing', '2008', '100m-hurdles-women')
womens_100H_08 = fn.olympic_scraper(beijing_100H_women)
womens_100H_08 = fn.content_cleaner(womens_100H_08)
womens_100H_08.columns = ['rank', 'name', 'womens_100H_08']
womens_100H_08.insert(3, "event_x", "Athletics Women's 100m Hurdles")
womens_100H_08.womens_100H_08 = [str(x) for x in womens_100H_08.womens_100H_08]
womens_100H_08.womens_100H_08 = [x[:5] for x in womens_100H_08.womens_100H_08]
womens_100H_08.womens_100H_08 = [x.strip() for x in womens_100H_08.womens_100H_08]
womens_100H_08.womens_100H_08 = [x.strip('/ DPG') for x in womens_100H_08.womens_100H_08]
womens_100H_08.name = [x.strip() for x in womens_100H_08.name]
womens_100H_08 = womens_100H_08.drop([21,22,23,31,39,40,79,80,88,120])


athlete_df = pd.merge(athlete_df, womens_100H_08[['name', 'womens_100H_08']], how='left', on='name', suffixes=(None, '_100H_womens_beijing'))


#### Scraping results from the women's 100m hurdles 2012-London
removing rows with missing values

In [330]:
london_100H_women = fn.olympic_query('london', '2012', '100m-hurdles-women')
womens_100H_12 = fn.olympic_scraper(london_100H_women)
womens_100H_12 = fn.content_cleaner(womens_100H_12)
womens_100H_12.columns = ['rank', 'name', 'womens_100H_12']
womens_100H_12.insert(3, "event_x", "Athletics Women's 100m Hurdles")
womens_100H_12.womens_100H_12 = [str(x) for x in womens_100H_12.womens_100H_12]
womens_100H_12.womens_100H_12 = [x[:7] for x in womens_100H_12.womens_100H_12]
womens_100H_12.womens_100H_12 = [x.strip() for x in womens_100H_12.womens_100H_12]
womens_100H_12.womens_100H_12 = [x.strip('/ DPG') for x in womens_100H_12.womens_100H_12]
womens_100H_12.name = [x.strip() for x in womens_100H_12.name]
womens_100H_12 = womens_100H_12.drop([30,31,32,80,81,82,83])
womens_100H_12

athlete_df = pd.merge(athlete_df, womens_100H_12[['name', 'womens_100H_12']], how='left', on='name', suffixes=(None, '_100H_womens_london'))


#### Scraping results from the women's 100m hurdles 2016-Rio


In [331]:
rio_100H_women = fn.olympic_query('rio', '2016', '100m-hurdles-women')
womens_100H_16 = fn.olympic_scraper(rio_100H_women)
womens_100H_16 = fn.content_cleaner(womens_100H_16)
womens_100H_16.columns = ['rank', 'name', 'womens_100H_16']
womens_100H_16.insert(3, "event_x", "Athletics Women's 100m Hurdles")
womens_100H_16.womens_100H_16 = [str(x) for x in womens_100H_16.womens_100H_16]
womens_100H_16.womens_100H_16 = [x[:7] for x in womens_100H_16.womens_100H_16]
womens_100H_16.womens_100H_16 = [x.strip() for x in womens_100H_16.womens_100H_16]
womens_100H_16.womens_100H_16 = [x.strip('/ DPG') for x in womens_100H_16.womens_100H_16]
womens_100H_16.name = [x.strip() for x in womens_100H_16.name]


athlete_df = pd.merge(athlete_df, womens_100H_16[['name', 'womens_100H_16']], how='left', on='name', suffixes=(None, '_100H_womens_rio'))


#### Scraping results from the women's 400m hurdles 2004-Athens
removing rows with missing values

In [332]:
athens_400H_women = fn.olympic_query('athens', '2004', '400m-hurdles-women')
womens_400H_04 = fn.olympic_scraper(athens_400H_women)
womens_400H_04 = fn.content_cleaner(womens_400H_04)
womens_400H_04.columns = ['rank', 'name', 'womens_400H_04']
womens_400H_04.insert(3, "event_x", "Athletics Women's 400m Hurdles")
womens_400H_04.womens_400H_04 = [str(x) for x in womens_400H_04.womens_400H_04]
womens_400H_04.womens_400H_04 = [x.strip() for x in womens_400H_04.womens_400H_04]
womens_400H_04.womens_400H_04 = [x.strip('/ DPG') for x in womens_400H_04.womens_400H_04]
womens_400H_04.name = [x.strip() for x in womens_400H_04.name]


athlete_df = pd.merge(athlete_df, womens_400H_04[['name', 'womens_400H_04']], how='left', on='name', suffixes=(None, '_400H_womens_athens'))


#### Scraping results from the women's 400m hurdles 2008-Beijing
removing rows with missing values

In [333]:
beijing_400H_women = fn.olympic_query('beijing', '2008', '400m-hurdles-women')
womens_400H_08 = fn.olympic_scraper(beijing_400H_women)
womens_400H_08 = fn.content_cleaner(womens_400H_08)
womens_400H_08.columns = ['rank', 'name', 'womens_400H_08']
womens_400H_08.insert(3, "event_x", "Athletics Women's 400m Hurdles")
womens_400H_08.womens_400H_08 = [str(x) for x in womens_400H_08.womens_400H_08]
womens_400H_08.womens_400H_08 = [x[:7] for x in womens_400H_08.womens_400H_08]
womens_400H_08.womens_400H_08 = [x.strip() for x in womens_400H_08.womens_400H_08]
womens_400H_08.womens_400H_08 = [x.strip('/ DPG') for x in womens_400H_08.womens_400H_08]
womens_400H_08.name = [x.strip() for x in womens_400H_08.name]
womens_400H_08 = womens_400H_08.drop([66,86])
womens_400H_08.womens_400H_08 = [float(x) for x in womens_400H_08.womens_400H_08]


athlete_df = pd.merge(athlete_df, womens_400H_08[['name', 'womens_400H_08']], how='left', on='name', suffixes=(None, '_400H_womens_beijing'))


#### Scraping results from the women's 400m hurdles 2012-London
removing rows with missing values

In [334]:
london_400H_women = fn.olympic_query('london', '2012', '400m-hurdles-women')
womens_400H_12 = fn.olympic_scraper(london_400H_women)
womens_400H_12 = fn.content_cleaner(womens_400H_12)
womens_400H_12.columns = ['rank', 'name', 'womens_400H_12']
womens_400H_12.insert(3, "event_x", "Athletics Women's 400m Hurdles")
womens_400H_12.womens_400H_12 = [str(x) for x in womens_400H_12.womens_400H_12]
womens_400H_12.womens_400H_12 = [x[:7] for x in womens_400H_12.womens_400H_12]
womens_400H_12.womens_400H_12 = [x.strip() for x in womens_400H_12.womens_400H_12]
womens_400H_12.womens_400H_12 = [x.strip('/ DPG') for x in womens_400H_12.womens_400H_12]
womens_400H_12.name = [x.strip() for x in womens_400H_12.name]
womens_400H_12 = womens_400H_12.drop([74,75,76,77])
womens_400H_12.womens_400H_12 = [float(x) for x in womens_400H_12.womens_400H_12]


athlete_df = pd.merge(athlete_df, womens_400H_12[['name', 'womens_400H_12']], how='left', on='name', suffixes=(None, '_400H_womens_london'))


#### Scraping results from the women's 400m hurdles 2016-Rio
removing rows with missing values

In [335]:
rio_400H_women = fn.olympic_query('rio', '2016', '400m-hurdles-women')
womens_400H_16 = fn.olympic_scraper(rio_400H_women)
womens_400H_16 = fn.content_cleaner(womens_400H_16)
womens_400H_16.columns = ['rank', 'name', 'womens_400H_16']
womens_400H_16.insert(3, "event_x", "Athletics Women's 400m Hurdles")
womens_400H_16.womens_400H_16 = [str(x) for x in womens_400H_16.womens_400H_16]
womens_400H_16.womens_400H_16 = [x[:7] for x in womens_400H_16.womens_400H_16]
womens_400H_16.womens_400H_16 = [x.strip() for x in womens_400H_16.womens_400H_16]
womens_400H_16.womens_400H_16 = [x.strip('/ DPG') for x in womens_400H_16.womens_400H_16]
womens_400H_16.name = [x.strip() for x in womens_400H_16.name]


athlete_df = pd.merge(athlete_df, womens_400H_16[['name', 'womens_400H_16']], how='left', on='name', suffixes=(None, '_400H_womens_rio'))


#### Scraping results from the women's 10000m run 2004-Athens


In [336]:
athens_10000m_women = fn.olympic_query('athens', '2004', '10000m-women')
womens_10000m_04 = fn.olympic_scraper(athens_10000m_women)
womens_10000m_04 = fn.content_cleaner(womens_10000m_04)
womens_10000m_04.columns = ['rank', 'name', 'womens_10000m_04']
womens_10000m_04.insert(3, "event_x", "Athletics Women's 10000 metres")
womens_10000m_04.womens_10000m_04 = [str(x) for x in womens_10000m_04.womens_10000m_04]
womens_10000m_04.womens_10000m_04 = [x.strip() for x in womens_10000m_04.womens_10000m_04]
womens_10000m_04.womens_10000m_04 = [x.replace('h','.')for x in womens_10000m_04.womens_10000m_04]
womens_10000m_04.womens_10000m_04 = [x.replace(':','.')for x in womens_10000m_04.womens_10000m_04]
womens_10000m_04.name = [x.strip() for x in womens_10000m_04.name]


athlete_df = pd.merge(athlete_df, womens_10000m_04[['name', 'womens_10000m_04']], how='left', on='name', suffixes=(None, '_10000m_womens_athens'))


#### Scraping results from the women's 10000m run 2008-Beijing
removing rows with missing values

In [337]:
beijing_10000m_women = fn.olympic_query('beijing', '2008', '10000m-women')
womens_10000m_08 = fn.olympic_scraper(beijing_10000m_women)
womens_10000m_08 = fn.content_cleaner(womens_10000m_08)
womens_10000m_08.columns = ['rank', 'name', 'womens_10000m_08']
womens_10000m_08.insert(3, "event_x", "Athletics Women's 10000 metres")
womens_10000m_08.womens_10000m_08 = [str(x) for x in womens_10000m_08.womens_10000m_08]
womens_10000m_08.womens_10000m_08 = [x.strip() for x in womens_10000m_08.womens_10000m_08]
womens_10000m_08.name = [x.strip() for x in womens_10000m_08.name]
womens_10000m_08.womens_10000m_08 = [x.strip('/ DPG') for x in womens_10000m_08.womens_10000m_08]
womens_10000m_08 = womens_10000m_08.drop([28,29,31])
womens_10000m_08.womens_10000m_08 = [x.replace('h','.')for x in womens_10000m_08.womens_10000m_08]
womens_10000m_08.womens_10000m_08 = [x.replace(':','.')for x in womens_10000m_08.womens_10000m_08]


athlete_df = pd.merge(athlete_df, womens_10000m_08[['name', 'womens_10000m_08']], how='left', on='name', suffixes=(None, '_10000m_womens_beijing'))


#### Scraping results from the women's 10000m run 2012-London
removing rows with missing values

In [338]:
london_10000m_women = fn.olympic_query('london', '2012', '10000m-women')
womens_10000m_12 = fn.olympic_scraper(london_10000m_women)
womens_10000m_12 = fn.content_cleaner(womens_10000m_12)
womens_10000m_12.columns = ['rank', 'name', 'womens_10000m_12']
womens_10000m_12.insert(3, "event_x", "Athletics Women's 10000 metres")
womens_10000m_12.womens_10000m_12 = [str(x) for x in womens_10000m_12.womens_10000m_12]
womens_10000m_12.womens_10000m_12 = [x.strip() for x in womens_10000m_12.womens_10000m_12]
womens_10000m_12.name = [x.strip() for x in womens_10000m_12.name]
womens_10000m_12 = womens_10000m_12.drop([21])
womens_10000m_12.womens_10000m_12 = [x.replace('h','.')for x in womens_10000m_12.womens_10000m_12]
womens_10000m_12.womens_10000m_12 = [x.replace(':','.')for x in womens_10000m_12.womens_10000m_12]


athlete_df = pd.merge(athlete_df, womens_10000m_12[['name', 'womens_10000m_12']], how='left', on='name', suffixes=(None, '_10000m_womens_london'))


#### Scraping results from the women's 10000m run 2016-Rio
removing rows with missing values

In [339]:
rio_10000m_women = fn.olympic_query('rio', '2016', '10000m-women')
womens_10000m_16 = fn.olympic_scraper(rio_10000m_women)
womens_10000m_16 = fn.content_cleaner(womens_10000m_16)
womens_10000m_16.columns = ['rank', 'name', 'womens_10000m_16']
womens_10000m_16.insert(3, "event_x", "Athletics Women's 10000 metres")
womens_10000m_16.womens_10000m_16 = [str(x) for x in womens_10000m_16.womens_10000m_16]
womens_10000m_16.womens_10000m_16 = [x.strip() for x in womens_10000m_16.womens_10000m_16]
womens_10000m_16.name = [x.strip() for x in womens_10000m_16.name]
womens_10000m_16 = womens_10000m_16.drop([35,36])
womens_10000m_16.womens_10000m_16 = [x.replace('h','.')for x in womens_10000m_16.womens_10000m_16]
womens_10000m_16.womens_10000m_16 = [x.replace(':','.')for x in womens_10000m_16.womens_10000m_16]


athlete_df = pd.merge(athlete_df, womens_10000m_16[['name', 'womens_10000m_16']], how='left', on='name', suffixes=(None, '_10000m_womens_rio'))


#### Scraping results from the women's 20km walk 2004-Athens
removing rows with missing values

In [340]:
athens_womens_20km = fn.olympic_query('athens', '2004', '20km-race-walk-women')
womens_20km = fn.olympic_scraper(athens_womens_20km)
womens_20km = fn.content_cleaner(womens_20km)
womens_20km.columns = ['rank', 'name', 'womens_20km']
womens_20km.insert(3, "event_x", "Athletics Women's 20km walk")
womens_20km.womens_20km = [str(x) for x in womens_20km.womens_20km]
womens_20km.womens_20km = [x.strip() for x in womens_20km.womens_20km]
womens_20km.womens_20km = [x.strip('/ DPG') for x in womens_20km.womens_20km]
womens_20km.womens_20km = [x.replace('h','.')for x in womens_20km.womens_20km]
womens_20km.womens_20km = [x.replace(':','.')for x in womens_20km.womens_20km]
womens_20km.name = [x.strip() for x in womens_20km.name]


athlete_df = pd.merge(athlete_df, womens_20km[['name', 'womens_20km']], how='left', on='name', suffixes=(None, '_20km_womens_athens'))


#### Scraping results from the women's 20km walk 2008-Beijing
removing rows with missing values

In [341]:
beijing_womens_20km = fn.olympic_query('beijing', '2008', '20km-race-walk-women')
womens_20km_08 = fn.olympic_scraper(beijing_womens_20km)
womens_20km_08 = fn.content_cleaner(womens_20km_08)
womens_20km_08.columns = ['rank', 'name', 'womens_20km_08']
womens_20km_08.insert(3, "event_x", "Athletics Women's 20km walk")
womens_20km_08.womens_20km_08 = [str(x) for x in womens_20km_08.womens_20km_08]
womens_20km_08.womens_20km_08 = [x.strip() for x in womens_20km_08.womens_20km_08]
womens_20km_08.womens_20km_08 = [x.strip('/ DPG') for x in womens_20km_08.womens_20km_08]
womens_20km_08.womens_20km_08 = [x.replace('h','.')for x in womens_20km_08.womens_20km_08]
womens_20km_08.womens_20km_08 = [x.replace(':','.')for x in womens_20km_08.womens_20km_08]
womens_20km_08.name = [x.strip() for x in womens_20km_08.name]
womens_20km_08 = womens_20km_08.drop(range(42,47))


athlete_df = pd.merge(athlete_df, womens_20km_08[['name', 'womens_20km_08']], how='left', on='name', suffixes=(None, '_20km_womens_beijing'))


#### Scraping results from the women's 20km walk 2012-London
removing rows with missing values

In [342]:
london_womens_20km = fn.olympic_query('london', '2012', '20km-race-walk-women')
womens_20km_12 = fn.olympic_scraper(london_womens_20km)
womens_20km_12 = fn.content_cleaner(womens_20km_12)
womens_20km_12.columns = ['rank', 'name', 'womens_20km_12']
womens_20km_12.insert(3, "event_x", "Athletics Womens's 20km walk")
womens_20km_12.womens_20km_12 = [str(x) for x in womens_20km_12.womens_20km_12]
womens_20km_12.womens_20km_12 = [x.strip() for x in womens_20km_12.womens_20km_12]
womens_20km_12.womens_20km_12 = [x.strip('/ DPG') for x in womens_20km_12.womens_20km_12]
womens_20km_12.womens_20km_12 = [x.replace('h','.')for x in womens_20km_12.womens_20km_12]
womens_20km_12.womens_20km_12 = [x.replace(':','.')for x in womens_20km_12.womens_20km_12]
womens_20km_12.name = [x.strip() for x in womens_20km_12.name]
womens_20km_12 = womens_20km_12.drop(range(54,60))


athlete_df = pd.merge(athlete_df, womens_20km_12[['name', 'womens_20km_12']], how='left', on='name', suffixes=(None, '_20km_womens_london'))


#### Scraping results from the women's 20km walk 2016-Rio
removing rows with missing values

In [343]:
rio_womens_20km = fn.olympic_query('rio', '2016', '20km-race-walk-women')
womens_20km_16 = fn.olympic_scraper(rio_womens_20km)
womens_20km_16 = fn.content_cleaner(womens_20km_16)
womens_20km_16.columns = ['rank', 'name', 'womens_20km_16']
womens_20km_16.insert(3, "event_x", "Athletics Women's 20km walk")
womens_20km_16.womens_20km_16 = [str(x) for x in womens_20km_16.womens_20km_16]
womens_20km_16.womens_20km_16 = [x.strip() for x in womens_20km_16.womens_20km_16]
womens_20km_16.womens_20km_16 = [x.strip('/ DPG') for x in womens_20km_16.womens_20km_16]
womens_20km_16.womens_20km_16 = [x.replace('h','.')for x in womens_20km_16.womens_20km_16]
womens_20km_16.womens_20km_16 = [x.replace(':','.')for x in womens_20km_16.womens_20km_16]
womens_20km_16.name = [x.strip() for x in womens_20km_16.name]
womens_20km_16 = womens_20km_16.drop(range(63,74))


athlete_df = pd.merge(athlete_df, womens_20km_16[['name', 'womens_20km_16']], how='left', on='name', suffixes=(None, '_20km_womens_rio'))


### There is no 50km walking race for women in any of the 4 Olympic Games that I am gathering data for. I am going to move on to the steeplechase beginning in 2008, there was no women's steeple-chase event during the 2004 Athens Summer Olympics Games

#### Scraping results for the women's 3000m Steeplechase 2008-Beijing
removing rows with missing values

In [344]:
beijing_womens_3000Steeple_08 = fn.olympic_query('beijing', '2008', '3000m-steeplechase-women')
womens_3000Steeple_08 = fn.olympic_scraper(beijing_womens_3000Steeple_08)
womens_3000Steeple_08 = fn.content_cleaner(womens_3000Steeple_08)
womens_3000Steeple_08.columns = ['rank', 'name', 'womens_3000Steeple_08']
womens_3000Steeple_08.insert(3, "event_x", "Athletics Womens's 3000Steeple_08")
womens_3000Steeple_08.womens_3000Steeple_08 = [str(x) for x in womens_3000Steeple_08.womens_3000Steeple_08]
womens_3000Steeple_08.womens_3000Steeple_08 = [x.strip() for x in womens_3000Steeple_08.womens_3000Steeple_08]
womens_3000Steeple_08.womens_3000Steeple_08 = [x.strip('/ DPG') for x in womens_3000Steeple_08.womens_3000Steeple_08]
womens_3000Steeple_08.womens_3000Steeple_08 = [x.replace('h','.')for x in womens_3000Steeple_08.womens_3000Steeple_08]
womens_3000Steeple_08.womens_3000Steeple_08 = [x.replace(':','.')for x in womens_3000Steeple_08.womens_3000Steeple_08]
womens_3000Steeple_08.name = [x.strip() for x in womens_3000Steeple_08.name]
womens_3000Steeple_08 = womens_3000Steeple_08.drop([13,61,63,64,65,99,113,114,115])


athlete_df = pd.merge(athlete_df, womens_3000Steeple_08[['name', 'womens_3000Steeple_08']], how='left', on='name', suffixes=(None, '_3000Steeple_womens_beijing'))


#### Scraping results for the women's 3000m Steeplechase 2012-London
removing row with missing value

In [345]:
london_womens_3000Steeple_12 = fn.olympic_query('london', '2012', '3000m-steeplechase-women')
womens_3000Steeple_12 = fn.olympic_scraper(london_womens_3000Steeple_12)
womens_3000Steeple_12 = fn.content_cleaner(womens_3000Steeple_12)
womens_3000Steeple_12.columns = ['rank', 'name', 'womens_3000Steeple_12']
womens_3000Steeple_12.insert(3, "event_x", "Athletics Women's 3000Steeple_12")
womens_3000Steeple_12.womens_3000Steeple_12 = [str(x) for x in womens_3000Steeple_12.womens_3000Steeple_12]
womens_3000Steeple_12.womens_3000Steeple_12 = [x.strip() for x in womens_3000Steeple_12.womens_3000Steeple_12]
womens_3000Steeple_12.womens_3000Steeple_12 = [x.strip('/ DPG') for x in womens_3000Steeple_12.womens_3000Steeple_12]
womens_3000Steeple_12.womens_3000Steeple_12 = [x.replace('h','.')for x in womens_3000Steeple_12.womens_3000Steeple_12]
womens_3000Steeple_12.womens_3000Steeple_12 = [x.replace(':','.')for x in womens_3000Steeple_12.womens_3000Steeple_12]
womens_3000Steeple_12.name = [x.strip() for x in womens_3000Steeple_12.name]
womens_3000Steeple_12 = womens_3000Steeple_12.drop(13)


athlete_df = pd.merge(athlete_df, womens_3000Steeple_12[['name', 'womens_3000Steeple_12']], how='left', on='name', suffixes=(None, '_3000Steeple_womens_london'))


#### Scraping results for the women's 3000m Steeplechase 2016-Rio


In [346]:
rio_womens_3000Steeple_16 = fn.olympic_query('rio', '2016', '3000m-steeplechase-women')
womens_3000Steeple_16 = fn.olympic_scraper(rio_womens_3000Steeple_16)
womens_3000Steeple_16 = fn.content_cleaner(womens_3000Steeple_16)
womens_3000Steeple_16.columns = ['rank', 'name', 'womens_3000Steeple_16']
womens_3000Steeple_16.insert(3, "event_x", "Athletics Women's 3000Steeple_16")
womens_3000Steeple_16.womens_3000Steeple_16 = [str(x) for x in womens_3000Steeple_16.womens_3000Steeple_16]
womens_3000Steeple_16.womens_3000Steeple_16 = [x.strip() for x in womens_3000Steeple_16.womens_3000Steeple_16]
womens_3000Steeple_16.womens_3000Steeple_16 = [x.strip('/ DPG') for x in womens_3000Steeple_16.womens_3000Steeple_16]
womens_3000Steeple_16.womens_3000Steeple_16 = [x.replace('h','.')for x in womens_3000Steeple_16.womens_3000Steeple_16]
womens_3000Steeple_16.womens_3000Steeple_16 = [x.replace(':','.')for x in womens_3000Steeple_16.womens_3000Steeple_16]
womens_3000Steeple_16.name = [x.strip() for x in womens_3000Steeple_16.name]


athlete_df = pd.merge(athlete_df, womens_3000Steeple_16[['name', 'womens_3000Steeple_16']], how='left', on='name', suffixes=(None, '_3000Steeple_womens_london'))


#### Scraping results for the women's discus throw 2004-Athens
removing rows with missing values

In [347]:
athens_womens_discus = fn.olympic_query('athens', '2004', 'discus-throw-women')
womens_discus_04 = fn.olympic_scraper(athens_womens_discus)
womens_discus_04 = fn.content_cleaner(womens_discus_04)
womens_discus_04.columns = ['rank', 'name', 'womens_discus_04']
womens_discus_04.insert(3, "event_x", "Athletics Women's Discus")
womens_discus_04.womens_discus_04 = [str(x) for x in womens_discus_04.womens_discus_04]
womens_discus_04.womens_discus_04 = [x.strip() for x in womens_discus_04.womens_discus_04]
womens_discus_04.womens_discus_04 = [x.strip('/ DPG') for x in womens_discus_04.womens_discus_04]
womens_discus_04.womens_discus_04 = [x.replace('h','.')for x in womens_discus_04.womens_discus_04]
womens_discus_04.womens_discus_04 = [x.replace(':','.')for x in womens_discus_04.womens_discus_04]
womens_discus_04.name = [x.strip() for x in womens_discus_04.name]


athlete_df = pd.merge(athlete_df, womens_discus_04[['name', 'womens_discus_04']], how='left', on='name', suffixes=(None, '_discus_womens_athens'))


#### Scraping results for the women's discus throw 2008-Beijing
removing rows with missing values

In [348]:
beijing_womens_discus = fn.olympic_query('beijing', '2008', 'discus-throw-women')
womens_discus_08 = fn.olympic_scraper(beijing_womens_discus)
womens_discus_08 = fn.content_cleaner(womens_discus_08)
womens_discus_08.columns = ['rank', 'name', 'womens_discus_08']
womens_discus_08.insert(3, "event_x", "Athletics Women's Discus")
womens_discus_08.womens_discus_08 = [str(x) for x in womens_discus_08.womens_discus_08]
womens_discus_08.womens_discus_08 = [x.strip() for x in womens_discus_08.womens_discus_08]
womens_discus_08.womens_discus_08 = [x.strip('/ DPG') for x in womens_discus_08.womens_discus_08]
womens_discus_08.womens_discus_08 = [x.replace('h','.')for x in womens_discus_08.womens_discus_08]
womens_discus_08.womens_discus_08 = [x.replace(':','.')for x in womens_discus_08.womens_discus_08]
womens_discus_08.name = [x.strip() for x in womens_discus_08.name]
womens_discus_08 = womens_discus_08.drop([48,87])


athlete_df = pd.merge(athlete_df, womens_discus_08[['name', 'womens_discus_08']], how='left', on='name', suffixes=(None, '_discus_womens_beijing'))


#### Scraping results for the women's discus throw 2012-London
removing rows with missing values

In [349]:
london_womens_discus = fn.olympic_query('london', '2012', 'discus-throw-women')
womens_discus_12 = fn.olympic_scraper(london_womens_discus)
womens_discus_12 = fn.content_cleaner(womens_discus_12)
womens_discus_12.columns = ['rank', 'name', 'womens_discus_12']
womens_discus_12.insert(3, "event_x", "Athletics Women's Discus")
womens_discus_12.womens_discus_12 = [str(x) for x in womens_discus_12.womens_discus_12]
womens_discus_12.womens_discus_12 = [x.strip() for x in womens_discus_12.womens_discus_12]
womens_discus_12.womens_discus_12 = [x.strip('/ DPG') for x in womens_discus_12.womens_discus_12]
womens_discus_12.womens_discus_12 = [x.replace('h','.')for x in womens_discus_12.womens_discus_12]
womens_discus_12.womens_discus_12 = [x.replace(':','.')for x in womens_discus_12.womens_discus_12]
womens_discus_12.name = [x.strip() for x in womens_discus_12.name]
womens_discus_12 = womens_discus_12.drop(44)


athlete_df = pd.merge(athlete_df, womens_discus_12[['name', 'womens_discus_12']], how='left', on='name', suffixes=(None, '_discus_womens_london'))


#### Scraping results for the women's discus throw 2016-Rio
removing rows with missing values

In [350]:
rio_womens_discus = fn.olympic_query('rio', '2016', 'discus-throw-women')
womens_discus_16 = fn.olympic_scraper(rio_womens_discus)
womens_discus_16 = fn.content_cleaner(womens_discus_16)
womens_discus_16.columns = ['rank', 'name', 'womens_discus_16']
womens_discus_16.insert(3, "event_x", "Athletics Women's Discus")
womens_discus_16.womens_discus_16 = [str(x) for x in womens_discus_16.womens_discus_16]
womens_discus_16.womens_discus_16 = [x.strip() for x in womens_discus_16.womens_discus_16]
womens_discus_16.womens_discus_16 = [x.strip('/ DPG') for x in womens_discus_16.womens_discus_16]
womens_discus_16.womens_discus_16 = [x.replace('h','.')for x in womens_discus_16.womens_discus_16]
womens_discus_16.womens_discus_16 = [x.replace(':','.')for x in womens_discus_16.womens_discus_16]
womens_discus_16.name = [x.strip() for x in womens_discus_16.name]
womens_discus_16 = womens_discus_16.drop(11)


athlete_df = pd.merge(athlete_df, womens_discus_16[['name', 'womens_discus_16']], how='left', on='name', suffixes=(None, '_discus_womens_rio'))


#### Scraping results for the women's hammer throw 2004-Athens


In [351]:
athens_womens_hammer = fn.olympic_query('athens', '2004', 'hammer-throw-women')
womens_hammer_04 = fn.olympic_scraper(athens_womens_hammer)
womens_hammer_04 = fn.content_cleaner(womens_hammer_04)
womens_hammer_04.columns = ['rank', 'name', 'womens_hammer_04']
womens_hammer_04.insert(3, "event_x", "Athletics Women's Hammer")
womens_hammer_04.womens_hammer_04 = [str(x) for x in womens_hammer_04.womens_hammer_04]
womens_hammer_04.womens_hammer_04 = [x.strip() for x in womens_hammer_04.womens_hammer_04]
womens_hammer_04.womens_hammer_04 = [x.strip('/ DPG') for x in womens_hammer_04.womens_hammer_04]
womens_hammer_04.womens_hammer_04 = [x.replace('h','.')for x in womens_hammer_04.womens_hammer_04]
womens_hammer_04.womens_hammer_04 = [x.replace(':','.')for x in womens_hammer_04.womens_hammer_04]
womens_hammer_04.name = [x.strip() for x in womens_hammer_04.name]


athlete_df = pd.merge(athlete_df, womens_hammer_04[['name', 'womens_hammer_04']], how='left', on='name', suffixes=(None, '_hammer_womens_athens'))


#### Scraping results for the women's hammer throw 2008-Beijing
removing rows with missing values

In [352]:
beijing_womens_hammer = fn.olympic_query('beijing', '2008', 'hammer-throw-women')
womens_hammer_08 = fn.olympic_scraper(beijing_womens_hammer)
womens_hammer_08 = fn.content_cleaner(womens_hammer_08)
womens_hammer_08.columns = ['rank', 'name', 'womens_hammer_08']
womens_hammer_08.insert(3, "event_x", "Athletics Women's Hammer")
womens_hammer_08.womens_hammer_08 = [str(x) for x in womens_hammer_08.womens_hammer_08]
womens_hammer_08.womens_hammer_08 = [x.strip() for x in womens_hammer_08.womens_hammer_08]
womens_hammer_08.womens_hammer_08 = [x.strip('/ DPG') for x in womens_hammer_08.womens_hammer_08]
womens_hammer_08.womens_hammer_08 = [x.replace('h','.')for x in womens_hammer_08.womens_hammer_08]
womens_hammer_08.womens_hammer_08 = [x.replace(':','.')for x in womens_hammer_08.womens_hammer_08]
womens_hammer_08.name = [x.strip() for x in womens_hammer_08.name]
womens_hammer_08 = womens_hammer_08.drop([10,57,58,59])


athlete_df = pd.merge(athlete_df, womens_hammer_08[['name', 'womens_hammer_08']], how='left', on='name', suffixes=(None, '_hammer_womens_beijing'))


#### Scraping results for the women's hammer throw 2012-London
removing row with missing value

In [353]:
london_womens_hammer = fn.olympic_query('london', '2012', 'hammer-throw-women')
womens_hammer_12 = fn.olympic_scraper(london_womens_hammer)
womens_hammer_12 = fn.content_cleaner(womens_hammer_12)
womens_hammer_12.columns = ['rank', 'name', 'womens_hammer_12']
womens_hammer_12.insert(3, "event_x", "Athletics Women's Hammer")
womens_hammer_12.womens_hammer_12 = [str(x) for x in womens_hammer_12.womens_hammer_12]
womens_hammer_12.womens_hammer_12 = [x.strip() for x in womens_hammer_12.womens_hammer_12]
womens_hammer_12.womens_hammer_12 = [x.strip('/ DPG') for x in womens_hammer_12.womens_hammer_12]
womens_hammer_12.womens_hammer_12 = [x.replace('h','.')for x in womens_hammer_12.womens_hammer_12]
womens_hammer_12.womens_hammer_12 = [x.replace(':','.')for x in womens_hammer_12.womens_hammer_12]
womens_hammer_12.name = [x.strip() for x in womens_hammer_12.name]
womens_hammer_12 = womens_hammer_12.drop(42)


athlete_df = pd.merge(athlete_df, womens_hammer_12[['name', 'womens_hammer_12']], how='left', on='name', suffixes=(None, '_hammer_womens_london'))


#### Scraping results for the women's hammer throw 2016-Rio
removing row with missing value

In [354]:
rio_womens_hammer = fn.olympic_query('rio', '2016', 'hammer-throw-women')
womens_hammer_16 = fn.olympic_scraper(rio_womens_hammer)
womens_hammer_16 = fn.content_cleaner(womens_hammer_16)
womens_hammer_16.columns = ['rank', 'name', 'womens_hammer_16']
womens_hammer_16.insert(3, "event_x", "Athletics Women's Hammer")
womens_hammer_16.womens_hammer_16 = [str(x) for x in womens_hammer_16.womens_hammer_16]
womens_hammer_16.womens_hammer_16 = [x.strip() for x in womens_hammer_16.womens_hammer_16]
womens_hammer_16.womens_hammer_16 = [x.strip('/ DPG') for x in womens_hammer_16.womens_hammer_16]
womens_hammer_16.womens_hammer_16 = [x.replace('h','.')for x in womens_hammer_16.womens_hammer_16]
womens_hammer_16.womens_hammer_16 = [x.replace(':','.')for x in womens_hammer_16.womens_hammer_16]
womens_hammer_16.name = [x.strip() for x in womens_hammer_16.name]
womens_hammer_16 = womens_hammer_16.drop(11)


athlete_df = pd.merge(athlete_df, womens_hammer_16[['name', 'womens_hammer_16']], how='left', on='name', suffixes=(None, '_hammer_womens_rio'))


#### Scraping results for the women's shot put throw 2014-Athens


In [355]:
athens_womens_shot = fn.olympic_query('athens', '2004', 'shot-put-women')
womens_shot_04 = fn.olympic_scraper(athens_womens_shot)
womens_shot_04 = fn.content_cleaner(womens_shot_04)
womens_shot_04.columns = ['rank', 'name', 'womens_shot_04']
womens_shot_04.insert(3, "event_x", "Athletics Women's shot")
womens_shot_04.womens_shot_04 = [str(x) for x in womens_shot_04.womens_shot_04]
womens_shot_04.womens_shot_04 = [x.strip() for x in womens_shot_04.womens_shot_04]
womens_shot_04.womens_shot_04 = [x.strip('/ DPG') for x in womens_shot_04.womens_shot_04]
womens_shot_04.womens_shot_04 = [x.replace('h','.')for x in womens_shot_04.womens_shot_04]
womens_shot_04.womens_shot_04 = [x.replace(':','.')for x in womens_shot_04.womens_shot_04]
womens_shot_04.name = [x.strip() for x in womens_shot_04.name]


athlete_df = pd.merge(athlete_df, womens_shot_04[['name', 'womens_shot_04']], how='left', on='name', suffixes=(None, '_shot_womens_athens'))


#### Scraping results for the women's shot put throw 2008-Beijing
removing rows with missing values

In [356]:
beijing_womens_shot = fn.olympic_query('beijing', '2008', 'shot-put-women')
womens_shot_08 = fn.olympic_scraper(beijing_womens_shot)
womens_shot_08 = fn.content_cleaner(womens_shot_08)
womens_shot_08.columns = ['rank', 'name', 'womens_shot_08']
womens_shot_08.insert(3, "event_x", "Athletics Women's Shot")
womens_shot_08.womens_shot_08 = [str(x) for x in womens_shot_08.womens_shot_08]
womens_shot_08.womens_shot_08 = [x.strip() for x in womens_shot_08.womens_shot_08]
womens_shot_08.womens_shot_08 = [x.strip('/ DPG') for x in womens_shot_08.womens_shot_08]
womens_shot_08.womens_shot_08 = [x.replace('h','.')for x in womens_shot_08.womens_shot_08]
womens_shot_08.womens_shot_08 = [x.replace(':','.')for x in womens_shot_08.womens_shot_08]
womens_shot_08.name = [x.strip() for x in womens_shot_08.name]
womens_shot_08 = womens_shot_08.drop([47,48,66,83])


athlete_df = pd.merge(athlete_df, womens_shot_08[['name', 'womens_shot_08']], how='left', on='name', suffixes=(None, '_shot_womens_beijing'))


#### Scraping results for the women's shot put throw 2012-London
removing row with missing value

In [357]:
london_womens_shot = fn.olympic_query('london', '2012', 'shot-put-women')
womens_shot_12 = fn.olympic_scraper(london_womens_shot)
womens_shot_12 = fn.content_cleaner(womens_shot_12)
womens_shot_12.columns = ['rank', 'name', 'womens_shot_12']
womens_shot_12.insert(3, "event_x", "Athletics Women's Shot")
womens_shot_12.womens_shot_12 = [str(x) for x in womens_shot_12.womens_shot_12]
womens_shot_12.womens_shot_12 = [x.strip() for x in womens_shot_12.womens_shot_12]
womens_shot_12.womens_shot_12 = [x.strip('/ DPG') for x in womens_shot_12.womens_shot_12]
womens_shot_12.womens_shot_12 = [x.replace('h','.')for x in womens_shot_12.womens_shot_12]
womens_shot_12.womens_shot_12 = [x.replace(':','.')for x in womens_shot_12.womens_shot_12]
womens_shot_12.name = [x.strip() for x in womens_shot_12.name]
womens_shot_12 = womens_shot_12.drop(41)


athlete_df = pd.merge(athlete_df, womens_shot_12[['name', 'womens_shot_12']], how='left', on='name', suffixes=(None, '_shot_womens_london'))


#### Scraping results for the women's shot put throw 2016-Rio


In [358]:
rio_womens_shot = fn.olympic_query('rio', '2016', 'shot-put-women')
womens_shot_16 = fn.olympic_scraper(rio_womens_shot)
womens_shot_16 = fn.content_cleaner(womens_shot_16)
womens_shot_16.columns = ['rank', 'name', 'womens_shot_16']
womens_shot_16.insert(3, "event_x", "Athletics Women's Shot")
womens_shot_16.womens_shot_16 = [str(x) for x in womens_shot_16.womens_shot_16]
womens_shot_16.womens_shot_16 = [x.strip() for x in womens_shot_16.womens_shot_16]
womens_shot_16.womens_shot_16 = [x.strip('/ DPG') for x in womens_shot_16.womens_shot_16]
womens_shot_16.womens_shot_16 = [x.replace('h','.')for x in womens_shot_16.womens_shot_16]
womens_shot_16.womens_shot_16 = [x.replace(':','.')for x in womens_shot_16.womens_shot_16]
womens_shot_16.name = [x.strip() for x in womens_shot_16.name]


athlete_df = pd.merge(athlete_df, womens_shot_16[['name', 'womens_shot_16']], how='left', on='name', suffixes=(None, '_shot_womens_rio'))


#### Scraping results for the women's javelin 2004-Athens


In [359]:
athens_womens_javelin = fn.olympic_query('athens', '2004', 'javelin-throw-women')
womens_javelin_04 = fn.olympic_scraper(athens_womens_javelin)
womens_javelin_04 = fn.content_cleaner(womens_javelin_04)
womens_javelin_04.columns = ['rank', 'name', 'womens_javelin_04']
womens_javelin_04.insert(3, "event_x", "Athletics Women's Javelin")
womens_javelin_04.womens_javelin_04 = [str(x) for x in womens_javelin_04.womens_javelin_04]
womens_javelin_04.womens_javelin_04 = [x.strip() for x in womens_javelin_04.womens_javelin_04]
womens_javelin_04.womens_javelin_04 = [x.strip('/ DPG') for x in womens_javelin_04.womens_javelin_04]
womens_javelin_04.womens_javelin_04 = [x.replace('h','.')for x in womens_javelin_04.womens_javelin_04]
womens_javelin_04.womens_javelin_04 = [x.replace(':','.')for x in womens_javelin_04.womens_javelin_04]
womens_javelin_04.name = [x.strip() for x in womens_javelin_04.name]


athlete_df = pd.merge(athlete_df, womens_javelin_04[['name', 'womens_javelin_04']], how='left', on='name', suffixes=(None, '_javelin_womens_athens'))


#### Scraping results for the women's javelin 2008-Beijing
removing rows with missing values

In [360]:
beijing_womens_javelin = fn.olympic_query('beijing', '2008', 'javelin-throw-women')
womens_javelin_08 = fn.olympic_scraper(beijing_womens_javelin)
womens_javelin_08 = fn.content_cleaner(womens_javelin_08)
womens_javelin_08.columns = ['rank', 'name', 'womens_javelin_08']
womens_javelin_08.insert(3, "event_x", "Athletics Women's Javelin")
womens_javelin_08.womens_javelin_08 = [str(x) for x in womens_javelin_08.womens_javelin_08]
womens_javelin_08.womens_javelin_08 = [x.strip() for x in womens_javelin_08.womens_javelin_08]
womens_javelin_08.womens_javelin_08 = [x.strip('/ DPG') for x in womens_javelin_08.womens_javelin_08]
womens_javelin_08.womens_javelin_08 = [x.replace('h','.')for x in womens_javelin_08.womens_javelin_08]
womens_javelin_08.womens_javelin_08 = [x.replace(':','.')for x in womens_javelin_08.womens_javelin_08]
womens_javelin_08.name = [x.strip() for x in womens_javelin_08.name]
womens_javelin_08 = womens_javelin_08.drop([64,65,92,119])


athlete_df = pd.merge(athlete_df, womens_javelin_08[['name', 'womens_javelin_08']], how='left', on='name', suffixes=(None, '_javelin_womens_beijing'))


#### Scraping results for the women's javelin 2012-London
removing rows with missing values

In [361]:
london_womens_javelin = fn.olympic_query('london', '2012', 'javelin-throw-women')
womens_javelin_12 = fn.olympic_scraper(london_womens_javelin)
womens_javelin_12 = fn.content_cleaner(womens_javelin_12)
womens_javelin_12.columns = ['rank', 'name', 'womens_javelin_12']
womens_javelin_12.insert(3, "event_x", "Athletics Women's Javelin")
womens_javelin_12.womens_javelin_12 = [str(x) for x in womens_javelin_12.womens_javelin_12]
womens_javelin_12.womens_javelin_12 = [x.strip() for x in womens_javelin_12.womens_javelin_12]
womens_javelin_12.womens_javelin_12 = [x.strip('/ DPG') for x in womens_javelin_12.womens_javelin_12]
womens_javelin_12.womens_javelin_12 = [x.replace('h','.')for x in womens_javelin_12.womens_javelin_12]
womens_javelin_12.womens_javelin_12 = [x.replace(':','.')for x in womens_javelin_12.womens_javelin_12]
womens_javelin_12.name = [x.strip() for x in womens_javelin_12.name]
womens_javelin_12 = womens_javelin_12.drop(range(50,54))


athlete_df = pd.merge(athlete_df, womens_javelin_12[['name', 'womens_javelin_12']], how='left', on='name', suffixes=(None, '_javelin_womens_london'))


#### Scraping results for the women's javelin 2016-Rio
removing rows with missing values

In [362]:
rio_womens_javelin = fn.olympic_query('rio', '2016', 'javelin-throw-women')
womens_javelin_16 = fn.olympic_scraper(rio_womens_javelin)
womens_javelin_16 = fn.content_cleaner(womens_javelin_16)
womens_javelin_16.columns = ['rank', 'name', 'womens_javelin_16']
womens_javelin_16.insert(3, "event_x", "Athletics Women's Javelin")
womens_javelin_16.womens_javelin_16 = [str(x) for x in womens_javelin_16.womens_javelin_16]
womens_javelin_16.womens_javelin_16 = [x.strip() for x in womens_javelin_16.womens_javelin_16]
womens_javelin_16.womens_javelin_16 = [x.strip('/ DPG') for x in womens_javelin_16.womens_javelin_16]
womens_javelin_16.womens_javelin_16 = [x.replace('h','.')for x in womens_javelin_16.womens_javelin_16]
womens_javelin_16.womens_javelin_16 = [x.replace(':','.')for x in womens_javelin_16.womens_javelin_16]
womens_javelin_16.name = [x.strip() for x in womens_javelin_16.name]
 

athlete_df = pd.merge(athlete_df, womens_javelin_16[['name', 'womens_javelin_16']], how='left', on='name', suffixes=(None, '_javelin_womens_rio'))


#### Scraping results from the women's pole vault 2004-Athens


In [363]:
athens_womens_pole = fn.olympic_query('athens', '2004', 'pole-vault-women')
womens_pole_04 = fn.olympic_scraper(athens_womens_pole)
womens_pole_04 = fn.content_cleaner(womens_pole_04)
womens_pole_04.columns = ['rank', 'name', 'womens_pole_04']
womens_pole_04.insert(3, "event_x", "Athletics Women's pole")
womens_pole_04.womens_pole_04 = [str(x) for x in womens_pole_04.womens_pole_04]
womens_pole_04.womens_pole_04 = [x.strip() for x in womens_pole_04.womens_pole_04]
womens_pole_04.womens_pole_04 = [x.strip('/ DPG') for x in womens_pole_04.womens_pole_04]
womens_pole_04.womens_pole_04 = [x.replace('h','.')for x in womens_pole_04.womens_pole_04]
womens_pole_04.womens_pole_04 = [x.replace(':','.')for x in womens_pole_04.womens_pole_04]
womens_pole_04.name = [x.strip() for x in womens_pole_04.name]


athlete_df = pd.merge(athlete_df, womens_pole_04[['name', 'womens_pole_04']], how='left', on='name', suffixes=(None, '_pole_womens_athens'))


#### Scraping results from the women's pole vault 2008-Beijing
removing rows with missing values

In [364]:
beijing_womens_pole = fn.olympic_query('beijing', '2008', 'pole-vault-women')
womens_pole_08 = fn.olympic_scraper(beijing_womens_pole)
womens_pole_08 = fn.content_cleaner(womens_pole_08)
womens_pole_08.columns = ['rank', 'name', 'womens_pole_08']
womens_pole_08.insert(3, "event_x", "Athletics Women's pole")
womens_pole_08.womens_pole_08 = [str(x) for x in womens_pole_08.womens_pole_08]
womens_pole_08.womens_pole_08 = [x.strip() for x in womens_pole_08.womens_pole_08]
womens_pole_08.womens_pole_08 = [x.strip('/ DPG') for x in womens_pole_08.womens_pole_08]
womens_pole_08.womens_pole_08 = [x.replace('h','.')for x in womens_pole_08.womens_pole_08]
womens_pole_08.womens_pole_08 = [x.replace(':','.')for x in womens_pole_08.womens_pole_08]
womens_pole_08.name = [x.strip() for x in womens_pole_08.name]
womens_pole_08 = womens_pole_08.drop([46,47,82,83])


athlete_df = pd.merge(athlete_df, womens_pole_08[['name', 'womens_pole_08']], how='left', on='name', suffixes=(None, '_pole_womens_beijing'))


#### Scraping results from the women's pole vault 2012-London
removing rows with missing values

In [365]:
london_womens_pole = fn.olympic_query('london', '2012', 'pole-vault-women')
womens_pole_12 = fn.olympic_scraper(london_womens_pole)
womens_pole_12 = fn.content_cleaner(womens_pole_12)
womens_pole_12.columns = ['rank', 'name', 'womens_pole_12']
womens_pole_12.insert(3, "event_x", "Athletics Women's pole")
womens_pole_12.womens_pole_12 = [str(x) for x in womens_pole_12.womens_pole_12]
womens_pole_12.womens_pole_12 = [x.strip() for x in womens_pole_12.womens_pole_12]
womens_pole_12.womens_pole_12 = [x.strip('/ DPG') for x in womens_pole_12.womens_pole_12]
womens_pole_12.womens_pole_12 = [x.replace('h','.')for x in womens_pole_12.womens_pole_12]
womens_pole_12.womens_pole_12 = [x.replace(':','.')for x in womens_pole_12.womens_pole_12]
womens_pole_12.name = [x.strip() for x in womens_pole_12.name]
womens_pole_12 = womens_pole_12.drop([11,47,48,49,50,51])


athlete_df = pd.merge(athlete_df, womens_pole_12[['name', 'womens_pole_12']], how='left', on='name', suffixes=(None, '_pole_womens_london'))


#### Scraping results from the women's pole vault 2016-Rio


In [366]:
rio_womens_pole = fn.olympic_query('rio', '2016', 'pole-vault-women')
womens_pole_16 = fn.olympic_scraper(rio_womens_pole)
womens_pole_16 = fn.content_cleaner(womens_pole_16)
womens_pole_16.columns = ['rank', 'name', 'womens_pole_16']
womens_pole_16.insert(3, "event_x", "Athletics Women's Pole")
womens_pole_16.womens_pole_16 = [str(x) for x in womens_pole_16.womens_pole_16]
womens_pole_16.womens_pole_16 = [x.strip() for x in womens_pole_16.womens_pole_16]
womens_pole_16.womens_pole_16 = [x.strip('/ DPG') for x in womens_pole_16.womens_pole_16]
womens_pole_16.womens_pole_16 = [x.replace('h','.')for x in womens_pole_16.womens_pole_16]
womens_pole_16.womens_pole_16 = [x.replace(':','.')for x in womens_pole_16.womens_pole_16]
womens_pole_16.name = [x.strip() for x in womens_pole_16.name]


athlete_df = pd.merge(athlete_df, womens_pole_16[['name', 'womens_pole_16']], how='left', on='name', suffixes=(None, '_pole_womens_rio'))


#### Scraping results from the women's long jump 2004-Athens


In [367]:
athens_womens_long = fn.olympic_query('athens', '2004', 'long-jump-women')
womens_long_04 = fn.olympic_scraper(athens_womens_long)
womens_long_04 = fn.content_cleaner(womens_long_04)
womens_long_04.columns = ['rank', 'name', 'womens_long_04']
womens_long_04.insert(3, "event_x", "Athletics Women's Long Jump")
womens_long_04.womens_long_04 = [str(x) for x in womens_long_04.womens_long_04]
womens_long_04.womens_long_04 = [x.strip() for x in womens_long_04.womens_long_04]
womens_long_04.womens_long_04 = [x.strip('/ DPG') for x in womens_long_04.womens_long_04]
womens_long_04.womens_long_04 = [x.replace('h','.')for x in womens_long_04.womens_long_04]
womens_long_04.womens_long_04 = [x.replace(':','.')for x in womens_long_04.womens_long_04]
womens_long_04.name = [x.strip() for x in womens_long_04.name]


athlete_df = pd.merge(athlete_df, womens_long_04[['name', 'womens_long_04']], how='left', on='name', suffixes=(None, '_long_womens_athens'))


#### Scraping results from the women's long jump 2008-Beijing
removing rows with missing values

In [368]:
beijing_womens_long = fn.olympic_query('beijing', '2008', 'long-jump-women')
womens_long_08 = fn.olympic_scraper(beijing_womens_long)
womens_long_08 = fn.content_cleaner(womens_long_08)
womens_long_08.columns = ['rank', 'name', 'womens_long_08']
womens_long_08.insert(3, "event_x", "Athletics Women's Long Jump")
womens_long_08.womens_long_08 = [str(x) for x in womens_long_08.womens_long_08]
womens_long_08.womens_long_08 = [x.strip() for x in womens_long_08.womens_long_08]
womens_long_08.womens_long_08 = [x.strip('/ DPG') for x in womens_long_08.womens_long_08]
womens_long_08.womens_long_08 = [x.replace('h','.')for x in womens_long_08.womens_long_08]
womens_long_08.womens_long_08 = [x.replace(':','.')for x in womens_long_08.womens_long_08]
womens_long_08.name = [x.strip() for x in womens_long_08.name]
womens_long_08 = womens_long_08.drop([48,49,51,52,73,74,93,94])


athlete_df = pd.merge(athlete_df, womens_long_08[['name', 'womens_long_08']], how='left', on='name', suffixes=(None, '_long_womens_beijing'))


#### Scraping results from the women's long jump 2012-London
removing rows with missing values

In [369]:
london_womens_long = fn.olympic_query('london', '2012', 'long-jump-women')
womens_long_12 = fn.olympic_scraper(london_womens_long)
womens_long_12 = fn.content_cleaner(womens_long_12)
womens_long_12.columns = ['rank', 'name', 'womens_long_12']
womens_long_12.insert(3, "event_x", "Athletics Women's Long Jump")
womens_long_12.womens_long_12 = [str(x) for x in womens_long_12.womens_long_12]
womens_long_12.womens_long_12 = [x.strip() for x in womens_long_12.womens_long_12]
womens_long_12.womens_long_12 = [x.strip('/ DPG') for x in womens_long_12.womens_long_12]
womens_long_12.womens_long_12 = [x.replace('h','.')for x in womens_long_12.womens_long_12]
womens_long_12.womens_long_12 = [x.replace(':','.')for x in womens_long_12.womens_long_12]
womens_long_12.name = [x.strip() for x in womens_long_12.name]
womens_long_12 = womens_long_12.drop([8,37,38,39,40])


athlete_df = pd.merge(athlete_df, womens_long_12[['name', 'womens_long_12']], how='left', on='name', suffixes=(None, '_long_womens_london'))


#### Scraping results from the women's long jump 2016-Rio
removing row with missing value

In [370]:
rio_womens_long = fn.olympic_query('rio', '2016', 'long-jump-women')
womens_long_16 = fn.olympic_scraper(rio_womens_long)
womens_long_16 = fn.content_cleaner(womens_long_16)
womens_long_16.columns = ['rank', 'name', 'womens_long_16']
womens_long_16.insert(3, "event_x", "Athletics Men's Long Jump")
womens_long_16.womens_long_16 = [str(x) for x in womens_long_16.womens_long_16]
womens_long_16.womens_long_16 = [x.strip() for x in womens_long_16.womens_long_16]
womens_long_16.womens_long_16 = [x.strip('/ DPG') for x in womens_long_16.womens_long_16]
womens_long_16.womens_long_16 = [x.replace('h','.')for x in womens_long_16.womens_long_16]
womens_long_16.womens_long_16 = [x.replace(':','.')for x in womens_long_16.womens_long_16]
womens_long_16.name = [x.strip() for x in womens_long_16.name]
womens_long_16 = womens_long_16.drop(11)


athlete_df = pd.merge(athlete_df, womens_long_16[['name', 'womens_long_16']], how='left', on='name', suffixes=(None, '_long_womens_rio'))


#### Scraping results from the women's high jump 2004-Athens
removing rows with missing values

In [371]:
athens_womens_high = fn.olympic_query('athens', '2004', 'high-jump-women')
womens_high_04 = fn.olympic_scraper(athens_womens_high)
womens_high_04 = fn.content_cleaner(womens_high_04)
womens_high_04.columns = ['rank', 'name', 'womens_high_04']
womens_high_04.insert(3, "event_x", "Athletics Women's High Jump")
womens_high_04.womens_high_04 = [str(x) for x in womens_high_04.womens_high_04]
womens_high_04.womens_high_04 = [x.strip() for x in womens_high_04.womens_high_04]
womens_high_04.womens_high_04 = [x.strip('/ DPG') for x in womens_high_04.womens_high_04]
womens_high_04.name = [x.strip() for x in womens_high_04.name]


athlete_df = pd.merge(athlete_df, womens_high_04[['name', 'womens_high_04']], how='left', on='name', suffixes=(None, '_high_womens_athens'))


#### Scraping results from the women's high jump 2008-Beijing
removing rows with missing values

In [372]:
beijing_womens_high = fn.olympic_query('beijing', '2008', 'high-jump-women')
womens_high_08 = fn.olympic_scraper(beijing_womens_high)
womens_high_08 = fn.content_cleaner(womens_high_08)
womens_high_08.columns = ['rank', 'name', 'womens_high_08']
womens_high_08.insert(3, "event_x", "Athletics Women's High Jump")
womens_high_08.womens_high_08 = [str(x) for x in womens_high_08.womens_high_08]
womens_high_08.womens_high_08 = [x.strip() for x in womens_high_08.womens_high_08]
womens_high_08.womens_high_08 = [x.strip('/ DPG') for x in womens_high_08.womens_high_08]
womens_high_08.name = [x.strip() for x in womens_high_08.name]
womens_high_08 = womens_high_08.drop([42,43,76,77])


athlete_df = pd.merge(athlete_df, womens_high_08[['name', 'womens_high_08']], how='left', on='name', suffixes=(None, '_high_womens_beijing'))


#### Scraping results from the women's high jump 2012-London
removing rows with missing values

In [373]:
london_womens_high = fn.olympic_query('london', '2012', 'high-jump-women')
womens_high_12 = fn.olympic_scraper(london_womens_high)
womens_high_12 = fn.content_cleaner(womens_high_12)
womens_high_12.columns = ['rank', 'name', 'womens_high_12']
womens_high_12.insert(3, "event_x", "Athletics Women's High Jump")
womens_high_12.womens_high_12 = [str(x) for x in womens_high_12.womens_high_12]
womens_high_12.womens_high_12 = [x.strip() for x in womens_high_12.womens_high_12]
womens_high_12.womens_high_12 = [x.strip('/ DPG') for x in womens_high_12.womens_high_12]
womens_high_12.name = [x.strip() for x in womens_high_12.name]
womens_high_12 = womens_high_12.drop(45)


athlete_df = pd.merge(athlete_df, womens_high_12[['name', 'womens_high_12']], how='left', on='name', suffixes=(None, '_high_womens_london'))


#### Scraping results from the women's high jump 2016-Rio
removing rows with missing values

In [374]:
rio_womens_high = fn.olympic_query('rio', '2016', 'high-jump-women')
womens_high_16 = fn.olympic_scraper(rio_womens_high)
womens_high_16 = fn.content_cleaner(womens_high_16)
womens_high_16.columns = ['rank', 'name', 'womens_high_16']
womens_high_16.insert(3, "event_x", "Athletics Men's High Jump")
womens_high_16.womens_high_16 = [str(x) for x in womens_high_16.womens_high_16]
womens_high_16.womens_high_16 = [x.strip() for x in womens_high_16.womens_high_16]
womens_high_16.womens_high_16 = [x.strip('/ DPG') for x in womens_high_16.womens_high_16]
womens_high_16.name = [x.strip() for x in womens_high_16.name]
womens_high_16



,rank,name,womens_high_16,event_x
0,1.,Ruth Beitia,,Athletics Men's High Jump
1,2.,Mirela Demireva,,Athletics Men's High Jump
2,3.,Blanka Vlasic,,Athletics Men's High Jump
3,4.,Chaunte Lowe,,Athletics Men's High Jump
4,5.,Alessia Trost,,Athletics Men's High Jump
5,6.,Levern Spencer,,Athletics Men's High Jump
6,7.,Marie-Laurence Jungfleisch,,Athletics Men's High Jump
7,7.,Sofie Skoog,,Athletics Men's High Jump


### There were no results for the women's high jump in the Rio Summer Olympic Games. I am going to continue with the other events for now. There are still rankings so I will see if I can retrieve the results from another site.

#### Scraping results from the women's triple jump 2004-Athens


In [375]:
athens_womens_triple = fn.olympic_query('athens', '2004', 'triple-jump-women')
womens_triple_04 = fn.olympic_scraper(athens_womens_triple)
womens_triple_04 = fn.content_cleaner(womens_triple_04)
womens_triple_04.columns = ['rank', 'name', 'womens_triple_04']
womens_triple_04.insert(3, "event_x", "Athletics Women's Triple Jump")
womens_triple_04.womens_triple_04 = [str(x) for x in womens_triple_04.womens_triple_04]
womens_triple_04.womens_triple_04 = [x.strip() for x in womens_triple_04.womens_triple_04]
womens_triple_04.womens_triple_04 = [x.strip('/ DPG') for x in womens_triple_04.womens_triple_04]
womens_triple_04.name = [x.strip() for x in womens_triple_04.name]


athlete_df = pd.merge(athlete_df, womens_triple_04[['name', 'womens_triple_04']], how='left', on='name', suffixes=(None, '_triple_womens_athens'))


#### Scraping results from the women's triple jump 2008-Beijing
removing rows with missing values

In [376]:
beijing_womens_triple = fn.olympic_query('beijing', '2008', 'triple-jump-women')
womens_triple_08 = fn.olympic_scraper(beijing_womens_triple)
womens_triple_08 = fn.content_cleaner(womens_triple_08)
womens_triple_08.columns = ['rank', 'name', 'womens_triple_08']
womens_triple_08.insert(3, "event_x", "Athletics Women's Triple Jump")
womens_triple_08.womens_triple_08 = [str(x) for x in womens_triple_08.womens_triple_08]
womens_triple_08.womens_triple_08 = [x.strip() for x in womens_triple_08.womens_triple_08]
womens_triple_08.womens_triple_08 = [x.strip('/ DPG') for x in womens_triple_08.womens_triple_08]
womens_triple_08.name = [x.strip() for x in womens_triple_08.name]
womens_triple_08 = womens_triple_08.drop([42,43,44,45,65,79,80,82,83])


athlete_df = pd.merge(athlete_df, womens_triple_08[['name', 'womens_triple_08']], how='left', on='name', suffixes=(None, '_triple_womens_beijing'))


#### Scraping results from the women's triple jump 2012-London
removing rows with missing values

In [377]:
london_womens_triple = fn.olympic_query('london', '2012', 'triple-jump-women')
womens_triple_12 = fn.olympic_scraper(london_womens_triple)
womens_triple_12 = fn.content_cleaner(womens_triple_12)
womens_triple_12.columns = ['rank', 'name', 'womens_triple_12']
womens_triple_12.insert(3, "event_x", "Athletics Men's High Jump")
womens_triple_12.womens_triple_12 = [str(x) for x in womens_triple_12.womens_triple_12]
womens_triple_12.womens_triple_12 = [x.strip() for x in womens_triple_12.womens_triple_12]
womens_triple_12.womens_triple_12 = [x.strip('/ DPG') for x in womens_triple_12.womens_triple_12]
womens_triple_12.name = [x.strip() for x in womens_triple_12.name]
womens_triple_12 = womens_triple_12.drop([44,45])


athlete_df = pd.merge(athlete_df, womens_triple_12[['name', 'womens_triple_12']], how='left', on='name', suffixes=(None, '_triple_womens_london'))


#### Scraping results from the women's triple jump 2016-Rio


In [378]:
rio_womens_triple = fn.olympic_query('rio', '2016', 'triple-jump-women')
womens_triple_16 = fn.olympic_scraper(rio_womens_triple)
womens_triple_16 = fn.content_cleaner(womens_triple_16)
womens_triple_16.columns = ['rank', 'name', 'womens_triple_16']
womens_triple_16.insert(3, "event_x", "Athletics Women's Triple Jump")
womens_triple_16.womens_triple_16 = [str(x) for x in womens_triple_16.womens_triple_16]
womens_triple_16.womens_triple_16 = [x.strip() for x in womens_triple_16.womens_triple_16]
womens_triple_16.womens_triple_16 = [x.strip('/ DPG') for x in womens_triple_16.womens_triple_16]
womens_triple_16.name = [x.strip() for x in womens_triple_16.name]


athlete_df = pd.merge(athlete_df, womens_triple_16[['name', 'womens_triple_16']], how='left', on='name', suffixes=(None, '_triple_womens_rio'))


In [379]:
athlete_df = athlete_df.drop_duplicates()

In [ ]:
athlete_df.to_csv('../data/model_df_4.csv', index=False)

# 

## Preparing dataframe for modeling

In [ ]:
athlete_df = pd.read_csv('../data/model_df_4.csv', low_memory=False)

In [459]:
len(athlete_df)

5880

#### Defining features that contain event results so I can get a count on how many events an athlete participated in.

In [461]:
result_cols = athlete_df.columns[16:170]

In [462]:
results_df = athlete_df[result_cols]
results_df.insert(153, 'events', 0)

The cell below may throw a 'setting with copy' warning but if you run it twice it works. I am counting the values across the columns of the dataframe and subtracting by 1 in order to see which athletes need to be dropped for now.

In [463]:
index = athlete_df.name
mask = list(results_df.columns)

#### Counting the number of event results by axis 1 (columns)

In [464]:
results_df['events'] = results_df.loc[:, (mask)].count(axis=1) -1
results_df['events'].value_counts()
results_df.index = index
results_df.reset_index(inplace=True)

/opt/anaconda3/envs/olympic-env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


#### Merging the event counts to the athlete dataframe

In [465]:
athlete_df = pd.merge(athlete_df, results_df[['name', 'events']], how='left', on='name')
athlete_df.events.value_counts()

0    3173
4    3078
5    2626
2    2500
1    2435
3    2058
6     256
Name: events, dtype: int64

In [466]:
athlete_df = athlete_df.drop_duplicates()
athlete_df.events.value_counts()

0    3173
1    1593
2     682
3     226
4     108
5      82
6      16
Name: events, dtype: int64

In [467]:
for x in athlete_df[result_cols]:
    athlete_df.loc[:,x] = [str(x) for x in athlete_df.loc[:,x]]
    athlete_df.loc[:,x] = [x.strip('\r\n') for x in athlete_df.loc[:,x]]

#### Defining the events with the format minute:seconds.milliseconds so I can convert the values to seconds. This is done in two for loops since the order of the events has times that aren't in this format between these columns

In [468]:
to_seconds_1 = athlete_df.columns[28:40]
to_seconds_1 = to_seconds_1.append(athlete_df.columns[107:119])
to_seconds_1

Index(['mens_800m_04', 'mens_800m_08', 'mens_800m_12', 'mens_800m_16', 'mens_1500m_04', 'mens_1500m_08', 'mens_1500m_12', 'mens_1500m_16', 'mens_5000m_04', 'mens_5000m_08', 'mens_5000m_12', 'mens_5000m_16', 'womens_800m_04', 'womens_800m_08', 'womens_800m_12', 'womens_800m_16', 'womens_1500m_04', 'womens_1500m_08', 'womens_1500m_12', 'womens_1500m_16', 'womens_5000m_04', 'womens_5000m_08', 'womens_5000m_12', 'womens_5000m_16'], dtype='object')

In [469]:
athlete_df.columns[48:64]

Index(['mens_10000m_04', 'mens_10000m_08', 'mens_10000m_12', 'mens_10000m_16', 'mens_20km', 'mens_20km_08', 'mens_20km_12', 'mens_20km_16', 'mens_50km_04', 'mens_50km_08', 'mens_50km_12', 'mens_50km_16', 'mens_3000Steeple_04', 'mens_3000Steeple_08', 'mens_3000Steeple_12', 'mens_3000Steeple_16'], dtype='object')

In [470]:
athlete_df.columns[127:138]

Index(['womens_10000m_04', 'womens_10000m_08', 'womens_10000m_12', 'womens_10000m_16', 'womens_20km', 'womens_20km_08', 'womens_20km_12', 'womens_20km_16', 'womens_3000Steeple_08', 'womens_3000Steeple_12', 'womens_3000Steeple_16'], dtype='object')

In [471]:
to_seconds_2 = athlete_df.columns[48:64]
to_seconds_2 = to_seconds_2.append(athlete_df.columns[127:138])
to_seconds_2

Index(['mens_10000m_04', 'mens_10000m_08', 'mens_10000m_12', 'mens_10000m_16', 'mens_20km', 'mens_20km_08', 'mens_20km_12', 'mens_20km_16', 'mens_50km_04', 'mens_50km_08', 'mens_50km_12', 'mens_50km_16', 'mens_3000Steeple_04', 'mens_3000Steeple_08', 'mens_3000Steeple_12', 'mens_3000Steeple_16', 'womens_10000m_04', 'womens_10000m_08', 'womens_10000m_12', 'womens_10000m_16', 'womens_20km', 'womens_20km_08', 'womens_20km_12', 'womens_20km_16', 'womens_3000Steeple_08', 'womens_3000Steeple_12',
       'womens_3000Steeple_16'],
      dtype='object')

#### Replacing the nan values with a sting time of 0.00.00 in order to convert the time to seconds.

In [472]:
for x in athlete_df[to_seconds_1]:
    #df[x] = df[x].fillna(value='0.00.00')
    athlete_df.loc[:,x] = [str(x) for x in athlete_df.loc[:,x]]
    athlete_df.loc[:,x] = [x.replace('nan', '0.00.00') for x in athlete_df.loc[:,x]]
for x in athlete_df[to_seconds_2]:
   # df[x] = df[x].fillna(value='0.00.00')
    athlete_df.loc[:,x] = [str(x) for x in athlete_df.loc[:,x]]
    athlete_df.loc[:,x] = [x.replace('nan', '0.00.00') for x in athlete_df.loc[:,x]]


In [473]:
for x in athlete_df[to_seconds_1]:
    athlete_df.loc[:, x] = [str(x) for x  in athlete_df.loc[:, x]]
    athlete_df.loc[:, x] = [(int(a) * 60 )+ int(b) + (int(c) / 1000) for a,b,c in athlete_df.loc[:, x].str.split('.')]
for x in athlete_df[to_seconds_2]:
    athlete_df.loc[:, x] = [str(x) for x  in athlete_df.loc[:, x]]
    athlete_df.loc[:, x] = [(int(a) * 60 )+ int(b) + (int(c) / 1000) for a,b,c in athlete_df.loc[:, x].str.split('.')]

In [474]:
in_seconds_1 = athlete_df.columns[16:28]
in_seconds_1 = in_seconds_1.append(athlete_df.columns[40:38])
in_seconds_2 = athlete_df.columns[95:107]
in_seconds_2 = in_seconds_2.append(athlete_df.columns[109:117])

#### Filling the missing values with 0 since the athlete did not compete in the event.

In [475]:
for x in athlete_df[in_seconds_1]:
    athlete_df[x] = athlete_df[x].fillna(value=0.00)
for x in athlete_df[in_seconds_2]:
    athlete_df[x] = athlete_df[x].fillna(value=0.00)

In [476]:
field = athlete_df.columns[64:95]
field = field.append(athlete_df.columns[128:169])

In [477]:
for x in athlete_df[field]:
    athlete_df.loc[:, x] = [str(x) for x in athlete_df.loc[:, x]]

In [478]:
athlete_df.flagged = [float(x) for x in athlete_df.flagged]
for x in athlete_df.loc[:, athlete_df.columns[16:]]:
    athlete_df.loc[:, x] = athlete_df.loc[:, x].fillna(float(0.0))
    athlete_df.loc[:, x] = [float(x) for x in athlete_df.loc[:, x]]

#### To fill in the missing values in the weight and height features, I am going to sort the dataframe by event and sex and then use the forward fill method, the last valid value.

In [479]:
athlete_df.loc[athlete_df.weight.isna()]['event'].value_counts()

Athletics Women's 100 metres                   16
Athletics Men's 100 metres                     16
Athletics Men's 5,000 metres                   13
Athletics Men's 1,500 metres                    9
Athletics Men's 200 metres                      9
Athletics Women's 800 metres                    7
Athletics Women's 400 metres                    7
Athletics Men's 800 metres                      6
Athletics Women's 200 metres                    5
Athletics Men's 10,000 metres                   5
Athletics Women's 10,000 metres                 4
Athletics Women's 1,500 metres                  4
Athletics Women's 5,000 metres                  4
Athletics Women's 100 metres Hurdles            4
Athletics Women's Long Jump                     4
Athletics Women's 3,000 metres Steeplechase     3
Athletics Men's 400 metres                      3
Athletics Men's 400 metres Hurdles              3
Athletics Women's Javelin Throw                 2
Athletics Women's 20 kilometres Walk            2


In [480]:
athlete_df = athlete_df.sort_values(['event', 'sex'])

In [481]:
athlete_df.weight.isna().sum()

143

In [482]:
athlete_df.weight = athlete_df.weight.fillna(method='ffill')
athlete_df.height = athlete_df.height.fillna(method='ffill')

#### I can fill in the remaining missing values with 0 since all of the event result features with missing values are due to either the athlete not competing in that event or I was unable to match on the name. The medal feature's mising values will be fill with zero to indicate the athlete did not win a medal during their event. 

In [483]:
athlete_df.isna().sum()

id                          0
name                        0
sex                         0
age                         0
height                      0
weight                      0
team                        0
noc                         0
games                       0
year                        0
season                      0
city                        0
sport                       0
event                       0
medal                    5406
flagged                     0
mens_100m_04             5856
mens_100m_08             5738
mens_100m_12             5763
mens_100m_16             5869
mens_200m_04             5878
mens_200m_08             5797
mens_200m_12             5803
mens_200m_16             5865
mens_400m_04             5874
mens_400m_08             5831
mens_400m_12             5840
mens_400m_16             5871
mens_800m_04                0
mens_800m_08                0
mens_800m_12                0
mens_800m_16                0
mens_1500m_04               0
mens_1500m

#### Converting the medal, sex, and event feature values to numeric so the model will be able to use it as a categorical variable

In [484]:
athlete_df.medal = athlete_df.medal.fillna(0.0)

In [485]:
medal_to_numeric = {'Gold': 1,
                    'Silver': 2,
                    'Bronze': 3}
athlete_df.medal = athlete_df.medal.replace(medal_to_numeric)

In [486]:
sex_to_numeric = {'M': 1,
                  'F': 0}
athlete_df.sex = athlete_df.sex.replace(sex_to_numeric)

In [487]:
athlete_df.medal.value_counts()

0.0    5406
2.0     194
3.0     145
1.0     135
Name: medal, dtype: int64

In [488]:
athlete_df.sex.value_counts()

1    3057
0    2823
Name: sex, dtype: int64

In [489]:
athlete_df.flagged = [float(x) for x in athlete_df.flagged]
athlete_df.medal = athlete_df.medal.fillna(float(0.0))
for x in athlete_df[athlete_df.columns[15:]]:
    athlete_df[x] = athlete_df[x].fillna(float(0.0))
    athlete_df[x] = [float(x) for x in athlete_df[x]]

In [490]:
athlete_df = athlete_df.drop('id', axis=1)

In [491]:
len(athlete_df[athlete_df.events == 0])

3173

In [492]:
len(athlete_df)

5880

In [493]:
len(athlete_df.name.unique())

4817

In [494]:
athlete_df[athlete_df.events != 0]['flagged'].value_counts()

0.0    2462
1.0     245
Name: flagged, dtype: int64

#### There are 3,173 athletes in the dataframe with no results, I am going to remove them for now. There are 5,880 records currently in the dataframe with many athletes appearing more than once, this is due to an athlete competing in more than one event as well as more than one Summer Olympics Games.

In [495]:
athlete_df = athlete_df[athlete_df.events != 0]

In [496]:
athlete_df.isna().sum()

name                     0
sex                      0
age                      0
height                   0
weight                   0
team                     0
noc                      0
games                    0
year                     0
season                   0
city                     0
sport                    0
event                    0
medal                    0
flagged                  0
mens_100m_04             0
mens_100m_08             0
mens_100m_12             0
mens_100m_16             0
mens_200m_04             0
mens_200m_08             0
mens_200m_12             0
mens_200m_16             0
mens_400m_04             0
mens_400m_08             0
mens_400m_12             0
mens_400m_16             0
mens_800m_04             0
mens_800m_08             0
mens_800m_12             0
mens_800m_16             0
mens_1500m_04            0
mens_1500m_08            0
mens_1500m_12            0
mens_1500m_16            0
mens_5000m_04            0
mens_5000m_08            0
m

In [497]:
athlete_df.event.value_counts(normalize=True, ascending=True)

Athletics Men's 50 kilometres Walk             0.007758
Athletics Women's Shot Put                     0.008496
Athletics Women's High Jump                    0.008866
Athletics Women's Discus Throw                 0.009235
Athletics Men's Triple Jump                    0.010344
Athletics Women's Triple Jump                  0.011452
Athletics Men's 10,000 metres                  0.013299
Athletics Men's Long Jump                      0.013299
Athletics Men's Hammer Throw                   0.013668
Athletics Women's Pole Vault                   0.014407
Athletics Men's Pole Vault                     0.015146
Athletics Men's High Jump                      0.015515
Athletics Women's 10,000 metres                0.016254
Athletics Women's Hammer Throw                 0.016624
Athletics Men's 20 kilometres Walk             0.018840
Athletics Men's Shot Put                       0.018840
Athletics Men's Javelin Throw                  0.019209
Athletics Women's 5,000 metres                 0

In [498]:
event_to_numeric = {"Athletics Women's High Jump": 0,
                    "Athletics Women's Discus Throw": 1,
                    "Athletics Women's Shot Put": 2,
                    "Athletics Women's Triple Jump": 3,
                    "Athletics Men's High Jump": 4,
                    "Athletics Men's 50 kilometres Walk": 5,
                    "Athletics Men's Long Jump": 6,
                    "Athletics Men's 10,000 metres": 7,
                    "Athletics Men's Triple Jump": 8,
                    "Athletics Men's Pole Vault": 9,
                    "Athletics Men's 20 kilometres Walk": 10,
                    "Athletics Women's 3,000 metres Steeplechase": 11,
                    "Athletics Women's Pole Vault": 12,
                    "Athletics Women's 10,000 metres": 13,
                    "Athletics Women's 20 kilometres Walk": 14,
                    "Athletics Women's Long Jump": 15,
                    "Athletics Women's 400 metres Hurdles": 16,
                    "Athletics Men's Hammer Throw": 17,
                    "Athletics Men's Javelin Throw": 18,
                    "Athletics Women's 5,000 metres": 19,
                    "Athletics Men's 400 metres Hurdles": 20,
                    "Athletics Women's Hammer Throw": 21,
                    "Athletics Women's Javelin Throw": 22,
                    "Athletics Men's Shot Put": 23,
                    "Athletics Men's 3,000 metres Steeplechase": 25,
                    "Athletics Women's 100 metres Hurdles": 26,
                    "Athletics Men's 110 metres Hurdles": 27,
                    "Athletics Men's 400 metres": 28,
                    "Athletics Women's 1,500 metres": 29,
                    "Athletics Men's Discus Throw": 30,
                    "Athletics Men's 5,000 metres": 31,
                    "Athletics Women's 800 metres": 32,
                    "Athletics Men's 800 metres": 33,
                    "Athletics Men's 1,500 metres": 34,
                    "Athletics Women's 400 metres": 35,
                    "Athletics Men's 200 metres": 36,
                    "Athletics Men's 100 metres": 37,
                    "Athletics Women's 100 metres": 38,
                    "Athletics Women's 200 metres": 39}

In [499]:
athlete_df.event = athlete_df.event.replace(event_to_numeric)

In [506]:
len(athlete_df)

2707

#### Saving the dataframe as a .csv file for modeling

In [507]:
athlete_df.to_csv('../data/model_df_5.csv', index=False)